In [ ]:
!nvidia-smi

Sun May 22 05:24:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
import sys
drive.mount('/content/drive')
#设置路径
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

Mounted at /content/drive


In [ ]:
! pip install torch==1.6.0

     |████████████████████████████████| 748.8 MB 18 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.


In [ ]:
! pip install transformers==4.0.1

     |████████████████████████████████| 1.4 MB 33.1 MB/s 
     |████████████████████████████████| 2.9 MB 33.8 MB/s 
     |████████████████████████████████| 880 kB 65.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=839ca8f6455b7e75b6a8b025b67c20d8ac44ec0224b3c98193daaa8bbfbd24c7
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [ ]:
! pip install torchvision == 0.7.0

ERROR: Invalid requirement: '=='


In [ ]:
import random
import numpy as np
import torch
config = {
        'train_file_path': '/content/drive/MyDrive/Colab Notebooks/dataset/ESIM/train.json',
        'dev_file_path': '/content/drive/MyDrive/Colab Notebooks/dataset/ESIM/dev.json',
        'test_file_path': '/content/drive/MyDrive/Colab Notebooks/dataset/ESIM/test.json',
        'output_path': '.',
        'model_path': '/content/drive/MyDrive/Colab Notebooks/dataset/BERT_model',
        'batch_size': 64,
        'num_epochs': 1,
        'max_seq_len': 64,
        'decay': 0.99,
        'kd_coeff': 1.0,
        'learning_rate': 2e-5,
        'warmup_ratio': 0.05,
        'weight_decay': 0.01,
        'use_bucket': True,
        'bucket_multiplier': 200,
        'device': 'cuda',
        'n_gpus': 0,
        'use_amp': True,
        'logging_step': 300,
        'ema_start_step': 500,
        'ema_start': False,
        'seed': 2022
    }

if not torch.cuda.is_available():
    config['device'] = 'cpu'
else:
    config['n_gpus'] = torch.cuda.device_count()
    config['batch_size'] *= config['n_gpus']

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(config['seed'])

2022

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(config['model_path'])

In [ ]:
def build_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer):
  inputs_dict = tokenizer.encode_plus(sentence_a, sentence_b, add_special_tokens = True, return_token_type_ids =True, return_attention_mask = True)

  inputs['input_ids'].append(inputs_dict['input_ids'])
  inputs['token_type_ids'].append(inputs_dict['token_type_ids'])
  inputs['attention_mask'].append(inputs_dict['attention_mask'])
  inputs['labels'].append(label)

In [ ]:
import pandas as pd
from tqdm import tqdm
import json
def parse_data(path, data_type='train'):
  sentence_a = []
  sentence_b = []
  labels = []

  with open(path, 'r', encoding = 'utf8') as f:
    for line in tqdm(f.readlines(), desc=f'Reading {data_type} data'):
      line = json.loads(line)
      sentence_a.append(line['sentence1'])
      sentence_b.append(line['sentence2'])
      if data_type != 'test':
        labels.append(int(line['label']))
      else:
        labels.append(0)

  df = pd.DataFrame(zip(sentence_a, sentence_b, labels), columns = ['text_a', 'text_b', 'labels'])
  return df

In [ ]:
from collections import defaultdict
def read_data(config, tokenizer):
  train_df = parse_data(config['train_file_path'], data_type = 'train')
  dev_df = parse_data(config['dev_file_path'], data_type = 'dev')
  test_df = parse_data(config['test_file_path'], data_type = 'test')

  data_df = {'train': train_df, 'dev': dev_df, 'test': test_df}
  processed_data = {}
  
  for data_type, df in data_df.items():
    inputs = defaultdict(list)

    for i, row in tqdm(df.iterrows(), desc=f'Preprocessing {data_type} data', total = len(df)):
      label = row[2]
      sentence_a, sentence_b = row[0], row[1]
      build_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer)

    processed_data[data_type] = inputs
  
  return processed_data


In [ ]:
data = read_data(config, tokenizer)

Preprocessing test data: 100%|██████████| 3861/3861 [00:01<00:00, 2043.64it/s]


In [ ]:
class Collator:
  def __init__(self, max_seq_len, tokenizer):
    self.max_seq_len = max_seq_len
    self.tokenizer = tokenizer

  def pad_and_truncate(self, input_ids_list, token_type_ids_list, attention_mask_list, labels_list, max_seq_len):
    input_ids = torch.zeros((len(input_ids_list), max_seq_len), dtype = torch.long)
    token_type_ids = torch.zeros_like(input_ids)
    attention_mask = torch.zeros_like(input_ids)
    for i in range(len(input_ids_list)):
      seq_len = len(input_ids_list[i])
      if seq_len <= max_seq_len:
        input_ids[i, :seq_len] = torch.tensor(input_ids_list[i], dtype = torch.long)
        token_type_ids[i, :seq_len] = torch.tensor(token_type_ids_list[i], dtype = torch.long)
        attention_mask[i, :seq_len] = torch.tensor(attention_mask_list[i], dtype = torch.long)

      else:
        input_ids[i] = torch.tensor(input_ids_list[i][:max_seq_len - 1] + [self.tokenizer.sep_token_id], dtype = torch.long)
        token_type_ids[i] = torch.tensor(token_type_ids_list[i][:max_seq_len], dtype = torch.long)
        attention_mask[i] = torch.tensor(attention_mask_list[i][:max_seq_len], dtype = torch.long)

    labels = torch.tensor(labels_list, dtype = torch.long)
    return input_ids, token_type_ids, attention_mask, labels

  def __call__(self, examples):
    input_ids_list, token_type_ids_list, attention_mask_list, labels_list = list(zip(*examples))
    cur_max_seq_len = max(len(input_ids) for input_ids in input_ids_list)
    max_seq_len = min(cur_max_seq_len, self.max_seq_len)

    input_ids, token_type_ids, attention_mask, labels = self.pad_and_truncate(input_ids_list, token_type_ids_list, attention_mask_list, labels_list, max_seq_len)

    data_dict = {
        'input_ids':input_ids,
        'token_type_ids':token_type_ids,
        'attention_mask':attention_mask,
        'labels':labels
    }
    return data_dict

In [ ]:
collate_fn = Collator(config['max_seq_len'], tokenizer)

In [ ]:
from torch.utils.data import Dataset
class AFQMCDataset(Dataset):
  def __init__(self, data_dict):
    super(AFQMCDataset, self).__init__()
    self.data_dict = data_dict

  def __getitem__(self, idx):
    data = (self.data_dict['input_ids'][idx],
         self.data_dict['token_type_ids'][idx],
         self.data_dict['attention_mask'][idx],
         self.data_dict['labels'][idx])
    return data
  
  def __len__(self):
    return len(self.data_dict['input_ids'])

In [ ]:
from torch.utils.data import Sampler
class SortedSampler(Sampler):
    """ Samples elements sequentially, always in the same order.

    Args:
        data (iterable): Iterable data.
        sort_key (callable): Specifies a function of one argument that is used to extract a
            numerical comparison key from each list element.

    Example:
        >>> list(SortedSampler(range(10), sort_key=lambda i: -i))
        [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

    """
    def __init__(self, data, sort_key):
      super().__init__(data)
      self.data = data
      self.sort_key = sort_key
      zip_ = [(i, self.sort_key(row)) for i, row in enumerate(self.data)]
      # print('init中zip_1:',zip_)
      zip_ = sorted(zip_, key = lambda r: r[1])
      # print('init中zip_2:',zip_)
      self.sorted_indexes = [item[0] for item in zip_]

    def __iter__(self):
      return iter(self.sorted_indexes)

    def __len__(self):
      return len(self.data)

  

```
init中zip_1: [(0, 17), (1, 37), (2, 33), (3, 33), (4, 26), (5, 47), (6, 25), (7, 27), (8, 28), (9, 22), (10, 25), (11, 30), (12, 32), (13, 28), (14, 18), (15, 26), (16, 28), (17, 25), (18, 22), (19, 33), (20, 26), (21, 28), (22, 30), (23, 32), (24, 17), (25, 43), (26, 26), (27, 24), (28, 22), (29, 20), (30, 25), (31, 31), (32, 26), (33, 29), (34, 33), (35, 26), (36, 31), (37, 24), (38, 44), (39, 23), (40, 22), (41, 26), (42, 33), (43, 33), (44, 35), (45, 28), (46, 37), (47, 25), (48, 27), (49, 34), (50, 24), (51, 64), (52, 25), (53, 23), (54, 21), (55, 52), (56, 23), (57, 32), (58, 46), (59, 29), (60, 31), (61, 22), (62, 36), (63, 37), (64, 40), (65, 31), (66, 26), (67, 39), (68, 19), (69, 32), (70, 39), (71, 28), (72, 30), (73, 33), (74, 32), (75, 23), (76, 38), (77, 31), (78, 31), (79, 18), (80, 19), (81, 20), (82, 23), (83, 26), (84, 36), (85, 24), (86, 31), (87, 36), (88, 28), (89, 24), (90, 66), (91, 35), (92, 34), (93, 25), (94, 34), (95, 36), (96, 35), (97, 19), (98, 18), (99, 29), (100, 17), (101, 15), (102, 27), (103, 18), (104, 33), (105, 21), (106, 21), (107, 47), (108, 20), (109, 27), (110, 26), (111, 29), (112, 37), (113, 34), (114, 24), (115, 27), (116, 16), (117, 29), (118, 24), (119, 25), (120, 29), (121, 55), (122, 56), (123, 24), (124, 34), (125, 18), (126, 34), (127, 28), (128, 27), (129, 23), (130, 27), (131, 18), (132, 28), (133, 43), (134, 37), (135, 49), (136, 32), (137, 19), (138, 23), (139, 62), (140, 31), (141, 27), (142, 24), (143, 38), (144, 16), (145, 26), (146, 36), (147, 45), (148, 24), (149, 24), (150, 27), (151, 29), (152, 36), (153, 32), (154, 42), (155, 18), (156, 46), (157, 27), (158, 19), (159, 29), (160, 81), (161, 27), (162, 22), (163, 24), (164, 23), (165, 31), (166, 22), (167, 28), (168, 38), (169, 29), (170, 29), (171, 26), (172, 30), (173, 42), (174, 43), (175, 28), (176, 18), (177, 39), (178, 37), (179, 42), (180, 25), (181, 34), (182, 21), (183, 25), (184, 37), (185, 31), (186, 38), (187, 31), (188, 16), (189, 22), (190, 44), (191, 21), (192, 47), (193, 21), (194, 25), (195, 37), (196, 27), (197, 25), (198, 23), (199, 26), (200, 25), (201, 38), (202, 41), (203, 28), (204, 43), (205, 41), (206, 27), (207, 22), (208, 25), (209, 55), (210, 17), (211, 30), (212, 41), (213, 20), (214, 32), (215, 45), (216, 34), (217, 17), (218, 37), (219, 33), (220, 19), (221, 33), (222, 29), (223, 16), (224, 27), (225, 40), (226, 21), (227, 18), (228, 32), (229, 31), (230, 31), (231, 22), (232, 22), (233, 38), (234, 17), (235, 38), (236, 35), (237, 43), (238, 45), (239, 45), (240, 58), (241, 22), (242, 30), (243, 20), (244, 21), (245, 22), (246, 20), (247, 21), (248, 21), (249, 32), (250, 26), (251, 31), (252, 26), (253, 48), (254, 23), (255, 28), (256, 26), (257, 26), (258, 24), (259, 23), (260, 21), (261, 25), (262, 16), (263, 43), (264, 25), (265, 28), (266, 28), (267, 23), (268, 29), (269, 24), (270, 37), (271, 35), (272, 26), (273, 48), (274, 27), (275, 22), (276, 43), (277, 18), (278, 38), (279, 26), (280, 27), (281, 27), (282, 31), (283, 43), (284, 57), (285, 19), (286, 37), (287, 45), (288, 35), (289, 28), (290, 41), (291, 26), (292, 27), (293, 36), (294, 29), (295, 29), (296, 31), (297, 33), (298, 31), (299, 24), (300, 18), (301, 31), (302, 26), (303, 18), (304, 26), (305, 30), (306, 24), (307, 32), (308, 32), (309, 36), (310, 20), (311, 24), (312, 42), (313, 21), (314, 19), (315, 30), (316, 23), (317, 35), (318, 15), (319, 22), (320, 19), (321, 27), (322, 17), (323, 28), (324, 32), (325, 21), (326, 26), (327, 27), (328, 21), (329, 18), (330, 32), (331, 19), (332, 16), (333, 33), (334, 26), (335, 21), (336, 28), (337, 23), (338, 33), (339, 23), (340, 18), (341, 23), (342, 32), (343, 19), (344, 33), (345, 32), (346, 29), (347, 25), (348, 53), (349, 28), (350, 17), (351, 33), (352, 22), (353, 16), (354, 29), (355, 20), (356, 19), (357, 25), (358, 26), (359, 46), (360, 35), (361, 38), (362, 24), (363, 21), (364, 49), (365, 35), (366, 24), (367, 20), (368, 21), (369, 46), (370, 27), (371, 35), (372, 21), (373, 46), (374, 26), (375, 26), (376, 28), (377, 26), (378, 27), (379, 30), (380, 23), (381, 88), (382, 29), (383, 32), (384, 25), (385, 49), (386, 34), (387, 32), (388, 23), (389, 31), (390, 23), (391, 32), (392, 18), (393, 17), (394, 23), (395, 38), (396, 20), (397, 17), (398, 107), (399, 18), (400, 26), (401, 29), (402, 47), (403, 23), (404, 19), (405, 35), (406, 42), (407, 19), (408, 34), (409, 25), (410, 40), (411, 25), (412, 22), (413, 30), (414, 32), (415, 27), (416, 20), (417, 23), (418, 36), (419, 25), (420, 28), (421, 24), (422, 24), (423, 49), (424, 18), (425, 34), (426, 27), (427, 27), (428, 32), (429, 31), (430, 30), (431, 17), (432, 32), (433, 27), (434, 16), (435, 32), (436, 25), (437, 22), (438, 25), (439, 25), (440, 25), (441, 18), (442, 28), (443, 27), (444, 23), (445, 34), (446, 21), (447, 19), (448, 25), (449, 22), (450, 29), (451, 29), (452, 35), (453, 24), (454, 34), (455, 33), (456, 43), (457, 27), (458, 32), (459, 25), (460, 30), (461, 45), (462, 25), (463, 25), (464, 57), (465, 30), (466, 48), (467, 25), (468, 22), (469, 25), (470, 42), (471, 32), (472, 18), (473, 55), (474, 18), (475, 24), (476, 28), (477, 32), (478, 21), (479, 22), (480, 21), (481, 28), (482, 30), (483, 44), (484, 20), (485, 22), (486, 24), (487, 21), (488, 22), (489, 47), (490, 30), (491, 19), (492, 34), (493, 24), (494, 21), (495, 19), (496, 45), (497, 21), (498, 33), (499, 24), (500, 42), (501, 25), (502, 25), (503, 20), (504, 29), (505, 26), (506, 21), (507, 21), (508, 26), (509, 46), (510, 21), (511, 22), (512, 24), (513, 26), (514, 27), (515, 45), (516, 21), (517, 52), (518, 35), (519, 31), (520, 29), (521, 33), (522, 39), (523, 42), (524, 22), (525, 23), (526, 23), (527, 26), (528, 30), (529, 39), (530, 23), (531, 29), (532, 32), (533, 40), (534, 19), (535, 30), (536, 22), (537, 30), (538, 33), (539, 26), (540, 31), (541, 24), (542, 43), (543, 17), (544, 32), (545, 17), (546, 33), (547, 73), (548, 27), (549, 40), (550, 36), (551, 26), (552, 25), (553, 38), (554, 20), (555, 32), (556, 46), (557, 43), (558, 21), (559, 36), (560, 22), (561, 26), (562, 33), (563, 35), (564, 24), (565, 26), (566, 20), (567, 32), (568, 27), (569, 41), (570, 30), (571, 27), (572, 21), (573, 24), (574, 26), (575, 21), (576, 19), (577, 49), (578, 17), (579, 26), (580, 57), (581, 23), (582, 40), (583, 38), (584, 23), (585, 31), (586, 41), (587, 27), (588, 21), (589, 22), (590, 42), (591, 28), (592, 28), (593, 24), (594, 29), (595, 27), (596, 20), (597, 30), (598, 25), (599, 58), (600, 32), (601, 26), (602, 48), (603, 29), (604, 24), (605, 29), (606, 50), (607, 59), (608, 29), (609, 28), (610, 21), (611, 28), (612, 31), (613, 20), (614, 32), (615, 24), (616, 21), (617, 42), (618, 29), (619, 31), (620, 31), (621, 25), (622, 36), (623, 42), (624, 19), (625, 20), (626, 22), (627, 25), (628, 38), (629, 22), (630, 30), (631, 30), (632, 48), (633, 18), (634, 25), (635, 34), (636, 22), (637, 48), (638, 30), (639, 24), (640, 24), (641, 28), (642, 27), (643, 24), (644, 28), (645, 21), (646, 33), (647, 27), (648, 28), (649, 19), (650, 26), (651, 22), (652, 32), (653, 24), (654, 49), (655, 28), (656, 33), (657, 33), (658, 24), (659, 21), (660, 24), (661, 40), (662, 24), (663, 19), (664, 26), (665, 53), (666, 23), (667, 25), (668, 25), (669, 38), (670, 31), (671, 27), (672, 30), (673, 35), (674, 23), (675, 25), (676, 27), (677, 34), (678, 35), (679, 23), (680, 24), (681, 25), (682, 47), (683, 38), (684, 21), (685, 28), (686, 26), (687, 21), (688, 31), (689, 30), (690, 22), (691, 50), (692, 23), (693, 30), (694, 18), (695, 22), (696, 33), (697, 20), (698, 26), (699, 23), (700, 19), (701, 20), (702, 28), (703, 35), (704, 29), (705, 23), (706, 40), (707, 21), (708, 22), (709, 28), (710, 46), (711, 25), (712, 22), (713, 22), (714, 25), (715, 32), (716, 20), (717, 16), (718, 31), (719, 21), (720, 31), (721, 49), (722, 26), (723, 30), (724, 34), (725, 36), (726, 23), (727, 24), (728, 36), (729, 26), (730, 24), (731, 49), (732, 29), (733, 20), (734, 24), (735, 33), (736, 40), (737, 49), (738, 38), (739, 49), (740, 24), (741, 58), (742, 26), (743, 26), (744, 18), (745, 33), (746, 50), (747, 22), (748, 33), (749, 27), (750, 18), (751, 40), (752, 29), (753, 27), (754, 27), (755, 30), (756, 28), (757, 35), (758, 22), (759, 30), (760, 22), (761, 34), (762, 18), (763, 23), (764, 34), (765, 20), (766, 19), (767, 17), (768, 21), (769, 26), (770, 42), (771, 48), (772, 26), (773, 38), (774, 16), (775, 29), (776, 21), (777, 15), (778, 35), (779, 35), (780, 39), (781, 19), (782, 23), (783, 25), (784, 28), (785, 20), (786, 30), (787, 23), (788, 19), (789, 22), (790, 24), (791, 25), (792, 37), (793, 22), (794, 43), (795, 14), (796, 41), (797, 53), (798, 40), (799, 18), (800, 28), (801, 35), (802, 56), (803, 28), (804, 30), (805, 20), (806, 23), (807, 37), (808, 23), (809, 29), (810, 32), (811, 21), (812, 23), (813, 26), (814, 26), (815, 17), (816, 22), (817, 34), (818, 25), (819, 48), (820, 47), (821, 23), (822, 32), (823, 32), (824, 29), (825, 27), (826, 23), (827, 47), (828, 29), (829, 31), (830, 24), (831, 46), (832, 38), (833, 22), (834, 83), (835, 34), (836, 33), (837, 31), (838, 25), (839, 37), (840, 21), (841, 23), (842, 34), (843, 20), (844, 54), (845, 64), (846, 23), (847, 43), (848, 21), (849, 30), (850, 31), (851, 15), (852, 24), (853, 24), (854, 24), (855, 37), (856, 39), (857, 37), (858, 54), (859, 22), (860, 34), (861, 28), (862, 21), (863, 31), (864, 27), (865, 27), (866, 44), (867, 42), (868, 30), (869, 26), (870, 34), (871, 51), (872, 17), (873, 55), (874, 21), (875, 18), (876, 23), (877, 18), (878, 30), (879, 21), (880, 40), (881, 19), (882, 29), (883, 15), (884, 19), (885, 54), (886, 22), (887, 25), (888, 24), (889, 35), (890, 30), (891, 42), (892, 32), (893, 28), (894, 20), (895, 28), (896, 37), (897, 28), (898, 23), (899, 37), (900, 22), (901, 24), (902, 35), (903, 29), (904, 31), (905, 26), (906, 21), (907, 32), (908, 20), (909, 21), (910, 63), (911, 26), (912, 45), (913, 19), (914, 17), (915, 23), (916, 46), (917, 40), (918, 18), (919, 18), (920, 36), (921, 40), (922, 22), (923, 20), (924, 20), (925, 36), (926, 31), (927, 20), (928, 34), (929, 39), (930, 24), (931, 32), (932, 28), (933, 16), (934, 46), (935, 23), (936, 25), (937, 27), (938, 19), (939, 29), (940, 22), (941, 32), (942, 25), (943, 33), (944, 20), (945, 30), (946, 19), (947, 25), (948, 25), (949, 22), (950, 49), (951, 24), (952, 20), (953, 39), (954, 23), (955, 28), (956, 17), (957, 37), (958, 30), (959, 32), (960, 33), (961, 41), (962, 29), (963, 34), (964, 28), (965, 28), (966, 26), (967, 36), (968, 53), (969, 31), (970, 23), (971, 25), (972, 28), (973, 24), (974, 21), (975, 42), (976, 37), (977, 27), (978, 67), (979, 38), (980, 38), (981, 29), (982, 20), (983, 21), (984, 40), (985, 20), (986, 32), (987, 21), (988, 46), (989, 38), (990, 22), (991, 30), (992, 68), (993, 28), (994, 24), (995, 16), (996, 28), (997, 27), (998, 32), (999, 36), (1000, 23), (1001, 17), (1002, 41), (1003, 25), (1004, 22), (1005, 33), (1006, 32), (1007, 24), (1008, 35), (1009, 43), (1010, 38), (1011, 30), (1012, 36), (1013, 23), (1014, 20), (1015, 24), (1016, 26), (1017, 24), (1018, 42), (1019, 19), (1020, 26), (1021, 21), (1022, 32), (1023, 30), (1024, 19), (1025, 46), (1026, 45), (1027, 17), (1028, 48), (1029, 36), (1030, 24), (1031, 22), (1032, 20), (1033, 43), (1034, 38), (1035, 30), (1036, 38), (1037, 51), (1038, 30), (1039, 26), (1040, 22), (1041, 36), (1042, 27), (1043, 28), (1044, 21), (1045, 25), (1046, 24), (1047, 19), (1048, 18), (1049, 28), (1050, 29), (1051, 37), (1052, 26), (1053, 19), (1054, 74), (1055, 28), (1056, 27), (1057, 52), (1058, 30), (1059, 30), (1060, 37), (1061, 32), (1062, 19), (1063, 40), (1064, 40), (1065, 24), (1066, 25), (1067, 24), (1068, 21), (1069, 34), (1070, 26), (1071, 21), (1072, 40), (1073, 23), (1074, 40), (1075, 20), (1076, 38), (1077, 29), (1078, 38), (1079, 24), (1080, 26), (1081, 28), (1082, 21), (1083, 20), (1084, 29), (1085, 35), (1086, 30), (1087, 26), (1088, 50), (1089, 23), (1090, 27), (1091, 28), (1092, 23), (1093, 26), (1094, 20), (1095, 30), (1096, 22), (1097, 36), (1098, 29), (1099, 22), (1100, 25), (1101, 29), (1102, 25), (1103, 27), (1104, 20), (1105, 28), (1106, 29), (1107, 37), (1108, 33), (1109, 26), (1110, 40), (1111, 23), (1112, 34), (1113, 18), (1114, 26), (1115, 31), (1116, 25), (1117, 31), (1118, 28), (1119, 25), (1120, 22), (1121, 24), (1122, 32), (1123, 24), (1124, 19), (1125, 22), (1126, 31), (1127, 39), (1128, 31), (1129, 29), (1130, 38), (1131, 26), (1132, 20), (1133, 24), (1134, 28), (1135, 23), (1136, 32), (1137, 31), (1138, 31), (1139, 30), (1140, 34), (1141, 23), (1142, 27), (1143, 32), (1144, 22), (1145, 22), (1146, 21), (1147, 22), (1148, 35), (1149, 35), (1150, 19), (1151, 32), (1152, 35), (1153, 25), (1154, 25), (1155, 32), (1156, 15), (1157, 21), (1158, 24), (1159, 26), (1160, 33), (1161, 33), (1162, 49), (1163, 23), (1164, 18), (1165, 20), (1166, 34), (1167, 30), (1168, 23), (1169, 23), (1170, 29), (1171, 22), (1172, 26), (1173, 29), (1174, 31), (1175, 19), (1176, 27), (1177, 30), (1178, 30), (1179, 27), (1180, 45), (1181, 29), (1182, 26), (1183, 24), (1184, 108), (1185, 22), (1186, 35), (1187, 26), (1188, 22), (1189, 24), (1190, 22), (1191, 24), (1192, 31), (1193, 28), (1194, 25), (1195, 24), (1196, 32), (1197, 20), (1198, 55), (1199, 30), (1200, 27), (1201, 32), (1202, 23), (1203, 17), (1204, 26), (1205, 32), (1206, 24), (1207, 22), (1208, 52), (1209, 58), (1210, 29), (1211, 35), (1212, 16), (1213, 36), (1214, 29), (1215, 30), (1216, 19), (1217, 19), (1218, 38), (1219, 27), (1220, 27), (1221, 21), (1222, 26), (1223, 26), (1224, 24), (1225, 22), (1226, 28), (1227, 26), (1228, 20), (1229, 26), (1230, 23), (1231, 29), (1232, 29), (1233, 17), (1234, 29), (1235, 21), (1236, 15), (1237, 30), (1238, 27), (1239, 27), (1240, 20), (1241, 20), (1242, 18), (1243, 51), (1244, 21), (1245, 35), (1246, 32), (1247, 39), (1248, 34), (1249, 36), (1250, 19), (1251, 25), (1252, 57), (1253, 25), (1254, 31), (1255, 24), (1256, 78), (1257, 29), (1258, 35), (1259, 39), (1260, 56), (1261, 27), (1262, 42), (1263, 35), (1264, 28), (1265, 20), (1266, 63), (1267, 37), (1268, 35), (1269, 19), (1270, 22), (1271, 29), (1272, 31), (1273, 30), (1274, 26), (1275, 26), (1276, 31), (1277, 30), (1278, 45), (1279, 31), (1280, 18), (1281, 28), (1282, 28), (1283, 26), (1284, 49), (1285, 35), (1286, 23), (1287, 25), (1288, 25), (1289, 19), (1290, 17), (1291, 22), (1292, 26), (1293, 40), (1294, 90), (1295, 34), (1296, 38), (1297, 26), (1298, 25), (1299, 21), (1300, 25), (1301, 25), (1302, 20), (1303, 38), (1304, 27), (1305, 24), (1306, 28), (1307, 21), (1308, 22), (1309, 20), (1310, 31), (1311, 43), (1312, 43), (1313, 28), (1314, 28), (1315, 24), (1316, 27), (1317, 19), (1318, 19), (1319, 23), (1320, 30), (1321, 20), (1322, 31), (1323, 42), (1324, 22), (1325, 28), (1326, 38), (1327, 24), (1328, 33), (1329, 67), (1330, 27), (1331, 31), (1332, 24), (1333, 29), (1334, 50), (1335, 19), (1336, 30), (1337, 31), (1338, 22), (1339, 44), (1340, 18), (1341, 20), (1342, 32), (1343, 24), (1344, 26), (1345, 19), (1346, 20), (1347, 37), (1348, 33), (1349, 30), (1350, 21), (1351, 22), (1352, 23), (1353, 88), (1354, 46), (1355, 25), (1356, 19), (1357, 28), (1358, 29), (1359, 23), (1360, 27), (1361, 43), (1362, 33), (1363, 26), (1364, 24), (1365, 25), (1366, 44), (1367, 23), (1368, 38), (1369, 26), (1370, 26), (1371, 18), (1372, 36), (1373, 58), (1374, 34), (1375, 25), (1376, 37), (1377, 22), (1378, 23), (1379, 26), (1380, 27), (1381, 31), (1382, 37), (1383, 22), (1384, 35), (1385, 34), (1386, 37), (1387, 22), (1388, 25), (1389, 19), (1390, 20), (1391, 25), (1392, 56), (1393, 19), (1394, 21), (1395, 26), (1396, 21), (1397, 77), (1398, 24), (1399, 26), (1400, 34), (1401, 43), (1402, 26), (1403, 28), (1404, 42), (1405, 21), (1406, 28), (1407, 28), (1408, 45), (1409, 19), (1410, 19), (1411, 24), (1412, 24), (1413, 24), (1414, 24), (1415, 38), (1416, 35), (1417, 27), (1418, 20), (1419, 23), (1420, 24), (1421, 48), (1422, 25), (1423, 31), (1424, 38), (1425, 40), (1426, 39), (1427, 30), (1428, 29), (1429, 30), (1430, 19), (1431, 24), (1432, 16), (1433, 23), (1434, 36), (1435, 53), (1436, 39), (1437, 41), (1438, 21), (1439, 37), (1440, 33), (1441, 33), (1442, 20), (1443, 66), (1444, 97), (1445, 17), (1446, 28), (1447, 19), (1448, 26), (1449, 28), (1450, 48), (1451, 50), (1452, 25), (1453, 38), (1454, 34), (1455, 24), (1456, 26), (1457, 24), (1458, 13), (1459, 20), (1460, 22), (1461, 35), (1462, 28), (1463, 18), (1464, 22), (1465, 35), (1466, 29), (1467, 19), (1468, 33), (1469, 32), (1470, 23), (1471, 22), (1472, 24), (1473, 23), (1474, 28), (1475, 22), (1476, 20), (1477, 19), (1478, 37), (1479, 39), (1480, 28), (1481, 25), (1482, 30), (1483, 38), (1484, 32), (1485, 30), (1486, 37), (1487, 59), (1488, 36), (1489, 27), (1490, 21), (1491, 53), (1492, 23), (1493, 29), (1494, 41), (1495, 21), (1496, 44), (1497, 44), (1498, 35), (1499, 27), (1500, 31), (1501, 37), (1502, 27), (1503, 21), (1504, 23), (1505, 32), (1506, 79), (1507, 37), (1508, 30), (1509, 21), (1510, 42), (1511, 38), (1512, 27), (1513, 28), (1514, 28), (1515, 26), (1516, 30), (1517, 43), (1518, 15), (1519, 22), (1520, 21), (1521, 26), (1522, 39), (1523, 36), (1524, 26), (1525, 72), (1526, 24), (1527, 19), (1528, 31), (1529, 34), (1530, 28), (1531, 32), (1532, 20), (1533, 26), (1534, 23), (1535, 32), (1536, 61), (1537, 36), (1538, 21), (1539, 30), (1540, 24), (1541, 53), (1542, 23), (1543, 38), (1544, 21), (1545, 19), (1546, 37), (1547, 19), (1548, 28), (1549, 30), (1550, 49), (1551, 26), (1552, 40), (1553, 27), (1554, 24), (1555, 22), (1556, 35), (1557, 39), (1558, 23), (1559, 32), (1560, 35), (1561, 25), (1562, 41), (1563, 21), (1564, 21), (1565, 25), (1566, 27), (1567, 28), (1568, 28), (1569, 24), (1570, 27), (1571, 18), (1572, 23), (1573, 47), (1574, 26), (1575, 44), (1576, 35), (1577, 25), (1578, 42), (1579, 41), (1580, 16), (1581, 37), (1582, 41), (1583, 49), (1584, 15), (1585, 39), (1586, 32), (1587, 36), (1588, 30), (1589, 22), (1590, 36), (1591, 21), (1592, 26), (1593, 33), (1594, 49), (1595, 21), (1596, 29), (1597, 34), (1598, 29), (1599, 35), (1600, 27), (1601, 22), (1602, 34), (1603, 25), (1604, 24), (1605, 29), (1606, 17), (1607, 27), (1608, 42), (1609, 29), (1610, 18), (1611, 21), (1612, 25), (1613, 61), (1614, 38), (1615, 19), (1616, 25), (1617, 34), (1618, 28), (1619, 22), (1620, 18), (1621, 47), (1622, 27), (1623, 25), (1624, 56), (1625, 39), (1626, 22), (1627, 39), (1628, 44), (1629, 15), (1630, 29), (1631, 20), (1632, 25), (1633, 21), (1634, 23), (1635, 24), (1636, 30), (1637, 40), (1638, 68), (1639, 31), (1640, 37), (1641, 23), (1642, 34), (1643, 28), (1644, 26), (1645, 28), (1646, 56), (1647, 27), (1648, 26), (1649, 26), (1650, 37), (1651, 20), (1652, 15), (1653, 32), (1654, 25), (1655, 32), (1656, 52), (1657, 18), (1658, 29), (1659, 28), (1660, 24), (1661, 25), (1662, 28), (1663, 35), (1664, 41), (1665, 23), (1666, 26), (1667, 16), (1668, 26), (1669, 28), (1670, 18), (1671, 26), (1672, 28), (1673, 28), (1674, 31), (1675, 24), (1676, 22), (1677, 16), (1678, 25), (1679, 19), (1680, 27), (1681, 33), (1682, 28), (1683, 18), (1684, 27), (1685, 29), (1686, 34), (1687, 44), (1688, 28), (1689, 27), (1690, 16), (1691, 19), (1692, 35), (1693, 31), (1694, 27), (1695, 21), (1696, 25), (1697, 32), (1698, 21), (1699, 29), (1700, 26), (1701, 24), (1702, 19), (1703, 22), (1704, 20), (1705, 24), (1706, 24), (1707, 35), (1708, 20), (1709, 35), (1710, 27), (1711, 59), (1712, 27), (1713, 27), (1714, 45), (1715, 20), (1716, 23), (1717, 21), (1718, 22), (1719, 29), (1720, 34), (1721, 30), (1722, 23), (1723, 21), (1724, 35), (1725, 41), (1726, 19), (1727, 24), (1728, 41), (1729, 23), (1730, 19), (1731, 24), (1732, 23), (1733, 25), (1734, 27), (1735, 14), (1736, 36), (1737, 27), (1738, 26), (1739, 18), (1740, 35), (1741, 24), (1742, 31), (1743, 39), (1744, 32), (1745, 22), (1746, 24), (1747, 41), (1748, 30), (1749, 28), (1750, 30), (1751, 24), (1752, 36), (1753, 19), (1754, 30), (1755, 37), (1756, 29), (1757, 21), (1758, 22), (1759, 26), (1760, 26), (1761, 39), (1762, 16), (1763, 31), (1764, 47), (1765, 21), (1766, 57), (1767, 19), (1768, 18), (1769, 33), (1770, 51), (1771, 23), (1772, 43), (1773, 38), (1774, 38), (1775, 32), (1776, 23), (1777, 29), (1778, 25), (1779, 17), (1780, 31), (1781, 18), (1782, 23), (1783, 29), (1784, 19), (1785, 30), (1786, 18), (1787, 19), (1788, 20), (1789, 22), (1790, 30), (1791, 28), (1792, 29), (1793, 41), (1794, 21), (1795, 33), (1796, 42), (1797, 35), (1798, 27), (1799, 19), (1800, 28), (1801, 36), (1802, 33), (1803, 23), (1804, 18), (1805, 35), (1806, 25), (1807, 20), (1808, 35), (1809, 46), (1810, 27), (1811, 28), (1812, 32), (1813, 28), (1814, 34), (1815, 28), (1816, 47), (1817, 100), (1818, 27), (1819, 19), (1820, 37), (1821, 20), (1822, 20), (1823, 41), (1824, 25), (1825, 24), (1826, 37), (1827, 37), (1828, 23), (1829, 55), (1830, 30), (1831, 38), (1832, 49), (1833, 23), (1834, 31), (1835, 24), (1836, 27), (1837, 46), (1838, 20), (1839, 24), (1840, 40), (1841, 32), (1842, 38), (1843, 27), (1844, 52), (1845, 36), (1846, 30), (1847, 32), (1848, 14), (1849, 30), (1850, 31), (1851, 28), (1852, 22), (1853, 22), (1854, 27), (1855, 28), (1856, 20), (1857, 24), (1858, 26), (1859, 31), (1860, 41), (1861, 31), (1862, 22), (1863, 32), (1864, 25), (1865, 27), (1866, 46), (1867, 29), (1868, 20), (1869, 34), (1870, 19), (1871, 34), (1872, 21), (1873, 32), (1874, 25), (1875, 21), (1876, 27), (1877, 35), (1878, 28), (1879, 19), (1880, 24), (1881, 23), (1882, 58), (1883, 23), (1884, 16), (1885, 29), (1886, 24), (1887, 38), (1888, 27), (1889, 37), (1890, 26), (1891, 23), (1892, 31), (1893, 21), (1894, 27), (1895, 28), (1896, 37), (1897, 32), (1898, 29), (1899, 17), (1900, 23), (1901, 20), (1902, 20), (1903, 24), (1904, 44), (1905, 25), (1906, 34), (1907, 47), (1908, 40), (1909, 40), (1910, 41), (1911, 32), (1912, 29), (1913, 35), (1914, 18), (1915, 26), (1916, 23), (1917, 35), (1918, 23), (1919, 58), (1920, 26), (1921, 36), (1922, 38), (1923, 127), (1924, 25), (1925, 29), (1926, 33), (1927, 29), (1928, 29), (1929, 31), (1930, 23), (1931, 31), (1932, 26), (1933, 30), (1934, 23), (1935, 27), (1936, 28), (1937, 22), (1938, 36), (1939, 34), (1940, 21), (1941, 16), (1942, 25), (1943, 22), (1944, 34), (1945, 20), (1946, 26), (1947, 33), (1948, 23), (1949, 32), (1950, 38), (1951, 28), (1952, 26), (1953, 29), (1954, 37), (1955, 33), (1956, 58), (1957, 15), (1958, 26), (1959, 23), (1960, 27), (1961, 22), (1962, 21), (1963, 26), (1964, 27), (1965, 18), (1966, 32), (1967, 32), (1968, 26), (1969, 31), (1970, 47), (1971, 26), (1972, 28), (1973, 44), (1974, 35), (1975, 33), (1976, 44), (1977, 23), (1978, 47), (1979, 39), (1980, 66), (1981, 28), (1982, 22), (1983, 23), (1984, 34), (1985, 31), (1986, 28), (1987, 43), (1988, 29), (1989, 30), (1990, 44), (1991, 25), (1992, 40), (1993, 24), (1994, 40), (1995, 24), (1996, 39), (1997, 66), (1998, 25), (1999, 36), (2000, 39), (2001, 35), (2002, 28), (2003, 35), (2004, 24), (2005, 34), (2006, 26), (2007, 29), (2008, 27), (2009, 45), (2010, 31), (2011, 24), (2012, 29), (2013, 22), (2014, 28), (2015, 27), (2016, 22), (2017, 75), (2018, 26), (2019, 22), (2020, 40), (2021, 22), (2022, 28), (2023, 23), (2024, 20), (2025, 21), (2026, 28), (2027, 23), (2028, 16), (2029, 25), (2030, 34), (2031, 20), (2032, 40), (2033, 28), (2034, 34), (2035, 38), (2036, 37), (2037, 27), (2038, 25), (2039, 21), (2040, 19), (2041, 36), (2042, 41), (2043, 31), (2044, 28), (2045, 31), (2046, 29), (2047, 32), (2048, 30), (2049, 21), (2050, 32), (2051, 45), (2052, 33), (2053, 26), (2054, 41), (2055, 31), (2056, 22), (2057, 27), (2058, 29), (2059, 50), (2060, 17), (2061, 28), (2062, 18), (2063, 32), (2064, 25), (2065, 24), (2066, 23), (2067, 23), (2068, 24), (2069, 30), (2070, 31), (2071, 25), (2072, 18), (2073, 25), (2074, 40), (2075, 26), (2076, 30), (2077, 36), (2078, 30), (2079, 22), (2080, 24), (2081, 29), (2082, 41), (2083, 39), (2084, 27), (2085, 53), (2086, 34), (2087, 74), (2088, 33), (2089, 32), (2090, 38), (2091, 36), (2092, 32), (2093, 14), (2094, 39), (2095, 19), (2096, 21), (2097, 28), (2098, 29), (2099, 34), (2100, 24), (2101, 30), (2102, 26), (2103, 17), (2104, 33), (2105, 27), (2106, 33), (2107, 41), (2108, 39), (2109, 28), (2110, 24), (2111, 35), (2112, 24), (2113, 45), (2114, 23), (2115, 22), (2116, 23), (2117, 24), (2118, 25), (2119, 26), (2120, 44), (2121, 47), (2122, 29), (2123, 24), (2124, 21), (2125, 43), (2126, 21), (2127, 21), (2128, 26), (2129, 26), (2130, 19), (2131, 23), (2132, 38), (2133, 20), (2134, 28), (2135, 24), (2136, 33), (2137, 24), (2138, 32), (2139, 22), (2140, 32), (2141, 38), (2142, 38), (2143, 24), (2144, 41), (2145, 25), (2146, 32), (2147, 23), (2148, 27), (2149, 32), (2150, 30), (2151, 34), (2152, 20), (2153, 43), (2154, 26), (2155, 22), (2156, 26), (2157, 34), (2158, 21), (2159, 31), (2160, 34), (2161, 24), (2162, 15), (2163, 23), (2164, 67), (2165, 21), (2166, 28), (2167, 29), (2168, 33), (2169, 19), (2170, 42), (2171, 33), (2172, 25), (2173, 20), (2174, 30), (2175, 19), (2176, 33), (2177, 34), (2178, 24), (2179, 25), (2180, 36), (2181, 76), (2182, 36), (2183, 50), (2184, 32), (2185, 32), (2186, 30), (2187, 37), (2188, 25), (2189, 29), (2190, 19), (2191, 32), (2192, 22), (2193, 27), (2194, 24), (2195, 14), (2196, 17), (2197, 35), (2198, 23), (2199, 23), (2200, 41), (2201, 20), (2202, 27), (2203, 34), (2204, 21), (2205, 19), (2206, 28), (2207, 43), (2208, 22), (2209, 25), (2210, 15), (2211, 27), (2212, 27), (2213, 21), (2214, 19), (2215, 27), (2216, 20), (2217, 35), (2218, 26), (2219, 35), (2220, 35), (2221, 35), (2222, 25), (2223, 27), (2224, 23), (2225, 67), (2226, 28), (2227, 27), (2228, 27), (2229, 36), (2230, 29), (2231, 21), (2232, 33), (2233, 37), (2234, 29), (2235, 34), (2236, 45), (2237, 41), (2238, 29), (2239, 26), (2240, 21), (2241, 30), (2242, 31), (2243, 20), (2244, 22), (2245, 31), (2246, 27), (2247, 36), (2248, 23), (2249, 25), (2250, 29), (2251, 25), (2252, 27), (2253, 21), (2254, 25), (2255, 27), (2256, 67), (2257, 33), (2258, 33), (2259, 35), (2260, 22), (2261, 19), (2262, 31), (2263, 25), (2264, 28), (2265, 27), (2266, 28), (2267, 23), (2268, 27), (2269, 18), (2270, 39), (2271, 18), (2272, 36), (2273, 32), (2274, 28), (2275, 46), (2276, 25), (2277, 24), (2278, 30), (2279, 33), (2280, 19), (2281, 24), (2282, 21), (2283, 18), (2284, 20), (2285, 39), (2286, 20), (2287, 19), (2288, 25), (2289, 27), (2290, 31), (2291, 40), (2292, 24), (2293, 19), (2294, 18), (2295, 36), (2296, 17), (2297, 29), (2298, 31), (2299, 41), (2300, 24), (2301, 24), (2302, 30), (2303, 36), (2304, 18), (2305, 28), (2306, 31), (2307, 32), (2308, 28), (2309, 34), (2310, 19), (2311, 27), (2312, 54), (2313, 25), (2314, 29), (2315, 29), (2316, 30), (2317, 26), (2318, 39), (2319, 29), (2320, 39), (2321, 27), (2322, 45), (2323, 25), (2324, 28), (2325, 32), (2326, 21), (2327, 27), (2328, 47), (2329, 28), (2330, 33), (2331, 40), (2332, 28), (2333, 32), (2334, 25), (2335, 48), (2336, 21), (2337, 20), (2338, 19), (2339, 28), (2340, 40), (2341, 30), (2342, 41), (2343, 20), (2344, 25), (2345, 33), (2346, 49), (2347, 28), (2348, 23), (2349, 22), (2350, 36), (2351, 27), (2352, 30), (2353, 23), (2354, 44), (2355, 20), (2356, 31), (2357, 21), (2358, 38), (2359, 19), (2360, 26), (2361, 28), (2362, 17), (2363, 28), (2364, 23), (2365, 41), (2366, 32), (2367, 34), (2368, 35), (2369, 31), (2370, 32), (2371, 21), (2372, 19), (2373, 17), (2374, 29), (2375, 24), (2376, 23), (2377, 75), (2378, 26), (2379, 40), (2380, 23), (2381, 32), (2382, 26), (2383, 36), (2384, 22), (2385, 56), (2386, 32), (2387, 26), (2388, 28), (2389, 20), (2390, 35), (2391, 29), (2392, 27), (2393, 40), (2394, 46), (2395, 45), (2396, 39), (2397, 51), (2398, 30), (2399, 36), (2400, 27), (2401, 80), (2402, 30), (2403, 24), (2404, 28), (2405, 33), (2406, 27), (2407, 19), (2408, 24), (2409, 59), (2410, 29), (2411, 31), (2412, 57), (2413, 44), (2414, 16), (2415, 33), (2416, 39), (2417, 20), (2418, 24), (2419, 15), (2420, 22), (2421, 43), (2422, 30), (2423, 28), (2424, 29), (2425, 34), (2426, 32), (2427, 31), (2428, 28), (2429, 34), (2430, 16), (2431, 37), (2432, 32), (2433, 27), (2434, 26), (2435, 38), (2436, 23), (2437, 46), (2438, 14), (2439, 18), (2440, 24), (2441, 21), (2442, 63), (2443, 25), (2444, 22), (2445, 38), (2446, 19), (2447, 43), (2448, 21), (2449, 41), (2450, 25), (2451, 23), (2452, 23), (2453, 25), (2454, 29), (2455, 25), (2456, 34), (2457, 32), (2458, 25), (2459, 41), (2460, 34), (2461, 29), (2462, 19), (2463, 23), (2464, 26), (2465, 27), (2466, 37), (2467, 31), (2468, 31), (2469, 26), (2470, 19), (2471, 29), (2472, 24), (2473, 26), (2474, 33), (2475, 41), (2476, 23), (2477, 22), (2478, 27), (2479, 24), (2480, 28), (2481, 20), (2482, 19), (2483, 20), (2484, 23), (2485, 48), (2486, 60), (2487, 39), (2488, 27), (2489, 30), (2490, 33), (2491, 30), (2492, 30), (2493, 24), (2494, 36), (2495, 30), (2496, 25), (2497, 34), (2498, 39), (2499, 30), (2500, 27), (2501, 27), (2502, 43), (2503, 24), (2504, 31), (2505, 29), (2506, 26), (2507, 21), (2508, 22), (2509, 23), (2510, 20), (2511, 17), (2512, 17), (2513, 23), (2514, 33), (2515, 34), (2516, 42), (2517, 21), (2518, 64), (2519, 24), (2520, 20), (2521, 24), (2522, 21), (2523, 27), (2524, 31), (2525, 26), (2526, 26), (2527, 45), (2528, 43), (2529, 22), (2530, 22), (2531, 28), (2532, 51), (2533, 29), (2534, 28), (2535, 28), (2536, 33), (2537, 34), (2538, 23), (2539, 25), (2540, 35), (2541, 30), (2542, 32), (2543, 42), (2544, 49), (2545, 51), (2546, 31), (2547, 32), (2548, 39), (2549, 45), (2550, 20), (2551, 29), (2552, 44), (2553, 28), (2554, 42), (2555, 26), (2556, 32), (2557, 21), (2558, 67), (2559, 34), (2560, 23), (2561, 29), (2562, 40), (2563, 31), (2564, 32), (2565, 18), (2566, 21), (2567, 24), (2568, 37), (2569, 24), (2570, 28), (2571, 23), (2572, 42), (2573, 38), (2574, 43), (2575, 23), (2576, 20), (2577, 34), (2578, 20), (2579, 20), (2580, 29), (2581, 25), (2582, 52), (2583, 36), (2584, 15), (2585, 39), (2586, 47), (2587, 52), (2588, 25), (2589, 28), (2590, 42), (2591, 27), (2592, 27), (2593, 21), (2594, 27), (2595, 23), (2596, 40), (2597, 18), (2598, 31), (2599, 37), (2600, 32), (2601, 68), (2602, 26), (2603, 22), (2604, 34), (2605, 36), (2606, 30), (2607, 20), (2608, 42), (2609, 45), (2610, 30), (2611, 32), (2612, 30), (2613, 19), (2614, 20), (2615, 23), (2616, 29), (2617, 24), (2618, 24), (2619, 23), (2620, 32), (2621, 30), (2622, 31), (2623, 27), (2624, 42), (2625, 24), (2626, 36), (2627, 47), (2628, 24), (2629, 37), (2630, 33), (2631, 36), (2632, 27), (2633, 30), (2634, 55), (2635, 22), (2636, 45), (2637, 26), (2638, 19), (2639, 23), (2640, 17), (2641, 26), (2642, 21), (2643, 24), (2644, 20), (2645, 37), (2646, 21), (2647, 38), (2648, 34), (2649, 28), (2650, 37), (2651, 41), (2652, 34), (2653, 26), (2654, 32), (2655, 23), (2656, 74), (2657, 20), (2658, 25), (2659, 31), (2660, 20), (2661, 29), (2662, 27), (2663, 28), (2664, 28), (2665, 17), (2666, 24), (2667, 20), (2668, 16), (2669, 19), (2670, 23), (2671, 20), (2672, 43), (2673, 27), (2674, 26), (2675, 24), (2676, 30), (2677, 26), (2678, 31), (2679, 19), (2680, 21), (2681, 20), (2682, 24), (2683, 20), (2684, 21), (2685, 29), (2686, 64), (2687, 26), (2688, 48), (2689, 41), (2690, 43), (2691, 30), (2692, 20), (2693, 32), (2694, 36), (2695, 31), (2696, 17), (2697, 41), (2698, 34), (2699, 16), (2700, 24), (2701, 22), (2702, 28), (2703, 30), (2704, 50), (2705, 21), (2706, 23), (2707, 24), (2708, 27), (2709, 25), (2710, 25), (2711, 30), (2712, 59), (2713, 36), (2714, 27), (2715, 19), (2716, 20), (2717, 30), (2718, 38), (2719, 34), (2720, 28), (2721, 19), (2722, 47), (2723, 22), (2724, 22), (2725, 35), (2726, 38), (2727, 25), (2728, 42), (2729, 18), (2730, 17), (2731, 29), (2732, 30), (2733, 26), (2734, 25), (2735, 25), (2736, 49), (2737, 35), (2738, 20), (2739, 27), (2740, 20), (2741, 20), (2742, 20), (2743, 41), (2744, 29), (2745, 47), (2746, 46), (2747, 18), (2748, 34), (2749, 47), (2750, 23), (2751, 38), (2752, 38), (2753, 21), (2754, 29), (2755, 33), (2756, 25), (2757, 30), (2758, 31), (2759, 34), (2760, 28), (2761, 30), (2762, 15), (2763, 39), (2764, 34), (2765, 30), (2766, 37), (2767, 17), (2768, 26), (2769, 29), (2770, 24), (2771, 34), (2772, 23), (2773, 30), (2774, 24), (2775, 29), (2776, 30), (2777, 19), (2778, 34), (2779, 30), (2780, 28), (2781, 24), (2782, 26), (2783, 32), (2784, 38), (2785, 26), (2786, 30), (2787, 21), (2788, 32), (2789, 44), (2790, 28), (2791, 36), (2792, 34), (2793, 30), (2794, 29), (2795, 19), (2796, 15), (2797, 17), (2798, 22), (2799, 37), (2800, 26), (2801, 26), (2802, 26), (2803, 21), (2804, 26), (2805, 117), (2806, 28), (2807, 21), (2808, 28), (2809, 47), (2810, 26), (2811, 28), (2812, 31), (2813, 31), (2814, 27), (2815, 18), (2816, 48), (2817, 24), (2818, 24), (2819, 54), (2820, 76), (2821, 38), (2822, 33), (2823, 27), (2824, 44), (2825, 20), (2826, 27), (2827, 26), (2828, 31), (2829, 48), (2830, 55), (2831, 22), (2832, 20), (2833, 23), (2834, 49), (2835, 22), (2836, 27), (2837, 26), (2838, 22), (2839, 40), (2840, 33), (2841, 37), (2842, 26), (2843, 19), (2844, 24), (2845, 19), (2846, 24), (2847, 25), (2848, 43), (2849, 33), (2850, 24), (2851, 20), (2852, 38), (2853, 22), (2854, 22), (2855, 30), (2856, 24), (2857, 23), (2858, 27), (2859, 27), (2860, 54), (2861, 47), (2862, 23), (2863, 28), (2864, 25), (2865, 24), (2866, 21), (2867, 32), (2868, 43), (2869, 23), (2870, 39), (2871, 31), (2872, 24), (2873, 23), (2874, 23), (2875, 19), (2876, 23), (2877, 29), (2878, 23), (2879, 30), (2880, 22), (2881, 29), (2882, 28), (2883, 28), (2884, 35), (2885, 29), (2886, 31), (2887, 50), (2888, 29), (2889, 24), (2890, 23), (2891, 23), (2892, 27), (2893, 28), (2894, 26), (2895, 22), (2896, 31), (2897, 28), (2898, 20), (2899, 26), (2900, 28), (2901, 19), (2902, 29), (2903, 49), (2904, 29), (2905, 28), (2906, 36), (2907, 41), (2908, 16), (2909, 34), (2910, 19), (2911, 24), (2912, 30), (2913, 29), (2914, 28), (2915, 27), (2916, 21), (2917, 31), (2918, 61), (2919, 25), (2920, 41), (2921, 20), (2922, 28), (2923, 27), (2924, 23), (2925, 24), (2926, 50), (2927, 37), (2928, 37), (2929, 20), (2930, 37), (2931, 38), (2932, 31), (2933, 20), (2934, 29), (2935, 26), (2936, 24), (2937, 28), (2938, 37), (2939, 31), (2940, 20), (2941, 33), (2942, 26), (2943, 24), (2944, 20), (2945, 25), (2946, 21), (2947, 43), (2948, 27), (2949, 21), (2950, 31), (2951, 25), (2952, 25), (2953, 29), (2954, 24), (2955, 27), (2956, 26), (2957, 27), (2958, 22), (2959, 25), (2960, 21), (2961, 31), (2962, 20), (2963, 23), (2964, 58), (2965, 22), (2966, 32), (2967, 61), (2968, 26), (2969, 21), (2970, 74), (2971, 25), (2972, 42), (2973, 25), (2974, 22), (2975, 27), (2976, 20), (2977, 26), (2978, 32), (2979, 21), (2980, 23), (2981, 46), (2982, 24), (2983, 19), (2984, 24), (2985, 28), (2986, 30), (2987, 31), (2988, 30), (2989, 36), (2990, 39), (2991, 41), (2992, 23), (2993, 26), (2994, 50), (2995, 71), (2996, 22), (2997, 23), (2998, 24), (2999, 28), (3000, 48), (3001, 21), (3002, 44), (3003, 61), (3004, 25), (3005, 25), (3006, 42), (3007, 21), (3008, 24), (3009, 28), (3010, 34), (3011, 33), (3012, 40), (3013, 18), (3014, 26), (3015, 20), (3016, 51), (3017, 19), (3018, 25), (3019, 23), (3020, 34), (3021, 22), (3022, 34), (3023, 19), (3024, 29), (3025, 31), (3026, 31), (3027, 28), (3028, 25), (3029, 39), (3030, 20), (3031, 26), (3032, 28), (3033, 29), (3034, 34), (3035, 30), (3036, 39), (3037, 28), (3038, 27), (3039, 26), (3040, 25), (3041, 23), (3042, 15), (3043, 21), (3044, 27), (3045, 23), (3046, 22), (3047, 28), (3048, 29), (3049, 26), (3050, 34), (3051, 30), (3052, 35), (3053, 21), (3054, 20), (3055, 23), (3056, 29), (3057, 24), (3058, 32), (3059, 29), (3060, 22), (3061, 38), (3062, 68), (3063, 27), (3064, 23), (3065, 33), (3066, 25), (3067, 36), (3068, 31), (3069, 19), (3070, 45), (3071, 74), (3072, 24), (3073, 40), (3074, 41), (3075, 41), (3076, 31), (3077, 31), (3078, 15), (3079, 26), (3080, 24), (3081, 21), (3082, 36), (3083, 27), (3084, 35), (3085, 22), (3086, 18), (3087, 22), (3088, 31), (3089, 23), (3090, 24), (3091, 24), (3092, 30), (3093, 21), (3094, 44), (3095, 25), (3096, 29), (3097, 30), (3098, 59), (3099, 22), (3100, 30), (3101, 24), (3102, 30), (3103, 23), (3104, 23), (3105, 20), (3106, 38), (3107, 32), (3108, 39), (3109, 24), (3110, 24), (3111, 27), (3112, 37), (3113, 34), (3114, 29), (3115, 20), (3116, 57), (3117, 33), (3118, 24), (3119, 25), (3120, 30), (3121, 20), (3122, 20), (3123, 16), (3124, 29), (3125, 47), (3126, 25), (3127, 20), (3128, 23), (3129, 28), (3130, 26), (3131, 28), (3132, 39), (3133, 21), (3134, 28), (3135, 26), (3136, 40), (3137, 24), (3138, 31), (3139, 28), (3140, 30), (3141, 24), (3142, 29), (3143, 28), (3144, 17), (3145, 25), (3146, 29), (3147, 16), (3148, 18), (3149, 27), (3150, 25), (3151, 22), (3152, 51), (3153, 25), (3154, 35), (3155, 39), (3156, 33), (3157, 43), (3158, 19), (3159, 22), (3160, 26), (3161, 21), (3162, 35), (3163, 26), (3164, 20), (3165, 24), (3166, 28), (3167, 28), (3168, 13), (3169, 20), (3170, 36), (3171, 25), (3172, 24), (3173, 21), (3174, 28), (3175, 29), (3176, 22), (3177, 28), (3178, 28), (3179, 33), (3180, 21), (3181, 33), (3182, 41), (3183, 28), (3184, 33), (3185, 37), (3186, 22), (3187, 25), (3188, 21), (3189, 30), (3190, 21), (3191, 36), (3192, 35), (3193, 35), (3194, 32), (3195, 17), (3196, 31), (3197, 39), (3198, 15), (3199, 30), (3200, 36), (3201, 69), (3202, 26), (3203, 30), (3204, 32), (3205, 31), (3206, 23), (3207, 26), (3208, 29), (3209, 29), (3210, 28), (3211, 23), (3212, 24), (3213, 28), (3214, 23), (3215, 30), (3216, 28), (3217, 22), (3218, 47), (3219, 24), (3220, 27), (3221, 19), (3222, 28), (3223, 25), (3224, 22), (3225, 24), (3226, 36), (3227, 32), (3228, 32), (3229, 38), (3230, 21), (3231, 23), (3232, 26), (3233, 33), (3234, 18), (3235, 24), (3236, 33), (3237, 36), (3238, 29), (3239, 23), (3240, 32), (3241, 28), (3242, 26), (3243, 59), (3244, 21), (3245, 27), (3246, 20), (3247, 20), (3248, 30), (3249, 24), (3250, 44), (3251, 21), (3252, 18), (3253, 25), (3254, 21), (3255, 22), (3256, 37), (3257, 27), (3258, 22), (3259, 49), (3260, 27), (3261, 18), (3262, 19), (3263, 24), (3264, 22), (3265, 23), (3266, 27), (3267, 29), (3268, 52), (3269, 20), (3270, 26), (3271, 28), (3272, 22), (3273, 30), (3274, 27), (3275, 26), (3276, 35), (3277, 25), (3278, 19), (3279, 23), (3280, 26), (3281, 33), (3282, 25), (3283, 21), (3284, 30), (3285, 26), (3286, 22), (3287, 18), (3288, 46), (3289, 20), (3290, 19), (3291, 46), (3292, 21), (3293, 24), (3294, 30), (3295, 23), (3296, 16), (3297, 23), (3298, 19), (3299, 23), (3300, 29), (3301, 33), (3302, 24), (3303, 25), (3304, 31), (3305, 20), (3306, 29), (3307, 29), (3308, 31), (3309, 26), (3310, 27), (3311, 31), (3312, 41), (3313, 18), (3314, 17), (3315, 49), (3316, 26), (3317, 24), (3318, 35), (3319, 32), (3320, 23), (3321, 30), (3322, 37), (3323, 27), (3324, 34), (3325, 25), (3326, 27), (3327, 27), (3328, 32), (3329, 20), (3330, 32), (3331, 17), (3332, 28), (3333, 21), (3334, 24), (3335, 23), (3336, 28), (3337, 22), (3338, 77), (3339, 24), (3340, 28), (3341, 37), (3342, 29), (3343, 30), (3344, 23), (3345, 20), (3346, 42), (3347, 21), (3348, 19), (3349, 23), (3350, 29), (3351, 36), (3352, 38), (3353, 59), (3354, 33), (3355, 35), (3356, 23), (3357, 19), (3358, 32), (3359, 20), (3360, 19), (3361, 21), (3362, 27), (3363, 35), (3364, 30), (3365, 45), (3366, 20), (3367, 26), (3368, 28), (3369, 23), (3370, 18), (3371, 47), (3372, 25), (3373, 17), (3374, 29), (3375, 28), (3376, 28), (3377, 20), (3378, 24), (3379, 30), (3380, 17), (3381, 31), (3382, 24), (3383, 25), (3384, 17), (3385, 27), (3386, 21), (3387, 29), (3388, 22), (3389, 24), (3390, 36), (3391, 41), (3392, 28), (3393, 23), (3394, 28), (3395, 33), (3396, 23), (3397, 19), (3398, 29), (3399, 22), (3400, 33), (3401, 38), (3402, 24), (3403, 25), (3404, 21), (3405, 25), (3406, 33), (3407, 27), (3408, 33), (3409, 21), (3410, 24), (3411, 33), (3412, 40), (3413, 33), (3414, 29), (3415, 31), (3416, 27), (3417, 24), (3418, 24), (3419, 22), (3420, 21), (3421, 24), (3422, 28), (3423, 33), (3424, 28), (3425, 58), (3426, 15), (3427, 22), (3428, 27), (3429, 41), (3430, 50), (3431, 19), (3432, 21), (3433, 42), (3434, 21), (3435, 22), (3436, 34), (3437, 33), (3438, 34), (3439, 18), (3440, 20), (3441, 36), (3442, 28), (3443, 22), (3444, 23), (3445, 26), (3446, 26), (3447, 56), (3448, 27), (3449, 31), (3450, 20), (3451, 28), (3452, 49), (3453, 35), (3454, 23), (3455, 18), (3456, 19), (3457, 20), (3458, 48), (3459, 33), (3460, 37), (3461, 27), (3462, 24), (3463, 32), (3464, 16), (3465, 36), (3466, 29), (3467, 31), (3468, 25), (3469, 20), (3470, 24), (3471, 25), (3472, 30), (3473, 33), (3474, 19), (3475, 32), (3476, 42), (3477, 31), (3478, 32), (3479, 20), (3480, 20), (3481, 22), (3482, 32), (3483, 26), (3484, 20), (3485, 33), (3486, 26), (3487, 26), (3488, 26), (3489, 26), (3490, 31), (3491, 20), (3492, 24), (3493, 23), (3494, 23), (3495, 22), (3496, 20), (3497, 32), (3498, 20), (3499, 24), (3500, 20), (3501, 21), (3502, 25), (3503, 22), (3504, 51), (3505, 29), (3506, 42), (3507, 43), (3508, 37), (3509, 22), (3510, 36), (3511, 22), (3512, 22), (3513, 20), (3514, 25), (3515, 35), (3516, 24), (3517, 31), (3518, 33), (3519, 28), (3520, 20), (3521, 25), (3522, 28), (3523, 26), (3524, 22), (3525, 26), (3526, 21), (3527, 31), (3528, 28), (3529, 28), (3530, 47), (3531, 31), (3532, 24), (3533, 19), (3534, 34), (3535, 27), (3536, 17), (3537, 34), (3538, 21), (3539, 44), (3540, 31), (3541, 35), (3542, 23), (3543, 18), (3544, 27), (3545, 24), (3546, 33), (3547, 47), (3548, 28), (3549, 24), (3550, 34), (3551, 33), (3552, 27), (3553, 23), (3554, 37), (3555, 15), (3556, 36), (3557, 25), (3558, 27), (3559, 30), (3560, 26), (3561, 19), (3562, 22), (3563, 29), (3564, 25), (3565, 21), (3566, 36), (3567, 23), (3568, 23), (3569, 26), (3570, 29), (3571, 22), (3572, 29), (3573, 31), (3574, 22), (3575, 42), (3576, 26), (3577, 31), (3578, 37), (3579, 34), (3580, 25), (3581, 26), (3582, 19), (3583, 85), (3584, 40), (3585, 27), (3586, 37), (3587, 21), (3588, 20), (3589, 37), (3590, 22), (3591, 30), (3592, 27), (3593, 24), (3594, 22), (3595, 23), (3596, 26), (3597, 71), (3598, 29), (3599, 29), (3600, 21), (3601, 23), (3602, 27), (3603, 22), (3604, 29), (3605, 52), (3606, 35), (3607, 21), (3608, 27), (3609, 24), (3610, 27), (3611, 17), (3612, 23), (3613, 32), (3614, 48), (3615, 28), (3616, 29), (3617, 32), (3618, 20), (3619, 38), (3620, 37), (3621, 36), (3622, 33), (3623, 23), (3624, 22), (3625, 41), (3626, 23), (3627, 23), (3628, 54), (3629, 28), (3630, 23), (3631, 15), (3632, 68), (3633, 22), (3634, 20), (3635, 31), (3636, 27), (3637, 21), (3638, 33), (3639, 31), (3640, 27), (3641, 22), (3642, 36), (3643, 22), (3644, 23), (3645, 31), (3646, 16), (3647, 20), (3648, 34), (3649, 52), (3650, 24), (3651, 24), (3652, 33), (3653, 29), (3654, 23), (3655, 43), (3656, 28), (3657, 29), (3658, 22), (3659, 29), (3660, 33), (3661, 35), (3662, 29), (3663, 29), (3664, 24), (3665, 30), (3666, 25), (3667, 34), (3668, 43), (3669, 20), (3670, 30), (3671, 26), (3672, 26), (3673, 29), (3674, 32), (3675, 21), (3676, 24), (3677, 26), (3678, 39), (3679, 34), (3680, 22), (3681, 27), (3682, 36), (3683, 35), (3684, 25), (3685, 16), (3686, 21), (3687, 22), (3688, 38), (3689, 34), (3690, 22), (3691, 22), (3692, 27), (3693, 30), (3694, 24), (3695, 34), (3696, 17), (3697, 25), (3698, 19), (3699, 43), (3700, 23), (3701, 16), (3702, 92), (3703, 29), (3704, 23), (3705, 28), (3706, 20), (3707, 26), (3708, 28), (3709, 54), (3710, 28), (3711, 22), (3712, 15), (3713, 49), (3714, 21), (3715, 30), (3716, 35), (3717, 21), (3718, 27), (3719, 19), (3720, 42), (3721, 21), (3722, 26), (3723, 19), (3724, 23), (3725, 20), (3726, 33), (3727, 24), (3728, 39), (3729, 38), (3730, 52), (3731, 31), (3732, 30), (3733, 23), (3734, 41), (3735, 29), (3736, 30), (3737, 22), (3738, 33), (3739, 34), (3740, 31), (3741, 22), (3742, 38), (3743, 26), (3744, 39), (3745, 23), (3746, 25), (3747, 25), (3748, 24), (3749, 30), (3750, 25), (3751, 36), (3752, 16), (3753, 23), (3754, 47), (3755, 24), (3756, 31), (3757, 39), (3758, 29), (3759, 23), (3760, 23), (3761, 30), (3762, 24), (3763, 22), (3764, 25), (3765, 28), (3766, 33), (3767, 34), (3768, 25), (3769, 25), (3770, 35), (3771, 22), (3772, 24), (3773, 24), (3774, 28), (3775, 18), (3776, 28), (3777, 44), (3778, 17), (3779, 24), (3780, 37), (3781, 39), (3782, 25), (3783, 31), (3784, 31), (3785, 25), (3786, 118), (3787, 26), (3788, 23), (3789, 34), (3790, 26), (3791, 33), (3792, 24), (3793, 27), (3794, 37), (3795, 39), (3796, 30), (3797, 28), (3798, 27), (3799, 32), (3800, 46), (3801, 19), (3802, 20), (3803, 31), (3804, 24), (3805, 64), (3806, 18), (3807, 23), (3808, 20), (3809, 29), (3810, 33), (3811, 30), (3812, 28), (3813, 41), (3814, 29), (3815, 31), (3816, 23), (3817, 39), (3818, 26), (3819, 36), (3820, 31), (3821, 25), (3822, 25), (3823, 28), (3824, 41), (3825, 22), (3826, 42), (3827, 31), (3828, 19), (3829, 21), (3830, 27), (3831, 19), (3832, 34), (3833, 44), (3834, 39), (3835, 25), (3836, 30), (3837, 22), (3838, 20), (3839, 25), (3840, 16), (3841, 27), (3842, 16), (3843, 42), (3844, 29), (3845, 43), (3846, 23), (3847, 20), (3848, 35), (3849, 38), (3850, 28), (3851, 31), (3852, 21), (3853, 18), (3854, 28), (3855, 33), (3856, 36), (3857, 33), (3858, 24), (3859, 36), (3860, 44), (3861, 24), (3862, 17), (3863, 41), (3864, 28), (3865, 24), (3866, 60), (3867, 38), (3868, 48), (3869, 22), (3870, 18), (3871, 35), (3872, 32), (3873, 25), (3874, 23), (3875, 26), (3876, 38), (3877, 51), (3878, 22), (3879, 27), (3880, 24), (3881, 24), (3882, 28), (3883, 29), (3884, 23), (3885, 34), (3886, 38), (3887, 17), (3888, 38), (3889, 24), (3890, 24), (3891, 21), (3892, 35), (3893, 20), (3894, 28), (3895, 53), (3896, 32), (3897, 26), (3898, 35), (3899, 30), (3900, 55), (3901, 41), (3902, 29), (3903, 23), (3904, 40), (3905, 44), (3906, 17), (3907, 29), (3908, 29), (3909, 65), (3910, 25), (3911, 39), (3912, 18), (3913, 33), (3914, 33), (3915, 33), (3916, 34), (3917, 33), (3918, 37), (3919, 23), (3920, 20), (3921, 40), (3922, 24), (3923, 41), (3924, 32), (3925, 28), (3926, 27), (3927, 30), (3928, 36), (3929, 27), (3930, 25), (3931, 57), (3932, 24), (3933, 18), (3934, 24), (3935, 31), (3936, 35), (3937, 24), (3938, 47), (3939, 27), (3940, 34), (3941, 39), (3942, 26), (3943, 27), (3944, 46), (3945, 21), (3946, 28), (3947, 24), (3948, 27), (3949, 32), (3950, 18), (3951, 26), (3952, 21), (3953, 26), (3954, 20), (3955, 43), (3956, 40), (3957, 33), (3958, 40), (3959, 33), (3960, 28), (3961, 26), (3962, 37), (3963, 24), (3964, 24), (3965, 30), (3966, 23), (3967, 26), (3968, 46), (3969, 26), (3970, 40), (3971, 26), (3972, 23), (3973, 34), (3974, 29), (3975, 25), (3976, 29), (3977, 24), (3978, 25), (3979, 34), (3980, 19), (3981, 33), (3982, 33), (3983, 20), (3984, 41), (3985, 29), (3986, 15), (3987, 33), (3988, 29), (3989, 32), (3990, 34), (3991, 34), (3992, 21), (3993, 24), (3994, 25), (3995, 24), (3996, 20), (3997, 38), (3998, 18), (3999, 30), (4000, 24), (4001, 16), (4002, 20), (4003, 24), (4004, 26), (4005, 42), (4006, 28), (4007, 38), (4008, 26), (4009, 27), (4010, 20), (4011, 23), (4012, 21), (4013, 28), (4014, 31), (4015, 32), (4016, 28), (4017, 18), (4018, 20), (4019, 32), (4020, 33), (4021, 32), (4022, 30), (4023, 17), (4024, 33), (4025, 28), (4026, 28), (4027, 27), (4028, 18), (4029, 21), (4030, 22), (4031, 37), (4032, 27), (4033, 28), (4034, 24), (4035, 37), (4036, 30), (4037, 32), (4038, 21), (4039, 20), (4040, 56), (4041, 31), (4042, 31), (4043, 25), (4044, 35), (4045, 34), (4046, 27), (4047, 37), (4048, 35), (4049, 31), (4050, 33), (4051, 21), (4052, 26), (4053, 26), (4054, 26), (4055, 28), (4056, 39), (4057, 24), (4058, 33), (4059, 56), (4060, 27), (4061, 38), (4062, 32), (4063, 38), (4064, 24), (4065, 26), (4066, 28), (4067, 32), (4068, 21), (4069, 39), (4070, 33), (4071, 25), (4072, 29), (4073, 29), (4074, 21), (4075, 16), (4076, 31), (4077, 21), (4078, 62), (4079, 29), (4080, 38), (4081, 40), (4082, 45), (4083, 24), (4084, 56), (4085, 27), (4086, 31), (4087, 24), (4088, 29), (4089, 27), (4090, 33), (4091, 20), (4092, 29), (4093, 28), (4094, 27), (4095, 32), (4096, 24), (4097, 35), (4098, 16), (4099, 17), (4100, 20), (4101, 36), (4102, 26), (4103, 16), (4104, 27), (4105, 40), (4106, 49), (4107, 23), (4108, 19), (4109, 64), (4110, 15), (4111, 29), (4112, 38), (4113, 29), (4114, 16), (4115, 23), (4116, 19), (4117, 24), (4118, 22), (4119, 35), (4120, 28), (4121, 17), (4122, 26), (4123, 20), (4124, 37), (4125, 15), (4126, 23), (4127, 21), (4128, 47), (4129, 20), (4130, 28), (4131, 21), (4132, 32), (4133, 25), (4134, 32), (4135, 16), (4136, 26), (4137, 30), (4138, 31), (4139, 31), (4140, 23), (4141, 31), (4142, 24), (4143, 22), (4144, 38), (4145, 29), (4146, 23), (4147, 35), (4148, 31), (4149, 17), (4150, 23), (4151, 20), (4152, 32), (4153, 21), (4154, 28), (4155, 33), (4156, 21), (4157, 22), (4158, 20), (4159, 38), (4160, 24), (4161, 41), (4162, 31), (4163, 33), (4164, 25), (4165, 28), (4166, 29), (4167, 20), (4168, 28), (4169, 22), (4170, 21), (4171, 25), (4172, 47), (4173, 33), (4174, 22), (4175, 18), (4176, 28), (4177, 31), (4178, 30), (4179, 38), (4180, 46), (4181, 37), (4182, 26), (4183, 22), (4184, 32), (4185, 23), (4186, 28), (4187, 30), (4188, 34), (4189, 35), (4190, 20), (4191, 49), (4192, 52), (4193, 24), (4194, 29), (4195, 32), (4196, 39), (4197, 33), (4198, 28), (4199, 25), (4200, 49), (4201, 29), (4202, 21), (4203, 32), (4204, 23), (4205, 50), (4206, 22), (4207, 20), (4208, 24), (4209, 24), (4210, 28), (4211, 25), (4212, 28), (4213, 36), (4214, 28), (4215, 26), (4216, 15), (4217, 46), (4218, 61), (4219, 73), (4220, 19), (4221, 70), (4222, 20), (4223, 60), (4224, 26), (4225, 23), (4226, 30), (4227, 23), (4228, 20), (4229, 25), (4230, 26), (4231, 24), (4232, 20), (4233, 26), (4234, 33), (4235, 34), (4236, 20), (4237, 25), (4238, 18), (4239, 32), (4240, 18), (4241, 23), (4242, 48), (4243, 20), (4244, 24), (4245, 40), (4246, 20), (4247, 34), (4248, 35), (4249, 39), (4250, 18), (4251, 22), (4252, 25), (4253, 33), (4254, 22), (4255, 36), (4256, 23), (4257, 54), (4258, 46), (4259, 73), (4260, 20), (4261, 32), (4262, 26), (4263, 31), (4264, 45), (4265, 20), (4266, 19), (4267, 33), (4268, 43), (4269, 23), (4270, 94), (4271, 27), (4272, 44), (4273, 34), (4274, 20), (4275, 26), (4276, 21), (4277, 28), (4278, 39), (4279, 27), (4280, 20), (4281, 27), (4282, 24), (4283, 39), (4284, 33), (4285, 29), (4286, 23), (4287, 27), (4288, 33), (4289, 21), (4290, 32), (4291, 21), (4292, 92), (4293, 23), (4294, 40), (4295, 21), (4296, 19), (4297, 19), (4298, 23), (4299, 46), (4300, 26), (4301, 28), (4302, 23), (4303, 32), (4304, 30), (4305, 27), (4306, 36), (4307, 32), (4308, 24), (4309, 26), (4310, 24), (4311, 20), (4312, 29), (4313, 15), (4314, 24), (4315, 23), (4316, 18), (4317, 29), (4318, 36), (4319, 28), (4320, 19), (4321, 40), (4322, 26), (4323, 23), (4324, 24), (4325, 36), (4326, 27), (4327, 23), (4328, 24), (4329, 18), (4330, 43), (4331, 26), (4332, 32), (4333, 29), (4334, 27), (4335, 28), (4336, 44), (4337, 26), (4338, 27), (4339, 31), (4340, 23), (4341, 30), (4342, 24), (4343, 20), (4344, 22), (4345, 24), (4346, 26), (4347, 29), (4348, 36), (4349, 29), (4350, 25), (4351, 15), (4352, 54), (4353, 35), (4354, 20), (4355, 30), (4356, 21), (4357, 27), (4358, 24), (4359, 29), (4360, 67), (4361, 56), (4362, 23), (4363, 20), (4364, 28), (4365, 36), (4366, 22), (4367, 21), (4368, 31), (4369, 30), (4370, 24), (4371, 30), (4372, 24), (4373, 31), (4374, 35), (4375, 23), (4376, 36), (4377, 86), (4378, 24), (4379, 25), (4380, 27), (4381, 43), (4382, 22), (4383, 21), (4384, 23), (4385, 36), (4386, 39), (4387, 20), (4388, 59), (4389, 24), (4390, 42), (4391, 35), (4392, 36), (4393, 38), (4394, 29), (4395, 36), (4396, 24), (4397, 27), (4398, 42), (4399, 32), (4400, 32), (4401, 27), (4402, 26), (4403, 25), (4404, 26), (4405, 22), (4406, 20), (4407, 21), (4408, 33), (4409, 21), (4410, 47), (4411, 31), (4412, 34), (4413, 37), (4414, 27), (4415, 27), (4416, 23), (4417, 27), (4418, 25), (4419, 32), (4420, 32), (4421, 24), (4422, 40), (4423, 28), (4424, 19), (4425, 18), (4426, 23), (4427, 27), (4428, 24), (4429, 31), (4430, 20), (4431, 26), (4432, 26), (4433, 32), (4434, 42), (4435, 37), (4436, 35), (4437, 42), (4438, 28), (4439, 68), (4440, 27), (4441, 24), (4442, 23), (4443, 21), (4444, 23), (4445, 34), (4446, 41), (4447, 32), (4448, 32), (4449, 27), (4450, 24), (4451, 24), (4452, 40), (4453, 23), (4454, 21), (4455, 41), (4456, 24), (4457, 20), (4458, 40), (4459, 20), (4460, 21), (4461, 29), (4462, 30), (4463, 21), (4464, 51), (4465, 21), (4466, 31), (4467, 39), (4468, 32), (4469, 26), (4470, 28), (4471, 28), (4472, 30), (4473, 26), (4474, 22), (4475, 60), (4476, 37), (4477, 31), (4478, 28), (4479, 33), (4480, 31), (4481, 27), (4482, 48), (4483, 24), (4484, 18), (4485, 28), (4486, 22), (4487, 27), (4488, 20), (4489, 28), (4490, 39), (4491, 40), (4492, 25), (4493, 31), (4494, 24), (4495, 44), (4496, 24), (4497, 24), (4498, 17), (4499, 36), (4500, 24), (4501, 32), (4502, 41), (4503, 21), (4504, 33), (4505, 31), (4506, 21), (4507, 19), (4508, 30), (4509, 33), (4510, 47), (4511, 33), (4512, 23), (4513, 29), (4514, 32), (4515, 19), (4516, 27), (4517, 25), (4518, 19), (4519, 17), (4520, 28), (4521, 37), (4522, 40), (4523, 25), (4524, 21), (4525, 22), (4526, 25), (4527, 26), (4528, 35), (4529, 30), (4530, 23), (4531, 18), (4532, 27), (4533, 15), (4534, 26), (4535, 28), (4536, 23), (4537, 29), (4538, 28), (4539, 15), (4540, 51), (4541, 28), (4542, 38), (4543, 28), (4544, 30), (4545, 34), (4546, 38), (4547, 31), (4548, 21), (4549, 26), (4550, 19), (4551, 20), (4552, 34), (4553, 20), (4554, 27), (4555, 26), (4556, 33), (4557, 49), (4558, 35), (4559, 24), (4560, 20), (4561, 35), (4562, 31), (4563, 22), (4564, 26), (4565, 34), (4566, 32), (4567, 21), (4568, 23), (4569, 23), (4570, 30), (4571, 38), (4572, 23), (4573, 41), (4574, 33), (4575, 25), (4576, 23), (4577, 26), (4578, 24), (4579, 30), (4580, 59), (4581, 26), (4582, 33), (4583, 24), (4584, 20), (4585, 24), (4586, 20), (4587, 19), (4588, 19), (4589, 32), (4590, 14), (4591, 24), (4592, 19), (4593, 33), (4594, 24), (4595, 28), (4596, 26), (4597, 33), (4598, 24), (4599, 22), (4600, 17), (4601, 26), (4602, 31), (4603, 45), (4604, 20), (4605, 22), (4606, 17), (4607, 24), (4608, 40), (4609, 41), (4610, 36), (4611, 25), (4612, 51), (4613, 25), (4614, 30), (4615, 28), (4616, 24), (4617, 30), (4618, 24), (4619, 26), (4620, 31), (4621, 20), (4622, 19), (4623, 23), (4624, 31), (4625, 21), (4626, 22), (4627, 23), (4628, 22), (4629, 27), (4630, 26), (4631, 38), (4632, 38), (4633, 43), (4634, 35), (4635, 29), (4636, 25), (4637, 23), (4638, 19), (4639, 28), (4640, 19), (4641, 22), (4642, 37), (4643, 28), (4644, 57), (4645, 17), (4646, 25), (4647, 28), (4648, 20), (4649, 29), (4650, 33), (4651, 30), (4652, 35), (4653, 40), (4654, 23), (4655, 25), (4656, 72), (4657, 38), (4658, 46), (4659, 30), (4660, 19), (4661, 27), (4662, 28), (4663, 23), (4664, 26), (4665, 26), (4666, 26), (4667, 32), (4668, 44), (4669, 22), (4670, 37), (4671, 33), (4672, 34), (4673, 24), (4674, 37), (4675, 21), (4676, 19), (4677, 28), (4678, 27), (4679, 23), (4680, 20), (4681, 28), (4682, 29), (4683, 34), (4684, 40), (4685, 16), (4686, 23), (4687, 22), (4688, 26), (4689, 29), (4690, 27), (4691, 19), (4692, 25), (4693, 35), (4694, 22), (4695, 31), (4696, 19), (4697, 33), (4698, 24), (4699, 22), (4700, 24), (4701, 28), (4702, 40), (4703, 23), (4704, 37), (4705, 42), (4706, 33), (4707, 59), (4708, 24), (4709, 21), (4710, 40), (4711, 29), (4712, 28), (4713, 20), (4714, 30), (4715, 23), (4716, 29), (4717, 29), (4718, 23), (4719, 28), (4720, 61), (4721, 37), (4722, 34), (4723, 30), (4724, 29), (4725, 25), (4726, 56), (4727, 20), (4728, 31), (4729, 23), (4730, 35), (4731, 20), (4732, 33), (4733, 17), (4734, 22), (4735, 29), (4736, 23), (4737, 24), (4738, 19), (4739, 28), (4740, 19), (4741, 30), (4742, 37), (4743, 23), (4744, 30), (4745, 26), (4746, 30), (4747, 56), (4748, 54), (4749, 26), (4750, 20), (4751, 39), (4752, 32), (4753, 37), (4754, 19), (4755, 31), (4756, 44), (4757, 24), (4758, 30), (4759, 23), (4760, 28), (4761, 26), (4762, 29), (4763, 18), (4764, 25), (4765, 23), (4766, 26), (4767, 22), (4768, 28), (4769, 30), (4770, 19), (4771, 29), (4772, 110), (4773, 24), (4774, 29), (4775, 31), (4776, 25), (4777, 23), (4778, 25), (4779, 50), (4780, 28), (4781, 34), (4782, 26), (4783, 56), (4784, 27), (4785, 20), (4786, 41), (4787, 27), (4788, 48), (4789, 24), (4790, 28), (4791, 19), (4792, 34), (4793, 19), (4794, 33), (4795, 32), (4796, 24), (4797, 33), (4798, 32), (4799, 24), (4800, 51), (4801, 35), (4802, 24), (4803, 25), (4804, 24), (4805, 29), (4806, 34), (4807, 32), (4808, 20), (4809, 20), (4810, 27), (4811, 42), (4812, 25), (4813, 18), (4814, 38), (4815, 49), (4816, 32), (4817, 28), (4818, 26), (4819, 30), (4820, 23), (4821, 45), (4822, 64), (4823, 33), (4824, 92), (4825, 30), (4826, 29), (4827, 27), (4828, 20), (4829, 27), (4830, 32), (4831, 25), (4832, 24), (4833, 29), (4834, 22), (4835, 29), (4836, 31), (4837, 25), (4838, 27), (4839, 42), (4840, 20), (4841, 29), (4842, 38), (4843, 23), (4844, 27), (4845, 22), (4846, 26), (4847, 20), (4848, 21), (4849, 33), (4850, 38), (4851, 23), (4852, 23), (4853, 45), (4854, 25), (4855, 26), (4856, 23), (4857, 29), (4858, 44), (4859, 24), (4860, 28), (4861, 43), (4862, 36), (4863, 21), (4864, 31), (4865, 30), (4866, 30), (4867, 26), (4868, 47), (4869, 22), (4870, 23), (4871, 38), (4872, 29), (4873, 40), (4874, 25), (4875, 61), (4876, 23), (4877, 19), (4878, 24), (4879, 40), (4880, 30), (4881, 27), (4882, 18), (4883, 34), (4884, 35), (4885, 25), (4886, 18), (4887, 24), (4888, 20), (4889, 36), (4890, 27), (4891, 25), (4892, 49), (4893, 16), (4894, 19), (4895, 23), (4896, 33), (4897, 24), (4898, 36), (4899, 25), (4900, 25), (4901, 19), (4902, 19), (4903, 36), (4904, 29), (4905, 23), (4906, 52), (4907, 24), (4908, 26), (4909, 20), (4910, 52), (4911, 19), (4912, 31), (4913, 21), (4914, 30), (4915, 18), (4916, 27), (4917, 27), (4918, 27), (4919, 29), (4920, 21), (4921, 30), (4922, 34), (4923, 28), (4924, 18), (4925, 36), (4926, 31), (4927, 19), (4928, 20), (4929, 32), (4930, 31), (4931, 33), (4932, 30), (4933, 33), (4934, 25), (4935, 27), (4936, 26), (4937, 23), (4938, 32), (4939, 45), (4940, 47), (4941, 31), (4942, 60), (4943, 24), (4944, 28), (4945, 28), (4946, 27), (4947, 23), (4948, 24), (4949, 23), (4950, 39), (4951, 14), (4952, 22), (4953, 24), (4954, 16), (4955, 21), (4956, 21), (4957, 28), (4958, 26), (4959, 19), (4960, 24), (4961, 38), (4962, 34), (4963, 41), (4964, 45), (4965, 24), (4966, 20), (4967, 32), (4968, 20), (4969, 31), (4970, 67), (4971, 24), (4972, 30), (4973, 27), (4974, 22), (4975, 27), (4976, 26), (4977, 27), (4978, 43), (4979, 29), (4980, 25), (4981, 22), (4982, 28), (4983, 50), (4984, 23), (4985, 24), (4986, 25), (4987, 17), (4988, 16), (4989, 30), (4990, 41), (4991, 34), (4992, 30), (4993, 24), (4994, 36), (4995, 22), (4996, 24), (4997, 22), (4998, 27), (4999, 31), (5000, 17), (5001, 26), (5002, 34), (5003, 24), (5004, 28), (5005, 29), (5006, 29), (5007, 40), (5008, 41), (5009, 35), (5010, 41), (5011, 24), (5012, 24), (5013, 25), (5014, 26), (5015, 27), (5016, 54), (5017, 30), (5018, 42), (5019, 17), (5020, 33), (5021, 30), (5022, 35), (5023, 43), (5024, 18), (5025, 28), (5026, 33), (5027, 28), (5028, 18), (5029, 14), (5030, 20), (5031, 23), (5032, 26), (5033, 32), (5034, 29), (5035, 26), (5036, 24), (5037, 35), (5038, 20), (5039, 28), (5040, 18), (5041, 25), (5042, 34), (5043, 32), (5044, 29), (5045, 23), (5046, 24), (5047, 15), (5048, 34), (5049, 30), (5050, 20), (5051, 33), (5052, 44), (5053, 32), (5054, 15), (5055, 29), (5056, 29), (5057, 23), (5058, 21), (5059, 25), (5060, 18), (5061, 47), (5062, 19), (5063, 31), (5064, 29), (5065, 47), (5066, 30), (5067, 18), (5068, 25), (5069, 27), (5070, 24), (5071, 21), (5072, 27), (5073, 48), (5074, 28), (5075, 27), (5076, 26), (5077, 22), (5078, 18), (5079, 24), (5080, 21), (5081, 20), (5082, 22), (5083, 46), (5084, 21), (5085, 34), (5086, 23), (5087, 38), (5088, 18), (5089, 22), (5090, 25), (5091, 25), (5092, 42), (5093, 22), (5094, 19), (5095, 19), (5096, 27), (5097, 28), (5098, 64), (5099, 23), (5100, 23), (5101, 20), (5102, 32), (5103, 31), (5104, 27), (5105, 83), (5106, 34), (5107, 26), (5108, 26), (5109, 37), (5110, 41), (5111, 63), (5112, 28), (5113, 21), (5114, 34), (5115, 28), (5116, 30), (5117, 30), (5118, 17), (5119, 57), (5120, 26), (5121, 30), (5122, 21), (5123, 29), (5124, 16), (5125, 28), (5126, 25), (5127, 21), (5128, 28), (5129, 22), (5130, 38), (5131, 22), (5132, 29), (5133, 29), (5134, 30), (5135, 27), (5136, 25), (5137, 30), (5138, 23), (5139, 25), (5140, 28), (5141, 21), (5142, 19), (5143, 26), (5144, 42), (5145, 19), (5146, 29), (5147, 24), (5148, 34), (5149, 34), (5150, 30), (5151, 32), (5152, 23), (5153, 56), (5154, 21), (5155, 29), (5156, 60), (5157, 19), (5158, 22), (5159, 24), (5160, 54), (5161, 25), (5162, 53), (5163, 24), (5164, 32), (5165, 26), (5166, 27), (5167, 25), (5168, 23), (5169, 24), (5170, 19), (5171, 40), (5172, 28), (5173, 44), (5174, 19), (5175, 20), (5176, 18), (5177, 28), (5178, 27), (5179, 30), (5180, 18), (5181, 25), (5182, 19), (5183, 23), (5184, 33), (5185, 29), (5186, 28), (5187, 35), (5188, 42), (5189, 33), (5190, 32), (5191, 28), (5192, 29), (5193, 33), (5194, 31), (5195, 28), (5196, 98), (5197, 50), (5198, 27), (5199, 20), (5200, 21), (5201, 36), (5202, 20), (5203, 32), (5204, 33), (5205, 21), (5206, 27), (5207, 26), (5208, 34), (5209, 18), (5210, 21), (5211, 20), (5212, 24), (5213, 63), (5214, 18), (5215, 25), (5216, 43), (5217, 41), (5218, 31), (5219, 24), (5220, 35), (5221, 27), (5222, 23), (5223, 21), (5224, 19), (5225, 18), (5226, 37), (5227, 26), (5228, 51), (5229, 36), (5230, 22), (5231, 21), (5232, 26), (5233, 35), (5234, 25), (5235, 18), (5236, 17), (5237, 41), (5238, 44), (5239, 63), (5240, 28), (5241, 24), (5242, 43), (5243, 25), (5244, 22), (5245, 21), (5246, 26), (5247, 20), (5248, 26), (5249, 30), (5250, 29), (5251, 31), (5252, 24), (5253, 32), (5254, 16), (5255, 32), (5256, 30), (5257, 15), (5258, 24), (5259, 22), (5260, 41), (5261, 21), (5262, 24), (5263, 29), (5264, 28), (5265, 27), (5266, 27), (5267, 22), (5268, 30), (5269, 26), (5270, 22), (5271, 44), (5272, 24), (5273, 27), (5274, 25), (5275, 21), (5276, 30), (5277, 35), (5278, 25), (5279, 24), (5280, 29), (5281, 20), (5282, 30), (5283, 28), (5284, 34), (5285, 40), (5286, 30), (5287, 41), (5288, 20), (5289, 31), (5290, 68), (5291, 34), (5292, 28), (5293, 38), (5294, 27), (5295, 20), (5296, 26), (5297, 25), (5298, 20), (5299, 27), (5300, 26), (5301, 35), (5302, 17), (5303, 50), (5304, 37), (5305, 41), (5306, 20), (5307, 25), (5308, 27), (5309, 32), (5310, 21), (5311, 24), (5312, 27), (5313, 82), (5314, 35), (5315, 22), (5316, 28), (5317, 26), (5318, 31), (5319, 25), (5320, 30), (5321, 32), (5322, 26), (5323, 20), (5324, 40), (5325, 24), (5326, 20), (5327, 21), (5328, 31), (5329, 41), (5330, 31), (5331, 15), (5332, 17), (5333, 18), (5334, 44), (5335, 29), (5336, 36), (5337, 30), (5338, 32), (5339, 34), (5340, 17), (5341, 24), (5342, 23), (5343, 23), (5344, 33), (5345, 20), (5346, 34), (5347, 39), (5348, 15), (5349, 22), (5350, 20), (5351, 35), (5352, 28), (5353, 36), (5354, 35), (5355, 28), (5356, 22), (5357, 27), (5358, 24), (5359, 22), (5360, 36), (5361, 37), (5362, 33), (5363, 24), (5364, 25), (5365, 24), (5366, 29), (5367, 25), (5368, 24), (5369, 25), (5370, 25), (5371, 22), (5372, 31), (5373, 25), (5374, 29), (5375, 30), (5376, 27), (5377, 47), (5378, 28), (5379, 25), (5380, 16), (5381, 20), (5382, 48), (5383, 26), (5384, 27), (5385, 19), (5386, 25), (5387, 28), (5388, 23), (5389, 23), (5390, 33), (5391, 23), (5392, 37), (5393, 30), (5394, 33), (5395, 26), (5396, 44), (5397, 25), (5398, 26), (5399, 24), (5400, 24), (5401, 20), (5402, 31), (5403, 23), (5404, 21), (5405, 30), (5406, 22), (5407, 59), (5408, 56), (5409, 27), (5410, 26), (5411, 28), (5412, 25), (5413, 25), (5414, 36), (5415, 25), (5416, 21), (5417, 35), (5418, 24), (5419, 22), (5420, 23), (5421, 35), (5422, 22), (5423, 30), (5424, 47), (5425, 55), (5426, 25), (5427, 16), (5428, 36), (5429, 23), (5430, 24), (5431, 21), (5432, 44), (5433, 19), (5434, 26), (5435, 21), (5436, 25), (5437, 33), (5438, 26), (5439, 29), (5440, 33), (5441, 19), (5442, 28), (5443, 22), (5444, 30), (5445, 28), (5446, 24), (5447, 81), (5448, 30), (5449, 27), (5450, 43), (5451, 34), (5452, 37), (5453, 22), (5454, 29), (5455, 31), (5456, 62), (5457, 49), (5458, 39), (5459, 48), (5460, 51), (5461, 28), (5462, 40), (5463, 22), (5464, 33), (5465, 24), (5466, 21), (5467, 31), (5468, 20), (5469, 26), (5470, 32), (5471, 22), (5472, 27), (5473, 25), (5474, 24), (5475, 16), (5476, 55), (5477, 29), (5478, 35), (5479, 28), (5480, 23), (5481, 63), (5482, 28), (5483, 20), (5484, 26), (5485, 36), (5486, 34), (5487, 17), (5488, 46), (5489, 35), (5490, 56), (5491, 41), (5492, 21), (5493, 39), (5494, 34), (5495, 48), (5496, 21), (5497, 35), (5498, 36), (5499, 16), (5500, 38), (5501, 21), (5502, 27), (5503, 21), (5504, 19), (5505, 38), (5506, 25), (5507, 23), (5508, 23), (5509, 28), (5510, 37), (5511, 18), (5512, 29), (5513, 24), (5514, 33), (5515, 29), (5516, 25), (5517, 34), (5518, 28), (5519, 31), (5520, 31), (5521, 29), (5522, 35), (5523, 26), (5524, 20), (5525, 33), (5526, 29), (5527, 44), (5528, 26), (5529, 26), (5530, 33), (5531, 40), (5532, 19), (5533, 32), (5534, 26), (5535, 27), (5536, 29), (5537, 57), (5538, 20), (5539, 16), (5540, 21), (5541, 26), (5542, 25), (5543, 31), (5544, 35), (5545, 65), (5546, 21), (5547, 20), (5548, 32), (5549, 25), (5550, 23), (5551, 25), (5552, 31), (5553, 61), (5554, 22), (5555, 23), (5556, 25), (5557, 20), (5558, 20), (5559, 33), (5560, 34), (5561, 42), (5562, 20), (5563, 23), (5564, 29), (5565, 31), (5566, 26), (5567, 20), (5568, 54), (5569, 30), (5570, 33), (5571, 32), (5572, 69), (5573, 34), (5574, 21), (5575, 39), (5576, 32), (5577, 24), (5578, 20), (5579, 31), (5580, 70), (5581, 30), (5582, 25), (5583, 21), (5584, 27), (5585, 31), (5586, 16), (5587, 38), (5588, 54), (5589, 27), (5590, 26), (5591, 28), (5592, 24), (5593, 42), (5594, 43), (5595, 27), (5596, 20), (5597, 27), (5598, 30), (5599, 36), (5600, 30), (5601, 24), (5602, 33), (5603, 18), (5604, 28), (5605, 59), (5606, 26), (5607, 23), (5608, 32), (5609, 37), (5610, 20), (5611, 20), (5612, 24), (5613, 36), (5614, 42), (5615, 24), (5616, 34), (5617, 41), (5618, 25), (5619, 25), (5620, 41), (5621, 23), (5622, 25), (5623, 24), (5624, 19), (5625, 37), (5626, 26), (5627, 20), (5628, 29), (5629, 42), (5630, 27), (5631, 55), (5632, 27), (5633, 18), (5634, 38), (5635, 30), (5636, 16), (5637, 26), (5638, 17), (5639, 24), (5640, 31), (5641, 38), (5642, 30), (5643, 24), (5644, 17), (5645, 20), (5646, 32), (5647, 83), (5648, 32), (5649, 30), (5650, 22), (5651, 32), (5652, 15), (5653, 32), (5654, 18), (5655, 46), (5656, 23), (5657, 25), (5658, 37), (5659, 22), (5660, 31), (5661, 23), (5662, 31), (5663, 28), (5664, 37), (5665, 43), (5666, 23), (5667, 29), (5668, 30), (5669, 20), (5670, 24), (5671, 33), (5672, 18), (5673, 30), (5674, 35), (5675, 33), (5676, 42), (5677, 29), (5678, 26), (5679, 31), (5680, 27), (5681, 28), (5682, 30), (5683, 35), (5684, 40), (5685, 26), (5686, 25), (5687, 25), (5688, 27), (5689, 24), (5690, 26), (5691, 44), (5692, 20), (5693, 37), (5694, 28), (5695, 32), (5696, 21), (5697, 22), (5698, 20), (5699, 51), (5700, 21), (5701, 31), (5702, 29), (5703, 25), (5704, 24), (5705, 22), (5706, 24), (5707, 32), (5708, 20), (5709, 67), (5710, 31), (5711, 23), (5712, 26), (5713, 27), (5714, 28), (5715, 19), (5716, 24), (5717, 29), (5718, 18), (5719, 21), (5720, 29), (5721, 41), (5722, 27), (5723, 43), (5724, 28), (5725, 26), (5726, 28), (5727, 27), (5728, 24), (5729, 24), (5730, 35), (5731, 33), (5732, 25), (5733, 35), (5734, 25), (5735, 59), (5736, 19), (5737, 32), (5738, 24), (5739, 41), (5740, 27), (5741, 19), (5742, 27), (5743, 31), (5744, 29), (5745, 38), (5746, 24), (5747, 23), (5748, 30), (5749, 23), (5750, 25), (5751, 30), (5752, 24), (5753, 45), (5754, 34), (5755, 25), (5756, 39), (5757, 24), (5758, 29), (5759, 39), (5760, 40), (5761, 34), (5762, 22), (5763, 32), (5764, 32), (5765, 26), (5766, 22), (5767, 22), (5768, 24), (5769, 24), (5770, 29), (5771, 24), (5772, 33), (5773, 27), (5774, 36), (5775, 36), (5776, 31), (5777, 29), (5778, 24), (5779, 19), (5780, 15), (5781, 28), (5782, 26), (5783, 42), (5784, 19), (5785, 27), (5786, 35), (5787, 23), (5788, 67), (5789, 39), (5790, 28), (5791, 26), (5792, 47), (5793, 25), (5794, 25), (5795, 70), (5796, 29), (5797, 28), (5798, 28), (5799, 24), (5800, 22), (5801, 20), (5802, 31), (5803, 20), (5804, 89), (5805, 26), (5806, 21), (5807, 28), (5808, 22), (5809, 23), (5810, 37), (5811, 38), (5812, 34), (5813, 27), (5814, 23), (5815, 32), (5816, 22), (5817, 20), (5818, 37), (5819, 20), (5820, 47), (5821, 27), (5822, 23), (5823, 31), (5824, 24), (5825, 34), (5826, 19), (5827, 35), (5828, 26), (5829, 19), (5830, 28), (5831, 24), (5832, 27), (5833, 45), (5834, 28), (5835, 23), (5836, 18), (5837, 32), (5838, 26), (5839, 38), (5840, 24), (5841, 19), (5842, 43), (5843, 22), (5844, 23), (5845, 29), (5846, 22), (5847, 20), (5848, 24), (5849, 45), (5850, 43), (5851, 49), (5852, 22), (5853, 39), (5854, 19), (5855, 14), (5856, 32), (5857, 14), (5858, 35), (5859, 27), (5860, 29), (5861, 35), (5862, 51), (5863, 23), (5864, 42), (5865, 30), (5866, 28), (5867, 31), (5868, 18), (5869, 91), (5870, 15), (5871, 25), (5872, 25), (5873, 34), (5874, 33), (5875, 26), (5876, 27), (5877, 28), (5878, 24), (5879, 42), (5880, 23), (5881, 17), (5882, 32), (5883, 27), (5884, 21), (5885, 34), (5886, 25), (5887, 21), (5888, 41), (5889, 32), (5890, 24), (5891, 29), (5892, 21), (5893, 56), (5894, 23), (5895, 25), (5896, 18), (5897, 30), (5898, 23), (5899, 22), (5900, 36), (5901, 36), (5902, 28), (5903, 20), (5904, 21), (5905, 25), (5906, 20), (5907, 38), (5908, 40), (5909, 37), (5910, 20), (5911, 22), (5912, 23), (5913, 13), (5914, 30), (5915, 47), (5916, 29), (5917, 40), (5918, 15), (5919, 23), (5920, 29), (5921, 28), (5922, 25), (5923, 22), (5924, 40), (5925, 25), (5926, 21), (5927, 41), (5928, 29), (5929, 28), (5930, 23), (5931, 24), (5932, 21), (5933, 39), (5934, 38), (5935, 28), (5936, 25), (5937, 34), (5938, 30), (5939, 34), (5940, 42), (5941, 23), (5942, 23), (5943, 36), (5944, 28), (5945, 22), (5946, 19), (5947, 24), (5948, 18), (5949, 23), (5950, 21), (5951, 25), (5952, 30), (5953, 27), (5954, 40), (5955, 24), (5956, 36), (5957, 29), (5958, 22), (5959, 26), (5960, 38), (5961, 27), (5962, 19), (5963, 28), (5964, 27), (5965, 34), (5966, 25), (5967, 18), (5968, 24), (5969, 49), (5970, 31), (5971, 19), (5972, 27), (5973, 33), (5974, 24), (5975, 35), (5976, 34), (5977, 27), (5978, 28), (5979, 29), (5980, 21), (5981, 35), (5982, 34), (5983, 29), (5984, 23), (5985, 23), (5986, 30), (5987, 19), (5988, 31), (5989, 28), (5990, 22), (5991, 41), (5992, 16), (5993, 37), (5994, 26), (5995, 33), (5996, 31), (5997, 31), (5998, 33), (5999, 30), (6000, 29), (6001, 20), (6002, 32), (6003, 16), (6004, 26), (6005, 98), (6006, 40), (6007, 21), (6008, 26), (6009, 18), (6010, 40), (6011, 24), (6012, 32), (6013, 37), (6014, 29), (6015, 18), (6016, 22), (6017, 24), (6018, 19), (6019, 28), (6020, 29), (6021, 32), (6022, 39), (6023, 32), (6024, 43), (6025, 28), (6026, 27), (6027, 23), (6028, 26), (6029, 32), (6030, 27), (6031, 17), (6032, 32), (6033, 50), (6034, 20), (6035, 44), (6036, 28), (6037, 29), (6038, 39), (6039, 20), (6040, 35), (6041, 26), (6042, 27), (6043, 38), (6044, 23), (6045, 31), (6046, 23), (6047, 25), (6048, 25), (6049, 32), (6050, 30), (6051, 20), (6052, 18), (6053, 26), (6054, 43), (6055, 15), (6056, 20), (6057, 29), (6058, 17), (6059, 27), (6060, 32), (6061, 19), (6062, 32), (6063, 28), (6064, 20), (6065, 20), (6066, 60), (6067, 31), (6068, 28), (6069, 24), (6070, 52), (6071, 26), (6072, 27), (6073, 31), (6074, 27), (6075, 21), (6076, 36), (6077, 29), (6078, 29), (6079, 20), (6080, 30), (6081, 34), (6082, 37), (6083, 23), (6084, 32), (6085, 37), (6086, 19), (6087, 39), (6088, 28), (6089, 33), (6090, 51), (6091, 36), (6092, 22), (6093, 38), (6094, 15), (6095, 25), (6096, 17), (6097, 29), (6098, 34), (6099, 44), (6100, 29), (6101, 21), (6102, 20), (6103, 24), (6104, 34), (6105, 34), (6106, 27), (6107, 19), (6108, 25), (6109, 25), (6110, 33), (6111, 32), (6112, 32), (6113, 24), (6114, 38), (6115, 15), (6116, 19), (6117, 21), (6118, 33), (6119, 25), (6120, 27), (6121, 17), (6122, 51), (6123, 49), (6124, 28), (6125, 20), (6126, 48), (6127, 35), (6128, 34), (6129, 20), (6130, 22), (6131, 24), (6132, 40), (6133, 30), (6134, 22), (6135, 24), (6136, 68), (6137, 34), (6138, 34), (6139, 22), (6140, 45), (6141, 22), (6142, 28), (6143, 30), (6144, 40), (6145, 25), (6146, 31), (6147, 25), (6148, 26), (6149, 27), (6150, 31), (6151, 28), (6152, 20), (6153, 25), (6154, 26), (6155, 25), (6156, 48), (6157, 26), (6158, 31), (6159, 25), (6160, 16), (6161, 24), (6162, 17), (6163, 22), (6164, 21), (6165, 17), (6166, 38), (6167, 37), (6168, 27), (6169, 38), (6170, 26), (6171, 49), (6172, 51), (6173, 25), (6174, 39), (6175, 25), (6176, 29), (6177, 23), (6178, 49), (6179, 31), (6180, 19), (6181, 23), (6182, 33), (6183, 31), (6184, 18), (6185, 24), (6186, 52), (6187, 29), (6188, 45), (6189, 33), (6190, 35), (6191, 28), (6192, 38), (6193, 44), (6194, 30), (6195, 24), (6196, 28), (6197, 32), (6198, 43), (6199, 24), (6200, 39), (6201, 17), (6202, 49), (6203, 23), (6204, 46), (6205, 26), (6206, 28), (6207, 25), (6208, 25), (6209, 36), (6210, 30), (6211, 30), (6212, 42), (6213, 26), (6214, 24), (6215, 23), (6216, 23), (6217, 23), (6218, 32), (6219, 33), (6220, 38), (6221, 20), (6222, 20), (6223, 23), (6224, 29), (6225, 25), (6226, 23), (6227, 24), (6228, 32), (6229, 69), (6230, 23), (6231, 21), (6232, 30), (6233, 26), (6234, 20), (6235, 20), (6236, 26), (6237, 23), (6238, 26), (6239, 42), (6240, 44), (6241, 20), (6242, 27), (6243, 23), (6244, 29), (6245, 45), (6246, 22), (6247, 38), (6248, 24), (6249, 27), (6250, 20), (6251, 16), (6252, 53), (6253, 35), (6254, 22), (6255, 24), (6256, 48), (6257, 18), (6258, 23), (6259, 30), (6260, 26), (6261, 21), (6262, 21), (6263, 23), (6264, 37), (6265, 34), (6266, 27), (6267, 24), (6268, 45), (6269, 19), (6270, 22), (6271, 21), (6272, 28), (6273, 28), (6274, 27), (6275, 21), (6276, 27), (6277, 18), (6278, 18), (6279, 25), (6280, 29), (6281, 25), (6282, 28), (6283, 26), (6284, 30), (6285, 34), (6286, 25), (6287, 16), (6288, 25), (6289, 26), (6290, 24), (6291, 33), (6292, 36), (6293, 25), (6294, 19), (6295, 23), (6296, 28), (6297, 26), (6298, 34), (6299, 21), (6300, 29), (6301, 19), (6302, 34), (6303, 31), (6304, 33), (6305, 30), (6306, 19), (6307, 49), (6308, 25), (6309, 22), (6310, 30), (6311, 29), (6312, 25), (6313, 28), (6314, 25), (6315, 29), (6316, 18), (6317, 33), (6318, 27), (6319, 18), (6320, 49), (6321, 20), (6322, 43), (6323, 38), (6324, 23), (6325, 24), (6326, 20), (6327, 34), (6328, 35), (6329, 44), (6330, 26), (6331, 33), (6332, 27), (6333, 19), (6334, 31), (6335, 27), (6336, 40), (6337, 25), (6338, 25), (6339, 27), (6340, 14), (6341, 36), (6342, 26), (6343, 27), (6344, 29), (6345, 24), (6346, 35), (6347, 28), (6348, 24), (6349, 24), (6350, 22), (6351, 31), (6352, 19), (6353, 24), (6354, 44), (6355, 32), (6356, 21), (6357, 23), (6358, 23), (6359, 35), (6360, 29), (6361, 38), (6362, 23), (6363, 23), (6364, 27), (6365, 20), (6366, 29), (6367, 22), (6368, 24), (6369, 23), (6370, 37), (6371, 22), (6372, 23), (6373, 20), (6374, 23), (6375, 29), (6376, 33), (6377, 23), (6378, 40), (6379, 26), (6380, 37), (6381, 20), (6382, 25), (6383, 32), (6384, 30), (6385, 15), (6386, 17), (6387, 61), (6388, 35), (6389, 26), (6390, 39), (6391, 33), (6392, 33), (6393, 16), (6394, 30), (6395, 19), (6396, 33), (6397, 27), (6398, 42), (6399, 21), (6400, 23), (6401, 28), (6402, 20), (6403, 30), (6404, 28), (6405, 22), (6406, 23), (6407, 39), (6408, 26), (6409, 27), (6410, 31), (6411, 23), (6412, 31), (6413, 40), (6414, 33), (6415, 39), (6416, 21), (6417, 19), (6418, 19), (6419, 43), (6420, 38), (6421, 18), (6422, 21), (6423, 24), (6424, 24), (6425, 25), (6426, 21), (6427, 29), (6428, 20), (6429, 28), (6430, 32), (6431, 31), (6432, 26), (6433, 31), (6434, 18), (6435, 41), (6436, 33), (6437, 43), (6438, 27), (6439, 24), (6440, 22), (6441, 41), (6442, 24), (6443, 29), (6444, 27), (6445, 23), (6446, 20), (6447, 19), (6448, 25), (6449, 27), (6450, 34), (6451, 42), (6452, 34), (6453, 36), (6454, 19), (6455, 30), (6456, 30), (6457, 22), (6458, 21), (6459, 46), (6460, 53), (6461, 41), (6462, 31), (6463, 19), (6464, 32), (6465, 27), (6466, 22), (6467, 18), (6468, 21), (6469, 34), (6470, 24), (6471, 16), (6472, 22), (6473, 36), (6474, 25), (6475, 21), (6476, 28), (6477, 22), (6478, 19), (6479, 114), (6480, 30), (6481, 40), (6482, 20), (6483, 24), (6484, 16), (6485, 29), (6486, 24), (6487, 31), (6488, 25), (6489, 28), (6490, 36), (6491, 21), (6492, 42), (6493, 32), (6494, 20), (6495, 34), (6496, 23), (6497, 30), (6498, 26), (6499, 33), (6500, 24), (6501, 22), (6502, 19), (6503, 30), (6504, 21), (6505, 38), (6506, 48), (6507, 37), (6508, 30), (6509, 35), (6510, 24), (6511, 21), (6512, 25), (6513, 23), (6514, 28), (6515, 29), (6516, 31), (6517, 19), (6518, 30), (6519, 25), (6520, 24), (6521, 20), (6522, 54), (6523, 24), (6524, 16), (6525, 22), (6526, 30), (6527, 29), (6528, 27), (6529, 21), (6530, 20), (6531, 25), (6532, 41), (6533, 28), (6534, 33), (6535, 25), (6536, 29), (6537, 41), (6538, 26), (6539, 41), (6540, 34), (6541, 44), (6542, 22), (6543, 32), (6544, 25), (6545, 19), (6546, 15), (6547, 30), (6548, 24), (6549, 20), (6550, 21), (6551, 31), (6552, 34), (6553, 16), (6554, 15), (6555, 38), (6556, 32), (6557, 36), (6558, 29), (6559, 34), (6560, 35), (6561, 31), (6562, 18), (6563, 24), (6564, 37), (6565, 26), (6566, 34), (6567, 24), (6568, 20), (6569, 28), (6570, 20), (6571, 27), (6572, 33), (6573, 23), (6574, 23), (6575, 24), (6576, 23), (6577, 29), (6578, 20), (6579, 36), (6580, 34), (6581, 27), (6582, 36), (6583, 39), (6584, 21), (6585, 27), (6586, 22), (6587, 27), (6588, 28), (6589, 52), (6590, 30), (6591, 31), (6592, 17), (6593, 17), (6594, 43), (6595, 22), (6596, 21), (6597, 21), (6598, 20), (6599, 23), (6600, 31), (6601, 48), (6602, 27), (6603, 24), (6604, 32), (6605, 26), (6606, 29), (6607, 33), (6608, 41), (6609, 34), (6610, 24), (6611, 24), (6612, 25), (6613, 34), (6614, 35), (6615, 25), (6616, 29), (6617, 27), (6618, 29), (6619, 34), (6620, 19), (6621, 25), (6622, 25), (6623, 24), (6624, 27), (6625, 23), (6626, 27), (6627, 30), (6628, 40), (6629, 33), (6630, 24), (6631, 18), (6632, 52), (6633, 35), (6634, 23), (6635, 24), (6636, 27), (6637, 27), (6638, 28), (6639, 37), (6640, 27), (6641, 24), (6642, 23), (6643, 20), (6644, 19), (6645, 24), (6646, 42), (6647, 23), (6648, 39), (6649, 37), (6650, 56), (6651, 24), (6652, 22), (6653, 26), (6654, 23), (6655, 19), (6656, 25), (6657, 30), (6658, 39), (6659, 26), (6660, 27), (6661, 40), (6662, 39), (6663, 36), (6664, 30), (6665, 30), (6666, 28), (6667, 20), (6668, 36), (6669, 26), (6670, 62), (6671, 21), (6672, 33), (6673, 29), (6674, 24), (6675, 16), (6676, 56), (6677, 31), (6678, 25), (6679, 36), (6680, 28), (6681, 26), (6682, 17), (6683, 32), (6684, 25), (6685, 21), (6686, 28), (6687, 25), (6688, 33), (6689, 42), (6690, 19), (6691, 34), (6692, 23), (6693, 22), (6694, 32), (6695, 71), (6696, 37), (6697, 28), (6698, 22), (6699, 24), (6700, 28), (6701, 26), (6702, 17), (6703, 22), (6704, 22), (6705, 36), (6706, 25), (6707, 32), (6708, 28), (6709, 27), (6710, 31), (6711, 22), (6712, 30), (6713, 27), (6714, 25), (6715, 25), (6716, 18), (6717, 24), (6718, 24), (6719, 18), (6720, 31), (6721, 15), (6722, 21), (6723, 24), (6724, 25), (6725, 24), (6726, 36), (6727, 23), (6728, 24), (6729, 29), (6730, 37), (6731, 19), (6732, 26), (6733, 69), (6734, 24), (6735, 30), (6736, 19), (6737, 42), (6738, 51), (6739, 23), (6740, 21), (6741, 27), (6742, 24), (6743, 32), (6744, 26), (6745, 23), (6746, 42), (6747, 37), (6748, 23), (6749, 24), (6750, 20), (6751, 35), (6752, 30), (6753, 27), (6754, 21), (6755, 16), (6756, 42), (6757, 22), (6758, 40), (6759, 30), (6760, 34), (6761, 24), (6762, 21), (6763, 22), (6764, 22), (6765, 31), (6766, 34), (6767, 30), (6768, 34), (6769, 21), (6770, 39), (6771, 28), (6772, 18), (6773, 25), (6774, 32), (6775, 22), (6776, 33), (6777, 26), (6778, 20), (6779, 22), (6780, 34), (6781, 26), (6782, 28), (6783, 20), (6784, 21), (6785, 34), (6786, 24), (6787, 25), (6788, 33), (6789, 67), (6790, 34), (6791, 25), (6792, 29), (6793, 29), (6794, 29), (6795, 33), (6796, 34), (6797, 22), (6798, 28), (6799, 23), (6800, 25), (6801, 25), (6802, 30), (6803, 28), (6804, 23), (6805, 20), (6806, 34), (6807, 33), (6808, 31), (6809, 27), (6810, 41), (6811, 21), (6812, 29), (6813, 28), (6814, 31), (6815, 26), (6816, 24), (6817, 33), (6818, 20), (6819, 42), (6820, 32), (6821, 35), (6822, 31), (6823, 33), (6824, 28), (6825, 27), (6826, 35), (6827, 26), (6828, 36), (6829, 20), (6830, 28), (6831, 35), (6832, 22), (6833, 19), (6834, 32), (6835, 27), (6836, 44), (6837, 22), (6838, 20), (6839, 86), (6840, 31), (6841, 37), (6842, 42), (6843, 37), (6844, 46), (6845, 19), (6846, 23), (6847, 25), (6848, 17), (6849, 23), (6850, 18), (6851, 24), (6852, 32), (6853, 27), (6854, 27), (6855, 26), (6856, 31), (6857, 36), (6858, 23), (6859, 43), (6860, 36), (6861, 26), (6862, 41), (6863, 18), (6864, 22), (6865, 30), (6866, 47), (6867, 60), (6868, 26), (6869, 27), (6870, 27), (6871, 20), (6872, 20), (6873, 33), (6874, 25), (6875, 33), (6876, 22), (6877, 35), (6878, 18), (6879, 37), (6880, 15), (6881, 28), (6882, 34), (6883, 18), (6884, 38), (6885, 30), (6886, 28), (6887, 25), (6888, 29), (6889, 35), (6890, 19), (6891, 18), (6892, 22), (6893, 27), (6894, 54), (6895, 75), (6896, 30), (6897, 26), (6898, 20), (6899, 24), (6900, 32), (6901, 81), (6902, 21), (6903, 36), (6904, 36), (6905, 27), (6906, 24), (6907, 24), (6908, 32), (6909, 20), (6910, 37), (6911, 27), (6912, 29), (6913, 24), (6914, 20), (6915, 23), (6916, 42), (6917, 24), (6918, 32), (6919, 28), (6920, 35), (6921, 32), (6922, 27), (6923, 24), (6924, 32), (6925, 33), (6926, 27), (6927, 31), (6928, 24), (6929, 27), (6930, 30), (6931, 40), (6932, 18), (6933, 20), (6934, 17), (6935, 155), (6936, 37), (6937, 28), (6938, 31), (6939, 36), (6940, 33), (6941, 32), (6942, 22), (6943, 24), (6944, 75), (6945, 20), (6946, 26), (6947, 23), (6948, 44), (6949, 23), (6950, 26), (6951, 29), (6952, 29), (6953, 30), (6954, 32), (6955, 43), (6956, 27), (6957, 22), (6958, 53), (6959, 25), (6960, 19), (6961, 26), (6962, 17), (6963, 23), (6964, 37), (6965, 22), (6966, 46), (6967, 30), (6968, 52), (6969, 21), (6970, 35), (6971, 20), (6972, 16), (6973, 21), (6974, 30), (6975, 50), (6976, 23), (6977, 15), (6978, 33), (6979, 26), (6980, 29), (6981, 28), (6982, 14), (6983, 21), (6984, 31), (6985, 22), (6986, 26), (6987, 44), (6988, 35), (6989, 23), (6990, 20), (6991, 53), (6992, 27), (6993, 33), (6994, 40), (6995, 20), (6996, 31), (6997, 29), (6998, 24), (6999, 30), (7000, 20), (7001, 26), (7002, 24), (7003, 26), (7004, 20), (7005, 40), (7006, 23), (7007, 34), (7008, 40), (7009, 27), (7010, 20), (7011, 26), (7012, 15), (7013, 24), (7014, 20), (7015, 23), (7016, 23), (7017, 24), (7018, 43), (7019, 28), (7020, 25), (7021, 35), (7022, 27), (7023, 28), (7024, 26), (7025, 24), (7026, 37), (7027, 27), (7028, 26), (7029, 20), (7030, 20), (7031, 29), (7032, 56), (7033, 31), (7034, 34), (7035, 25), (7036, 24), (7037, 27), (7038, 30), (7039, 36), (7040, 28), (7041, 21), (7042, 32), (7043, 22), (7044, 23), (7045, 24), (7046, 20), (7047, 17), (7048, 50), (7049, 23), (7050, 25), (7051, 40), (7052, 29), (7053, 24), (7054, 33), (7055, 27), (7056, 34), (7057, 36), (7058, 19), (7059, 23), (7060, 28), (7061, 26), (7062, 25), (7063, 31), (7064, 40), (7065, 22), (7066, 26), (7067, 25), (7068, 26), (7069, 23), (7070, 25), (7071, 38), (7072, 27), (7073, 25), (7074, 28), (7075, 28), (7076, 21), (7077, 24), (7078, 49), (7079, 25), (7080, 28), (7081, 34), (7082, 35), (7083, 24), (7084, 36), (7085, 22), (7086, 30), (7087, 39), (7088, 36), (7089, 23), (7090, 22), (7091, 26), (7092, 29), (7093, 62), (7094, 27), (7095, 35), (7096, 36), (7097, 29), (7098, 26), (7099, 28), (7100, 38), (7101, 34), (7102, 31), (7103, 23), (7104, 20), (7105, 42), (7106, 31), (7107, 51), (7108, 23), (7109, 27), (7110, 25), (7111, 34), (7112, 19), (7113, 22), (7114, 25), (7115, 50), (7116, 29), (7117, 29), (7118, 35), (7119, 52), (7120, 43), (7121, 36), (7122, 33), (7123, 22), (7124, 35), (7125, 43), (7126, 42), (7127, 42), (7128, 25), (7129, 29), (7130, 45), (7131, 22), (7132, 28), (7133, 23), (7134, 24), (7135, 27), (7136, 42), (7137, 35), (7138, 18), (7139, 38), (7140, 47), (7141, 44), (7142, 33), (7143, 24), (7144, 28), (7145, 29), (7146, 21), (7147, 63), (7148, 30), (7149, 25), (7150, 39), (7151, 27), (7152, 24), (7153, 22), (7154, 22), (7155, 22), (7156, 45), (7157, 25), (7158, 29), (7159, 42), (7160, 23), (7161, 23), (7162, 36), (7163, 19), (7164, 28), (7165, 24), (7166, 18), (7167, 60), (7168, 27), (7169, 22), (7170, 50), (7171, 22), (7172, 40), (7173, 23), (7174, 27), (7175, 34), (7176, 44), (7177, 40), (7178, 19), (7179, 51), (7180, 31), (7181, 18), (7182, 24), (7183, 29), (7184, 34), (7185, 25), (7186, 25), (7187, 24), (7188, 58), (7189, 18), (7190, 26), (7191, 22), (7192, 29), (7193, 24), (7194, 24), (7195, 29), (7196, 20), (7197, 25), (7198, 26), (7199, 31), (7200, 19), (7201, 27), (7202, 15), (7203, 14), (7204, 20), (7205, 36), (7206, 35), (7207, 23), (7208, 44), (7209, 20), (7210, 27), (7211, 33), (7212, 35), (7213, 31), (7214, 23), (7215, 22), (7216, 27), (7217, 32), (7218, 14), (7219, 42), (7220, 22), (7221, 26), (7222, 23), (7223, 42), (7224, 23), (7225, 23), (7226, 24), (7227, 33), (7228, 29), (7229, 29), (7230, 22), (7231, 51), (7232, 26), (7233, 22), (7234, 39), (7235, 24), (7236, 20), (7237, 24), (7238, 30), (7239, 19), (7240, 34), (7241, 22), (7242, 30), (7243, 24), (7244, 68), (7245, 45), (7246, 26), (7247, 21), (7248, 31), (7249, 21), (7250, 28), (7251, 33), (7252, 27), (7253, 30), (7254, 27), (7255, 25), (7256, 19), (7257, 18), (7258, 18), (7259, 28), (7260, 31), (7261, 30), (7262, 24), (7263, 26), (7264, 24), (7265, 52), (7266, 22), (7267, 29), (7268, 23), (7269, 47), (7270, 31), (7271, 26), (7272, 49), (7273, 21), (7274, 56), (7275, 38), (7276, 50), (7277, 18), (7278, 25), (7279, 33), (7280, 24), (7281, 27), (7282, 15), (7283, 26), (7284, 24), (7285, 39), (7286, 20), (7287, 26), (7288, 25), (7289, 47), (7290, 42), (7291, 28), (7292, 22), (7293, 23), (7294, 23), (7295, 32), (7296, 22), (7297, 29), (7298, 44), (7299, 34), (7300, 27), (7301, 25), (7302, 30), (7303, 21), (7304, 25), (7305, 26), (7306, 28), (7307, 30), (7308, 23), (7309, 33), (7310, 33), (7311, 68), (7312, 36), (7313, 62), (7314, 18), (7315, 30), (7316, 29), (7317, 20), (7318, 29), (7319, 28), (7320, 32), (7321, 53), (7322, 28), (7323, 25), (7324, 28), (7325, 19), (7326, 66), (7327, 22), (7328, 31), (7329, 28), (7330, 23), (7331, 21), (7332, 54), (7333, 30), (7334, 26), (7335, 26), (7336, 26), (7337, 20), (7338, 27), (7339, 42), (7340, 25), (7341, 24), (7342, 25), (7343, 43), (7344, 25), (7345, 24), (7346, 38), (7347, 26), (7348, 27), (7349, 47), (7350, 29), (7351, 25), (7352, 24), (7353, 52), (7354, 30), (7355, 28), (7356, 27), (7357, 24), (7358, 26), (7359, 28), (7360, 24), (7361, 35), (7362, 40), (7363, 49), (7364, 19), (7365, 25), (7366, 24), (7367, 39), (7368, 39), (7369, 29), (7370, 22), (7371, 26), (7372, 32), (7373, 40), (7374, 27), (7375, 19), (7376, 36), (7377, 62), (7378, 41), (7379, 35), (7380, 26), (7381, 34), (7382, 30), (7383, 35), (7384, 30), (7385, 36), (7386, 38), (7387, 30), (7388, 17), (7389, 22), (7390, 29), (7391, 30), (7392, 32), (7393, 44), (7394, 20), (7395, 56), (7396, 19), (7397, 35), (7398, 23), (7399, 23), (7400, 20), (7401, 23), (7402, 29), (7403, 146), (7404, 24), (7405, 33), (7406, 37), (7407, 33), (7408, 53), (7409, 22), (7410, 26), (7411, 44), (7412, 20), (7413, 58), (7414, 26), (7415, 18), (7416, 26), (7417, 25), (7418, 31), (7419, 21), (7420, 36), (7421, 25), (7422, 24), (7423, 17), (7424, 25), (7425, 33), (7426, 34), (7427, 37), (7428, 26), (7429, 24), (7430, 22), (7431, 35), (7432, 24), (7433, 20), (7434, 27), (7435, 39), (7436, 32), (7437, 20), (7438, 19), (7439, 33), (7440, 44), (7441, 54), (7442, 25), (7443, 22), (7444, 23), (7445, 23), (7446, 38), (7447, 19), (7448, 43), (7449, 25), (7450, 39), (7451, 49), (7452, 29), (7453, 27), (7454, 17), (7455, 30), (7456, 34), (7457, 29), (7458, 19), (7459, 21), (7460, 33), (7461, 24), (7462, 33), (7463, 23), (7464, 33), (7465, 31), (7466, 130), (7467, 23), (7468, 29), (7469, 41), (7470, 21), (7471, 38), (7472, 39), (7473, 27), (7474, 33), (7475, 28), (7476, 24), (7477, 21), (7478, 32), (7479, 19), (7480, 23), (7481, 37), (7482, 30), (7483, 20), (7484, 33), (7485, 23), (7486, 24), (7487, 31), (7488, 23), (7489, 31), (7490, 33), (7491, 26), (7492, 29), (7493, 17), (7494, 20), (7495, 24), (7496, 26), (7497, 31), (7498, 51), (7499, 24), (7500, 26), (7501, 22), (7502, 23), (7503, 24), (7504, 21), (7505, 27), (7506, 28), (7507, 42), (7508, 41), (7509, 32), (7510, 34), (7511, 26), (7512, 37), (7513, 19), (7514, 26), (7515, 27), (7516, 28), (7517, 28), (7518, 24), (7519, 30), (7520, 30), (7521, 38), (7522, 23), (7523, 18), (7524, 23), (7525, 22), (7526, 31), (7527, 32), (7528, 27), (7529, 23), (7530, 34), (7531, 21), (7532, 49), (7533, 30), (7534, 21), (7535, 33), (7536, 25), (7537, 30), (7538, 28), (7539, 27), (7540, 23), (7541, 32), (7542, 24), (7543, 36), (7544, 17), (7545, 17), (7546, 25), (7547, 40), (7548, 15), (7549, 28), (7550, 46), (7551, 25), (7552, 24), (7553, 34), (7554, 35), (7555, 37), (7556, 21), (7557, 25), (7558, 26), (7559, 21), (7560, 26), (7561, 34), (7562, 33), (7563, 27), (7564, 20), (7565, 28), (7566, 46), (7567, 18), (7568, 31), (7569, 60), (7570, 29), (7571, 31), (7572, 28), (7573, 25), (7574, 20), (7575, 27), (7576, 28), (7577, 25), (7578, 35), (7579, 24), (7580, 17), (7581, 34), (7582, 27), (7583, 29), (7584, 36), (7585, 24), (7586, 24), (7587, 29), (7588, 30), (7589, 36), (7590, 37), (7591, 36), (7592, 25), (7593, 27), (7594, 38), (7595, 42), (7596, 32), (7597, 23), (7598, 32), (7599, 14), (7600, 34), (7601, 29), (7602, 21), (7603, 36), (7604, 31), (7605, 25), (7606, 20), (7607, 21), (7608, 40), (7609, 26), (7610, 26), (7611, 27), (7612, 23), (7613, 42), (7614, 30), (7615, 33), (7616, 28), (7617, 28), (7618, 19), (7619, 20), (7620, 30), (7621, 29), (7622, 33), (7623, 23), (7624, 33), (7625, 49), (7626, 44), (7627, 25), (7628, 25), (7629, 42), (7630, 25), (7631, 35), (7632, 84), (7633, 51), (7634, 36), (7635, 24), (7636, 37), (7637, 38), (7638, 33), (7639, 19), (7640, 24), (7641, 24), (7642, 28), (7643, 26), (7644, 31), (7645, 40), (7646, 27), (7647, 31), (7648, 35), (7649, 49), (7650, 35), (7651, 36), (7652, 23), (7653, 53), (7654, 40), (7655, 23), (7656, 30), (7657, 31), (7658, 78), (7659, 29), (7660, 27), (7661, 24), (7662, 34), (7663, 25), (7664, 33), (7665, 20), (7666, 22), (7667, 37), (7668, 23), (7669, 33), (7670, 18), (7671, 42), (7672, 35), (7673, 25), (7674, 30), (7675, 23), (7676, 26), (7677, 17), (7678, 42), (7679, 25), (7680, 40), (7681, 34), (7682, 35), (7683, 37), (7684, 23), (7685, 32), (7686, 26), (7687, 24), (7688, 26), (7689, 41), (7690, 62), (7691, 18), (7692, 23), (7693, 26), (7694, 37), (7695, 33), (7696, 23), (7697, 25), (7698, 17), (7699, 36), (7700, 22), (7701, 39), (7702, 20), (7703, 20), (7704, 33), (7705, 34), (7706, 33), (7707, 23), (7708, 19), (7709, 21), (7710, 24), (7711, 45), (7712, 16), (7713, 79), (7714, 20), (7715, 22), (7716, 18), (7717, 23), (7718, 43), (7719, 27), (7720, 26), (7721, 22), (7722, 32), (7723, 30), (7724, 28), (7725, 22), (7726, 35), (7727, 21), (7728, 32), (7729, 25), (7730, 20), (7731, 19), (7732, 26), (7733, 26), (7734, 24), (7735, 34), (7736, 29), (7737, 34), (7738, 28), (7739, 35), (7740, 37), (7741, 27), (7742, 37), (7743, 45), (7744, 32), (7745, 32), (7746, 28), (7747, 23), (7748, 22), (7749, 29), (7750, 37), (7751, 27), (7752, 23), (7753, 35), (7754, 30), (7755, 50), (7756, 78), (7757, 22), (7758, 25), (7759, 30), (7760, 23), (7761, 30), (7762, 27), (7763, 48), (7764, 40), (7765, 23), (7766, 25), (7767, 21), (7768, 25), (7769, 31), (7770, 24), (7771, 35), (7772, 49), (7773, 32), (7774, 39), (7775, 33), (7776, 17), (7777, 20), (7778, 23), (7779, 24), (7780, 20), (7781, 26), (7782, 23), (7783, 51), (7784, 36), (7785, 19), (7786, 26), (7787, 28), (7788, 39), (7789, 21), (7790, 26), (7791, 28), (7792, 89), (7793, 25), (7794, 31), (7795, 32), (7796, 40), (7797, 25), (7798, 30), (7799, 17), (7800, 27), (7801, 26), (7802, 32), (7803, 28), (7804, 28), (7805, 24), (7806, 21), (7807, 38), (7808, 30), (7809, 24), (7810, 45), (7811, 40), (7812, 28), (7813, 50), (7814, 31), (7815, 29), (7816, 36), (7817, 35), (7818, 23), (7819, 19), (7820, 24), (7821, 30), (7822, 25), (7823, 23), (7824, 38), (7825, 36), (7826, 17), (7827, 23), (7828, 27), (7829, 26), (7830, 28), (7831, 31), (7832, 25), (7833, 38), (7834, 41), (7835, 27), (7836, 52), (7837, 26), (7838, 36), (7839, 16), (7840, 24), (7841, 24), (7842, 28), (7843, 21), (7844, 34), (7845, 27), (7846, 36), (7847, 37), (7848, 21), (7849, 22), (7850, 27), (7851, 20), (7852, 19), (7853, 28), (7854, 29), (7855, 38), (7856, 28), (7857, 34), (7858, 20), (7859, 29), (7860, 20), (7861, 88), (7862, 30), (7863, 35), (7864, 29), (7865, 23), (7866, 28), (7867, 44), (7868, 26), (7869, 28), (7870, 38), (7871, 36), (7872, 21), (7873, 18), (7874, 20), (7875, 28), (7876, 37), (7877, 23), (7878, 60), (7879, 27), (7880, 36), (7881, 24), (7882, 30), (7883, 23), (7884, 34), (7885, 30), (7886, 22), (7887, 23), (7888, 19), (7889, 27), (7890, 32), (7891, 16), (7892, 20), (7893, 31), (7894, 22), (7895, 28), (7896, 56), (7897, 16), (7898, 28), (7899, 54), (7900, 43), (7901, 16), (7902, 28), (7903, 30), (7904, 29), (7905, 27), (7906, 37), (7907, 26), (7908, 27), (7909, 35), (7910, 35), (7911, 30), (7912, 23), (7913, 31), (7914, 62), (7915, 21), (7916, 31), (7917, 24), (7918, 25), (7919, 23), (7920, 23), (7921, 35), (7922, 31), (7923, 25), (7924, 48), (7925, 26), (7926, 25), (7927, 22), (7928, 26), (7929, 21), (7930, 21), (7931, 57), (7932, 50), (7933, 27), (7934, 40), (7935, 32), (7936, 28), (7937, 24), (7938, 41), (7939, 24), (7940, 21), (7941, 31), (7942, 25), (7943, 34), (7944, 42), (7945, 72), (7946, 36), (7947, 27), (7948, 22), (7949, 30), (7950, 56), (7951, 19), (7952, 30), (7953, 25), (7954, 27), (7955, 25), (7956, 22), (7957, 24), (7958, 23), (7959, 27), (7960, 28), (7961, 47), (7962, 41), (7963, 35), (7964, 24), (7965, 32), (7966, 21), (7967, 22), (7968, 38), (7969, 36), (7970, 26), (7971, 28), (7972, 28), (7973, 30), (7974, 31), (7975, 25), (7976, 32), (7977, 19), (7978, 26), (7979, 31), (7980, 23), (7981, 28), (7982, 23), (7983, 25), (7984, 31), (7985, 24), (7986, 33), (7987, 38), (7988, 26), (7989, 25), (7990, 22), (7991, 24), (7992, 25), (7993, 20), (7994, 19), (7995, 19), (7996, 26), (7997, 43), (7998, 23), (7999, 17), (8000, 40), (8001, 27), (8002, 36), (8003, 15), (8004, 24), (8005, 21), (8006, 60), (8007, 15), (8008, 17), (8009, 18), (8010, 26), (8011, 65), (8012, 31), (8013, 30), (8014, 22), (8015, 20), (8016, 34), (8017, 20), (8018, 23), (8019, 35), (8020, 24), (8021, 30), (8022, 36), (8023, 36), (8024, 29), (8025, 38), (8026, 35), (8027, 25), (8028, 23), (8029, 27), (8030, 20), (8031, 33), (8032, 23), (8033, 26), (8034, 25), (8035, 67), (8036, 36), (8037, 44), (8038, 22), (8039, 34), (8040, 63), (8041, 21), (8042, 23), (8043, 24), (8044, 25), (8045, 20), (8046, 25), (8047, 28), (8048, 21), (8049, 21), (8050, 22), (8051, 19), (8052, 36), (8053, 18), (8054, 32), (8055, 39), (8056, 18), (8057, 22), (8058, 21), (8059, 25), (8060, 23), (8061, 40), (8062, 21), (8063, 33), (8064, 18), (8065, 30), (8066, 31), (8067, 30), (8068, 34), (8069, 30), (8070, 34), (8071, 25), (8072, 25), (8073, 26), (8074, 24), (8075, 60), (8076, 18), (8077, 30), (8078, 19), (8079, 19), (8080, 20), (8081, 39), (8082, 34), (8083, 43), (8084, 29), (8085, 35), (8086, 29), (8087, 35), (8088, 16), (8089, 24), (8090, 51), (8091, 29), (8092, 27), (8093, 21), (8094, 37), (8095, 24), (8096, 29), (8097, 28), (8098, 27), (8099, 15), (8100, 23), (8101, 26), (8102, 28), (8103, 38), (8104, 42), (8105, 28), (8106, 42), (8107, 24), (8108, 21), (8109, 26), (8110, 33), (8111, 44), (8112, 30), (8113, 22), (8114, 19), (8115, 24), (8116, 28), (8117, 32), (8118, 40), (8119, 44), (8120, 61), (8121, 22), (8122, 60), (8123, 34), (8124, 46), (8125, 17), (8126, 41), (8127, 23), (8128, 23), (8129, 25), (8130, 51), (8131, 40), (8132, 30), (8133, 36), (8134, 41), (8135, 31), (8136, 28), (8137, 28), (8138, 16), (8139, 43), (8140, 25), (8141, 32), (8142, 25), (8143, 24), (8144, 23), (8145, 26), (8146, 43), (8147, 23), (8148, 29), (8149, 37), (8150, 24), (8151, 37), (8152, 27), (8153, 27), (8154, 22), (8155, 41), (8156, 33), (8157, 46), (8158, 31), (8159, 30), (8160, 30), (8161, 20), (8162, 31), (8163, 32), (8164, 48), (8165, 23), (8166, 30), (8167, 22), (8168, 48), (8169, 48), (8170, 26), (8171, 24), (8172, 20), (8173, 27), (8174, 29), (8175, 26), (8176, 22), (8177, 25), (8178, 57), (8179, 29), (8180, 21), (8181, 39), (8182, 29), (8183, 24), (8184, 19), (8185, 23), (8186, 18), (8187, 25), (8188, 27), (8189, 35), (8190, 26), (8191, 19), (8192, 29), (8193, 27), (8194, 18), (8195, 27), (8196, 20), (8197, 24), (8198, 23), (8199, 25), (8200, 41), (8201, 28), (8202, 20), (8203, 28), (8204, 29), (8205, 53), (8206, 28), (8207, 23), (8208, 18), (8209, 20), (8210, 51), (8211, 37), (8212, 95), (8213, 20), (8214, 27), (8215, 41), (8216, 22), (8217, 31), (8218, 27), (8219, 28), (8220, 40), (8221, 26), (8222, 23), (8223, 26), (8224, 26), (8225, 34), (8226, 19), (8227, 88), (8228, 41), (8229, 23), (8230, 26), (8231, 28), (8232, 26), (8233, 18), (8234, 38), (8235, 50), (8236, 40), (8237, 20), (8238, 26), (8239, 21), (8240, 31), (8241, 20), (8242, 26), (8243, 33), (8244, 22), (8245, 31), (8246, 25), (8247, 29), (8248, 30), (8249, 23), (8250, 29), (8251, 20), (8252, 27), (8253, 25), (8254, 60), (8255, 30), (8256, 27), (8257, 36), (8258, 24), (8259, 19), (8260, 20), (8261, 37), (8262, 27), (8263, 28), (8264, 29), (8265, 28), (8266, 27), (8267, 31), (8268, 19), (8269, 22), (8270, 19), (8271, 26), (8272, 56), (8273, 22), (8274, 31), (8275, 31), (8276, 42), (8277, 31), (8278, 23), (8279, 21), (8280, 20), (8281, 33), (8282, 48), (8283, 20), (8284, 27), (8285, 55), (8286, 33), (8287, 22), (8288, 32), (8289, 19), (8290, 24), (8291, 38), (8292, 22), (8293, 30), (8294, 24), (8295, 25), (8296, 60), (8297, 30), (8298, 119), (8299, 19), (8300, 28), (8301, 30), (8302, 21), (8303, 52), (8304, 29), (8305, 24), (8306, 24), (8307, 21), (8308, 28), (8309, 37), (8310, 32), (8311, 39), (8312, 22), (8313, 31), (8314, 22), (8315, 21), (8316, 41), (8317, 30), (8318, 18), (8319, 27), (8320, 21), (8321, 28), (8322, 29), (8323, 34), (8324, 23), (8325, 25), (8326, 49), (8327, 26), (8328, 20), (8329, 41), (8330, 27), (8331, 22), (8332, 19), (8333, 27), (8334, 21), (8335, 62), (8336, 20), (8337, 82), (8338, 22), (8339, 25), (8340, 27), (8341, 40), (8342, 19), (8343, 28), (8344, 18), (8345, 18), (8346, 24), (8347, 39), (8348, 19), (8349, 30), (8350, 35), (8351, 31), (8352, 25), (8353, 27), (8354, 39), (8355, 19), (8356, 36), (8357, 47), (8358, 25), (8359, 21), (8360, 62), (8361, 30), (8362, 26), (8363, 34), (8364, 23), (8365, 34), (8366, 24), (8367, 29), (8368, 32), (8369, 24), (8370, 34), (8371, 27), (8372, 24), (8373, 25), (8374, 44), (8375, 31), (8376, 20), (8377, 22), (8378, 43), (8379, 21), (8380, 28), (8381, 22), (8382, 38), (8383, 28), (8384, 34), (8385, 40), (8386, 23), (8387, 36), (8388, 25), (8389, 55), (8390, 35), (8391, 24), (8392, 30), (8393, 17), (8394, 17), (8395, 21), (8396, 26), (8397, 18), (8398, 24), (8399, 29), (8400, 32), (8401, 27), (8402, 30), (8403, 23), (8404, 30), (8405, 25), (8406, 29), (8407, 25), (8408, 23), (8409, 33), (8410, 29), (8411, 41), (8412, 41), (8413, 37), (8414, 30), (8415, 38), (8416, 37), (8417, 27), (8418, 20), (8419, 39), (8420, 18), (8421, 38), (8422, 19), (8423, 38), (8424, 39), (8425, 27), (8426, 23), (8427, 27), (8428, 29), (8429, 37), (8430, 17), (8431, 33), (8432, 49), (8433, 23), (8434, 20), (8435, 18), (8436, 26), (8437, 25), (8438, 20), (8439, 49), (8440, 17), (8441, 26), (8442, 19), (8443, 24), (8444, 19), (8445, 21), (8446, 33), (8447, 32), (8448, 36), (8449, 28), (8450, 65), (8451, 33), (8452, 32), (8453, 37), (8454, 44), (8455, 26), (8456, 34), (8457, 23), (8458, 37), (8459, 25), (8460, 21), (8461, 19), (8462, 21), (8463, 26), (8464, 23), (8465, 41), (8466, 29), (8467, 76), (8468, 20), (8469, 27), (8470, 21), (8471, 55), (8472, 29), (8473, 48), (8474, 22), (8475, 28), (8476, 22), (8477, 36), (8478, 20), (8479, 48), (8480, 21), (8481, 36), (8482, 32), (8483, 33), (8484, 27), (8485, 30), (8486, 22), (8487, 30), (8488, 35), (8489, 23), (8490, 18), (8491, 29), (8492, 26), (8493, 26), (8494, 30), (8495, 40), (8496, 21), (8497, 33), (8498, 28), (8499, 34), (8500, 33), (8501, 22), (8502, 25), (8503, 22), (8504, 26), (8505, 48), (8506, 38), (8507, 29), (8508, 29), (8509, 21), (8510, 33), (8511, 36), (8512, 50), (8513, 32), (8514, 28), (8515, 20), (8516, 23), (8517, 27), (8518, 32), (8519, 34), (8520, 27), (8521, 30), (8522, 29), (8523, 22), (8524, 32), (8525, 35), (8526, 29), (8527, 27), (8528, 31), (8529, 25), (8530, 23), (8531, 45), (8532, 25), (8533, 27), (8534, 31), (8535, 32), (8536, 42), (8537, 24), (8538, 23), (8539, 19), (8540, 30), (8541, 27), (8542, 25), (8543, 28), (8544, 33), (8545, 30), (8546, 28), (8547, 24), (8548, 22), (8549, 34), (8550, 29), (8551, 39), (8552, 23), (8553, 32), (8554, 33), (8555, 21), (8556, 20), (8557, 38), (8558, 33), (8559, 23), (8560, 19), (8561, 44), (8562, 33), (8563, 42), (8564, 15), (8565, 19), (8566, 23), (8567, 55), (8568, 28), (8569, 64), (8570, 19), (8571, 20), (8572, 35), (8573, 25), (8574, 88), (8575, 23), (8576, 38), (8577, 21), (8578, 37), (8579, 28), (8580, 17), (8581, 33), (8582, 36), (8583, 29), (8584, 29), (8585, 42), (8586, 29), (8587, 31), (8588, 22), (8589, 31), (8590, 23), (8591, 27), (8592, 22), (8593, 26), (8594, 61), (8595, 25), (8596, 22), (8597, 29), (8598, 25), (8599, 29), (8600, 26), (8601, 30), (8602, 20), (8603, 30), (8604, 23), (8605, 82), (8606, 64), (8607, 15), (8608, 28), (8609, 29), (8610, 52), (8611, 23), (8612, 31), (8613, 26), (8614, 28), (8615, 20), (8616, 27), (8617, 24), (8618, 28), (8619, 37), (8620, 25), (8621, 19), (8622, 25), (8623, 23), (8624, 31), (8625, 22), (8626, 37), (8627, 32), (8628, 43), (8629, 46), (8630, 27), (8631, 32), (8632, 26), (8633, 38), (8634, 30), (8635, 45), (8636, 17), (8637, 28), (8638, 33), (8639, 47), (8640, 38), (8641, 32), (8642, 17), (8643, 19), (8644, 31), (8645, 26), (8646, 20), (8647, 28), (8648, 42), (8649, 31), (8650, 33), (8651, 17), (8652, 23), (8653, 23), (8654, 26), (8655, 19), (8656, 25), (8657, 24), (8658, 55), (8659, 31), (8660, 23), (8661, 19), (8662, 27), (8663, 38), (8664, 25), (8665, 28), (8666, 23), (8667, 24), (8668, 24), (8669, 25), (8670, 25), (8671, 36), (8672, 42), (8673, 35), (8674, 24), (8675, 21), (8676, 27), (8677, 35), (8678, 25), (8679, 49), (8680, 32), (8681, 17), (8682, 20), (8683, 24), (8684, 30), (8685, 19), (8686, 18), (8687, 57), (8688, 28), (8689, 32), (8690, 29), (8691, 27), (8692, 43), (8693, 33), (8694, 34), (8695, 18), (8696, 28), (8697, 22), (8698, 25), (8699, 32), (8700, 60), (8701, 15), (8702, 30), (8703, 40), (8704, 63), (8705, 24), (8706, 22), (8707, 47), (8708, 20), (8709, 30), (8710, 40), (8711, 37), (8712, 46), (8713, 32), (8714, 38), (8715, 28), (8716, 26), (8717, 28), (8718, 56), (8719, 28), (8720, 29), (8721, 26), (8722, 21), (8723, 20), (8724, 33), (8725, 31), (8726, 30), (8727, 35), (8728, 20), (8729, 23), (8730, 33), (8731, 34), (8732, 58), (8733, 24), (8734, 19), (8735, 27), (8736, 18), (8737, 27), (8738, 31), (8739, 50), (8740, 32), (8741, 31), (8742, 113), (8743, 34), (8744, 26), (8745, 26), (8746, 28), (8747, 30), (8748, 21), (8749, 35), (8750, 45), (8751, 32), (8752, 41), (8753, 23), (8754, 20), (8755, 34), (8756, 26), (8757, 19), (8758, 25), (8759, 21), (8760, 31), (8761, 28), (8762, 50), (8763, 20), (8764, 22), (8765, 26), (8766, 37), (8767, 28), (8768, 32), (8769, 50), (8770, 50), (8771, 25), (8772, 21), (8773, 28), (8774, 24), (8775, 28), (8776, 18), (8777, 25), (8778, 22), (8779, 25), (8780, 20), (8781, 20), (8782, 40), (8783, 24), (8784, 19), (8785, 37), (8786, 23), (8787, 26), (8788, 31), (8789, 18), (8790, 27), (8791, 23), (8792, 26), (8793, 32), (8794, 38), (8795, 21), (8796, 23), (8797, 30), (8798, 24), (8799, 80), (8800, 17), (8801, 22), (8802, 25), (8803, 32), (8804, 36), (8805, 31), (8806, 23), (8807, 25), (8808, 33), (8809, 32), (8810, 21), (8811, 27), (8812, 17), (8813, 29), (8814, 43), (8815, 39), (8816, 26), (8817, 41), (8818, 35), (8819, 26), (8820, 24), (8821, 23), (8822, 30), (8823, 24), (8824, 36), (8825, 27), (8826, 30), (8827, 18), (8828, 32), (8829, 28), (8830, 18), (8831, 32), (8832, 18), (8833, 27), (8834, 31), (8835, 58), (8836, 36), (8837, 20), (8838, 32), (8839, 20), (8840, 64), (8841, 26), (8842, 26), (8843, 30), (8844, 70), (8845, 38), (8846, 22), (8847, 29), (8848, 36), (8849, 45), (8850, 57), (8851, 36), (8852, 27), (8853, 37), (8854, 32), (8855, 19), (8856, 34), (8857, 23), (8858, 20), (8859, 39), (8860, 24), (8861, 19), (8862, 54), (8863, 25), (8864, 37), (8865, 22), (8866, 28), (8867, 19), (8868, 23), (8869, 24), (8870, 33), (8871, 41), (8872, 49), (8873, 35), (8874, 35), (8875, 21), (8876, 24), (8877, 28), (8878, 36), (8879, 27), (8880, 15), (8881, 42), (8882, 24), (8883, 53), (8884, 34), (8885, 31), (8886, 32), (8887, 21), (8888, 26), (8889, 30), (8890, 25), (8891, 19), (8892, 22), (8893, 57), (8894, 35), (8895, 27), (8896, 41), (8897, 21), (8898, 20), (8899, 32), (8900, 38), (8901, 22), (8902, 30), (8903, 24), (8904, 28), (8905, 39), (8906, 20), (8907, 24), (8908, 41), (8909, 23), (8910, 24), (8911, 65), (8912, 20), (8913, 31), (8914, 31), (8915, 45), (8916, 19), (8917, 25), (8918, 59), (8919, 27), (8920, 31), (8921, 25), (8922, 24), (8923, 27), (8924, 29), (8925, 20), (8926, 22), (8927, 28), (8928, 29), (8929, 35), (8930, 16), (8931, 15), (8932, 20), (8933, 24), (8934, 30), (8935, 31), (8936, 23), (8937, 23), (8938, 26), (8939, 66), (8940, 20), (8941, 28), (8942, 33), (8943, 63), (8944, 20), (8945, 42), (8946, 24), (8947, 22), (8948, 39), (8949, 67), (8950, 22), (8951, 31), (8952, 26), (8953, 36), (8954, 24), (8955, 27), (8956, 34), (8957, 31), (8958, 30), (8959, 35), (8960, 20), (8961, 50), (8962, 21), (8963, 24), (8964, 70), (8965, 25), (8966, 30), (8967, 26), (8968, 28), (8969, 20), (8970, 30), (8971, 26), (8972, 27), (8973, 17), (8974, 24), (8975, 41), (8976, 29), (8977, 30), (8978, 35), (8979, 38), (8980, 32), (8981, 79), (8982, 29), (8983, 22), (8984, 50), (8985, 20), (8986, 32), (8987, 24), (8988, 24), (8989, 34), (8990, 29), (8991, 29), (8992, 32), (8993, 20), (8994, 19), (8995, 26), (8996, 21), (8997, 24), (8998, 19), (8999, 19), (9000, 33), (9001, 68), (9002, 21), (9003, 31), (9004, 45), (9005, 19), (9006, 18), (9007, 19), (9008, 36), (9009, 21), (9010, 27), (9011, 32), (9012, 25), (9013, 39), (9014, 33), (9015, 31), (9016, 35), (9017, 29), (9018, 17), (9019, 32), (9020, 22), (9021, 28), (9022, 38), (9023, 45), (9024, 22), (9025, 30), (9026, 35), (9027, 23), (9028, 49), (9029, 35), (9030, 35), (9031, 21), (9032, 18), (9033, 35), (9034, 42), (9035, 22), (9036, 24), (9037, 21), (9038, 20), (9039, 17), (9040, 59), (9041, 50), (9042, 42), (9043, 39), (9044, 48), (9045, 19), (9046, 72), (9047, 26), (9048, 29), (9049, 26), (9050, 35), (9051, 29), (9052, 27), (9053, 29), (9054, 41), (9055, 21), (9056, 30), (9057, 53), (9058, 37), (9059, 26), (9060, 28), (9061, 28), (9062, 31), (9063, 29), (9064, 25), (9065, 27), (9066, 18), (9067, 34), (9068, 21), (9069, 29), (9070, 38), (9071, 39), (9072, 64), (9073, 27), (9074, 20), (9075, 33), (9076, 19), (9077, 28), (9078, 21), (9079, 38), (9080, 32), (9081, 41), (9082, 16), (9083, 24), (9084, 35), (9085, 33), (9086, 26), (9087, 27), (9088, 32), (9089, 17), (9090, 37), (9091, 26), (9092, 20), (9093, 17), (9094, 34), (9095, 21), (9096, 20), (9097, 31), (9098, 30), (9099, 35), (9100, 25), (9101, 38), (9102, 35), (9103, 17), (9104, 37), (9105, 22), (9106, 20), (9107, 21), (9108, 24), (9109, 39), (9110, 33), (9111, 31), (9112, 25), (9113, 23), (9114, 32), (9115, 19), (9116, 34), (9117, 19), (9118, 32), (9119, 18), (9120, 24), (9121, 57), (9122, 22), (9123, 24), (9124, 37), (9125, 22), (9126, 35), (9127, 31), (9128, 25), (9129, 37), (9130, 17), (9131, 22), (9132, 43), (9133, 38), (9134, 19), (9135, 25), (9136, 24), (9137, 16), (9138, 32), (9139, 20), (9140, 21), (9141, 36), (9142, 22), (9143, 28), (9144, 24), (9145, 23), (9146, 28), (9147, 26), (9148, 29), (9149, 32), (9150, 19), (9151, 23), (9152, 25), (9153, 19), (9154, 29), (9155, 31), (9156, 21), (9157, 33), (9158, 29), (9159, 56), (9160, 35), (9161, 19), (9162, 17), (9163, 29), (9164, 31), (9165, 22), (9166, 20), (9167, 22), (9168, 22), (9169, 37), (9170, 25), (9171, 44), (9172, 20), (9173, 44), (9174, 21), (9175, 45), (9176, 46), (9177, 23), (9178, 20), (9179, 44), (9180, 31), (9181, 22), (9182, 26), (9183, 31), (9184, 17), (9185, 21), (9186, 37), (9187, 19), (9188, 35), (9189, 24), (9190, 28), (9191, 34), (9192, 31), (9193, 26), (9194, 27), (9195, 29), (9196, 28), (9197, 24), (9198, 23), (9199, 20), (9200, 55), (9201, 22), (9202, 61), (9203, 31), (9204, 21), (9205, 32), (9206, 22), (9207, 30), (9208, 23), (9209, 25), (9210, 33), (9211, 26), (9212, 26), (9213, 14), (9214, 33), (9215, 22), (9216, 23), (9217, 28), (9218, 34), (9219, 28), (9220, 32), (9221, 23), (9222, 31), (9223, 22), (9224, 33), (9225, 14), (9226, 26), (9227, 24), (9228, 32), (9229, 26), (9230, 44), (9231, 30), (9232, 18), (9233, 19), (9234, 26), (9235, 19), (9236, 28), (9237, 38), (9238, 29), (9239, 23), (9240, 25), (9241, 22), (9242, 28), (9243, 25), (9244, 39), (9245, 19), (9246, 38), (9247, 26), (9248, 24), (9249, 30), (9250, 30), (9251, 29), (9252, 29), (9253, 34), (9254, 25), (9255, 39), (9256, 55), (9257, 26), (9258, 35), (9259, 27), (9260, 26), (9261, 49), (9262, 26), (9263, 30), (9264, 36), (9265, 27), (9266, 25), (9267, 14), (9268, 20), (9269, 25), (9270, 23), (9271, 27), (9272, 25), (9273, 18), (9274, 23), (9275, 30), (9276, 25), (9277, 31), (9278, 31), (9279, 44), (9280, 31), (9281, 34), (9282, 26), (9283, 33), (9284, 30), (9285, 36), (9286, 34), (9287, 22), (9288, 33), (9289, 28), (9290, 32), (9291, 29), (9292, 20), (9293, 40), (9294, 23), (9295, 19), (9296, 25), (9297, 21), (9298, 39), (9299, 34), (9300, 23), (9301, 25), (9302, 20), (9303, 25), (9304, 38), (9305, 37), (9306, 62), (9307, 28), (9308, 60), (9309, 25), (9310, 20), (9311, 30), (9312, 32), (9313, 22), (9314, 24), (9315, 25), (9316, 25), (9317, 47), (9318, 42), (9319, 29), (9320, 54), (9321, 37), (9322, 30), (9323, 40), (9324, 27), (9325, 17), (9326, 31), (9327, 19), (9328, 37), (9329, 23), (9330, 28), (9331, 19), (9332, 18), (9333, 21), (9334, 26), (9335, 24), (9336, 27), (9337, 23), (9338, 38), (9339, 15), (9340, 24), (9341, 24), (9342, 29), (9343, 19), (9344, 46), (9345, 29), (9346, 25), (9347, 36), (9348, 36), (9349, 16), (9350, 16), (9351, 71), (9352, 33), (9353, 58), (9354, 42), (9355, 26), (9356, 23), (9357, 28), (9358, 44), (9359, 19), (9360, 24), (9361, 40), (9362, 27), (9363, 24), (9364, 38), (9365, 25), (9366, 20), (9367, 19), (9368, 27), (9369, 27), (9370, 17), (9371, 27), (9372, 46), (9373, 23), (9374, 20), (9375, 29), (9376, 19), (9377, 30), (9378, 24), (9379, 28), (9380, 29), (9381, 31), (9382, 25), (9383, 25), (9384, 51), (9385, 17), (9386, 32), (9387, 26), (9388, 24), (9389, 26), (9390, 26), (9391, 25), (9392, 28), (9393, 30), (9394, 21), (9395, 29), (9396, 46), (9397, 34), (9398, 24), (9399, 29), (9400, 56), (9401, 29), (9402, 34), (9403, 20), (9404, 32), (9405, 25), (9406, 41), (9407, 31), (9408, 23), (9409, 31), (9410, 20), (9411, 18), (9412, 45), (9413, 19), (9414, 21), (9415, 54), (9416, 19), (9417, 28), (9418, 27), (9419, 21), (9420, 17), (9421, 29), (9422, 73), (9423, 52), (9424, 22), (9425, 27), (9426, 30), (9427, 18), (9428, 26), (9429, 26), (9430, 32), (9431, 33), (9432, 15), (9433, 24), (9434, 34), (9435, 20), (9436, 27), (9437, 32), (9438, 26), (9439, 27), (9440, 21), (9441, 24), (9442, 36), (9443, 41), (9444, 28), (9445, 23), (9446, 43), (9447, 25), (9448, 25), (9449, 40), (9450, 26), (9451, 23), (9452, 32), (9453, 38), (9454, 53), (9455, 39), (9456, 47), (9457, 26), (9458, 31), (9459, 22), (9460, 40), (9461, 22), (9462, 20), (9463, 24), (9464, 24), (9465, 29), (9466, 36), (9467, 31), (9468, 39), (9469, 20), (9470, 24), (9471, 19), (9472, 22), (9473, 19), (9474, 27), (9475, 24), (9476, 27), (9477, 17), (9478, 27), (9479, 27), (9480, 35), (9481, 57), (9482, 21), (9483, 24), (9484, 23), (9485, 25), (9486, 24), (9487, 25), (9488, 30), (9489, 25), (9490, 60), (9491, 25), (9492, 23), (9493, 21), (9494, 28), (9495, 45), (9496, 38), (9497, 19), (9498, 23), (9499, 20), (9500, 36), (9501, 51), (9502, 26), (9503, 28), (9504, 23), (9505, 19), (9506, 37), (9507, 32), (9508, 28), (9509, 25), (9510, 19), (9511, 27), (9512, 30), (9513, 42), (9514, 26), (9515, 24), (9516, 18), (9517, 35), (9518, 29), (9519, 17), (9520, 24), (9521, 16), (9522, 18), (9523, 32), (9524, 37), (9525, 35), (9526, 33), (9527, 24), (9528, 26), (9529, 17), (9530, 42), (9531, 28), (9532, 77), (9533, 27), (9534, 18), (9535, 26), (9536, 26), (9537, 27), (9538, 29), (9539, 20), (9540, 39), (9541, 31), (9542, 15), (9543, 21), (9544, 27), (9545, 15), (9546, 17), (9547, 35), (9548, 27), (9549, 27), (9550, 24), (9551, 29), (9552, 29), (9553, 18), (9554, 41), (9555, 20), (9556, 39), (9557, 36), (9558, 23), (9559, 26), (9560, 34), (9561, 23), (9562, 25), (9563, 42), (9564, 20), (9565, 26), (9566, 25), (9567, 46), (9568, 27), (9569, 36), (9570, 20), (9571, 21), (9572, 26), (9573, 23), (9574, 39), (9575, 28), (9576, 23), (9577, 30), (9578, 24), (9579, 42), (9580, 27), (9581, 39), (9582, 31), (9583, 21), (9584, 21), (9585, 30), (9586, 27), (9587, 42), (9588, 26), (9589, 34), (9590, 36), (9591, 33), (9592, 20), (9593, 28), (9594, 20), (9595, 16), (9596, 30), (9597, 33), (9598, 47), (9599, 21), (9600, 28), (9601, 26), (9602, 37), (9603, 24), (9604, 24), (9605, 22), (9606, 26), (9607, 35), (9608, 39), (9609, 31), (9610, 23), (9611, 29), (9612, 38), (9613, 21), (9614, 27), (9615, 38), (9616, 30), (9617, 37), (9618, 19), (9619, 21), (9620, 21), (9621, 53), (9622, 16), (9623, 29), (9624, 28), (9625, 42), (9626, 36), (9627, 20), (9628, 39), (9629, 18), (9630, 19), (9631, 27), (9632, 25), (9633, 32), (9634, 17), (9635, 27), (9636, 27), (9637, 22), (9638, 80), (9639, 30), (9640, 31), (9641, 49), (9642, 20), (9643, 29), (9644, 22), (9645, 23), (9646, 13), (9647, 19), (9648, 21), (9649, 34), (9650, 24), (9651, 38), (9652, 29), (9653, 27), (9654, 21), (9655, 41), (9656, 37), (9657, 32), (9658, 37), (9659, 22), (9660, 23), (9661, 48), (9662, 26), (9663, 37), (9664, 23), (9665, 16), (9666, 22), (9667, 33), (9668, 25), (9669, 47), (9670, 20), (9671, 32), (9672, 28), (9673, 30), (9674, 41), (9675, 51), (9676, 48), (9677, 36), (9678, 28), (9679, 28), (9680, 24), (9681, 25), (9682, 29), (9683, 44), (9684, 26), (9685, 26), (9686, 21), (9687, 42), (9688, 19), (9689, 39), (9690, 29), (9691, 32), (9692, 29), (9693, 49), (9694, 45), (9695, 18), (9696, 28), (9697, 22), (9698, 23), (9699, 23), (9700, 55), (9701, 28), (9702, 25), (9703, 29), (9704, 34), (9705, 28), (9706, 50), (9707, 34), (9708, 33), (9709, 23), (9710, 21), (9711, 29), (9712, 24), (9713, 50), (9714, 36), (9715, 30), (9716, 22), (9717, 20), (9718, 28), (9719, 22), (9720, 32), (9721, 28), (9722, 23), (9723, 20), (9724, 34), (9725, 24), (9726, 25), (9727, 31), (9728, 40), (9729, 40), (9730, 25), (9731, 21), (9732, 26), (9733, 20), (9734, 20), (9735, 24), (9736, 25), (9737, 27), (9738, 40), (9739, 37), (9740, 25), (9741, 30), (9742, 29), (9743, 24), (9744, 30), (9745, 21), (9746, 31), (9747, 26), (9748, 19), (9749, 21), (9750, 27), (9751, 25), (9752, 56), (9753, 26), (9754, 22), (9755, 28), (9756, 24), (9757, 28), (9758, 37), (9759, 21), (9760, 31), (9761, 30), (9762, 30), (9763, 28), (9764, 20), (9765, 23), (9766, 33), (9767, 27), (9768, 20), (9769, 43), (9770, 28), (9771, 24), (9772, 24), (9773, 30), (9774, 38), (9775, 23), (9776, 27), (9777, 29), (9778, 31), (9779, 24), (9780, 25), (9781, 32), (9782, 17), (9783, 21), (9784, 41), (9785, 26), (9786, 34), (9787, 27), (9788, 24), (9789, 28), (9790, 29), (9791, 26), (9792, 22), (9793, 46), (9794, 21), (9795, 35), (9796, 36), (9797, 31), (9798, 41), (9799, 36), (9800, 31), (9801, 33), (9802, 27), (9803, 31), (9804, 24), (9805, 23), (9806, 28), (9807, 39), (9808, 33), (9809, 30), (9810, 27), (9811, 26), (9812, 25), (9813, 24), (9814, 24), (9815, 38), (9816, 29), (9817, 38), (9818, 38), (9819, 21), (9820, 18), (9821, 20), (9822, 20), (9823, 21), (9824, 27), (9825, 29), (9826, 51), (9827, 26), (9828, 21), (9829, 20), (9830, 29), (9831, 25), (9832, 24), (9833, 19), (9834, 69), (9835, 28), (9836, 29), (9837, 25), (9838, 36), (9839, 30), (9840, 39), (9841, 15), (9842, 21), (9843, 30), (9844, 37), (9845, 32), (9846, 23), (9847, 21), (9848, 17), (9849, 29), (9850, 24), (9851, 29), (9852, 76), (9853, 16), (9854, 41), (9855, 30), (9856, 45), (9857, 34), (9858, 34), (9859, 26), (9860, 36), (9861, 30), (9862, 27), (9863, 38), (9864, 27), (9865, 22), (9866, 47), (9867, 19), (9868, 26), (9869, 22), (9870, 16), (9871, 17), (9872, 27), (9873, 26), (9874, 24), (9875, 29), (9876, 27), (9877, 33), (9878, 21), (9879, 18), (9880, 29), (9881, 24), (9882, 30), (9883, 24), (9884, 31), (9885, 27), (9886, 23), (9887, 23), (9888, 20), (9889, 23), (9890, 38), (9891, 32), (9892, 40), (9893, 20), (9894, 33), (9895, 28), (9896, 40), (9897, 27), (9898, 28), (9899, 35), (9900, 20), (9901, 24), (9902, 33), (9903, 33), (9904, 18), (9905, 30), (9906, 18), (9907, 19), (9908, 39), (9909, 28), (9910, 30), (9911, 28), (9912, 33), (9913, 23), (9914, 28), (9915, 35), (9916, 31), (9917, 33), (9918, 58), (9919, 17), (9920, 28), (9921, 20), (9922, 23), (9923, 24), (9924, 16), (9925, 35), (9926, 27), (9927, 38), (9928, 23), (9929, 34), (9930, 24), (9931, 50), (9932, 22), (9933, 34), (9934, 25), (9935, 22), (9936, 68), (9937, 20), (9938, 15), (9939, 58), (9940, 28), (9941, 29), (9942, 38), (9943, 32), (9944, 57), (9945, 27), (9946, 53), (9947, 25), (9948, 29), (9949, 17), (9950, 23), (9951, 20), (9952, 70), (9953, 36), (9954, 42), (9955, 27), (9956, 42), (9957, 23), (9958, 48), (9959, 36), (9960, 23), (9961, 36), (9962, 23), (9963, 21), (9964, 43), (9965, 21), (9966, 18), (9967, 60), (9968, 29), (9969, 31), (9970, 30), (9971, 22), (9972, 32), (9973, 30), (9974, 25), (9975, 27), (9976, 26), (9977, 29), (9978, 31), (9979, 32), (9980, 24), (9981, 35), (9982, 15), (9983, 20), (9984, 26), (9985, 23), (9986, 46), (9987, 46), (9988, 28), (9989, 32), (9990, 35), (9991, 19), (9992, 28), (9993, 33), (9994, 46), (9995, 22), (9996, 22), (9997, 29), (9998, 27), (9999, 24), (10000, 19), (10001, 37), (10002, 21), (10003, 22), (10004, 41), (10005, 54), (10006, 28), (10007, 33), (10008, 33), (10009, 31), (10010, 29), (10011, 24), (10012, 27), (10013, 25), (10014, 20), (10015, 28), (10016, 35), (10017, 16), (10018, 29), (10019, 25), (10020, 24), (10021, 27), (10022, 26), (10023, 29), (10024, 26), (10025, 31), (10026, 26), (10027, 36), (10028, 51), (10029, 31), (10030, 25), (10031, 25), (10032, 29), (10033, 30), (10034, 24), (10035, 27), (10036, 50), (10037, 26), (10038, 46), (10039, 19), (10040, 29), (10041, 30), (10042, 48), (10043, 46), (10044, 39), (10045, 25), (10046, 33), (10047, 33), (10048, 21), (10049, 28), (10050, 38), (10051, 30), (10052, 38), (10053, 22), (10054, 25), (10055, 26), (10056, 42), (10057, 30), (10058, 35), (10059, 24), (10060, 37), (10061, 36), (10062, 34), (10063, 38), (10064, 26), (10065, 19), (10066, 25), (10067, 29), (10068, 23), (10069, 21), (10070, 28), (10071, 48), (10072, 30), (10073, 22), (10074, 25), (10075, 28), (10076, 36), (10077, 23), (10078, 26), (10079, 20), (10080, 24), (10081, 28), (10082, 40), (10083, 21), (10084, 19), (10085, 20), (10086, 32), (10087, 21), (10088, 21), (10089, 16), (10090, 30), (10091, 31), (10092, 27), (10093, 28), (10094, 17), (10095, 28), (10096, 29), (10097, 38), (10098, 52), (10099, 30), (10100, 23), (10101, 18), (10102, 23), (10103, 27), (10104, 20), (10105, 36), (10106, 73), (10107, 30), (10108, 36), (10109, 28), (10110, 28), (10111, 42), (10112, 28), (10113, 32), (10114, 22), (10115, 29), (10116, 24), (10117, 24), (10118, 23), (10119, 25), (10120, 38), (10121, 27), (10122, 20), (10123, 27), (10124, 26), (10125, 25), (10126, 28), (10127, 99), (10128, 42), (10129, 23), (10130, 33), (10131, 29), (10132, 21), (10133, 22), (10134, 21), (10135, 26), (10136, 28), (10137, 23), (10138, 35), (10139, 38), (10140, 50), (10141, 24), (10142, 18), (10143, 32), (10144, 21), (10145, 31), (10146, 19), (10147, 28), (10148, 25), (10149, 31), (10150, 29), (10151, 27), (10152, 46), (10153, 37), (10154, 37), (10155, 47), (10156, 29), (10157, 39), (10158, 22), (10159, 33), (10160, 27), (10161, 24), (10162, 24), (10163, 39), (10164, 27), (10165, 24), (10166, 27), (10167, 20), (10168, 28), (10169, 34), (10170, 19), (10171, 29), (10172, 39), (10173, 33), (10174, 25), (10175, 30), (10176, 19), (10177, 22), (10178, 22), (10179, 21), (10180, 31), (10181, 22), (10182, 18), (10183, 24), (10184, 21), (10185, 19), (10186, 36), (10187, 24), (10188, 36), (10189, 24), (10190, 28), (10191, 26), (10192, 18), (10193, 29), (10194, 28), (10195, 50), (10196, 19), (10197, 32), (10198, 20), (10199, 26), (10200, 24), (10201, 28), (10202, 26), (10203, 23), (10204, 26), (10205, 34), (10206, 17), (10207, 31), (10208, 26), (10209, 28), (10210, 21), (10211, 29), (10212, 25), (10213, 32), (10214, 38), (10215, 24), (10216, 33), (10217, 33), (10218, 25), (10219, 35), (10220, 21), (10221, 29), (10222, 22), (10223, 38), (10224, 45), (10225, 25), (10226, 20), (10227, 24), (10228, 32), (10229, 22), (10230, 58), (10231, 35), (10232, 24), (10233, 34), (10234, 36), (10235, 27), (10236, 21), (10237, 48), (10238, 37), (10239, 26), (10240, 43), (10241, 28), (10242, 27), (10243, 25), (10244, 27), (10245, 20), (10246, 28), (10247, 28), (10248, 26), (10249, 20), (10250, 29), (10251, 28), (10252, 33), (10253, 19), (10254, 15), (10255, 31), (10256, 28), (10257, 30), (10258, 27), (10259, 49), (10260, 20), (10261, 26), (10262, 42), (10263, 31), (10264, 28), (10265, 37), (10266, 26), (10267, 27), (10268, 23), (10269, 20), (10270, 19), (10271, 39), (10272, 31), (10273, 21), (10274, 22), (10275, 29), (10276, 43), (10277, 55), (10278, 22), (10279, 28), (10280, 24), (10281, 19), (10282, 27), (10283, 37), (10284, 24), (10285, 15), (10286, 30), (10287, 25), (10288, 21), (10289, 32), (10290, 30), (10291, 19), (10292, 20), (10293, 27), (10294, 23), (10295, 31), (10296, 38), (10297, 20), (10298, 27), (10299, 29), (10300, 25), (10301, 23), (10302, 20), (10303, 22), (10304, 31), (10305, 33), (10306, 32), (10307, 54), (10308, 27), (10309, 22), (10310, 22), (10311, 46), (10312, 20), (10313, 37), (10314, 27), (10315, 22), (10316, 19), (10317, 34), (10318, 51), (10319, 31), (10320, 28), (10321, 17), (10322, 15), (10323, 32), (10324, 36), (10325, 33), (10326, 31), (10327, 31), (10328, 19), (10329, 28), (10330, 34), (10331, 25), (10332, 40), (10333, 20), (10334, 28), (10335, 52), (10336, 33), (10337, 34), (10338, 30), (10339, 31), (10340, 31), (10341, 62), (10342, 20), (10343, 29), (10344, 29), (10345, 15), (10346, 21), (10347, 31), (10348, 17), (10349, 19), (10350, 27), (10351, 33), (10352, 20), (10353, 25), (10354, 21), (10355, 24), (10356, 22), (10357, 51), (10358, 24), (10359, 36), (10360, 31), (10361, 21), (10362, 17), (10363, 25), (10364, 19), (10365, 22), (10366, 24), (10367, 18), (10368, 22), (10369, 28), (10370, 33), (10371, 31), (10372, 27), (10373, 45), (10374, 24), (10375, 41), (10376, 30), (10377, 32), (10378, 32), (10379, 21), (10380, 28), (10381, 44), (10382, 38), (10383, 24), (10384, 18), (10385, 26), (10386, 39), (10387, 25), (10388, 29), (10389, 24), (10390, 29), (10391, 23), (10392, 36), (10393, 43), (10394, 45), (10395, 30), (10396, 30), (10397, 20), (10398, 27), (10399, 35), (10400, 32), (10401, 37), (10402, 30), (10403, 36), (10404, 22), (10405, 24), (10406, 22), (10407, 38), (10408, 52), (10409, 25), (10410, 53), (10411, 23), (10412, 43), (10413, 20), (10414, 25), (10415, 28), (10416, 32), (10417, 25), (10418, 21), (10419, 44), (10420, 20), (10421, 27), (10422, 21), (10423, 25), (10424, 22), (10425, 38), (10426, 32), (10427, 24), (10428, 26), (10429, 27), (10430, 17), (10431, 28), (10432, 20), (10433, 28), (10434, 23), (10435, 33), (10436, 30), (10437, 24), (10438, 18), (10439, 33), (10440, 22), (10441, 27), (10442, 27), (10443, 42), (10444, 38), (10445, 31), (10446, 25), (10447, 39), (10448, 60), (10449, 28), (10450, 33), (10451, 22), (10452, 32), (10453, 20), (10454, 27), (10455, 24), (10456, 18), (10457, 21), (10458, 21), (10459, 25), (10460, 28), (10461, 38), (10462, 26), (10463, 24), (10464, 24), (10465, 24), (10466, 17), (10467, 19), (10468, 26), (10469, 22), (10470, 16), (10471, 29), (10472, 32), (10473, 32), (10474, 38), (10475, 22), (10476, 30), (10477, 24), (10478, 36), (10479, 22), (10480, 23), (10481, 26), (10482, 20), (10483, 27), (10484, 25), (10485, 28), (10486, 29), (10487, 25), (10488, 26), (10489, 32), (10490, 26), (10491, 25), (10492, 21), (10493, 19), (10494, 44), (10495, 26), (10496, 26), (10497, 34), (10498, 32), (10499, 18), (10500, 24), (10501, 24), (10502, 24), (10503, 23), (10504, 35), (10505, 38), (10506, 37), (10507, 28), (10508, 45), (10509, 25), (10510, 28), (10511, 34), (10512, 27), (10513, 57), (10514, 27), (10515, 24), (10516, 28), (10517, 51), (10518, 21), (10519, 25), (10520, 30), (10521, 30), (10522, 21), (10523, 43), (10524, 35), (10525, 30), (10526, 35), (10527, 33), (10528, 30), (10529, 35), (10530, 23), (10531, 22), (10532, 23), (10533, 41), (10534, 24), (10535, 23), (10536, 18), (10537, 28), (10538, 35), (10539, 42), (10540, 44), (10541, 24), (10542, 23), (10543, 34), (10544, 32), (10545, 27), (10546, 30), (10547, 15), (10548, 30), (10549, 30), (10550, 20), (10551, 23), (10552, 38), (10553, 21), (10554, 23), (10555, 29), (10556, 17), (10557, 28), (10558, 52), (10559, 37), (10560, 23), (10561, 32), (10562, 22), (10563, 25), (10564, 24), (10565, 25), (10566, 34), (10567, 39), (10568, 17), (10569, 29), (10570, 60), (10571, 31), (10572, 18), (10573, 30), (10574, 24), (10575, 21), (10576, 30), (10577, 22), (10578, 27), (10579, 34), (10580, 34), (10581, 25), (10582, 22), (10583, 14), (10584, 44), (10585, 23), (10586, 35), (10587, 28), (10588, 36), (10589, 36), (10590, 26), (10591, 31), (10592, 25), (10593, 20), (10594, 34), (10595, 25), (10596, 41), (10597, 23), (10598, 40), (10599, 36), (10600, 26), (10601, 31), (10602, 18), (10603, 36), (10604, 31), (10605, 41), (10606, 23), (10607, 36), (10608, 32), (10609, 36), (10610, 24), (10611, 26), (10612, 25), (10613, 32), (10614, 19), (10615, 29), (10616, 34), (10617, 24), (10618, 25), (10619, 34), (10620, 21), (10621, 45), (10622, 20), (10623, 33), (10624, 24), (10625, 23), (10626, 33), (10627, 25), (10628, 24), (10629, 28), (10630, 24), (10631, 34), (10632, 39), (10633, 26), (10634, 27), (10635, 16), (10636, 31), (10637, 15), (10638, 20), (10639, 20), (10640, 30), (10641, 19), (10642, 29), (10643, 31), (10644, 18), (10645, 20), (10646, 19), (10647, 29), (10648, 22), (10649, 31), (10650, 21), (10651, 30), (10652, 47), (10653, 32), (10654, 39), (10655, 30), (10656, 27), (10657, 22), (10658, 39), (10659, 36), (10660, 85), (10661, 33), (10662, 23), (10663, 36), (10664, 31), (10665, 32), (10666, 25), (10667, 29), (10668, 59), (10669, 28), (10670, 25), (10671, 34), (10672, 18), (10673, 31), (10674, 27), (10675, 22), (10676, 24), (10677, 37), (10678, 28), (10679, 28), (10680, 24), (10681, 26), (10682, 49), (10683, 29), (10684, 23), (10685, 32), (10686, 31), (10687, 19), (10688, 24), (10689, 27), (10690, 25), (10691, 34), (10692, 36), (10693, 57), (10694, 26), (10695, 24), (10696, 32), (10697, 29), (10698, 19), (10699, 21), (10700, 25), (10701, 26), (10702, 22), (10703, 31), (10704, 15), (10705, 21), (10706, 37), (10707, 26), (10708, 27), (10709, 28), (10710, 23), (10711, 22), (10712, 21), (10713, 27), (10714, 35), (10715, 33), (10716, 16), (10717, 26), (10718, 30), (10719, 20), (10720, 22), (10721, 19), (10722, 19), (10723, 23), (10724, 38), (10725, 28), (10726, 22), (10727, 29), (10728, 22), (10729, 34), (10730, 27), (10731, 21), (10732, 30), (10733, 21), (10734, 28), (10735, 23), (10736, 40), (10737, 20), (10738, 29), (10739, 20), (10740, 41), (10741, 25), (10742, 27), (10743, 26), (10744, 32), (10745, 29), (10746, 35), (10747, 47), (10748, 23), (10749, 18), (10750, 27), (10751, 28), (10752, 25), (10753, 39), (10754, 20), (10755, 27), (10756, 19), (10757, 20), (10758, 25), (10759, 35), (10760, 27), (10761, 48), (10762, 22), (10763, 33), (10764, 20), (10765, 55), (10766, 24), (10767, 23), (10768, 25), (10769, 29), (10770, 38), (10771, 16), (10772, 29), (10773, 38), (10774, 28), (10775, 42), (10776, 27), (10777, 23), (10778, 38), (10779, 40), (10780, 21), (10781, 36), (10782, 27), (10783, 30), (10784, 37), (10785, 34), (10786, 33), (10787, 37), (10788, 26), (10789, 24), (10790, 22), (10791, 31), (10792, 27), (10793, 48), (10794, 37), (10795, 23), (10796, 28), (10797, 28), (10798, 42), (10799, 28), (10800, 58), (10801, 36), (10802, 29), (10803, 23), (10804, 35), (10805, 32), (10806, 24), (10807, 30), (10808, 27), (10809, 28), (10810, 37), (10811, 19), (10812, 25), (10813, 35), (10814, 27), (10815, 40), (10816, 23), (10817, 27), (10818, 21), (10819, 22), (10820, 26), (10821, 65), (10822, 31), (10823, 24), (10824, 23), (10825, 19), (10826, 25), (10827, 34), (10828, 28), (10829, 22), (10830, 30), (10831, 24), (10832, 52), (10833, 29), (10834, 21), (10835, 28), (10836, 39), (10837, 32), (10838, 19), (10839, 28), (10840, 27), (10841, 20), (10842, 40), (10843, 24), (10844, 47), (10845, 24), (10846, 18), (10847, 30), (10848, 39), (10849, 29), (10850, 35), (10851, 45), (10852, 42), (10853, 23), (10854, 20), (10855, 39), (10856, 30), (10857, 23), (10858, 20), (10859, 19), (10860, 16), (10861, 30), (10862, 59), (10863, 20), (10864, 36), (10865, 35), (10866, 42), (10867, 38), (10868, 48), (10869, 27), (10870, 24), (10871, 30), (10872, 24), (10873, 44), (10874, 30), (10875, 34), (10876, 43), (10877, 26), (10878, 26), (10879, 22), (10880, 36), (10881, 19), (10882, 25), (10883, 35), (10884, 19), (10885, 27), (10886, 33), (10887, 23), (10888, 33), (10889, 32), (10890, 23), (10891, 56), (10892, 38), (10893, 22), (10894, 27), (10895, 27), (10896, 21), (10897, 27), (10898, 27), (10899, 26), (10900, 60), (10901, 37), (10902, 24), (10903, 30), (10904, 27), (10905, 28), (10906, 37), (10907, 31), (10908, 30), (10909, 36), (10910, 20), (10911, 33), (10912, 32), (10913, 30), (10914, 25), (10915, 22), (10916, 20), (10917, 40), (10918, 21), (10919, 34), (10920, 31), (10921, 20), (10922, 20), (10923, 20), (10924, 98), (10925, 26), (10926, 49), (10927, 45), (10928, 29), (10929, 28), (10930, 29), (10931, 25), (10932, 28), (10933, 25), (10934, 22), (10935, 17), (10936, 29), (10937, 52), (10938, 19), (10939, 45), (10940, 37), (10941, 35), (10942, 29), (10943, 27), (10944, 27), (10945, 27), (10946, 25), (10947, 32), (10948, 25), (10949, 86), (10950, 22), (10951, 46), (10952, 35), (10953, 39), (10954, 37), (10955, 22), (10956, 23), (10957, 30), (10958, 28), (10959, 44), (10960, 24), (10961, 31), (10962, 27), (10963, 27), (10964, 21), (10965, 49), (10966, 36), (10967, 22), (10968, 32), (10969, 16), (10970, 28), (10971, 36), (10972, 29), (10973, 24), (10974, 36), (10975, 35), (10976, 31), (10977, 20), (10978, 28), (10979, 21), (10980, 27), (10981, 35), (10982, 24), (10983, 19), (10984, 23), (10985, 31), (10986, 24), (10987, 29), (10988, 28), (10989, 28), (10990, 29), (10991, 23), (10992, 27), (10993, 26), (10994, 22), (10995, 30), (10996, 27), (10997, 32), (10998, 23), (10999, 27), (11000, 27), (11001, 45), (11002, 20), (11003, 22), (11004, 29), (11005, 50), (11006, 36), (11007, 27), (11008, 29), (11009, 36), (11010, 24), (11011, 41), (11012, 25), (11013, 26), (11014, 39), (11015, 50), (11016, 22), (11017, 19), (11018, 30), (11019, 33), (11020, 52), (11021, 36), (11022, 22), (11023, 24), (11024, 26), (11025, 30), (11026, 28), (11027, 28), (11028, 23), (11029, 24), (11030, 44), (11031, 22), (11032, 36), (11033, 31), (11034, 28), (11035, 27), (11036, 29), (11037, 20), (11038, 35), (11039, 41), (11040, 29), (11041, 32), (11042, 18), (11043, 19), (11044, 43), (11045, 27), (11046, 18), (11047, 23), (11048, 25), (11049, 26), (11050, 18), (11051, 20), (11052, 26), (11053, 34), (11054, 25), (11055, 27), (11056, 27), (11057, 34), (11058, 27), (11059, 46), (11060, 29), (11061, 22), (11062, 21), (11063, 27), (11064, 24), (11065, 52), (11066, 33), (11067, 25), (11068, 25), (11069, 22), (11070, 27), (11071, 34), (11072, 28), (11073, 23), (11074, 47), (11075, 33), (11076, 34), (11077, 52), (11078, 27), (11079, 16), (11080, 18), (11081, 48), (11082, 28), (11083, 34), (11084, 25), (11085, 17), (11086, 26), (11087, 31), (11088, 39), (11089, 32), (11090, 28), (11091, 50), (11092, 16), (11093, 19), (11094, 20), (11095, 25), (11096, 20), (11097, 22), (11098, 26), (11099, 44), (11100, 30), (11101, 30), (11102, 29), (11103, 20), (11104, 57), (11105, 31), (11106, 20), (11107, 29), (11108, 21), (11109, 18), (11110, 23), (11111, 22), (11112, 21), (11113, 18), (11114, 33), (11115, 20), (11116, 19), (11117, 30), (11118, 22), (11119, 17), (11120, 19), (11121, 30), (11122, 38), (11123, 30), (11124, 17), (11125, 31), (11126, 26), (11127, 19), (11128, 21), (11129, 19), (11130, 32), (11131, 30), (11132, 22), (11133, 27), (11134, 27), (11135, 25), (11136, 35), (11137, 28), (11138, 26), (11139, 33), (11140, 17), (11141, 25), (11142, 14), (11143, 31), (11144, 26), (11145, 25), (11146, 37), (11147, 26), (11148, 39), (11149, 25), (11150, 31), (11151, 17), (11152, 32), (11153, 31), (11154, 30), (11155, 41), (11156, 28), (11157, 40), (11158, 27), (11159, 32), (11160, 20), (11161, 21), (11162, 29), (11163, 19), (11164, 26), (11165, 29), (11166, 24), (11167, 34), (11168, 46), (11169, 25), (11170, 32), (11171, 20), (11172, 27), (11173, 38), (11174, 24), (11175, 30), (11176, 15), (11177, 37), (11178, 29), (11179, 23), (11180, 21), (11181, 39), (11182, 23), (11183, 31), (11184, 19), (11185, 33), (11186, 28), (11187, 24), (11188, 32), (11189, 26), (11190, 41), (11191, 23), (11192, 25), (11193, 29), (11194, 30), (11195, 25), (11196, 22), (11197, 47), (11198, 29), (11199, 22), (11200, 18), (11201, 23), (11202, 30), (11203, 25), (11204, 31), (11205, 30), (11206, 22), (11207, 26), (11208, 16), (11209, 22), (11210, 26), (11211, 32), (11212, 28), (11213, 19), (11214, 37), (11215, 17), (11216, 48), (11217, 20), (11218, 37), (11219, 19), (11220, 33), (11221, 23), (11222, 36), (11223, 22), (11224, 27), (11225, 40), (11226, 41), (11227, 54), (11228, 46), (11229, 24), (11230, 21), (11231, 38), (11232, 29), (11233, 26), (11234, 30), (11235, 20), (11236, 20), (11237, 29), (11238, 32), (11239, 56), (11240, 37), (11241, 30), (11242, 36), (11243, 27), (11244, 37), (11245, 27), (11246, 35), (11247, 33), (11248, 26), (11249, 30), (11250, 24), (11251, 39), (11252, 31), (11253, 21), (11254, 24), (11255, 22), (11256, 31), (11257, 22), (11258, 36), (11259, 42), (11260, 18), (11261, 50), (11262, 17), (11263, 24), (11264, 19), (11265, 31), (11266, 28), (11267, 15), (11268, 30), (11269, 24), (11270, 23), (11271, 23), (11272, 29), (11273, 24), (11274, 20), (11275, 25), (11276, 19), (11277, 20), (11278, 34), (11279, 21), (11280, 20), (11281, 71), (11282, 23), (11283, 17), (11284, 26), (11285, 39), (11286, 25), (11287, 21), (11288, 20), (11289, 21), (11290, 25), (11291, 23), (11292, 27), (11293, 24), (11294, 26), (11295, 25), (11296, 20), (11297, 43), (11298, 23), (11299, 31), (11300, 16), (11301, 38), (11302, 74), (11303, 43), (11304, 25), (11305, 24), (11306, 30), (11307, 25), (11308, 34), (11309, 22), (11310, 42), (11311, 22), (11312, 22), (11313, 26), (11314, 34), (11315, 35), (11316, 28), (11317, 30), (11318, 26), (11319, 61), (11320, 27), (11321, 32), (11322, 18), (11323, 29), (11324, 27), (11325, 26), (11326, 19), (11327, 26), (11328, 74), (11329, 32), (11330, 31), (11331, 20), (11332, 28), (11333, 26), (11334, 19), (11335, 30), (11336, 30), (11337, 23), (11338, 52), (11339, 29), (11340, 29), (11341, 57), (11342, 46), (11343, 29), (11344, 33), (11345, 22), (11346, 20), (11347, 36), (11348, 28), (11349, 25), (11350, 30), (11351, 32), (11352, 23), (11353, 48), (11354, 107), (11355, 51), (11356, 35), (11357, 44), (11358, 28), (11359, 55), (11360, 27), (11361, 30), (11362, 42), (11363, 64), (11364, 18), (11365, 19), (11366, 25), (11367, 26), (11368, 36), (11369, 48), (11370, 35), (11371, 26), (11372, 78), (11373, 43), (11374, 26), (11375, 32), (11376, 21), (11377, 30), (11378, 43), (11379, 25), (11380, 37), (11381, 24), (11382, 20), (11383, 34), (11384, 38), (11385, 19), (11386, 32), (11387, 25), (11388, 36), (11389, 36), (11390, 32), (11391, 39), (11392, 22), (11393, 25), (11394, 41), (11395, 24), (11396, 20), (11397, 20), (11398, 20), (11399, 26), (11400, 25), (11401, 26), (11402, 42), (11403, 41), (11404, 22), (11405, 33), (11406, 24), (11407, 19), (11408, 29), (11409, 29), (11410, 28), (11411, 29), (11412, 27), (11413, 18), (11414, 25), (11415, 26), (11416, 23), (11417, 35), (11418, 26), (11419, 24), (11420, 29), (11421, 21), (11422, 18), (11423, 26), (11424, 45), (11425, 30), (11426, 16), (11427, 37), (11428, 29), (11429, 34), (11430, 33), (11431, 24), (11432, 22), (11433, 24), (11434, 37), (11435, 25), (11436, 47), (11437, 38), (11438, 26), (11439, 22), (11440, 45), (11441, 26), (11442, 27), (11443, 20), (11444, 43), (11445, 33), (11446, 19), (11447, 28), (11448, 28), (11449, 21), (11450, 31), (11451, 26), (11452, 31), (11453, 24), (11454, 27), (11455, 24), (11456, 33), (11457, 37), (11458, 21), (11459, 25), (11460, 19), (11461, 23), (11462, 26), (11463, 23), (11464, 16), (11465, 26), (11466, 26), (11467, 30), (11468, 29), (11469, 28), (11470, 24), (11471, 27), (11472, 30), (11473, 18), (11474, 26), (11475, 35), (11476, 29), (11477, 17), (11478, 21), (11479, 23), (11480, 24), (11481, 18), (11482, 27), (11483, 25), (11484, 35), (11485, 21), (11486, 40), (11487, 25), (11488, 29), (11489, 25), (11490, 23), (11491, 21), (11492, 22), (11493, 21), (11494, 24), (11495, 29), (11496, 23), (11497, 27), (11498, 17), (11499, 31), (11500, 36), (11501, 21), (11502, 41), (11503, 22), (11504, 33), (11505, 20), (11506, 49), (11507, 18), (11508, 19), (11509, 35), (11510, 23), (11511, 19), (11512, 28), (11513, 32), (11514, 47), (11515, 19), (11516, 42), (11517, 28), (11518, 29), (11519, 25), (11520, 29), (11521, 34), (11522, 28), (11523, 33), (11524, 29), (11525, 29), (11526, 23), (11527, 24), (11528, 23), (11529, 26), (11530, 23), (11531, 32), (11532, 21), (11533, 22), (11534, 21), (11535, 41), (11536, 49), (11537, 36), (11538, 43), (11539, 28), (11540, 32), (11541, 20), (11542, 39), (11543, 88), (11544, 26), (11545, 29), (11546, 22), (11547, 25), (11548, 35), (11549, 31), (11550, 26), (11551, 22), (11552, 34), (11553, 23), (11554, 30), (11555, 18), (11556, 22), (11557, 35), (11558, 36), (11559, 22), (11560, 24), (11561, 27), (11562, 29), (11563, 24), (11564, 26), (11565, 31), (11566, 25), (11567, 27), (11568, 41), (11569, 33), (11570, 24), (11571, 32), (11572, 20), (11573, 35), (11574, 46), (11575, 60), (11576, 17), (11577, 81), (11578, 25), (11579, 28), (11580, 38), (11581, 25), (11582, 27), (11583, 22), (11584, 23), (11585, 23), (11586, 22), (11587, 20), (11588, 48), (11589, 28), (11590, 37), (11591, 25), (11592, 32), (11593, 20), (11594, 26), (11595, 30), (11596, 22), (11597, 27), (11598, 19), (11599, 17), (11600, 26), (11601, 23), (11602, 42), (11603, 18), (11604, 26), (11605, 35), (11606, 25), (11607, 31), (11608, 29), (11609, 27), (11610, 35), (11611, 23), (11612, 29), (11613, 35), (11614, 31), (11615, 28), (11616, 32), (11617, 38), (11618, 29), (11619, 22), (11620, 23), (11621, 19), (11622, 45), (11623, 25), (11624, 16), (11625, 18), (11626, 55), (11627, 20), (11628, 37), (11629, 30), (11630, 23), (11631, 30), (11632, 29), (11633, 28), (11634, 26), (11635, 34), (11636, 19), (11637, 18), (11638, 19), (11639, 70), (11640, 24), (11641, 22), (11642, 23), (11643, 36), (11644, 35), (11645, 18), (11646, 25), (11647, 25), (11648, 46), (11649, 33), (11650, 29), (11651, 35), (11652, 30), (11653, 17), (11654, 23), (11655, 29), (11656, 31), (11657, 33), (11658, 26), (11659, 28), (11660, 26), (11661, 34), (11662, 23), (11663, 41), (11664, 18), (11665, 22), (11666, 40), (11667, 30), (11668, 28), (11669, 28), (11670, 27), (11671, 36), (11672, 29), (11673, 33), (11674, 29), (11675, 29), (11676, 21), (11677, 42), (11678, 27), (11679, 25), (11680, 30), (11681, 30), (11682, 24), (11683, 29), (11684, 19), (11685, 24), (11686, 13), (11687, 26), (11688, 32), (11689, 19), (11690, 35), (11691, 29), (11692, 24), (11693, 18), (11694, 24), (11695, 25), (11696, 33), (11697, 24), (11698, 50), (11699, 31), (11700, 25), (11701, 25), (11702, 31), (11703, 26), (11704, 37), (11705, 34), (11706, 19), (11707, 27), (11708, 26), (11709, 37), (11710, 23), (11711, 29), (11712, 19), (11713, 26), (11714, 26), (11715, 45), (11716, 23), (11717, 26), (11718, 52), (11719, 19), (11720, 41), (11721, 25), (11722, 26), (11723, 30), (11724, 34), (11725, 27), (11726, 24), (11727, 24), (11728, 30), (11729, 23), (11730, 28), (11731, 26), (11732, 30), (11733, 25), (11734, 16), (11735, 40), (11736, 30), (11737, 30), (11738, 34), (11739, 27), (11740, 23), (11741, 20), (11742, 17), (11743, 26), (11744, 16), (11745, 21), (11746, 33), (11747, 25), (11748, 27), (11749, 33), (11750, 24), (11751, 31), (11752, 28), (11753, 20), (11754, 25), (11755, 22), (11756, 19), (11757, 24), (11758, 27), (11759, 24), (11760, 22), (11761, 28), (11762, 32), (11763, 28), (11764, 27), (11765, 22), (11766, 23), (11767, 35), (11768, 26), (11769, 27), (11770, 53), (11771, 39), (11772, 28), (11773, 18), (11774, 24), (11775, 32), (11776, 32), (11777, 27), (11778, 38), (11779, 25), (11780, 34), (11781, 28), (11782, 21), (11783, 24), (11784, 40), (11785, 42), (11786, 50), (11787, 35), (11788, 30), (11789, 24), (11790, 26), (11791, 19), (11792, 32), (11793, 28), (11794, 39), (11795, 30), (11796, 25), (11797, 39), (11798, 24), (11799, 24), (11800, 45), (11801, 18), (11802, 19), (11803, 38), (11804, 27), (11805, 30), (11806, 20), (11807, 19), (11808, 35), (11809, 32), (11810, 27), (11811, 26), (11812, 19), (11813, 48), (11814, 36), (11815, 19), (11816, 30), (11817, 22), (11818, 21), (11819, 52), (11820, 18), (11821, 21), (11822, 32), (11823, 23), (11824, 17), (11825, 23), (11826, 26), (11827, 21), (11828, 41), (11829, 44), (11830, 23), (11831, 22), (11832, 24), (11833, 35), (11834, 24), (11835, 22), (11836, 23), (11837, 25), (11838, 21), (11839, 20), (11840, 26), (11841, 26), (11842, 35), (11843, 30), (11844, 25), (11845, 14), (11846, 23), (11847, 25), (11848, 33), (11849, 30), (11850, 30), (11851, 30), (11852, 22), (11853, 31), (11854, 42), (11855, 28), (11856, 18), (11857, 22), (11858, 31), (11859, 39), (11860, 29), (11861, 26), (11862, 22), (11863, 15), (11864, 34), (11865, 34), (11866, 25), (11867, 20), (11868, 39), (11869, 26), (11870, 31), (11871, 28), (11872, 35), (11873, 26), (11874, 29), (11875, 31), (11876, 22), (11877, 23), (11878, 19), (11879, 35), (11880, 19), (11881, 24), (11882, 22), (11883, 22), (11884, 23), (11885, 33), (11886, 23), (11887, 25), (11888, 23), (11889, 37), (11890, 28), (11891, 24), (11892, 26), (11893, 48), (11894, 25), (11895, 30), (11896, 44), (11897, 35), (11898, 28), (11899, 28), (11900, 29), (11901, 28), (11902, 40), (11903, 17), (11904, 43), (11905, 26), (11906, 22), (11907, 55), (11908, 19), (11909, 27), (11910, 20), (11911, 26), (11912, 26), (11913, 37), (11914, 29), (11915, 22), (11916, 29), (11917, 19), (11918, 16), (11919, 20), (11920, 23), (11921, 43), (11922, 34), (11923, 26), (11924, 29), (11925, 40), (11926, 25), (11927, 30), (11928, 30), (11929, 32), (11930, 22), (11931, 19), (11932, 33), (11933, 38), (11934, 28), (11935, 52), (11936, 30), (11937, 28), (11938, 34), (11939, 22), (11940, 25), (11941, 72), (11942, 27), (11943, 35), (11944, 27), (11945, 25), (11946, 118), (11947, 26), (11948, 27), (11949, 23), (11950, 23), (11951, 33), (11952, 28), (11953, 24), (11954, 26), (11955, 33), (11956, 20), (11957, 30), (11958, 20), (11959, 29), (11960, 26), (11961, 24), (11962, 40), (11963, 26), (11964, 23), (11965, 24), (11966, 21), (11967, 30), (11968, 29), (11969, 28), (11970, 25), (11971, 32), (11972, 24), (11973, 23), (11974, 27), (11975, 17), (11976, 28), (11977, 31), (11978, 34), (11979, 23), (11980, 43), (11981, 36), (11982, 20), (11983, 28), (11984, 27), (11985, 19), (11986, 51), (11987, 33), (11988, 32), (11989, 23), (11990, 27), (11991, 36), (11992, 21), (11993, 28), (11994, 28), (11995, 20), (11996, 39), (11997, 51), (11998, 27), (11999, 35), (12000, 27), (12001, 41), (12002, 20), (12003, 39), (12004, 31), (12005, 31), (12006, 26), (12007, 22), (12008, 34), (12009, 27), (12010, 23), (12011, 26), (12012, 18), (12013, 37), (12014, 27), (12015, 24), (12016, 39), (12017, 27), (12018, 22), (12019, 28), (12020, 21), (12021, 20), (12022, 30), (12023, 59), (12024, 20), (12025, 54), (12026, 23), (12027, 38), (12028, 27), (12029, 23), (12030, 30), (12031, 32), (12032, 75), (12033, 35), (12034, 23), (12035, 28), (12036, 25), (12037, 22), (12038, 45), (12039, 34), (12040, 32), (12041, 18), (12042, 31), (12043, 49), (12044, 33), (12045, 32), (12046, 33), (12047, 33), (12048, 20), (12049, 24), (12050, 28), (12051, 39), (12052, 32), (12053, 37), (12054, 46), (12055, 44), (12056, 23), (12057, 27), (12058, 30), (12059, 30), (12060, 15), (12061, 30), (12062, 33), (12063, 24), (12064, 42), (12065, 26), (12066, 51), (12067, 29), (12068, 37), (12069, 24), (12070, 22), (12071, 37), (12072, 25), (12073, 18), (12074, 32), (12075, 24), (12076, 35), (12077, 16), (12078, 28), (12079, 32), (12080, 21), (12081, 35), (12082, 21), (12083, 21), (12084, 25), (12085, 39), (12086, 20), (12087, 41), (12088, 29), (12089, 22), (12090, 23), (12091, 34), (12092, 26), (12093, 29), (12094, 19), (12095, 29), (12096, 26), (12097, 26), (12098, 32), (12099, 20), (12100, 34), (12101, 22), (12102, 26), (12103, 17), (12104, 16), (12105, 20), (12106, 20), (12107, 45), (12108, 35), (12109, 20), (12110, 28), (12111, 26), (12112, 43), (12113, 25), (12114, 23), (12115, 33), (12116, 15), (12117, 25), (12118, 42), (12119, 29), (12120, 54), (12121, 35), (12122, 33), (12123, 23), (12124, 27), (12125, 26), (12126, 31), (12127, 20), (12128, 25), (12129, 23), (12130, 86), (12131, 26), (12132, 22), (12133, 30), (12134, 18), (12135, 31), (12136, 26), (12137, 29), (12138, 35), (12139, 35), (12140, 29), (12141, 45), (12142, 45), (12143, 25), (12144, 15), (12145, 36), (12146, 28), (12147, 31), (12148, 25), (12149, 24), (12150, 25), (12151, 23), (12152, 27), (12153, 30), (12154, 74), (12155, 27), (12156, 43), (12157, 19), (12158, 24), (12159, 21), (12160, 44), (12161, 72), (12162, 39), (12163, 23), (12164, 45), (12165, 26), (12166, 63), (12167, 37), (12168, 36), (12169, 19), (12170, 26), (12171, 28), (12172, 34), (12173, 37), (12174, 68), (12175, 33), (12176, 35), (12177, 31), (12178, 47), (12179, 27), (12180, 20), (12181, 53), (12182, 38), (12183, 24), (12184, 32), (12185, 20), (12186, 21), (12187, 21), (12188, 30), (12189, 19), (12190, 25), (12191, 42), (12192, 27), (12193, 24), (12194, 55), (12195, 27), (12196, 52), (12197, 22), (12198, 32), (12199, 20), (12200, 39), (12201, 25), (12202, 26), (12203, 20), (12204, 16), (12205, 22), (12206, 24), (12207, 30), (12208, 23), (12209, 40), (12210, 15), (12211, 27), (12212, 40), (12213, 36), (12214, 18), (12215, 19), (12216, 26), (12217, 37), (12218, 27), (12219, 21), (12220, 25), (12221, 33), (12222, 41), (12223, 45), (12224, 30), (12225, 50), (12226, 35), (12227, 21), (12228, 29), (12229, 20), (12230, 26), (12231, 22), (12232, 26), (12233, 23), (12234, 25), (12235, 32), (12236, 26), (12237, 32), (12238, 54), (12239, 28), (12240, 33), (12241, 26), (12242, 23), (12243, 40), (12244, 19), (12245, 41), (12246, 42), (12247, 39), (12248, 20), (12249, 129), (12250, 25), (12251, 25), (12252, 29), (12253, 26), (12254, 28), (12255, 23), (12256, 29), (12257, 24), (12258, 33), (12259, 30), (12260, 19), (12261, 32), (12262, 20), (12263, 29), (12264, 25), (12265, 35), (12266, 45), (12267, 21), (12268, 28), (12269, 31), (12270, 29), (12271, 27), (12272, 33), (12273, 27), (12274, 32), (12275, 24), (12276, 28), (12277, 23), (12278, 25), (12279, 48), (12280, 26), (12281, 19), (12282, 22), (12283, 66), (12284, 24), (12285, 19), (12286, 28), (12287, 40), (12288, 27), (12289, 23), (12290, 33), (12291, 27), (12292, 26), (12293, 19), (12294, 36), (12295, 36), (12296, 17), (12297, 29), (12298, 24), (12299, 25), (12300, 31), (12301, 127), (12302, 27), (12303, 24), (12304, 24), (12305, 26), (12306, 31), (12307, 23), (12308, 18), (12309, 22), (12310, 31), (12311, 28), (12312, 32), (12313, 29), (12314, 27), (12315, 28), (12316, 19), (12317, 22), (12318, 28), (12319, 19), (12320, 24), (12321, 42), (12322, 28), (12323, 22), (12324, 56), (12325, 27), (12326, 80), (12327, 25), (12328, 31), (12329, 29), (12330, 25), (12331, 32), (12332, 39), (12333, 29), (12334, 23), (12335, 21), (12336, 33), (12337, 27), (12338, 31), (12339, 43), (12340, 34), (12341, 18), (12342, 26), (12343, 33), (12344, 33), (12345, 17), (12346, 35), (12347, 25), (12348, 26), (12349, 23), (12350, 31), (12351, 23), (12352, 23), (12353, 28), (12354, 26), (12355, 32), (12356, 20), (12357, 37), (12358, 70), (12359, 34), (12360, 26), (12361, 33), (12362, 38), (12363, 21), (12364, 27), (12365, 31), (12366, 21), (12367, 47), (12368, 28), (12369, 42), (12370, 46), (12371, 30), (12372, 22), (12373, 20), (12374, 28), (12375, 19), (12376, 21), (12377, 20), (12378, 23), (12379, 18), (12380, 29), (12381, 27), (12382, 29), (12383, 26), (12384, 25), (12385, 31), (12386, 28), (12387, 28), (12388, 34), (12389, 17), (12390, 24), (12391, 15), (12392, 31), (12393, 37), (12394, 25), (12395, 33), (12396, 33), (12397, 20), (12398, 33), (12399, 24), (12400, 32), (12401, 31), (12402, 26), (12403, 22), (12404, 23), (12405, 48), (12406, 40), (12407, 24), (12408, 24), (12409, 25), (12410, 18), (12411, 21), (12412, 31), (12413, 22), (12414, 27), (12415, 32), (12416, 19), (12417, 27), (12418, 22), (12419, 37), (12420, 38), (12421, 24), (12422, 33), (12423, 30), (12424, 46), (12425, 19), (12426, 27), (12427, 22), (12428, 42), (12429, 33), (12430, 26), (12431, 35), (12432, 28), (12433, 46), (12434, 20), (12435, 52), (12436, 28), (12437, 19), (12438, 28), (12439, 20), (12440, 46), (12441, 38), (12442, 19), (12443, 23), (12444, 43), (12445, 27), (12446, 33), (12447, 23), (12448, 25), (12449, 38), (12450, 36), (12451, 30), (12452, 24), (12453, 27), (12454, 28), (12455, 49), (12456, 17), (12457, 34), (12458, 31), (12459, 30), (12460, 21), (12461, 21), (12462, 23), (12463, 35), (12464, 38), (12465, 25), (12466, 33), (12467, 59), (12468, 29), (12469, 27), (12470, 22), (12471, 36), (12472, 55), (12473, 34), (12474, 23), (12475, 47), (12476, 29), (12477, 22), (12478, 28), (12479, 38), (12480, 24), (12481, 19), (12482, 19), (12483, 23), (12484, 37), (12485, 29), (12486, 42), (12487, 23), (12488, 31), (12489, 52), (12490, 42), (12491, 25), (12492, 25), (12493, 33), (12494, 29), (12495, 24), (12496, 25), (12497, 29), (12498, 43), (12499, 30), (12500, 28), (12501, 24), (12502, 31), (12503, 25), (12504, 19), (12505, 61), (12506, 40), (12507, 29), (12508, 31), (12509, 24), (12510, 23), (12511, 31), (12512, 18), (12513, 31), (12514, 31), (12515, 21), (12516, 39), (12517, 26), (12518, 21), (12519, 28), (12520, 64), (12521, 55), (12522, 27), (12523, 28), (12524, 26), (12525, 21), (12526, 26), (12527, 50), (12528, 35), (12529, 26), (12530, 29), (12531, 48), (12532, 26), (12533, 23), (12534, 49), (12535, 18), (12536, 21), (12537, 54), (12538, 29), (12539, 19), (12540, 26), (12541, 27), (12542, 27), (12543, 29), (12544, 28), (12545, 27), (12546, 38), (12547, 18), (12548, 21), (12549, 21), (12550, 39), (12551, 21), (12552, 24), (12553, 28), (12554, 20), (12555, 19), (12556, 30), (12557, 28), (12558, 37), (12559, 24), (12560, 30), (12561, 24), (12562, 21), (12563, 26), (12564, 21), (12565, 20), (12566, 21), (12567, 28), (12568, 29), (12569, 21), (12570, 51), (12571, 24), (12572, 42), (12573, 18), (12574, 42), (12575, 27), (12576, 23), (12577, 33), (12578, 38), (12579, 26), (12580, 33), (12581, 35), (12582, 29), (12583, 35), (12584, 32), (12585, 14), (12586, 20), (12587, 67), (12588, 28), (12589, 39), (12590, 28), (12591, 18), (12592, 21), (12593, 21), (12594, 19), (12595, 18), (12596, 32), (12597, 30), (12598, 26), (12599, 23), (12600, 34), (12601, 19), (12602, 27), (12603, 36), (12604, 23), (12605, 16), (12606, 15), (12607, 17), (12608, 58), (12609, 30), (12610, 23), (12611, 40), (12612, 32), (12613, 27), (12614, 23), (12615, 25), (12616, 29), (12617, 59), (12618, 29), (12619, 43), (12620, 29), (12621, 30), (12622, 27), (12623, 26), (12624, 34), (12625, 29), (12626, 27), (12627, 25), (12628, 36), (12629, 26), (12630, 34), (12631, 27), (12632, 40), (12633, 22), (12634, 35), (12635, 33), (12636, 38), (12637, 34), (12638, 23), (12639, 39), (12640, 40), (12641, 20), (12642, 13), (12643, 30), (12644, 25), (12645, 56), (12646, 31), (12647, 19), (12648, 28), (12649, 25), (12650, 42), (12651, 30), (12652, 32), (12653, 34), (12654, 41), (12655, 28), (12656, 23), (12657, 41), (12658, 22), (12659, 35), (12660, 24), (12661, 36), (12662, 23), (12663, 27), (12664, 22), (12665, 27), (12666, 29), (12667, 21), (12668, 17), (12669, 30), (12670, 17), (12671, 19), (12672, 21), (12673, 28), (12674, 25), (12675, 26), (12676, 26), (12677, 33), (12678, 26), (12679, 30), (12680, 32), (12681, 29), (12682, 28), (12683, 39), (12684, 46), (12685, 24), (12686, 34), (12687, 26), (12688, 25), (12689, 24), (12690, 28), (12691, 31), (12692, 34), (12693, 19), (12694, 29), (12695, 31), (12696, 25), (12697, 22), (12698, 28), (12699, 24), (12700, 23), (12701, 23), (12702, 22), (12703, 33), (12704, 23), (12705, 21), (12706, 23), (12707, 35), (12708, 35), (12709, 60), (12710, 20), (12711, 34), (12712, 34), (12713, 29), (12714, 29), (12715, 21), (12716, 27), (12717, 33), (12718, 35), (12719, 29), (12720, 19), (12721, 35), (12722, 24), (12723, 27), (12724, 44), (12725, 24), (12726, 32), (12727, 26), (12728, 59), (12729, 16), (12730, 25), (12731, 18), (12732, 24), (12733, 21), (12734, 29), (12735, 30), (12736, 61), (12737, 24), (12738, 23), (12739, 28), (12740, 38), (12741, 26), (12742, 25), (12743, 26), (12744, 23), (12745, 25), (12746, 28), (12747, 29), (12748, 41), (12749, 22), (12750, 24), (12751, 24), (12752, 20), (12753, 59), (12754, 25), (12755, 23), (12756, 23), (12757, 51), (12758, 29), (12759, 53), (12760, 40), (12761, 23), (12762, 27), (12763, 20), (12764, 38), (12765, 33), (12766, 47), (12767, 23), (12768, 22), (12769, 17), (12770, 30), (12771, 32), (12772, 23), (12773, 27), (12774, 46), (12775, 25), (12776, 39), (12777, 24), (12778, 15), (12779, 32), (12780, 28), (12781, 29), (12782, 57), (12783, 23), (12784, 30), (12785, 20), (12786, 24), (12787, 30), (12788, 25), (12789, 29), (12790, 21), (12791, 30), (12792, 21), (12793, 27), (12794, 51), (12795, 22), (12796, 42), (12797, 25), (12798, 62), (12799, 35)]
init中zip_2: [(1458, 13), (3168, 13), (5913, 13), (9646, 13), (11686, 13), (12642, 13), (795, 14), (1735, 14), (1848, 14), (2093, 14), (2195, 14), (2438, 14), (4590, 14), (4951, 14), (5029, 14), (5855, 14), (5857, 14), (6340, 14), (6982, 14), (7203, 14), (7218, 14), (7599, 14), (9213, 14), (9225, 14), (9267, 14), (10583, 14), (11142, 14), (11845, 14), (12585, 14), (101, 15), (318, 15), (777, 15), (851, 15), (883, 15), (1156, 15), (1236, 15), (1518, 15), (1584, 15), (1629, 15), (1652, 15), (1957, 15), (2162, 15), (2210, 15), (2419, 15), (2584, 15), (2762, 15), (2796, 15), (3042, 15), (3078, 15), (3198, 15), (3426, 15), (3555, 15), (3631, 15), (3712, 15), (3986, 15), (4110, 15), (4125, 15), (4216, 15), (4313, 15), (4351, 15), (4533, 15), (4539, 15), (5047, 15), (5054, 15), (5257, 15), (5331, 15), (5348, 15), (5652, 15), (5780, 15), (5870, 15), (5918, 15), (6055, 15), (6094, 15), (6115, 15), (6385, 15), (6546, 15), (6554, 15), (6721, 15), (6880, 15), (6977, 15), (7012, 15), (7202, 15), (7282, 15), (7548, 15), (8003, 15), (8007, 15), (8099, 15), (8564, 15), (8607, 15), (8701, 15), (8880, 15), (8931, 15), (9339, 15), (9432, 15), (9542, 15), (9545, 15), (9841, 15), (9938, 15), (9982, 15), (10254, 15), (10285, 15), (10322, 15), (10345, 15), (10547, 15), (10637, 15), (10704, 15), (11176, 15), (11267, 15), (11863, 15), (12060, 15), (12116, 15), (12144, 15), (12210, 15), (12391, 15), (12606, 15), (12778, 15), (116, 16), (144, 16), (188, 16), (223, 16), (262, 16), (332, 16), (353, 16), (434, 16), (717, 16), (774, 16), (933, 16), (995, 16), (1212, 16), (1432, 16), (1580, 16), (1667, 16), (1677, 16), (1690, 16), (1762, 16), (1884, 16), (1941, 16), (2028, 16), (2414, 16), (2430, 16), (2668, 16), (2699, 16), (2908, 16), (3123, 16), (3147, 16), (3296, 16), (3464, 16), (3646, 16), (3685, 16), (3701, 16), (3752, 16), (3840, 16), (3842, 16), (4001, 16), (4075, 16), (4098, 16), (4103, 16), (4114, 16), (4135, 16), (4685, 16), (4893, 16), (4954, 16), (4988, 16), (5124, 16), (5254, 16), (5380, 16), (5427, 16), (5475, 16), (5499, 16), (5539, 16), (5586, 16), (5636, 16), (5992, 16), (6003, 16), (6160, 16), (6251, 16), (6287, 16), (6393, 16), (6471, 16), (6484, 16), (6524, 16), (6553, 16), (6675, 16), (6755, 16), (6972, 16), (7712, 16), (7839, 16), (7891, 16), (7897, 16), (7901, 16), (8088, 16), (8138, 16), (8930, 16), (9082, 16), (9137, 16), (9349, 16), (9350, 16), (9521, 16), (9595, 16), (9622, 16), (9665, 16), (9853, 16), (9870, 16), (9924, 16), (10017, 16), (10089, 16), (10470, 16), (10635, 16), (10716, 16), (10771, 16), (10860, 16), (10969, 16), (11079, 16), (11092, 16), (11208, 16), (11300, 16), (11426, 16), (11464, 16), (11624, 16), (11734, 16), (11744, 16), (11918, 16), (12077, 16), (12104, 16), (12204, 16), (12605, 16), (12729, 16), (0, 17), (24, 17), (100, 17), (210, 17), (217, 17), (234, 17), (322, 17), (350, 17), (393, 17), (397, 17), (431, 17), (543, 17), (545, 17), (578, 17), (767, 17), (815, 17), (872, 17), (914, 17), (956, 17), (1001, 17), (1027, 17), (1203, 17), (1233, 17), (1290, 17), (1445, 17), (1606, 17), (1779, 17), (1899, 17), (2060, 17), (2103, 17), (2196, 17), (2296, 17), (2362, 17), (2373, 17), (2511, 17), (2512, 17), (2640, 17), (2665, 17), (2696, 17), (2730, 17), (2767, 17), (2797, 17), (3144, 17), (3195, 17), (3314, 17), (3331, 17), (3373, 17), (3380, 17), (3384, 17), (3536, 17), (3611, 17), (3696, 17), (3778, 17), (3862, 17), (3887, 17), (3906, 17), (4023, 17), (4099, 17), (4121, 17), (4149, 17), (4498, 17), (4519, 17), (4600, 17), (4606, 17), (4645, 17), (4733, 17), (4987, 17), (5000, 17), (5019, 17), (5118, 17), (5236, 17), (5302, 17), (5332, 17), (5340, 17), (5487, 17), (5638, 17), (5644, 17), (5881, 17), (6031, 17), (6058, 17), (6096, 17), (6121, 17), (6162, 17), (6165, 17), (6201, 17), (6386, 17), (6592, 17), (6593, 17), (6682, 17), (6702, 17), (6848, 17), (6934, 17), (6962, 17), (7047, 17), (7388, 17), (7423, 17), (7454, 17), (7493, 17), (7544, 17), (7545, 17), (7580, 17), (7677, 17), (7698, 17), (7776, 17), (7799, 17), (7826, 17), (7999, 17), (8008, 17), (8125, 17), (8393, 17), (8394, 17), (8430, 17), (8440, 17), (8580, 17), (8636, 17), (8642, 17), (8651, 17), (8681, 17), (8800, 17), (8812, 17), (8973, 17), (9018, 17), (9039, 17), (9089, 17), (9093, 17), (9103, 17), (9130, 17), (9162, 17), (9184, 17), (9325, 17), (9370, 17), (9385, 17), (9420, 17), (9477, 17), (9519, 17), (9529, 17), (9546, 17), (9634, 17), (9782, 17), (9848, 17), (9871, 17), (9919, 17), (9949, 17), (10094, 17), (10206, 17), (10321, 17), (10348, 17), (10362, 17), (10430, 17), (10466, 17), (10556, 17), (10568, 17), (10935, 17), (11085, 17), (11119, 17), (11124, 17), (11140, 17), (11151, 17), (11215, 17), (11262, 17), (11283, 17), (11477, 17), (11498, 17), (11576, 17), (11599, 17), (11653, 17), (11742, 17), (11824, 17), (11903, 17), (11975, 17), (12103, 17), (12296, 17), (12345, 17), (12389, 17), (12456, 17), (12607, 17), (12668, 17), (12670, 17), (12769, 17), (14, 18), (79, 18), (98, 18), (103, 18), (125, 18), (131, 18), (155, 18), (176, 18), (227, 18), (277, 18), (300, 18), (303, 18), (329, 18), (340, 18), (392, 18), (399, 18), (424, 18), (441, 18), (472, 18), (474, 18), (633, 18), (694, 18), (744, 18), (750, 18), (762, 18), (799, 18), (875, 18), (877, 18), (918, 18), (919, 18), (1048, 18), (1113, 18), (1164, 18), (1242, 18), (1280, 18), (1340, 18), (1371, 18), (1463, 18), (1571, 18), (1610, 18), (1620, 18), (1657, 18), (1670, 18), (1683, 18), (1739, 18), (1768, 18), (1781, 18), (1786, 18), (1804, 18), (1914, 18), (1965, 18), (2062, 18), (2072, 18), (2269, 18), (2271, 18), (2283, 18), (2294, 18), (2304, 18), (2439, 18), (2565, 18), (2597, 18), (2729, 18), (2747, 18), (2815, 18), (3013, 18), (3086, 18), (3148, 18), (3234, 18), (3252, 18), (3261, 18), (3287, 18), (3313, 18), (3370, 18), (3439, 18), (3455, 18), (3543, 18), (3775, 18), (3806, 18), (3853, 18), (3870, 18), (3912, 18), (3933, 18), (3950, 18), (3998, 18), (4017, 18), (4028, 18), (4175, 18), (4238, 18), (4240, 18), (4250, 18), (4316, 18), (4329, 18), (4425, 18), (4484, 18), (4531, 18), (4763, 18), (4813, 18), (4882, 18), (4886, 18), (4915, 18), (4924, 18), (5024, 18), (5028, 18), (5040, 18), (5060, 18), (5067, 18), (5078, 18), (5088, 18), (5176, 18), (5180, 18), (5209, 18), (5214, 18), (5225, 18), (5235, 18), (5333, 18), (5511, 18), (5603, 18), (5633, 18), (5654, 18), (5672, 18), (5718, 18), (5836, 18), (5868, 18), (5896, 18), (5948, 18), (5967, 18), (6009, 18), (6015, 18), (6052, 18), (6184, 18), (6257, 18), (6277, 18), (6278, 18), (6316, 18), (6319, 18), (6421, 18), (6434, 18), (6467, 18), (6562, 18), (6631, 18), (6716, 18), (6719, 18), (6772, 18), (6850, 18), (6863, 18), (6878, 18), (6883, 18), (6891, 18), (6932, 18), (7138, 18), (7166, 18), (7181, 18), (7189, 18), (7257, 18), (7258, 18), (7277, 18), (7314, 18), (7415, 18), (7523, 18), (7567, 18), (7670, 18), (7691, 18), (7716, 18), (7873, 18), (8009, 18), (8053, 18), (8056, 18), (8064, 18), (8076, 18), (8186, 18), (8194, 18), (8208, 18), (8233, 18), (8318, 18), (8344, 18), (8345, 18), (8397, 18), (8420, 18), (8435, 18), (8490, 18), (8686, 18), (8695, 18), (8736, 18), (8776, 18), (8789, 18), (8827, 18), (8830, 18), (8832, 18), (9006, 18), (9032, 18), (9066, 18), (9119, 18), (9232, 18), (9273, 18), (9332, 18), (9411, 18), (9427, 18), (9516, 18), (9522, 18), (9534, 18), (9553, 18), (9629, 18), (9695, 18), (9820, 18), (9879, 18), (9904, 18), (9906, 18), (9966, 18), (10101, 18), (10142, 18), (10182, 18), (10192, 18), (10367, 18), (10384, 18), (10438, 18), (10456, 18), (10499, 18), (10536, 18), (10572, 18), (10602, 18), (10644, 18), (10672, 18), (10749, 18), (10846, 18), (11042, 18), (11046, 18), (11050, 18), (11080, 18), (11109, 18), (11113, 18), (11200, 18), (11260, 18), (11322, 18), (11364, 18), (11413, 18), (11422, 18), (11473, 18), (11481, 18), (11507, 18), (11555, 18), (11603, 18), (11625, 18), (11637, 18), (11645, 18), (11664, 18), (11693, 18), (11773, 18), (11801, 18), (11820, 18), (11856, 18), (12012, 18), (12041, 18), (12073, 18), (12134, 18), (12214, 18), (12308, 18), (12341, 18), (12379, 18), (12410, 18), (12512, 18), (12535, 18), (12547, 18), (12573, 18), (12591, 18), (12595, 18), (12731, 18), (68, 19), (80, 19), (97, 19), (137, 19), (158, 19), (220, 19), (285, 19), (314, 19), (320, 19), (331, 19), (343, 19), (356, 19), (404, 19), (407, 19), (447, 19), (491, 19), (495, 19), (534, 19), (576, 19), (624, 19), (649, 19), (663, 19), (700, 19), (766, 19), (781, 19), (788, 19), (881, 19), (884, 19), (913, 19), (938, 19), (946, 19), (1019, 19), (1024, 19), (1047, 19), (1053, 19), (1062, 19), (1124, 19), (1150, 19), (1175, 19), (1216, 19), (1217, 19), (1250, 19), (1269, 19), (1289, 19), (1317, 19), (1318, 19), (1335, 19), (1345, 19), (1356, 19), (1389, 19), (1393, 19), (1409, 19), (1410, 19), (1430, 19), (1447, 19), (1467, 19), (1477, 19), (1527, 19), (1545, 19), (1547, 19), (1615, 19), (1679, 19), (1691, 19), (1702, 19), (1726, 19), (1730, 19), (1753, 19), (1767, 19), (1784, 19), (1787, 19), (1799, 19), (1819, 19), (1870, 19), (1879, 19), (2040, 19), (2095, 19), (2130, 19), (2169, 19), (2175, 19), (2190, 19), (2205, 19), (2214, 19), (2261, 19), (2280, 19), (2287, 19), (2293, 19), (2310, 19), (2338, 19), (2359, 19), (2372, 19), (2407, 19), (2446, 19), (2462, 19), (2470, 19), (2482, 19), (2613, 19), (2638, 19), (2669, 19), (2679, 19), (2715, 19), (2721, 19), (2777, 19), (2795, 19), (2843, 19), (2845, 19), (2875, 19), (2901, 19), (2910, 19), (2983, 19), (3017, 19), (3023, 19), (3069, 19), (3158, 19), (3221, 19), (3262, 19), (3278, 19), (3290, 19), (3298, 19), (3348, 19), (3357, 19), (3360, 19), (3397, 19), (3431, 19), (3456, 19), (3474, 19), (3533, 19), (3561, 19), (3582, 19), (3698, 19), (3719, 19), (3723, 19), (3801, 19), (3828, 19), (3831, 19), (3980, 19), (4108, 19), (4116, 19), (4220, 19), (4266, 19), (4296, 19), (4297, 19), (4320, 19), (4424, 19), (4507, 19), (4515, 19), (4518, 19), (4550, 19), (4587, 19), (4588, 19), (4592, 19), (4622, 19), (4638, 19), (4640, 19), (4660, 19), (4676, 19), (4691, 19), (4696, 19), (4738, 19), (4740, 19), (4754, 19), (4770, 19), (4791, 19), (4793, 19), (4877, 19), (4894, 19), (4901, 19), (4902, 19), (4911, 19), (4927, 19), (4959, 19), (5062, 19), (5094, 19), (5095, 19), (5142, 19), (5145, 19), (5157, 19), (5170, 19), (5174, 19), (5182, 19), (5224, 19), (5385, 19), (5433, 19), (5441, 19), (5504, 19), (5532, 19), (5624, 19), (5715, 19), (5736, 19), (5741, 19), (5779, 19), (5784, 19), (5826, 19), (5829, 19), (5841, 19), (5854, 19), (5946, 19), (5962, 19), (5971, 19), (5987, 19), (6018, 19), (6061, 19), (6086, 19), (6107, 19), (6116, 19), (6180, 19), (6269, 19), (6294, 19), (6301, 19), (6306, 19), (6333, 19), (6352, 19), (6395, 19), (6417, 19), (6418, 19), (6447, 19), (6454, 19), (6463, 19), (6478, 19), (6502, 19), (6517, 19), (6545, 19), (6620, 19), (6644, 19), (6655, 19), (6690, 19), (6731, 19), (6736, 19), (6833, 19), (6845, 19), (6890, 19), (6960, 19), (7058, 19), (7112, 19), (7163, 19), (7178, 19), (7200, 19), (7239, 19), (7256, 19), (7325, 19), (7364, 19), (7375, 19), (7396, 19), (7438, 19), (7447, 19), (7458, 19), (7479, 19), (7513, 19), (7618, 19), (7639, 19), (7708, 19), (7731, 19), (7785, 19), (7819, 19), (7852, 19), (7888, 19), (7951, 19), (7977, 19), (7994, 19), (7995, 19), (8051, 19), (8078, 19), (8079, 19), (8114, 19), (8184, 19), (8191, 19), (8226, 19), (8259, 19), (8268, 19), (8270, 19), (8289, 19), (8299, 19), (8332, 19), (8342, 19), (8348, 19), (8355, 19), (8422, 19), (8442, 19), (8444, 19), (8461, 19), (8539, 19), (8560, 19), (8565, 19), (8570, 19), (8621, 19), (8643, 19), (8655, 19), (8661, 19), (8685, 19), (8734, 19), (8757, 19), (8784, 19), (8855, 19), (8861, 19), (8867, 19), (8891, 19), (8916, 19), (8994, 19), (8998, 19), (8999, 19), (9005, 19), (9007, 19), (9045, 19), (9076, 19), (9115, 19), (9117, 19), (9134, 19), (9150, 19), (9153, 19), (9161, 19), (9187, 19), (9233, 19), (9235, 19), (9245, 19), (9295, 19), (9327, 19), (9331, 19), (9343, 19), (9359, 19), (9367, 19), (9376, 19), (9413, 19), (9416, 19), (9471, 19), (9473, 19), (9497, 19), (9505, 19), (9510, 19), (9618, 19), (9630, 19), (9647, 19), (9688, 19), (9748, 19), (9833, 19), (9867, 19), (9907, 19), (9991, 19), (10000, 19), (10039, 19), (10065, 19), (10084, 19), (10146, 19), (10170, 19), (10176, 19), (10185, 19), (10196, 19), (10253, 19), (10270, 19), (10281, 19), (10291, 19), (10316, 19), (10328, 19), (10349, 19), (10364, 19), (10467, 19), (10493, 19), (10614, 19), (10641, 19), (10646, 19), (10687, 19), (10698, 19), (10721, 19), (10722, 19), (10756, 19), (10811, 19), (10825, 19), (10838, 19), (10859, 19), (10881, 19), (10884, 19), (10938, 19), (10983, 19), (11017, 19), (11043, 19), (11093, 19), (11116, 19), (11120, 19), (11127, 19), (11129, 19), (11163, 19), (11184, 19), (11213, 19), (11219, 19), (11264, 19), (11276, 19), (11326, 19), (11334, 19), (11365, 19), (11385, 19), (11407, 19), (11446, 19), (11460, 19), (11508, 19), (11511, 19), (11515, 19), (11598, 19), (11621, 19), (11636, 19), (11638, 19), (11684, 19), (11689, 19), (11706, 19), (11712, 19), (11719, 19), (11756, 19), (11791, 19), (11802, 19), (11807, 19), (11812, 19), (11815, 19), (11878, 19), (11880, 19), (11908, 19), (11917, 19), (11931, 19), (11985, 19), (12094, 19), (12157, 19), (12169, 19), (12189, 19), (12215, 19), (12244, 19), (12260, 19), (12281, 19), (12285, 19), (12293, 19), (12316, 19), (12319, 19), (12375, 19), (12416, 19), (12425, 19), (12437, 19), (12442, 19), (12481, 19), (12482, 19), (12504, 19), (12539, 19), (12555, 19), (12594, 19), (12601, 19), (12647, 19), (12671, 19), (12693, 19), (12720, 19), (29, 20), (81, 20), (108, 20), (213, 20), (243, 20), (246, 20), (310, 20), (355, 20), (367, 20), (396, 20), (416, 20), (484, 20), (503, 20), (554, 20), (566, 20), (596, 20), (613, 20), (625, 20), (697, 20), (701, 20), (716, 20), (733, 20), (765, 20), (785, 20), (805, 20), (843, 20), (894, 20), (908, 20), (923, 20), (924, 20), (927, 20), (944, 20), (952, 20), (982, 20), (985, 20), (1014, 20), (1032, 20), (1075, 20), (1083, 20), (1094, 20), (1104, 20), (1132, 20), (1165, 20), (1197, 20), (1228, 20), (1240, 20), (1241, 20), (1265, 20), (1302, 20), (1309, 20), (1321, 20), (1341, 20), (1346, 20), (1390, 20), (1418, 20), (1442, 20), (1459, 20), (1476, 20), (1532, 20), (1631, 20), (1651, 20), (1704, 20), (1708, 20), (1715, 20), (1788, 20), (1807, 20), (1821, 20), (1822, 20), (1838, 20), (1856, 20), (1868, 20), (1901, 20), (1902, 20), (1945, 20), (2024, 20), (2031, 20), (2133, 20), (2152, 20), (2173, 20), (2201, 20), (2216, 20), (2243, 20), (2284, 20), (2286, 20), (2337, 20), (2343, 20), (2355, 20), (2389, 20), (2417, 20), (2481, 20), (2483, 20), (2510, 20), (2520, 20), (2550, 20), (2576, 20), (2578, 20), (2579, 20), (2607, 20), (2614, 20), (2644, 20), (2657, 20), (2660, 20), (2667, 20), (2671, 20), (2681, 20), (2683, 20), (2692, 20), (2716, 20), (2738, 20), (2740, 20), (2741, 20), (2742, 20), (2825, 20), (2832, 20), (2851, 20), (2898, 20), (2921, 20), (2929, 20), (2933, 20), (2940, 20), (2944, 20), (2962, 20), (2976, 20), (3015, 20), (3030, 20), (3054, 20), (3105, 20), (3115, 20), (3121, 20), (3122, 20), (3127, 20), (3164, 20), (3169, 20), (3246, 20), (3247, 20), (3269, 20), (3289, 20), (3305, 20), (3329, 20), (3345, 20), (3359, 20), (3366, 20), (3377, 20), (3440, 20), (3450, 20), (3457, 20), (3469, 20), (3479, 20), (3480, 20), (3484, 20), (3491, 20), (3496, 20), (3498, 20), (3500, 20), (3513, 20), (3520, 20), (3588, 20), (3618, 20), (3634, 20), (3647, 20), (3669, 20), (3706, 20), (3725, 20), (3802, 20), (3808, 20), (3838, 20), (3847, 20), (3893, 20), (3920, 20), (3954, 20), (3983, 20), (3996, 20), (4002, 20), (4010, 20), (4018, 20), (4039, 20), (4091, 20), (4100, 20), (4123, 20), (4129, 20), (4151, 20), (4158, 20), (4167, 20), (4190, 20), (4207, 20), (4222, 20), (4228, 20), (4232, 20), (4236, 20), (4243, 20), (4246, 20), (4260, 20), (4265, 20), (4274, 20), (4280, 20), (4311, 20), (4343, 20), (4354, 20), (4363, 20), (4387, 20), (4406, 20), (4430, 20), (4457, 20), (4459, 20), (4488, 20), (4551, 20), (4553, 20), (4560, 20), (4584, 20), (4586, 20), (4604, 20), (4621, 20), (4648, 20), (4680, 20), (4713, 20), (4727, 20), (4731, 20), (4750, 20), (4785, 20), (4808, 20), (4809, 20), (4828, 20), (4840, 20), (4847, 20), (4888, 20), (4909, 20), (4928, 20), (4966, 20), (4968, 20), (5030, 20), (5038, 20), (5050, 20), (5081, 20), (5101, 20), (5175, 20), (5199, 20), (5202, 20), (5211, 20), (5247, 20), (5281, 20), (5288, 20), (5295, 20), (5298, 20), (5306, 20), (5323, 20), (5326, 20), (5345, 20), (5350, 20), (5381, 20), (5401, 20), (5468, 20), (5483, 20), (5524, 20), (5538, 20), (5547, 20), (5557, 20), (5558, 20), (5562, 20), (5567, 20), (5578, 20), (5596, 20), (5610, 20), (5611, 20), (5627, 20), (5645, 20), (5669, 20), (5692, 20), (5698, 20), (5708, 20), (5801, 20), (5803, 20), (5817, 20), (5819, 20), (5847, 20), (5903, 20), (5906, 20), (5910, 20), (6001, 20), (6034, 20), (6039, 20), (6051, 20), (6056, 20), (6064, 20), (6065, 20), (6079, 20), (6102, 20), (6125, 20), (6129, 20), (6152, 20), (6221, 20), (6222, 20), (6234, 20), (6235, 20), (6241, 20), (6250, 20), (6321, 20), (6326, 20), (6365, 20), (6373, 20), (6381, 20), (6402, 20), (6428, 20), (6446, 20), (6482, 20), (6494, 20), (6521, 20), (6530, 20), (6549, 20), (6568, 20), (6570, 20), (6578, 20), (6598, 20), (6643, 20), (6667, 20), (6750, 20), (6778, 20), (6783, 20), (6805, 20), (6818, 20), (6829, 20), (6838, 20), (6871, 20), (6872, 20), (6898, 20), (6909, 20), (6914, 20), (6933, 20), (6945, 20), (6971, 20), (6990, 20), (6995, 20), (7000, 20), (7004, 20), (7010, 20), (7014, 20), (7029, 20), (7030, 20), (7046, 20), (7104, 20), (7196, 20), (7204, 20), (7209, 20), (7236, 20), (7286, 20), (7317, 20), (7337, 20), (7394, 20), (7400, 20), (7412, 20), (7433, 20), (7437, 20), (7483, 20), (7494, 20), (7564, 20), (7574, 20), (7606, 20), (7619, 20), (7665, 20), (7702, 20), (7703, 20), (7714, 20), (7730, 20), (7777, 20), (7780, 20), (7851, 20), (7858, 20), (7860, 20), (7874, 20), (7892, 20), (7993, 20), (8015, 20), (8017, 20), (8030, 20), (8045, 20), (8080, 20), (8161, 20), (8172, 20), (8196, 20), (8202, 20), (8209, 20), (8213, 20), (8237, 20), (8241, 20), (8251, 20), (8260, 20), (8280, 20), (8283, 20), (8328, 20), (8336, 20), (8376, 20), (8418, 20), (8434, 20), (8438, 20), (8468, 20), (8478, 20), (8515, 20), (8556, 20), (8571, 20), (8602, 20), (8615, 20), (8646, 20), (8682, 20), (8708, 20), (8723, 20), (8728, 20), (8754, 20), (8763, 20), (8780, 20), (8781, 20), (8837, 20), (8839, 20), (8858, 20), (8898, 20), (8906, 20), (8912, 20), (8925, 20), (8932, 20), (8940, 20), (8944, 20), (8960, 20), (8969, 20), (8985, 20), (8993, 20), (9038, 20), (9074, 20), (9092, 20), (9096, 20), (9106, 20), (9139, 20), (9166, 20), (9172, 20), (9178, 20), (9199, 20), (9268, 20), (9292, 20), (9302, 20), (9310, 20), (9366, 20), (9374, 20), (9403, 20), (9410, 20), (9435, 20), (9462, 20), (9469, 20), (9499, 20), (9539, 20), (9555, 20), (9564, 20), (9570, 20), (9592, 20), (9594, 20), (9627, 20), (9642, 20), (9670, 20), (9717, 20), (9723, 20), (9733, 20), (9734, 20), (9764, 20), (9768, 20), (9821, 20), (9822, 20), (9829, 20), (9888, 20), (9893, 20), (9900, 20), (9921, 20), (9937, 20), (9951, 20), (9983, 20), (10014, 20), (10079, 20), (10085, 20), (10104, 20), (10122, 20), (10167, 20), (10198, 20), (10226, 20), (10245, 20), (10249, 20), (10260, 20), (10269, 20), (10292, 20), (10297, 20), (10302, 20), (10312, 20), (10333, 20), (10342, 20), (10352, 20), (10397, 20), (10413, 20), (10420, 20), (10432, 20), (10453, 20), (10482, 20), (10550, 20), (10593, 20), (10622, 20), (10638, 20), (10639, 20), (10645, 20), (10719, 20), (10737, 20), (10739, 20), (10754, 20), (10757, 20), (10764, 20), (10841, 20), (10854, 20), (10858, 20), (10863, 20), (10910, 20), (10916, 20), (10921, 20), (10922, 20), (10923, 20), (10977, 20), (11002, 20), (11037, 20), (11051, 20), (11094, 20), (11096, 20), (11103, 20), (11106, 20), (11115, 20), (11160, 20), (11171, 20), (11217, 20), (11235, 20), (11236, 20), (11274, 20), (11277, 20), (11280, 20), (11288, 20), (11296, 20), (11331, 20), (11346, 20), (11382, 20), (11396, 20), (11397, 20), (11398, 20), (11443, 20), (11505, 20), (11541, 20), (11572, 20), (11587, 20), (11593, 20), (11627, 20), (11741, 20), (11753, 20), (11806, 20), (11839, 20), (11867, 20), (11910, 20), (11919, 20), (11956, 20), (11958, 20), (11982, 20), (11995, 20), (12002, 20), (12021, 20), (12024, 20), (12048, 20), (12086, 20), (12099, 20), (12105, 20), (12106, 20), (12109, 20), (12127, 20), (12180, 20), (12185, 20), (12199, 20), (12203, 20), (12229, 20), (12248, 20), (12262, 20), (12356, 20), (12373, 20), (12377, 20), (12397, 20), (12434, 20), (12439, 20), (12554, 20), (12565, 20), (12586, 20), (12641, 20), (12710, 20), (12752, 20), (12763, 20), (12785, 20), (54, 21), (105, 21), (106, 21), (182, 21), (191, 21), (193, 21), (226, 21), (244, 21), (247, 21), (248, 21), (260, 21), (313, 21), (325, 21), (328, 21), (335, 21), (363, 21), (368, 21), (372, 21), (446, 21), (478, 21), (480, 21), (487, 21), (494, 21), (497, 21), (506, 21), (507, 21), (510, 21), (516, 21), (558, 21), (572, 21), (575, 21), (588, 21), (610, 21), (616, 21), (645, 21), (659, 21), (684, 21), (687, 21), (707, 21), (719, 21), (768, 21), (776, 21), (811, 21), (840, 21), (848, 21), (862, 21), (874, 21), (879, 21), (906, 21), (909, 21), (974, 21), (983, 21), (987, 21), (1021, 21), (1044, 21), (1068, 21), (1071, 21), (1082, 21), (1146, 21), (1157, 21), (1221, 21), (1235, 21), (1244, 21), (1299, 21), (1307, 21), (1350, 21), (1394, 21), (1396, 21), (1405, 21), (1438, 21), (1490, 21), (1495, 21), (1503, 21), (1509, 21), (1520, 21), (1538, 21), (1544, 21), (1563, 21), (1564, 21), (1591, 21), (1595, 21), (1611, 21), (1633, 21), (1695, 21), (1698, 21), (1717, 21), (1723, 21), (1757, 21), (1765, 21), (1794, 21), (1872, 21), (1875, 21), (1893, 21), (1940, 21), (1962, 21), (2025, 21), (2039, 21), (2049, 21), (2096, 21), (2124, 21), (2126, 21), (2127, 21), (2158, 21), (2165, 21), (2204, 21), (2213, 21), (2231, 21), (2240, 21), (2253, 21), (2282, 21), (2326, 21), (2336, 21), (2357, 21), (2371, 21), (2441, 21), (2448, 21), (2507, 21), (2517, 21), (2522, 21), (2557, 21), (2566, 21), (2593, 21), (2642, 21), (2646, 21), (2680, 21), (2684, 21), (2705, 21), (2753, 21), (2787, 21), (2803, 21), (2807, 21), (2866, 21), (2916, 21), (2946, 21), (2949, 21), (2960, 21), (2969, 21), (2979, 21), (3001, 21), (3007, 21), (3043, 21), (3053, 21), (3081, 21), (3093, 21), (3133, 21), (3161, 21), (3173, 21), (3180, 21), (3188, 21), (3190, 21), (3230, 21), (3244, 21), (3251, 21), (3254, 21), (3283, 21), (3292, 21), (3333, 21), (3347, 21), (3361, 21), (3386, 21), (3404, 21), (3409, 21), (3420, 21), (3432, 21), (3434, 21), (3501, 21), (3526, 21), (3538, 21), (3565, 21), (3587, 21), (3600, 21), (3607, 21), (3637, 21), (3675, 21), (3686, 21), (3714, 21), (3717, 21), (3721, 21), (3829, 21), (3852, 21), (3891, 21), (3945, 21), (3952, 21), (3992, 21), (4012, 21), (4029, 21), (4038, 21), (4051, 21), (4068, 21), (4074, 21), (4077, 21), (4127, 21), (4131, 21), (4153, 21), (4156, 21), (4170, 21), (4202, 21), (4276, 21), (4289, 21), (4291, 21), (4295, 21), (4356, 21), (4367, 21), (4383, 21), (4407, 21), (4409, 21), (4443, 21), (4454, 21), (4460, 21), (4463, 21), (4465, 21), (4503, 21), (4506, 21), (4524, 21), (4548, 21), (4567, 21), (4625, 21), (4675, 21), (4709, 21), (4848, 21), (4863, 21), (4913, 21), (4920, 21), (4955, 21), (4956, 21), (5058, 21), (5071, 21), (5080, 21), (5084, 21), (5113, 21), (5122, 21), (5127, 21), (5141, 21), (5154, 21), (5200, 21), (5205, 21), (5210, 21), (5223, 21), (5231, 21), (5245, 21), (5261, 21), (5275, 21), (5310, 21), (5327, 21), (5404, 21), (5416, 21), (5431, 21), (5435, 21), (5466, 21), (5492, 21), (5496, 21), (5501, 21), (5503, 21), (5540, 21), (5546, 21), (5574, 21), (5583, 21), (5696, 21), (5700, 21), (5719, 21), (5806, 21), (5884, 21), (5887, 21), (5892, 21), (5904, 21), (5926, 21), (5932, 21), (5950, 21), (5980, 21), (6007, 21), (6075, 21), (6101, 21), (6117, 21), (6164, 21), (6231, 21), (6261, 21), (6262, 21), (6271, 21), (6275, 21), (6299, 21), (6356, 21), (6399, 21), (6416, 21), (6422, 21), (6426, 21), (6458, 21), (6468, 21), (6475, 21), (6491, 21), (6504, 21), (6511, 21), (6529, 21), (6550, 21), (6584, 21), (6596, 21), (6597, 21), (6671, 21), (6685, 21), (6722, 21), (6740, 21), (6754, 21), (6762, 21), (6769, 21), (6784, 21), (6811, 21), (6902, 21), (6969, 21), (6973, 21), (6983, 21), (7041, 21), (7076, 21), (7146, 21), (7247, 21), (7249, 21), (7273, 21), (7303, 21), (7331, 21), (7419, 21), (7459, 21), (7470, 21), (7477, 21), (7504, 21), (7531, 21), (7534, 21), (7556, 21), (7559, 21), (7602, 21), (7607, 21), (7709, 21), (7727, 21), (7767, 21), (7789, 21), (7806, 21), (7843, 21), (7848, 21), (7872, 21), (7915, 21), (7929, 21), (7930, 21), (7940, 21), (7966, 21), (8005, 21), (8041, 21), (8048, 21), (8049, 21), (8058, 21), (8062, 21), (8093, 21), (8108, 21), (8180, 21), (8239, 21), (8279, 21), (8302, 21), (8307, 21), (8315, 21), (8320, 21), (8334, 21), (8359, 21), (8379, 21), (8395, 21), (8445, 21), (8460, 21), (8462, 21), (8470, 21), (8480, 21), (8496, 21), (8509, 21), (8555, 21), (8577, 21), (8675, 21), (8722, 21), (8748, 21), (8759, 21), (8772, 21), (8795, 21), (8810, 21), (8875, 21), (8887, 21), (8897, 21), (8962, 21), (8996, 21), (9002, 21), (9009, 21), (9031, 21), (9037, 21), (9055, 21), (9068, 21), (9078, 21), (9095, 21), (9107, 21), (9140, 21), (9156, 21), (9174, 21), (9185, 21), (9204, 21), (9297, 21), (9333, 21), (9394, 21), (9414, 21), (9419, 21), (9440, 21), (9482, 21), (9493, 21), (9543, 21), (9571, 21), (9583, 21), (9584, 21), (9599, 21), (9613, 21), (9619, 21), (9620, 21), (9648, 21), (9654, 21), (9686, 21), (9710, 21), (9731, 21), (9745, 21), (9749, 21), (9759, 21), (9783, 21), (9794, 21), (9819, 21), (9823, 21), (9828, 21), (9842, 21), (9847, 21), (9878, 21), (9963, 21), (9965, 21), (10002, 21), (10048, 21), (10069, 21), (10083, 21), (10087, 21), (10088, 21), (10132, 21), (10134, 21), (10144, 21), (10179, 21), (10184, 21), (10210, 21), (10220, 21), (10236, 21), (10273, 21), (10288, 21), (10346, 21), (10354, 21), (10361, 21), (10379, 21), (10418, 21), (10422, 21), (10457, 21), (10458, 21), (10492, 21), (10518, 21), (10522, 21), (10553, 21), (10575, 21), (10620, 21), (10650, 21), (10699, 21), (10705, 21), (10712, 21), (10731, 21), (10733, 21), (10780, 21), (10818, 21), (10834, 21), (10896, 21), (10918, 21), (10964, 21), (10979, 21), (11062, 21), (11108, 21), (11112, 21), (11128, 21), (11161, 21), (11180, 21), (11230, 21), (11253, 21), (11279, 21), (11287, 21), (11289, 21), (11376, 21), (11421, 21), (11449, 21), (11458, 21), (11478, 21), (11485, 21), (11491, 21), (11493, 21), (11501, 21), (11532, 21), (11534, 21), (11676, 21), (11745, 21), (11782, 21), (11818, 21), (11821, 21), (11827, 21), (11838, 21), (11966, 21), (11992, 21), (12020, 21), (12080, 21), (12082, 21), (12083, 21), (12159, 21), (12186, 21), (12187, 21), (12219, 21), (12227, 21), (12267, 21), (12335, 21), (12363, 21), (12366, 21), (12376, 21), (12411, 21), (12460, 21), (12461, 21), (12515, 21), (12518, 21), (12525, 21), (12536, 21), (12548, 21), (12549, 21), (12551, 21), (12562, 21), (12564, 21), (12566, 21), (12569, 21), (12592, 21), (12593, 21), (12667, 21), (12672, 21), (12705, 21), (12715, 21), (12733, 21), (12790, 21), (12792, 21), (9, 22), (18, 22), (28, 22), (40, 22), (61, 22), (162, 22), (166, 22), (189, 22), (207, 22), (231, 22), (232, 22), (241, 22), (245, 22), (275, 22), (319, 22), (352, 22), (412, 22), (437, 22), (449, 22), (468, 22), (479, 22), (485, 22), (488, 22), (511, 22), (524, 22), (536, 22), (560, 22), (589, 22), (626, 22), (629, 22), (636, 22), (651, 22), (690, 22), (695, 22), (708, 22), (712, 22), (713, 22), (747, 22), (758, 22), (760, 22), (789, 22), (793, 22), (816, 22), (833, 22), (859, 22), (886, 22), (900, 22), (922, 22), (940, 22), (949, 22), (990, 22), (1004, 22), (1031, 22), (1040, 22), (1096, 22), (1099, 22), (1120, 22), (1125, 22), (1144, 22), (1145, 22), (1147, 22), (1171, 22), (1185, 22), (1188, 22), (1190, 22), (1207, 22), (1225, 22), (1270, 22), (1291, 22), (1308, 22), (1324, 22), (1338, 22), (1351, 22), (1377, 22), (1383, 22), (1387, 22), (1460, 22), (1464, 22), (1471, 22), (1475, 22), (1519, 22), (1555, 22), (1589, 22), (1601, 22), (1619, 22), (1626, 22), (1676, 22), (1703, 22), (1718, 22), (1745, 22), (1758, 22), (1789, 22), (1852, 22), (1853, 22), (1862, 22), (1937, 22), (1943, 22), (1961, 22), (1982, 22), (2013, 22), (2016, 22), (2019, 22), (2021, 22), (2056, 22), (2079, 22), (2115, 22), (2139, 22), (2155, 22), (2192, 22), (2208, 22), (2244, 22), (2260, 22), (2349, 22), (2384, 22), (2420, 22), (2444, 22), (2477, 22), (2508, 22), (2529, 22), (2530, 22), (2603, 22), (2635, 22), (2701, 22), (2723, 22), (2724, 22), (2798, 22), (2831, 22), (2835, 22), (2838, 22), (2853, 22), (2854, 22), (2880, 22), (2895, 22), (2958, 22), (2965, 22), (2974, 22), (2996, 22), (3021, 22), (3046, 22), (3060, 22), (3085, 22), (3087, 22), (3099, 22), (3151, 22), (3159, 22), (3176, 22), (3186, 22), (3217, 22), (3224, 22), (3255, 22), (3258, 22), (3264, 22), (3272, 22), (3286, 22), (3337, 22), (3388, 22), (3399, 22), (3419, 22), (3427, 22), (3435, 22), (3443, 22), (3481, 22), (3495, 22), (3503, 22), (3509, 22), (3511, 22), (3512, 22), (3524, 22), (3562, 22), (3571, 22), (3574, 22), (3590, 22), (3594, 22), (3603, 22), (3624, 22), (3633, 22), (3641, 22), (3643, 22), (3658, 22), (3680, 22), (3687, 22), (3690, 22), (3691, 22), (3711, 22), (3737, 22), (3741, 22), (3763, 22), (3771, 22), (3825, 22), (3837, 22), (3869, 22), (3878, 22), (4030, 22), (4118, 22), (4143, 22), (4157, 22), (4169, 22), (4174, 22), (4183, 22), (4206, 22), (4251, 22), (4254, 22), (4344, 22), (4366, 22), (4382, 22), (4405, 22), (4474, 22), (4486, 22), (4525, 22), (4563, 22), (4599, 22), (4605, 22), (4626, 22), (4628, 22), (4641, 22), (4669, 22), (4687, 22), (4694, 22), (4699, 22), (4734, 22), (4767, 22), (4834, 22), (4845, 22), (4869, 22), (4952, 22), (4974, 22), (4981, 22), (4995, 22), (4997, 22), (5077, 22), (5082, 22), (5089, 22), (5093, 22), (5129, 22), (5131, 22), (5158, 22), (5230, 22), (5244, 22), (5259, 22), (5267, 22), (5270, 22), (5315, 22), (5349, 22), (5356, 22), (5359, 22), (5371, 22), (5406, 22), (5419, 22), (5422, 22), (5443, 22), (5453, 22), (5463, 22), (5471, 22), (5554, 22), (5650, 22), (5659, 22), (5697, 22), (5705, 22), (5762, 22), (5766, 22), (5767, 22), (5800, 22), (5808, 22), (5816, 22), (5843, 22), (5846, 22), (5852, 22), (5899, 22), (5911, 22), (5923, 22), (5945, 22), (5958, 22), (5990, 22), (6016, 22), (6092, 22), (6130, 22), (6134, 22), (6139, 22), (6141, 22), (6163, 22), (6246, 22), (6254, 22), (6270, 22), (6309, 22), (6350, 22), (6367, 22), (6371, 22), (6405, 22), (6440, 22), (6457, 22), (6466, 22), (6472, 22), (6477, 22), (6501, 22), (6525, 22), (6542, 22), (6586, 22), (6595, 22), (6652, 22), (6693, 22), (6698, 22), (6703, 22), (6704, 22), (6711, 22), (6757, 22), (6763, 22), (6764, 22), (6775, 22), (6779, 22), (6797, 22), (6832, 22), (6837, 22), (6864, 22), (6876, 22), (6892, 22), (6942, 22), (6957, 22), (6965, 22), (6985, 22), (7043, 22), (7065, 22), (7085, 22), (7090, 22), (7113, 22), (7123, 22), (7131, 22), (7153, 22), (7154, 22), (7155, 22), (7169, 22), (7171, 22), (7191, 22), (7215, 22), (7220, 22), (7230, 22), (7233, 22), (7241, 22), (7266, 22), (7292, 22), (7296, 22), (7327, 22), (7370, 22), (7389, 22), (7409, 22), (7430, 22), (7443, 22), (7501, 22), (7525, 22), (7666, 22), (7700, 22), (7715, 22), (7721, 22), (7725, 22), (7748, 22), (7757, 22), (7849, 22), (7886, 22), (7894, 22), (7927, 22), (7948, 22), (7956, 22), (7967, 22), (7990, 22), (8014, 22), (8038, 22), (8050, 22), (8057, 22), (8113, 22), (8121, 22), (8154, 22), (8167, 22), (8176, 22), (8216, 22), (8244, 22), (8269, 22), (8273, 22), (8287, 22), (8292, 22), (8312, 22), (8314, 22), (8331, 22), (8338, 22), (8377, 22), (8381, 22), (8474, 22), (8476, 22), (8486, 22), (8501, 22), (8503, 22), (8523, 22), (8548, 22), (8588, 22), (8592, 22), (8596, 22), (8625, 22), (8697, 22), (8706, 22), (8764, 22), (8778, 22), (8801, 22), (8846, 22), (8865, 22), (8892, 22), (8901, 22), (8926, 22), (8947, 22), (8950, 22), (8983, 22), (9020, 22), (9024, 22), (9035, 22), (9105, 22), (9122, 22), (9125, 22), (9131, 22), (9142, 22), (9165, 22), (9167, 22), (9168, 22), (9181, 22), (9201, 22), (9206, 22), (9215, 22), (9223, 22), (9241, 22), (9287, 22), (9313, 22), (9424, 22), (9459, 22), (9461, 22), (9472, 22), (9605, 22), (9637, 22), (9644, 22), (9659, 22), (9666, 22), (9697, 22), (9716, 22), (9719, 22), (9754, 22), (9792, 22), (9865, 22), (9869, 22), (9932, 22), (9935, 22), (9971, 22), (9995, 22), (9996, 22), (10003, 22), (10053, 22), (10073, 22), (10114, 22), (10133, 22), (10158, 22), (10177, 22), (10178, 22), (10181, 22), (10222, 22), (10229, 22), (10274, 22), (10278, 22), (10303, 22), (10309, 22), (10310, 22), (10315, 22), (10356, 22), (10365, 22), (10368, 22), (10404, 22), (10406, 22), (10424, 22), (10440, 22), (10451, 22), (10469, 22), (10475, 22), (10479, 22), (10531, 22), (10562, 22), (10577, 22), (10582, 22), (10648, 22), (10657, 22), (10675, 22), (10702, 22), (10711, 22), (10720, 22), (10726, 22), (10728, 22), (10762, 22), (10790, 22), (10819, 22), (10829, 22), (10879, 22), (10893, 22), (10915, 22), (10934, 22), (10950, 22), (10955, 22), (10967, 22), (10994, 22), (11003, 22), (11016, 22), (11022, 22), (11031, 22), (11061, 22), (11069, 22), (11097, 22), (11111, 22), (11118, 22), (11132, 22), (11196, 22), (11199, 22), (11206, 22), (11209, 22), (11223, 22), (11255, 22), (11257, 22), (11309, 22), (11311, 22), (11312, 22), (11345, 22), (11392, 22), (11404, 22), (11432, 22), (11439, 22), (11492, 22), (11503, 22), (11533, 22), (11546, 22), (11551, 22), (11556, 22), (11559, 22), (11583, 22), (11586, 22), (11596, 22), (11619, 22), (11641, 22), (11665, 22), (11755, 22), (11760, 22), (11765, 22), (11817, 22), (11831, 22), (11835, 22), (11852, 22), (11857, 22), (11862, 22), (11876, 22), (11882, 22), (11883, 22), (11906, 22), (11915, 22), (11930, 22), (11939, 22), (12007, 22), (12018, 22), (12037, 22), (12070, 22), (12089, 22), (12101, 22), (12132, 22), (12197, 22), (12205, 22), (12231, 22), (12282, 22), (12309, 22), (12317, 22), (12323, 22), (12372, 22), (12403, 22), (12413, 22), (12418, 22), (12427, 22), (12470, 22), (12477, 22), (12633, 22), (12658, 22), (12664, 22), (12697, 22), (12702, 22), (12749, 22), (12768, 22), (12795, 22), (39, 23), (53, 23), (56, 23), (75, 23), (82, 23), (129, 23), (138, 23), (164, 23), (198, 23), (254, 23), (259, 23), (267, 23), (316, 23), (337, 23), (339, 23), (341, 23), (380, 23), (388, 23), (390, 23), (394, 23), (403, 23), (417, 23), (444, 23), (525, 23), (526, 23), (530, 23), (581, 23), (584, 23), (666, 23), (674, 23), (679, 23), (692, 23), (699, 23), (705, 23), (726, 23), (763, 23), (782, 23), (787, 23), (806, 23), (808, 23), (812, 23), (821, 23), (826, 23), (841, 23), (846, 23), (876, 23), (898, 23), (915, 23), (935, 23), (954, 23), (970, 23), (1000, 23), (1013, 23), (1073, 23), (1089, 23), (1092, 23), (1111, 23), (1135, 23), (1141, 23), (1163, 23), (1168, 23), (1169, 23), (1202, 23), (1230, 23), (1286, 23), (1319, 23), (1352, 23), (1359, 23), (1367, 23), (1378, 23), (1419, 23), (1433, 23), (1470, 23), (1473, 23), (1492, 23), (1504, 23), (1534, 23), (1542, 23), (1558, 23), (1572, 23), (1634, 23), (1641, 23), (1665, 23), (1716, 23), (1722, 23), (1729, 23), (1732, 23), (1771, 23), (1776, 23), (1782, 23), (1803, 23), (1828, 23), (1833, 23), (1881, 23), (1883, 23), (1891, 23), (1900, 23), (1916, 23), (1918, 23), (1930, 23), (1934, 23), (1948, 23), (1959, 23), (1977, 23), (1983, 23), (2023, 23), (2027, 23), (2066, 23), (2067, 23), (2114, 23), (2116, 23), (2131, 23), (2147, 23), (2163, 23), (2198, 23), (2199, 23), (2224, 23), (2248, 23), (2267, 23), (2348, 23), (2353, 23), (2364, 23), (2376, 23), (2380, 23), (2436, 23), (2451, 23), (2452, 23), (2463, 23), (2476, 23), (2484, 23), (2509, 23), (2513, 23), (2538, 23), (2560, 23), (2571, 23), (2575, 23), (2595, 23), (2615, 23), (2619, 23), (2639, 23), (2655, 23), (2670, 23), (2706, 23), (2750, 23), (2772, 23), (2833, 23), (2857, 23), (2862, 23), (2869, 23), (2873, 23), (2874, 23), (2876, 23), (2878, 23), (2890, 23), (2891, 23), (2924, 23), (2963, 23), (2980, 23), (2992, 23), (2997, 23), (3019, 23), (3041, 23), (3045, 23), (3055, 23), (3064, 23), (3089, 23), (3103, 23), (3104, 23), (3128, 23), (3206, 23), (3211, 23), (3214, 23), (3231, 23), (3239, 23), (3265, 23), (3279, 23), (3295, 23), (3297, 23), (3299, 23), (3320, 23), (3335, 23), (3344, 23), (3349, 23), (3356, 23), (3369, 23), (3393, 23), (3396, 23), (3444, 23), (3454, 23), (3493, 23), (3494, 23), (3542, 23), (3553, 23), (3567, 23), (3568, 23), (3595, 23), (3601, 23), (3612, 23), (3623, 23), (3626, 23), (3627, 23), (3630, 23), (3644, 23), (3654, 23), (3700, 23), (3704, 23), (3724, 23), (3733, 23), (3745, 23), (3753, 23), (3759, 23), (3760, 23), (3788, 23), (3807, 23), (3816, 23), (3846, 23), (3874, 23), (3884, 23), (3903, 23), (3919, 23), (3966, 23), (3972, 23), (4011, 23), (4107, 23), (4115, 23), (4126, 23), (4140, 23), (4146, 23), (4150, 23), (4185, 23), (4204, 23), (4225, 23), (4227, 23), (4241, 23), (4256, 23), (4269, 23), (4286, 23), (4293, 23), (4298, 23), (4302, 23), (4315, 23), (4323, 23), (4327, 23), (4340, 23), (4362, 23), (4375, 23), (4384, 23), (4416, 23), (4426, 23), (4442, 23), (4444, 23), (4453, 23), (4512, 23), (4530, 23), (4536, 23), (4568, 23), (4569, 23), (4572, 23), (4576, 23), (4623, 23), (4627, 23), (4637, 23), (4654, 23), (4663, 23), (4679, 23), (4686, 23), (4703, 23), (4715, 23), (4718, 23), (4729, 23), (4736, 23), (4743, 23), (4759, 23), (4765, 23), (4777, 23), (4820, 23), (4843, 23), (4851, 23), (4852, 23), (4856, 23), (4870, 23), (4876, 23), (4895, 23), (4905, 23), (4937, 23), (4947, 23), (4949, 23), (4984, 23), (5031, 23), (5045, 23), (5057, 23), (5086, 23), (5099, 23), (5100, 23), (5138, 23), (5152, 23), (5168, 23), (5183, 23), (5222, 23), (5342, 23), (5343, 23), (5388, 23), (5389, 23), (5391, 23), (5403, 23), (5420, 23), (5429, 23), (5480, 23), (5507, 23), (5508, 23), (5550, 23), (5555, 23), (5563, 23), (5607, 23), (5621, 23), (5656, 23), (5661, 23), (5666, 23), (5711, 23), (5747, 23), (5749, 23), (5787, 23), (5809, 23), (5814, 23), (5822, 23), (5835, 23), (5844, 23), (5863, 23), (5880, 23), (5894, 23), (5898, 23), (5912, 23), (5919, 23), (5930, 23), (5941, 23), (5942, 23), (5949, 23), (5984, 23), (5985, 23), (6027, 23), (6044, 23), (6046, 23), (6083, 23), (6177, 23), (6181, 23), (6203, 23), (6215, 23), (6216, 23), (6217, 23), (6223, 23), (6226, 23), (6230, 23), (6237, 23), (6243, 23), (6258, 23), (6263, 23), (6295, 23), (6324, 23), (6357, 23), (6358, 23), (6362, 23), (6363, 23), (6369, 23), (6372, 23), (6374, 23), (6377, 23), (6400, 23), (6406, 23), (6411, 23), (6445, 23), (6496, 23), (6513, 23), (6573, 23), (6574, 23), (6576, 23), (6599, 23), (6625, 23), (6634, 23), (6642, 23), (6647, 23), (6654, 23), (6692, 23), (6727, 23), (6739, 23), (6745, 23), (6748, 23), (6799, 23), (6804, 23), (6846, 23), (6849, 23), (6858, 23), (6915, 23), (6947, 23), (6949, 23), (6963, 23), (6976, 23), (6989, 23), (7006, 23), (7015, 23), (7016, 23), (7044, 23), (7049, 23), (7059, 23), (7069, 23), (7089, 23), (7103, 23), (7108, 23), (7133, 23), (7160, 23), (7161, 23), (7173, 23), (7207, 23), (7214, 23), (7222, 23), (7224, 23), (7225, 23), (7268, 23), (7293, 23), (7294, 23), (7308, 23), (7330, 23), (7398, 23), (7399, 23), (7401, 23), (7444, 23), (7445, 23), (7463, 23), (7467, 23), (7480, 23), (7485, 23), (7488, 23), (7502, 23), (7522, 23), (7524, 23), (7529, 23), (7540, 23), (7597, 23), (7612, 23), (7623, 23), (7652, 23), (7655, 23), (7668, 23), (7675, 23), (7684, 23), (7692, 23), (7696, 23), (7707, 23), (7717, 23), (7747, 23), (7752, 23), (7760, 23), (7765, 23), (7778, 23), (7782, 23), (7818, 23), (7823, 23), (7827, 23), (7865, 23), (7877, 23), (7883, 23), (7887, 23), (7912, 23), (7919, 23), (7920, 23), (7958, 23), (7980, 23), (7982, 23), (7998, 23), (8018, 23), (8028, 23), (8032, 23), (8042, 23), (8060, 23), (8100, 23), (8127, 23), (8128, 23), (8144, 23), (8147, 23), (8165, 23), (8185, 23), (8198, 23), (8207, 23), (8222, 23), (8229, 23), (8249, 23), (8278, 23), (8324, 23), (8364, 23), (8386, 23), (8403, 23), (8408, 23), (8426, 23), (8433, 23), (8457, 23), (8464, 23), (8489, 23), (8516, 23), (8530, 23), (8538, 23), (8552, 23), (8559, 23), (8566, 23), (8575, 23), (8590, 23), (8604, 23), (8611, 23), (8623, 23), (8652, 23), (8653, 23), (8660, 23), (8666, 23), (8729, 23), (8753, 23), (8786, 23), (8791, 23), (8796, 23), (8806, 23), (8821, 23), (8857, 23), (8868, 23), (8909, 23), (8936, 23), (8937, 23), (9027, 23), (9113, 23), (9145, 23), (9151, 23), (9177, 23), (9198, 23), (9208, 23), (9216, 23), (9221, 23), (9239, 23), (9270, 23), (9274, 23), (9294, 23), (9300, 23), (9329, 23), (9337, 23), (9356, 23), (9373, 23), (9408, 23), (9445, 23), (9451, 23), (9484, 23), (9492, 23), (9498, 23), (9504, 23), (9558, 23), (9561, 23), (9573, 23), (9576, 23), (9610, 23), (9645, 23), (9660, 23), (9664, 23), (9698, 23), (9699, 23), (9709, 23), (9722, 23), (9765, 23), (9775, 23), (9805, 23), (9846, 23), (9886, 23), (9887, 23), (9889, 23), (9913, 23), (9922, 23), (9928, 23), (9950, 23), (9957, 23), (9960, 23), (9962, 23), (9985, 23), (10068, 23), (10077, 23), (10100, 23), (10102, 23), (10118, 23), (10129, 23), (10137, 23), (10203, 23), (10268, 23), (10294, 23), (10301, 23), (10391, 23), (10411, 23), (10434, 23), (10480, 23), (10503, 23), (10530, 23), (10532, 23), (10535, 23), (10542, 23), (10551, 23), (10554, 23), (10560, 23), (10585, 23), (10597, 23), (10606, 23), (10625, 23), (10662, 23), (10684, 23), (10710, 23), (10723, 23), (10735, 23), (10748, 23), (10767, 23), (10777, 23), (10795, 23), (10803, 23), (10816, 23), (10824, 23), (10853, 23), (10857, 23), (10887, 23), (10890, 23), (10956, 23), (10984, 23), (10991, 23), (10998, 23), (11028, 23), (11047, 23), (11073, 23), (11110, 23), (11179, 23), (11182, 23), (11191, 23), (11201, 23), (11221, 23), (11270, 23), (11271, 23), (11282, 23), (11291, 23), (11298, 23), (11337, 23), (11352, 23), (11416, 23), (11461, 23), (11463, 23), (11479, 23), (11490, 23), (11496, 23), (11510, 23), (11526, 23), (11528, 23), (11530, 23), (11553, 23), (11584, 23), (11585, 23), (11601, 23), (11611, 23), (11620, 23), (11630, 23), (11642, 23), (11654, 23), (11662, 23), (11710, 23), (11716, 23), (11729, 23), (11740, 23), (11766, 23), (11823, 23), (11825, 23), (11830, 23), (11836, 23), (11846, 23), (11877, 23), (11884, 23), (11886, 23), (11888, 23), (11920, 23), (11949, 23), (11950, 23), (11964, 23), (11973, 23), (11979, 23), (11989, 23), (12010, 23), (12026, 23), (12029, 23), (12034, 23), (12056, 23), (12090, 23), (12114, 23), (12123, 23), (12129, 23), (12151, 23), (12163, 23), (12208, 23), (12233, 23), (12242, 23), (12255, 23), (12277, 23), (12289, 23), (12307, 23), (12334, 23), (12349, 23), (12351, 23), (12352, 23), (12378, 23), (12404, 23), (12443, 23), (12447, 23), (12462, 23), (12474, 23), (12483, 23), (12487, 23), (12510, 23), (12533, 23), (12576, 23), (12599, 23), (12604, 23), (12610, 23), (12614, 23), (12638, 23), (12656, 23), (12662, 23), (12700, 23), (12701, 23), (12704, 23), (12706, 23), (12738, 23), (12744, 23), (12755, 23), (12756, 23), (12761, 23), (12767, 23), (12772, 23), (12783, 23), (27, 24), (37, 24), (50, 24), (85, 24), (89, 24), (114, 24), (118, 24), (123, 24), (142, 24), (148, 24), (149, 24), (163, 24), (258, 24), (269, 24), (299, 24), (306, 24), (311, 24), (362, 24), (366, 24), (421, 24), (422, 24), (453, 24), (475, 24), (486, 24), (493, 24), (499, 24), (512, 24), (541, 24), (564, 24), (573, 24), (593, 24), (604, 24), (615, 24), (639, 24), (640, 24), (643, 24), (653, 24), (658, 24), (660, 24), (662, 24), (680, 24), (727, 24), (730, 24), (734, 24), (740, 24), (790, 24), (830, 24), (852, 24), (853, 24), (854, 24), (888, 24), (901, 24), (930, 24), (951, 24), (973, 24), (994, 24), (1007, 24), (1015, 24), (1017, 24), (1030, 24), (1046, 24), (1065, 24), (1067, 24), (1079, 24), (1121, 24), (1123, 24), (1133, 24), (1158, 24), (1183, 24), (1189, 24), (1191, 24), (1195, 24), (1206, 24), (1224, 24), (1255, 24), (1305, 24), (1315, 24), (1327, 24), (1332, 24), (1343, 24), (1364, 24), (1398, 24), (1411, 24), (1412, 24), (1413, 24), (1414, 24), (1420, 24), (1431, 24), (1455, 24), (1457, 24), (1472, 24), (1526, 24), (1540, 24), (1554, 24), (1569, 24), (1604, 24), (1635, 24), (1660, 24), (1675, 24), (1701, 24), (1705, 24), (1706, 24), (1727, 24), (1731, 24), (1741, 24), (1746, 24), (1751, 24), (1825, 24), (1835, 24), (1839, 24), (1857, 24), (1880, 24), (1886, 24), (1903, 24), (1993, 24), (1995, 24), (2004, 24), (2011, 24), (2065, 24), (2068, 24), (2080, 24), (2100, 24), (2110, 24), (2112, 24), (2117, 24), (2123, 24), (2135, 24), (2137, 24), (2143, 24), (2161, 24), (2178, 24), (2194, 24), (2277, 24), (2281, 24), (2292, 24), (2300, 24), (2301, 24), (2375, 24), (2403, 24), (2408, 24), (2418, 24), (2440, 24), (2472, 24), (2479, 24), (2493, 24), (2503, 24), (2519, 24), (2521, 24), (2567, 24), (2569, 24), (2617, 24), (2618, 24), (2625, 24), (2628, 24), (2643, 24), (2666, 24), (2675, 24), (2682, 24), (2700, 24), (2707, 24), (2770, 24), (2774, 24), (2781, 24), (2817, 24), (2818, 24), (2844, 24), (2846, 24), (2850, 24), (2856, 24), (2865, 24), (2872, 24), (2889, 24), (2911, 24), (2925, 24), (2936, 24), (2943, 24), (2954, 24), (2982, 24), (2984, 24), (2998, 24), (3008, 24), (3057, 24), (3072, 24), (3080, 24), (3090, 24), (3091, 24), (3101, 24), (3109, 24), (3110, 24), (3118, 24), (3137, 24), (3141, 24), (3165, 24), (3172, 24), (3212, 24), (3219, 24), (3225, 24), (3235, 24), (3249, 24), (3263, 24), (3293, 24), (3302, 24), (3317, 24), (3334, 24), (3339, 24), (3378, 24), (3382, 24), (3389, 24), (3402, 24), (3410, 24), (3417, 24), (3418, 24), (3421, 24), (3462, 24), (3470, 24), (3492, 24), (3499, 24), (3516, 24), (3532, 24), (3545, 24), (3549, 24), (3593, 24), (3609, 24), (3650, 24), (3651, 24), (3664, 24), (3676, 24), (3694, 24), (3727, 24), (3748, 24), (3755, 24), (3762, 24), (3772, 24), (3773, 24), (3779, 24), (3792, 24), (3804, 24), (3858, 24), (3861, 24), (3865, 24), (3880, 24), (3881, 24), (3889, 24), (3890, 24), (3922, 24), (3932, 24), (3934, 24), (3937, 24), (3947, 24), (3963, 24), (3964, 24), (3977, 24), (3993, 24), (3995, 24), (4000, 24), (4003, 24), (4034, 24), (4057, 24), (4064, 24), (4083, 24), (4087, 24), (4096, 24), (4117, 24), (4142, 24), (4160, 24), (4193, 24), (4208, 24), (4209, 24), (4231, 24), (4244, 24), (4282, 24), (4308, 24), (4310, 24), (4314, 24), (4324, 24), (4328, 24), (4342, 24), (4345, 24), (4358, 24), (4370, 24), (4372, 24), (4378, 24), (4389, 24), (4396, 24), (4421, 24), (4428, 24), (4441, 24), (4450, 24), (4451, 24), (4456, 24), (4483, 24), (4494, 24), (4496, 24), (4497, 24), (4500, 24), (4559, 24), (4578, 24), (4583, 24), (4585, 24), (4591, 24), (4594, 24), (4598, 24), (4607, 24), (4616, 24), (4618, 24), (4673, 24), (4698, 24), (4700, 24), (4708, 24), (4737, 24), (4757, 24), (4773, 24), (4789, 24), (4796, 24), (4799, 24), (4802, 24), (4804, 24), (4832, 24), (4859, 24), (4878, 24), (4887, 24), (4897, 24), (4907, 24), (4943, 24), (4948, 24), (4953, 24), (4960, 24), (4965, 24), (4971, 24), (4985, 24), (4993, 24), (4996, 24), (5003, 24), (5011, 24), (5012, 24), (5036, 24), (5046, 24), (5070, 24), (5079, 24), (5147, 24), (5159, 24), (5163, 24), (5169, 24), (5212, 24), (5219, 24), (5241, 24), (5252, 24), (5258, 24), (5262, 24), (5272, 24), (5279, 24), (5311, 24), (5325, 24), (5341, 24), (5358, 24), (5363, 24), (5365, 24), (5368, 24), (5399, 24), (5400, 24), (5418, 24), (5430, 24), (5446, 24), (5465, 24), (5474, 24), (5513, 24), (5577, 24), (5592, 24), (5601, 24), (5612, 24), (5615, 24), (5623, 24), (5639, 24), (5643, 24), (5670, 24), (5689, 24), (5704, 24), (5706, 24), (5716, 24), (5728, 24), (5729, 24), (5738, 24), (5746, 24), (5752, 24), (5757, 24), (5768, 24), (5769, 24), (5771, 24), (5778, 24), (5799, 24), (5824, 24), (5831, 24), (5840, 24), (5848, 24), (5878, 24), (5890, 24), (5931, 24), (5947, 24), (5955, 24), (5968, 24), (5974, 24), (6011, 24), (6017, 24), (6069, 24), (6103, 24), (6113, 24), (6131, 24), (6135, 24), (6161, 24), (6185, 24), (6195, 24), (6199, 24), (6214, 24), (6227, 24), (6248, 24), (6255, 24), (6267, 24), (6290, 24), (6325, 24), (6345, 24), (6348, 24), (6349, 24), (6353, 24), (6368, 24), (6423, 24), (6424, 24), (6439, 24), (6442, 24), (6470, 24), (6483, 24), (6486, 24), (6500, 24), (6510, 24), (6520, 24), (6523, 24), (6548, 24), (6563, 24), (6567, 24), (6575, 24), (6603, 24), (6610, 24), (6611, 24), (6623, 24), (6630, 24), (6635, 24), (6641, 24), (6645, 24), (6651, 24), (6674, 24), (6699, 24), (6717, 24), (6718, 24), (6723, 24), (6725, 24), (6728, 24), (6734, 24), (6742, 24), (6749, 24), (6761, 24), (6786, 24), (6816, 24), (6851, 24), (6899, 24), (6906, 24), (6907, 24), (6913, 24), (6917, 24), (6923, 24), (6928, 24), (6943, 24), (6998, 24), (7002, 24), (7013, 24), (7017, 24), (7025, 24), (7036, 24), (7045, 24), (7053, 24), (7077, 24), (7083, 24), (7134, 24), (7143, 24), (7152, 24), (7165, 24), (7182, 24), (7187, 24), (7193, 24), (7194, 24), (7226, 24), (7235, 24), (7237, 24), (7243, 24), (7262, 24), (7264, 24), (7280, 24), (7284, 24), (7341, 24), (7345, 24), (7352, 24), (7357, 24), (7360, 24), (7366, 24), (7404, 24), (7422, 24), (7429, 24), (7432, 24), (7461, 24), (7476, 24), (7486, 24), (7495, 24), (7499, 24), (7503, 24), (7518, 24), (7542, 24), (7552, 24), (7579, 24), (7585, 24), (7586, 24), (7635, 24), (7640, 24), (7641, 24), (7661, 24), (7687, 24), (7710, 24), (7734, 24), (7770, 24), (7779, 24), (7805, 24), (7809, 24), (7820, 24), (7840, 24), (7841, 24), (7881, 24), (7917, 24), (7937, 24), (7939, 24), (7957, 24), (7964, 24), (7985, 24), (7991, 24), (8004, 24), (8020, 24), (8043, 24), (8074, 24), (8089, 24), (8095, 24), (8107, 24), (8115, 24), (8143, 24), (8150, 24), (8171, 24), (8183, 24), (8197, 24), (8258, 24), (8290, 24), (8294, 24), (8305, 24), (8306, 24), (8346, 24), (8366, 24), (8369, 24), (8372, 24), (8391, 24), (8398, 24), (8443, 24), (8537, 24), (8547, 24), (8617, 24), (8657, 24), (8667, 24), (8668, 24), (8674, 24), (8683, 24), (8705, 24), (8733, 24), (8774, 24), (8783, 24), (8798, 24), (8820, 24), (8823, 24), (8860, 24), (8869, 24), (8876, 24), (8882, 24), (8903, 24), (8907, 24), (8910, 24), (8922, 24), (8933, 24), (8946, 24), (8954, 24), (8963, 24), (8974, 24), (8987, 24), (8988, 24), (8997, 24), (9036, 24), (9083, 24), (9108, 24), (9120, 24), (9123, 24), (9136, 24), (9144, 24), (9189, 24), (9197, 24), (9227, 24), (9248, 24), (9314, 24), (9335, 24), (9340, 24), (9341, 24), (9360, 24), (9363, 24), (9378, 24), (9388, 24), (9398, 24), (9433, 24), (9441, 24), (9463, 24), (9464, 24), (9470, 24), (9475, 24), (9483, 24), (9486, 24), (9515, 24), (9520, 24), (9527, 24), (9550, 24), (9578, 24), (9603, 24), (9604, 24), (9650, 24), (9680, 24), (9712, 24), (9725, 24), (9735, 24), (9743, 24), (9756, 24), (9771, 24), (9772, 24), (9779, 24), (9788, 24), (9804, 24), (9813, 24), (9814, 24), (9832, 24), (9850, 24), (9874, 24), (9881, 24), (9883, 24), (9901, 24), (9923, 24), (9930, 24), (9980, 24), (9999, 24), (10011, 24), (10020, 24), (10034, 24), (10059, 24), (10080, 24), (10116, 24), (10117, 24), (10141, 24), (10161, 24), (10162, 24), (10165, 24), (10183, 24), (10187, 24), (10189, 24), (10200, 24), (10215, 24), (10227, 24), (10232, 24), (10280, 24), (10284, 24), (10355, 24), (10358, 24), (10366, 24), (10374, 24), (10383, 24), (10389, 24), (10405, 24), (10427, 24), (10437, 24), (10455, 24), (10463, 24), (10464, 24), (10465, 24), (10477, 24), (10500, 24), (10501, 24), (10502, 24), (10515, 24), (10534, 24), (10541, 24), (10564, 24), (10574, 24), (10610, 24), (10617, 24), (10624, 24), (10628, 24), (10630, 24), (10676, 24), (10680, 24), (10688, 24), (10695, 24), (10766, 24), (10789, 24), (10806, 24), (10823, 24), (10831, 24), (10843, 24), (10845, 24), (10870, 24), (10872, 24), (10902, 24), (10960, 24), (10973, 24), (10982, 24), (10986, 24), (11010, 24), (11023, 24), (11029, 24), (11064, 24), (11166, 24), (11174, 24), (11187, 24), (11229, 24), (11250, 24), (11254, 24), (11263, 24), (11269, 24), (11273, 24), (11293, 24), (11305, 24), (11381, 24), (11395, 24), (11406, 24), (11419, 24), (11431, 24), (11433, 24), (11453, 24), (11455, 24), (11470, 24), (11480, 24), (11494, 24), (11527, 24), (11560, 24), (11563, 24), (11570, 24), (11640, 24), (11682, 24), (11685, 24), (11692, 24), (11694, 24), (11697, 24), (11726, 24), (11727, 24), (11750, 24), (11757, 24), (11759, 24), (11774, 24), (11783, 24), (11789, 24), (11798, 24), (11799, 24), (11832, 24), (11834, 24), (11881, 24), (11891, 24), (11953, 24), (11961, 24), (11965, 24), (11972, 24), (12015, 24), (12049, 24), (12063, 24), (12069, 24), (12075, 24), (12149, 24), (12158, 24), (12183, 24), (12193, 24), (12206, 24), (12257, 24), (12275, 24), (12284, 24), (12298, 24), (12303, 24), (12304, 24), (12320, 24), (12390, 24), (12399, 24), (12407, 24), (12408, 24), (12421, 24), (12452, 24), (12480, 24), (12495, 24), (12501, 24), (12509, 24), (12552, 24), (12559, 24), (12561, 24), (12571, 24), (12660, 24), (12685, 24), (12689, 24), (12699, 24), (12722, 24), (12725, 24), (12732, 24), (12737, 24), (12750, 24), (12751, 24), (12777, 24), (12786, 24), (6, 25), (10, 25), (17, 25), (30, 25), (47, 25), (52, 25), (93, 25), (119, 25), (180, 25), (183, 25), (194, 25), (197, 25), (200, 25), (208, 25), (261, 25), (264, 25), (347, 25), (357, 25), (384, 25), (409, 25), (411, 25), (419, 25), (436, 25), (438, 25), (439, 25), (440, 25), (448, 25), (459, 25), (462, 25), (463, 25), (467, 25), (469, 25), (501, 25), (502, 25), (552, 25), (598, 25), (621, 25), (627, 25), (634, 25), (667, 25), (668, 25), (675, 25), (681, 25), (711, 25), (714, 25), (783, 25), (791, 25), (818, 25), (838, 25), (887, 25), (936, 25), (942, 25), (947, 25), (948, 25), (971, 25), (1003, 25), (1045, 25), (1066, 25), (1100, 25), (1102, 25), (1116, 25), (1119, 25), (1153, 25), (1154, 25), (1194, 25), (1251, 25), (1253, 25), (1287, 25), (1288, 25), (1298, 25), (1300, 25), (1301, 25), (1355, 25), (1365, 25), (1375, 25), (1388, 25), (1391, 25), (1422, 25), (1452, 25), (1481, 25), (1561, 25), (1565, 25), (1577, 25), (1603, 25), (1612, 25), (1616, 25), (1623, 25), (1632, 25), (1654, 25), (1661, 25), (1678, 25), (1696, 25), (1733, 25), (1778, 25), (1806, 25), (1824, 25), (1864, 25), (1874, 25), (1905, 25), (1924, 25), (1942, 25), (1991, 25), (1998, 25), (2029, 25), (2038, 25), (2064, 25), (2071, 25), (2073, 25), (2118, 25), (2145, 25), (2172, 25), (2179, 25), (2188, 25), (2209, 25), (2222, 25), (2249, 25), (2251, 25), (2254, 25), (2263, 25), (2276, 25), (2288, 25), (2313, 25), (2323, 25), (2334, 25), (2344, 25), (2443, 25), (2450, 25), (2453, 25), (2455, 25), (2458, 25), (2496, 25), (2539, 25), (2581, 25), (2588, 25), (2658, 25), (2709, 25), (2710, 25), (2727, 25), (2734, 25), (2735, 25), (2756, 25), (2847, 25), (2864, 25), (2919, 25), (2945, 25), (2951, 25), (2952, 25), (2959, 25), (2971, 25), (2973, 25), (3004, 25), (3005, 25), (3018, 25), (3028, 25), (3040, 25), (3066, 25), (3095, 25), (3119, 25), (3126, 25), (3145, 25), (3150, 25), (3153, 25), (3171, 25), (3187, 25), (3223, 25), (3253, 25), (3277, 25), (3282, 25), (3303, 25), (3325, 25), (3372, 25), (3383, 25), (3403, 25), (3405, 25), (3468, 25), (3471, 25), (3502, 25), (3514, 25), (3521, 25), (3557, 25), (3564, 25), (3580, 25), (3666, 25), (3684, 25), (3697, 25), (3746, 25), (3747, 25), (3750, 25), (3764, 25), (3768, 25), (3769, 25), (3782, 25), (3785, 25), (3821, 25), (3822, 25), (3835, 25), (3839, 25), (3873, 25), (3910, 25), (3930, 25), (3975, 25), (3978, 25), (3994, 25), (4043, 25), (4071, 25), (4133, 25), (4164, 25), (4171, 25), (4199, 25), (4211, 25), (4229, 25), (4237, 25), (4252, 25), (4350, 25), (4379, 25), (4403, 25), (4418, 25), (4492, 25), (4517, 25), (4523, 25), (4526, 25), (4575, 25), (4611, 25), (4613, 25), (4636, 25), (4646, 25), (4655, 25), (4692, 25), (4725, 25), (4764, 25), (4776, 25), (4778, 25), (4803, 25), (4812, 25), (4831, 25), (4837, 25), (4854, 25), (4874, 25), (4885, 25), (4891, 25), (4899, 25), (4900, 25), (4934, 25), (4980, 25), (4986, 25), (5013, 25), (5041, 25), (5059, 25), (5068, 25), (5090, 25), (5091, 25), (5126, 25), (5136, 25), (5139, 25), (5161, 25), (5167, 25), (5181, 25), (5215, 25), (5234, 25), (5243, 25), (5274, 25), (5278, 25), (5297, 25), (5307, 25), (5319, 25), (5364, 25), (5367, 25), (5369, 25), (5370, 25), (5373, 25), (5379, 25), (5386, 25), (5397, 25), (5412, 25), (5413, 25), (5415, 25), (5426, 25), (5436, 25), (5473, 25), (5506, 25), (5516, 25), (5542, 25), (5549, 25), (5551, 25), (5556, 25), (5582, 25), (5618, 25), (5619, 25), (5622, 25), (5657, 25), (5686, 25), (5687, 25), (5703, 25), (5732, 25), (5734, 25), (5750, 25), (5755, 25), (5793, 25), (5794, 25), (5871, 25), (5872, 25), (5886, 25), (5895, 25), (5905, 25), (5922, 25), (5925, 25), (5936, 25), (5951, 25), (5966, 25), (6047, 25), (6048, 25), (6095, 25), (6108, 25), (6109, 25), (6119, 25), (6145, 25), (6147, 25), (6153, 25), (6155, 25), (6159, 25), (6173, 25), (6175, 25), (6207, 25), (6208, 25), (6225, 25), (6279, 25), (6281, 25), (6286, 25), (6288, 25), (6293, 25), (6308, 25), (6312, 25), (6314, 25), (6337, 25), (6338, 25), (6382, 25), (6425, 25), (6448, 25), (6474, 25), (6488, 25), (6512, 25), (6519, 25), (6531, 25), (6535, 25), (6544, 25), (6612, 25), (6615, 25), (6621, 25), (6622, 25), (6656, 25), (6678, 25), (6684, 25), (6687, 25), (6706, 25), (6714, 25), (6715, 25), (6724, 25), (6773, 25), (6787, 25), (6791, 25), (6800, 25), (6801, 25), (6847, 25), (6874, 25), (6887, 25), (6959, 25), (7020, 25), (7035, 25), (7050, 25), (7062, 25), (7067, 25), (7070, 25), (7073, 25), (7079, 25), (7110, 25), (7114, 25), (7128, 25), (7149, 25), (7157, 25), (7185, 25), (7186, 25), (7197, 25), (7255, 25), (7278, 25), (7288, 25), (7301, 25), (7304, 25), (7323, 25), (7340, 25), (7342, 25), (7344, 25), (7351, 25), (7365, 25), (7417, 25), (7421, 25), (7424, 25), (7442, 25), (7449, 25), (7536, 25), (7546, 25), (7551, 25), (7557, 25), (7573, 25), (7577, 25), (7592, 25), (7605, 25), (7627, 25), (7628, 25), (7630, 25), (7663, 25), (7673, 25), (7679, 25), (7697, 25), (7729, 25), (7758, 25), (7766, 25), (7768, 25), (7793, 25), (7797, 25), (7822, 25), (7832, 25), (7918, 25), (7923, 25), (7926, 25), (7942, 25), (7953, 25), (7955, 25), (7975, 25), (7983, 25), (7989, 25), (7992, 25), (8027, 25), (8034, 25), (8044, 25), (8046, 25), (8059, 25), (8071, 25), (8072, 25), (8129, 25), (8140, 25), (8142, 25), (8177, 25), (8187, 25), (8199, 25), (8246, 25), (8253, 25), (8295, 25), (8325, 25), (8339, 25), (8352, 25), (8358, 25), (8373, 25), (8388, 25), (8405, 25), (8407, 25), (8437, 25), (8459, 25), (8502, 25), (8529, 25), (8532, 25), (8542, 25), (8573, 25), (8595, 25), (8598, 25), (8620, 25), (8622, 25), (8656, 25), (8664, 25), (8669, 25), (8670, 25), (8678, 25), (8698, 25), (8758, 25), (8771, 25), (8777, 25), (8779, 25), (8802, 25), (8807, 25), (8863, 25), (8890, 25), (8917, 25), (8921, 25), (8965, 25), (9012, 25), (9064, 25), (9100, 25), (9112, 25), (9128, 25), (9135, 25), (9152, 25), (9170, 25), (9209, 25), (9240, 25), (9243, 25), (9254, 25), (9266, 25), (9269, 25), (9272, 25), (9276, 25), (9296, 25), (9301, 25), (9303, 25), (9309, 25), (9315, 25), (9316, 25), (9346, 25), (9365, 25), (9382, 25), (9383, 25), (9391, 25), (9405, 25), (9447, 25), (9448, 25), (9485, 25), (9487, 25), (9489, 25), (9491, 25), (9509, 25), (9562, 25), (9566, 25), (9632, 25), (9668, 25), (9681, 25), (9702, 25), (9726, 25), (9730, 25), (9736, 25), (9740, 25), (9751, 25), (9780, 25), (9812, 25), (9831, 25), (9837, 25), (9934, 25), (9947, 25), (9974, 25), (10013, 25), (10019, 25), (10030, 25), (10031, 25), (10045, 25), (10054, 25), (10066, 25), (10074, 25), (10119, 25), (10125, 25), (10148, 25), (10174, 25), (10212, 25), (10218, 25), (10225, 25), (10243, 25), (10287, 25), (10300, 25), (10331, 25), (10353, 25), (10363, 25), (10387, 25), (10409, 25), (10414, 25), (10417, 25), (10423, 25), (10446, 25), (10459, 25), (10484, 25), (10487, 25), (10491, 25), (10509, 25), (10519, 25), (10563, 25), (10565, 25), (10581, 25), (10592, 25), (10595, 25), (10612, 25), (10618, 25), (10627, 25), (10666, 25), (10670, 25), (10690, 25), (10700, 25), (10741, 25), (10752, 25), (10758, 25), (10768, 25), (10812, 25), (10826, 25), (10882, 25), (10914, 25), (10931, 25), (10933, 25), (10946, 25), (10948, 25), (11012, 25), (11048, 25), (11054, 25), (11067, 25), (11068, 25), (11084, 25), (11095, 25), (11135, 25), (11141, 25), (11145, 25), (11149, 25), (11169, 25), (11192, 25), (11195, 25), (11203, 25), (11275, 25), (11286, 25), (11290, 25), (11295, 25), (11304, 25), (11307, 25), (11349, 25), (11366, 25), (11379, 25), (11387, 25), (11393, 25), (11400, 25), (11414, 25), (11435, 25), (11459, 25), (11483, 25), (11487, 25), (11489, 25), (11519, 25), (11547, 25), (11566, 25), (11578, 25), (11581, 25), (11591, 25), (11606, 25), (11623, 25), (11646, 25), (11647, 25), (11679, 25), (11695, 25), (11700, 25), (11701, 25), (11721, 25), (11733, 25), (11747, 25), (11754, 25), (11779, 25), (11796, 25), (11837, 25), (11844, 25), (11847, 25), (11866, 25), (11887, 25), (11894, 25), (11926, 25), (11940, 25), (11945, 25), (11970, 25), (12036, 25), (12072, 25), (12084, 25), (12113, 25), (12117, 25), (12128, 25), (12143, 25), (12148, 25), (12150, 25), (12190, 25), (12201, 25), (12220, 25), (12234, 25), (12250, 25), (12251, 25), (12264, 25), (12278, 25), (12299, 25), (12327, 25), (12330, 25), (12347, 25), (12384, 25), (12394, 25), (12409, 25), (12448, 25), (12465, 25), (12491, 25), (12492, 25), (12496, 25), (12503, 25), (12615, 25), (12627, 25), (12644, 25), (12649, 25), (12674, 25), (12688, 25), (12696, 25), (12730, 25), (12742, 25), (12745, 25), (12754, 25), (12775, 25), (12788, 25), (12797, 25), (4, 26), (15, 26), (20, 26), (26, 26), (32, 26), (35, 26), (41, 26), (66, 26), (83, 26), (110, 26), (145, 26), (171, 26), (199, 26), (250, 26), (252, 26), (256, 26), (257, 26), (272, 26), (279, 26), (291, 26), (302, 26), (304, 26), (326, 26), (334, 26), (358, 26), (374, 26), (375, 26), (377, 26), (400, 26), (505, 26), (508, 26), (513, 26), (527, 26), (539, 26), (551, 26), (561, 26), (565, 26), (574, 26), (579, 26), (601, 26), (650, 26), (664, 26), (686, 26), (698, 26), (722, 26), (729, 26), (742, 26), (743, 26), (769, 26), (772, 26), (813, 26), (814, 26), (869, 26), (905, 26), (911, 26), (966, 26), (1016, 26), (1020, 26), (1039, 26), (1052, 26), (1070, 26), (1080, 26), (1087, 26), (1093, 26), (1109, 26), (1114, 26), (1131, 26), (1159, 26), (1172, 26), (1182, 26), (1187, 26), (1204, 26), (1222, 26), (1223, 26), (1227, 26), (1229, 26), (1274, 26), (1275, 26), (1283, 26), (1292, 26), (1297, 26), (1344, 26), (1363, 26), (1369, 26), (1370, 26), (1379, 26), (1395, 26), (1399, 26), (1402, 26), (1448, 26), (1456, 26), (1515, 26), (1521, 26), (1524, 26), (1533, 26), (1551, 26), (1574, 26), (1592, 26), (1644, 26), (1648, 26), (1649, 26), (1666, 26), (1668, 26), (1671, 26), (1700, 26), (1738, 26), (1759, 26), (1760, 26), (1858, 26), (1890, 26), (1915, 26), (1920, 26), (1932, 26), (1946, 26), (1952, 26), (1958, 26), (1963, 26), (1968, 26), (1971, 26), (2006, 26), (2018, 26), (2053, 26), (2075, 26), (2102, 26), (2119, 26), (2128, 26), (2129, 26), (2154, 26), (2156, 26), (2218, 26), (2239, 26), (2317, 26), (2360, 26), (2378, 26), (2382, 26), (2387, 26), (2434, 26), (2464, 26), (2469, 26), (2473, 26), (2506, 26), (2525, 26), (2526, 26), (2555, 26), (2602, 26), (2637, 26), (2641, 26), (2653, 26), (2674, 26), (2677, 26), (2687, 26), (2733, 26), (2768, 26), (2782, 26), (2785, 26), (2800, 26), (2801, 26), (2802, 26), (2804, 26), (2810, 26), (2827, 26), (2837, 26), (2842, 26), (2894, 26), (2899, 26), (2935, 26), (2942, 26), (2956, 26), (2968, 26), (2977, 26), (2993, 26), (3014, 26), (3031, 26), (3039, 26), (3049, 26), (3079, 26), (3130, 26), (3135, 26), (3160, 26), (3163, 26), (3202, 26), (3207, 26), (3232, 26), (3242, 26), (3270, 26), (3275, 26), (3280, 26), (3285, 26), (3309, 26), (3316, 26), (3367, 26), (3445, 26), (3446, 26), (3483, 26), (3486, 26), (3487, 26), (3488, 26), (3489, 26), (3523, 26), (3525, 26), (3560, 26), (3569, 26), (3576, 26), (3581, 26), (3596, 26), (3671, 26), (3672, 26), (3677, 26), (3707, 26), (3722, 26), (3743, 26), (3787, 26), (3790, 26), (3818, 26), (3875, 26), (3897, 26), (3942, 26), (3951, 26), (3953, 26), (3961, 26), (3967, 26), (3969, 26), (3971, 26), (4004, 26), (4008, 26), (4052, 26), (4053, 26), (4054, 26), (4065, 26), (4102, 26), (4122, 26), (4136, 26), (4182, 26), (4215, 26), (4224, 26), (4230, 26), (4233, 26), (4262, 26), (4275, 26), (4300, 26), (4309, 26), (4322, 26), (4331, 26), (4337, 26), (4346, 26), (4402, 26), (4404, 26), (4431, 26), (4432, 26), (4469, 26), (4473, 26), (4527, 26), (4534, 26), (4549, 26), (4555, 26), (4564, 26), (4577, 26), (4581, 26), (4596, 26), (4601, 26), (4619, 26), (4630, 26), (4664, 26), (4665, 26), (4666, 26), (4688, 26), (4745, 26), (4749, 26), (4761, 26), (4766, 26), (4782, 26), (4818, 26), (4846, 26), (4855, 26), (4867, 26), (4908, 26), (4936, 26), (4958, 26), (4976, 26), (5001, 26), (5014, 26), (5032, 26), (5035, 26), (5076, 26), (5107, 26), (5108, 26), (5120, 26), (5143, 26), (5165, 26), (5207, 26), (5227, 26), (5232, 26), (5246, 26), (5248, 26), (5269, 26), (5296, 26), (5300, 26), (5317, 26), (5322, 26), (5383, 26), (5395, 26), (5398, 26), (5410, 26), (5434, 26), (5438, 26), (5469, 26), (5484, 26), (5523, 26), (5528, 26), (5529, 26), (5534, 26), (5541, 26), (5566, 26), (5590, 26), (5606, 26), (5626, 26), (5637, 26), (5678, 26), (5685, 26), (5690, 26), (5712, 26), (5725, 26), (5765, 26), (5782, 26), (5791, 26), (5805, 26), (5828, 26), (5838, 26), (5875, 26), (5959, 26), (5994, 26), (6004, 26), (6008, 26), (6028, 26), (6041, 26), (6053, 26), (6071, 26), (6148, 26), (6154, 26), (6157, 26), (6170, 26), (6205, 26), (6213, 26), (6233, 26), (6236, 26), (6238, 26), (6260, 26), (6283, 26), (6289, 26), (6297, 26), (6330, 26), (6342, 26), (6379, 26), (6389, 26), (6408, 26), (6432, 26), (6498, 26), (6538, 26), (6565, 26), (6605, 26), (6653, 26), (6659, 26), (6669, 26), (6681, 26), (6701, 26), (6732, 26), (6744, 26), (6777, 26), (6781, 26), (6815, 26), (6827, 26), (6855, 26), (6861, 26), (6868, 26), (6897, 26), (6946, 26), (6950, 26), (6961, 26), (6979, 26), (6986, 26), (7001, 26), (7003, 26), (7011, 26), (7024, 26), (7028, 26), (7061, 26), (7066, 26), (7068, 26), (7091, 26), (7098, 26), (7190, 26), (7198, 26), (7221, 26), (7232, 26), (7246, 26), (7263, 26), (7271, 26), (7283, 26), (7287, 26), (7305, 26), (7334, 26), (7335, 26), (7336, 26), (7347, 26), (7358, 26), (7371, 26), (7380, 26), (7410, 26), (7414, 26), (7416, 26), (7428, 26), (7491, 26), (7496, 26), (7500, 26), (7511, 26), (7514, 26), (7558, 26), (7560, 26), (7609, 26), (7610, 26), (7643, 26), (7676, 26), (7686, 26), (7688, 26), (7693, 26), (7720, 26), (7732, 26), (7733, 26), (7781, 26), (7786, 26), (7790, 26), (7801, 26), (7829, 26), (7837, 26), (7868, 26), (7907, 26), (7925, 26), (7928, 26), (7970, 26), (7978, 26), (7988, 26), (7996, 26), (8010, 26), (8033, 26), (8073, 26), (8101, 26), (8109, 26), (8145, 26), (8170, 26), (8175, 26), (8190, 26), (8221, 26), (8223, 26), (8224, 26), (8230, 26), (8232, 26), (8238, 26), (8242, 26), (8271, 26), (8327, 26), (8362, 26), (8396, 26), (8436, 26), (8441, 26), (8455, 26), (8463, 26), (8492, 26), (8493, 26), (8504, 26), (8593, 26), (8600, 26), (8613, 26), (8632, 26), (8645, 26), (8654, 26), (8716, 26), (8721, 26), (8744, 26), (8745, 26), (8756, 26), (8765, 26), (8787, 26), (8792, 26), (8816, 26), (8819, 26), (8841, 26), (8842, 26), (8888, 26), (8938, 26), (8952, 26), (8967, 26), (8971, 26), (8995, 26), (9047, 26), (9049, 26), (9059, 26), (9086, 26), (9091, 26), (9147, 26), (9182, 26), (9193, 26), (9211, 26), (9212, 26), (9226, 26), (9229, 26), (9234, 26), (9247, 26), (9257, 26), (9260, 26), (9262, 26), (9282, 26), (9334, 26), (9355, 26), (9387, 26), (9389, 26), (9390, 26), (9428, 26), (9429, 26), (9438, 26), (9450, 26), (9457, 26), (9502, 26), (9514, 26), (9528, 26), (9535, 26), (9536, 26), (9559, 26), (9565, 26), (9572, 26), (9588, 26), (9601, 26), (9606, 26), (9662, 26), (9684, 26), (9685, 26), (9732, 26), (9747, 26), (9753, 26), (9785, 26), (9791, 26), (9811, 26), (9827, 26), (9859, 26), (9868, 26), (9873, 26), (9976, 26), (9984, 26), (10022, 26), (10024, 26), (10026, 26), (10037, 26), (10055, 26), (10064, 26), (10078, 26), (10124, 26), (10135, 26), (10191, 26), (10199, 26), (10202, 26), (10204, 26), (10208, 26), (10239, 26), (10248, 26), (10261, 26), (10266, 26), (10385, 26), (10428, 26), (10462, 26), (10468, 26), (10481, 26), (10488, 26), (10490, 26), (10495, 26), (10496, 26), (10590, 26), (10600, 26), (10611, 26), (10633, 26), (10681, 26), (10694, 26), (10701, 26), (10707, 26), (10717, 26), (10743, 26), (10788, 26), (10820, 26), (10877, 26), (10878, 26), (10899, 26), (10925, 26), (10993, 26), (11013, 26), (11024, 26), (11049, 26), (11052, 26), (11086, 26), (11098, 26), (11126, 26), (11138, 26), (11144, 26), (11147, 26), (11164, 26), (11189, 26), (11207, 26), (11210, 26), (11233, 26), (11248, 26), (11284, 26), (11294, 26), (11313, 26), (11318, 26), (11325, 26), (11327, 26), (11333, 26), (11367, 26), (11371, 26), (11374, 26), (11399, 26), (11401, 26), (11415, 26), (11418, 26), (11423, 26), (11438, 26), (11441, 26), (11451, 26), (11462, 26), (11465, 26), (11466, 26), (11474, 26), (11529, 26), (11544, 26), (11550, 26), (11564, 26), (11594, 26), (11600, 26), (11604, 26), (11634, 26), (11658, 26), (11660, 26), (11687, 26), (11703, 26), (11708, 26), (11713, 26), (11714, 26), (11717, 26), (11722, 26), (11731, 26), (11743, 26), (11768, 26), (11790, 26), (11811, 26), (11826, 26), (11840, 26), (11841, 26), (11861, 26), (11869, 26), (11873, 26), (11892, 26), (11905, 26), (11911, 26), (11912, 26), (11923, 26), (11947, 26), (11954, 26), (11960, 26), (11963, 26), (12006, 26), (12011, 26), (12065, 26), (12092, 26), (12096, 26), (12097, 26), (12102, 26), (12111, 26), (12125, 26), (12131, 26), (12136, 26), (12165, 26), (12170, 26), (12202, 26), (12216, 26), (12230, 26), (12232, 26), (12236, 26), (12241, 26), (12253, 26), (12280, 26), (12292, 26), (12305, 26), (12342, 26), (12348, 26), (12354, 26), (12360, 26), (12383, 26), (12402, 26), (12430, 26), (12517, 26), (12524, 26), (12526, 26), (12529, 26), (12532, 26), (12540, 26), (12563, 26), (12579, 26), (12598, 26), (12623, 26), (12629, 26), (12675, 26), (12676, 26), (12678, 26), (12687, 26), (12727, 26), (12741, 26), (12743, 26), (7, 27), (48, 27), (102, 27), (109, 27), (115, 27), (128, 27), (130, 27), (141, 27), (150, 27), (157, 27), (161, 27), (196, 27), (206, 27), (224, 27), (274, 27), (280, 27), (281, 27), (292, 27), (321, 27), (327, 27), (370, 27), (378, 27), (415, 27), (426, 27), (427, 27), (433, 27), (443, 27), (457, 27), (514, 27), (548, 27), (568, 27), (571, 27), (587, 27), (595, 27), (642, 27), (647, 27), (671, 27), (676, 27), (749, 27), (753, 27), (754, 27), (825, 27), (864, 27), (865, 27), (937, 27), (977, 27), (997, 27), (1042, 27), (1056, 27), (1090, 27), (1103, 27), (1142, 27), (1176, 27), (1179, 27), (1200, 27), (1219, 27), (1220, 27), (1238, 27), (1239, 27), (1261, 27), (1304, 27), (1316, 27), (1330, 27), (1360, 27), (1380, 27), (1417, 27), (1489, 27), (1499, 27), (1502, 27), (1512, 27), (1553, 27), (1566, 27), (1570, 27), (1600, 27), (1607, 27), (1622, 27), (1647, 27), (1680, 27), (1684, 27), (1689, 27), (1694, 27), (1710, 27), (1712, 27), (1713, 27), (1734, 27), (1737, 27), (1798, 27), (1810, 27), (1818, 27), (1836, 27), (1843, 27), (1854, 27), (1865, 27), (1876, 27), (1888, 27), (1894, 27), (1935, 27), (1960, 27), (1964, 27), (2008, 27), (2015, 27), (2037, 27), (2057, 27), (2084, 27), (2105, 27), (2148, 27), (2193, 27), (2202, 27), (2211, 27), (2212, 27), (2215, 27), (2223, 27), (2227, 27), (2228, 27), (2246, 27), (2252, 27), (2255, 27), (2265, 27), (2268, 27), (2289, 27), (2311, 27), (2321, 27), (2327, 27), (2351, 27), (2392, 27), (2400, 27), (2406, 27), (2433, 27), (2465, 27), (2478, 27), (2488, 27), (2500, 27), (2501, 27), (2523, 27), (2591, 27), (2592, 27), (2594, 27), (2623, 27), (2632, 27), (2662, 27), (2673, 27), (2708, 27), (2714, 27), (2739, 27), (2814, 27), (2823, 27), (2826, 27), (2836, 27), (2858, 27), (2859, 27), (2892, 27), (2915, 27), (2923, 27), (2948, 27), (2955, 27), (2957, 27), (2975, 27), (3038, 27), (3044, 27), (3063, 27), (3083, 27), (3111, 27), (3149, 27), (3220, 27), (3245, 27), (3257, 27), (3260, 27), (3266, 27), (3274, 27), (3310, 27), (3323, 27), (3326, 27), (3327, 27), (3362, 27), (3385, 27), (3407, 27), (3416, 27), (3428, 27), (3448, 27), (3461, 27), (3535, 27), (3544, 27), (3552, 27), (3558, 27), (3585, 27), (3592, 27), (3602, 27), (3608, 27), (3610, 27), (3636, 27), (3640, 27), (3681, 27), (3692, 27), (3718, 27), (3793, 27), (3798, 27), (3830, 27), (3841, 27), (3879, 27), (3926, 27), (3929, 27), (3939, 27), (3943, 27), (3948, 27), (4009, 27), (4027, 27), (4032, 27), (4046, 27), (4060, 27), (4085, 27), (4089, 27), (4094, 27), (4104, 27), (4271, 27), (4279, 27), (4281, 27), (4287, 27), (4305, 27), (4326, 27), (4334, 27), (4338, 27), (4357, 27), (4380, 27), (4397, 27), (4401, 27), (4414, 27), (4415, 27), (4417, 27), (4427, 27), (4440, 27), (4449, 27), (4481, 27), (4487, 27), (4516, 27), (4532, 27), (4554, 27), (4629, 27), (4661, 27), (4678, 27), (4690, 27), (4784, 27), (4787, 27), (4810, 27), (4827, 27), (4829, 27), (4838, 27), (4844, 27), (4881, 27), (4890, 27), (4916, 27), (4917, 27), (4918, 27), (4935, 27), (4946, 27), (4973, 27), (4975, 27), (4977, 27), (4998, 27), (5015, 27), (5069, 27), (5072, 27), (5075, 27), (5096, 27), (5104, 27), (5135, 27), (5166, 27), (5178, 27), (5198, 27), (5206, 27), (5221, 27), (5265, 27), (5266, 27), (5273, 27), (5294, 27), (5299, 27), (5308, 27), (5312, 27), (5357, 27), (5376, 27), (5384, 27), (5409, 27), (5449, 27), (5472, 27), (5502, 27), (5535, 27), (5584, 27), (5589, 27), (5595, 27), (5597, 27), (5630, 27), (5632, 27), (5680, 27), (5688, 27), (5713, 27), (5722, 27), (5727, 27), (5740, 27), (5742, 27), (5773, 27), (5785, 27), (5813, 27), (5821, 27), (5832, 27), (5859, 27), (5876, 27), (5883, 27), (5953, 27), (5961, 27), (5964, 27), (5972, 27), (5977, 27), (6026, 27), (6030, 27), (6042, 27), (6059, 27), (6072, 27), (6074, 27), (6106, 27), (6120, 27), (6149, 27), (6168, 27), (6242, 27), (6249, 27), (6266, 27), (6274, 27), (6276, 27), (6318, 27), (6332, 27), (6335, 27), (6339, 27), (6343, 27), (6364, 27), (6397, 27), (6409, 27), (6438, 27), (6444, 27), (6449, 27), (6465, 27), (6528, 27), (6571, 27), (6581, 27), (6585, 27), (6587, 27), (6602, 27), (6617, 27), (6624, 27), (6626, 27), (6636, 27), (6637, 27), (6640, 27), (6660, 27), (6709, 27), (6713, 27), (6741, 27), (6753, 27), (6809, 27), (6825, 27), (6835, 27), (6853, 27), (6854, 27), (6869, 27), (6870, 27), (6893, 27), (6905, 27), (6911, 27), (6922, 27), (6926, 27), (6929, 27), (6956, 27), (6992, 27), (7009, 27), (7022, 27), (7027, 27), (7037, 27), (7055, 27), (7072, 27), (7094, 27), (7109, 27), (7135, 27), (7151, 27), (7168, 27), (7174, 27), (7201, 27), (7210, 27), (7216, 27), (7252, 27), (7254, 27), (7281, 27), (7300, 27), (7338, 27), (7348, 27), (7356, 27), (7374, 27), (7434, 27), (7453, 27), (7473, 27), (7505, 27), (7515, 27), (7528, 27), (7539, 27), (7563, 27), (7575, 27), (7582, 27), (7593, 27), (7611, 27), (7646, 27), (7660, 27), (7719, 27), (7741, 27), (7751, 27), (7762, 27), (7800, 27), (7828, 27), (7835, 27), (7845, 27), (7850, 27), (7879, 27), (7889, 27), (7905, 27), (7908, 27), (7933, 27), (7947, 27), (7954, 27), (7959, 27), (8001, 27), (8029, 27), (8092, 27), (8098, 27), (8152, 27), (8153, 27), (8173, 27), (8188, 27), (8193, 27), (8195, 27), (8214, 27), (8218, 27), (8252, 27), (8256, 27), (8262, 27), (8266, 27), (8284, 27), (8319, 27), (8330, 27), (8333, 27), (8340, 27), (8353, 27), (8371, 27), (8401, 27), (8417, 27), (8425, 27), (8427, 27), (8469, 27), (8484, 27), (8517, 27), (8520, 27), (8527, 27), (8533, 27), (8541, 27), (8591, 27), (8616, 27), (8630, 27), (8662, 27), (8676, 27), (8691, 27), (8735, 27), (8737, 27), (8790, 27), (8811, 27), (8825, 27), (8833, 27), (8852, 27), (8879, 27), (8895, 27), (8919, 27), (8923, 27), (8955, 27), (8972, 27), (9010, 27), (9052, 27), (9065, 27), (9073, 27), (9087, 27), (9194, 27), (9259, 27), (9265, 27), (9271, 27), (9324, 27), (9336, 27), (9362, 27), (9368, 27), (9369, 27), (9371, 27), (9418, 27), (9425, 27), (9436, 27), (9439, 27), (9474, 27), (9476, 27), (9478, 27), (9479, 27), (9511, 27), (9533, 27), (9537, 27), (9544, 27), (9548, 27), (9549, 27), (9568, 27), (9580, 27), (9586, 27), (9614, 27), (9631, 27), (9635, 27), (9636, 27), (9653, 27), (9737, 27), (9750, 27), (9767, 27), (9776, 27), (9787, 27), (9802, 27), (9810, 27), (9824, 27), (9862, 27), (9864, 27), (9872, 27), (9876, 27), (9885, 27), (9897, 27), (9926, 27), (9945, 27), (9955, 27), (9975, 27), (9998, 27), (10012, 27), (10021, 27), (10035, 27), (10092, 27), (10103, 27), (10121, 27), (10123, 27), (10151, 27), (10160, 27), (10164, 27), (10166, 27), (10235, 27), (10242, 27), (10244, 27), (10258, 27), (10267, 27), (10282, 27), (10293, 27), (10298, 27), (10308, 27), (10314, 27), (10350, 27), (10372, 27), (10398, 27), (10421, 27), (10429, 27), (10441, 27), (10442, 27), (10454, 27), (10483, 27), (10512, 27), (10514, 27), (10545, 27), (10578, 27), (10634, 27), (10656, 27), (10674, 27), (10689, 27), (10708, 27), (10713, 27), (10730, 27), (10742, 27), (10750, 27), (10755, 27), (10760, 27), (10776, 27), (10782, 27), (10792, 27), (10808, 27), (10814, 27), (10817, 27), (10840, 27), (10869, 27), (10885, 27), (10894, 27), (10895, 27), (10897, 27), (10898, 27), (10904, 27), (10943, 27), (10944, 27), (10945, 27), (10962, 27), (10963, 27), (10980, 27), (10992, 27), (10996, 27), (10999, 27), (11000, 27), (11007, 27), (11035, 27), (11045, 27), (11055, 27), (11056, 27), (11058, 27), (11063, 27), (11070, 27), (11078, 27), (11133, 27), (11134, 27), (11158, 27), (11172, 27), (11224, 27), (11243, 27), (11245, 27), (11292, 27), (11320, 27), (11324, 27), (11360, 27), (11412, 27), (11442, 27), (11454, 27), (11471, 27), (11482, 27), (11497, 27), (11561, 27), (11567, 27), (11582, 27), (11597, 27), (11609, 27), (11670, 27), (11678, 27), (11707, 27), (11725, 27), (11739, 27), (11748, 27), (11758, 27), (11764, 27), (11769, 27), (11777, 27), (11804, 27), (11810, 27), (11909, 27), (11942, 27), (11944, 27), (11948, 27), (11974, 27), (11984, 27), (11990, 27), (11998, 27), (12000, 27), (12009, 27), (12014, 27), (12017, 27), (12028, 27), (12057, 27), (12124, 27), (12152, 27), (12155, 27), (12179, 27), (12192, 27), (12195, 27), (12211, 27), (12218, 27), (12271, 27), (12273, 27), (12288, 27), (12291, 27), (12302, 27), (12314, 27), (12325, 27), (12337, 27), (12364, 27), (12381, 27), (12414, 27), (12417, 27), (12426, 27), (12445, 27), (12453, 27), (12469, 27), (12522, 27), (12541, 27), (12542, 27), (12545, 27), (12575, 27), (12602, 27), (12613, 27), (12622, 27), (12626, 27), (12631, 27), (12663, 27), (12665, 27), (12716, 27), (12723, 27), (12762, 27), (12773, 27), (12793, 27), (8, 28), (13, 28), (16, 28), (21, 28), (45, 28), (71, 28), (88, 28), (127, 28), (132, 28), (167, 28), (175, 28), (203, 28), (255, 28), (265, 28), (266, 28), (289, 28), (323, 28), (336, 28), (349, 28), (376, 28), (420, 28), (442, 28), (476, 28), (481, 28), (591, 28), (592, 28), (609, 28), (611, 28), (641, 28), (644, 28), (648, 28), (655, 28), (685, 28), (702, 28), (709, 28), (756, 28), (784, 28), (800, 28), (803, 28), (861, 28), (893, 28), (895, 28), (897, 28), (932, 28), (955, 28), (964, 28), (965, 28), (972, 28), (993, 28), (996, 28), (1043, 28), (1049, 28), (1055, 28), (1081, 28), (1091, 28), (1105, 28), (1118, 28), (1134, 28), (1193, 28), (1226, 28), (1264, 28), (1281, 28), (1282, 28), (1306, 28), (1313, 28), (1314, 28), (1325, 28), (1357, 28), (1403, 28), (1406, 28), (1407, 28), (1446, 28), (1449, 28), (1462, 28), (1474, 28), (1480, 28), (1513, 28), (1514, 28), (1530, 28), (1548, 28), (1567, 28), (1568, 28), (1618, 28), (1643, 28), (1645, 28), (1659, 28), (1662, 28), (1669, 28), (1672, 28), (1673, 28), (1682, 28), (1688, 28), (1749, 28), (1791, 28), (1800, 28), (1811, 28), (1813, 28), (1815, 28), (1851, 28), (1855, 28), (1878, 28), (1895, 28), (1936, 28), (1951, 28), (1972, 28), (1981, 28), (1986, 28), (2002, 28), (2014, 28), (2022, 28), (2026, 28), (2033, 28), (2044, 28), (2061, 28), (2097, 28), (2109, 28), (2134, 28), (2166, 28), (2206, 28), (2226, 28), (2264, 28), (2266, 28), (2274, 28), (2305, 28), (2308, 28), (2324, 28), (2329, 28), (2332, 28), (2339, 28), (2347, 28), (2361, 28), (2363, 28), (2388, 28), (2404, 28), (2423, 28), (2428, 28), (2480, 28), (2531, 28), (2534, 28), (2535, 28), (2553, 28), (2570, 28), (2589, 28), (2649, 28), (2663, 28), (2664, 28), (2702, 28), (2720, 28), (2760, 28), (2780, 28), (2790, 28), (2806, 28), (2808, 28), (2811, 28), (2863, 28), (2882, 28), (2883, 28), (2893, 28), (2897, 28), (2900, 28), (2905, 28), (2914, 28), (2922, 28), (2937, 28), (2985, 28), (2999, 28), (3009, 28), (3027, 28), (3032, 28), (3037, 28), (3047, 28), (3129, 28), (3131, 28), (3134, 28), (3139, 28), (3143, 28), (3166, 28), (3167, 28), (3174, 28), (3177, 28), (3178, 28), (3183, 28), (3210, 28), (3213, 28), (3216, 28), (3222, 28), (3241, 28), (3271, 28), (3332, 28), (3336, 28), (3340, 28), (3368, 28), (3375, 28), (3376, 28), (3392, 28), (3394, 28), (3422, 28), (3424, 28), (3442, 28), (3451, 28), (3519, 28), (3522, 28), (3528, 28), (3529, 28), (3548, 28), (3615, 28), (3629, 28), (3656, 28), (3705, 28), (3708, 28), (3710, 28), (3765, 28), (3774, 28), (3776, 28), (3797, 28), (3812, 28), (3823, 28), (3850, 28), (3854, 28), (3864, 28), (3882, 28), (3894, 28), (3925, 28), (3946, 28), (3960, 28), (4006, 28), (4013, 28), (4016, 28), (4025, 28), (4026, 28), (4033, 28), (4055, 28), (4066, 28), (4093, 28), (4120, 28), (4130, 28), (4154, 28), (4165, 28), (4168, 28), (4176, 28), (4186, 28), (4198, 28), (4210, 28), (4212, 28), (4214, 28), (4277, 28), (4301, 28), (4319, 28), (4335, 28), (4364, 28), (4423, 28), (4438, 28), (4470, 28), (4471, 28), (4478, 28), (4485, 28), (4489, 28), (4520, 28), (4535, 28), (4538, 28), (4541, 28), (4543, 28), (4595, 28), (4615, 28), (4639, 28), (4643, 28), (4647, 28), (4662, 28), (4677, 28), (4681, 28), (4701, 28), (4712, 28), (4719, 28), (4739, 28), (4760, 28), (4768, 28), (4780, 28), (4790, 28), (4817, 28), (4860, 28), (4923, 28), (4944, 28), (4945, 28), (4957, 28), (4982, 28), (5004, 28), (5025, 28), (5027, 28), (5039, 28), (5074, 28), (5097, 28), (5112, 28), (5115, 28), (5125, 28), (5128, 28), (5140, 28), (5172, 28), (5177, 28), (5186, 28), (5191, 28), (5195, 28), (5240, 28), (5264, 28), (5283, 28), (5292, 28), (5316, 28), (5352, 28), (5355, 28), (5378, 28), (5387, 28), (5411, 28), (5442, 28), (5445, 28), (5461, 28), (5479, 28), (5482, 28), (5509, 28), (5518, 28), (5591, 28), (5604, 28), (5663, 28), (5681, 28), (5694, 28), (5714, 28), (5724, 28), (5726, 28), (5781, 28), (5790, 28), (5797, 28), (5798, 28), (5807, 28), (5830, 28), (5834, 28), (5866, 28), (5877, 28), (5902, 28), (5921, 28), (5929, 28), (5935, 28), (5944, 28), (5963, 28), (5978, 28), (5989, 28), (6019, 28), (6025, 28), (6036, 28), (6063, 28), (6068, 28), (6088, 28), (6124, 28), (6142, 28), (6151, 28), (6191, 28), (6196, 28), (6206, 28), (6272, 28), (6273, 28), (6282, 28), (6296, 28), (6313, 28), (6347, 28), (6401, 28), (6404, 28), (6429, 28), (6476, 28), (6489, 28), (6514, 28), (6533, 28), (6569, 28), (6588, 28), (6638, 28), (6666, 28), (6680, 28), (6686, 28), (6697, 28), (6700, 28), (6708, 28), (6771, 28), (6782, 28), (6798, 28), (6803, 28), (6813, 28), (6824, 28), (6830, 28), (6881, 28), (6886, 28), (6919, 28), (6937, 28), (6981, 28), (7019, 28), (7023, 28), (7040, 28), (7060, 28), (7074, 28), (7075, 28), (7080, 28), (7099, 28), (7132, 28), (7144, 28), (7164, 28), (7250, 28), (7259, 28), (7291, 28), (7306, 28), (7319, 28), (7322, 28), (7324, 28), (7329, 28), (7355, 28), (7359, 28), (7475, 28), (7506, 28), (7516, 28), (7517, 28), (7538, 28), (7549, 28), (7565, 28), (7572, 28), (7576, 28), (7616, 28), (7617, 28), (7642, 28), (7724, 28), (7738, 28), (7746, 28), (7787, 28), (7791, 28), (7803, 28), (7804, 28), (7812, 28), (7830, 28), (7842, 28), (7853, 28), (7856, 28), (7866, 28), (7869, 28), (7875, 28), (7895, 28), (7898, 28), (7902, 28), (7936, 28), (7960, 28), (7971, 28), (7972, 28), (7981, 28), (8047, 28), (8097, 28), (8102, 28), (8105, 28), (8116, 28), (8136, 28), (8137, 28), (8201, 28), (8203, 28), (8206, 28), (8219, 28), (8231, 28), (8263, 28), (8265, 28), (8300, 28), (8308, 28), (8321, 28), (8343, 28), (8380, 28), (8383, 28), (8449, 28), (8475, 28), (8498, 28), (8514, 28), (8543, 28), (8546, 28), (8568, 28), (8579, 28), (8608, 28), (8614, 28), (8618, 28), (8637, 28), (8647, 28), (8665, 28), (8688, 28), (8696, 28), (8715, 28), (8717, 28), (8719, 28), (8746, 28), (8761, 28), (8767, 28), (8773, 28), (8775, 28), (8829, 28), (8866, 28), (8877, 28), (8904, 28), (8927, 28), (8941, 28), (8968, 28), (9021, 28), (9060, 28), (9061, 28), (9077, 28), (9143, 28), (9146, 28), (9190, 28), (9196, 28), (9217, 28), (9219, 28), (9236, 28), (9242, 28), (9289, 28), (9307, 28), (9330, 28), (9357, 28), (9379, 28), (9392, 28), (9417, 28), (9444, 28), (9494, 28), (9503, 28), (9508, 28), (9531, 28), (9575, 28), (9593, 28), (9600, 28), (9624, 28), (9672, 28), (9678, 28), (9679, 28), (9696, 28), (9701, 28), (9705, 28), (9718, 28), (9721, 28), (9755, 28), (9757, 28), (9763, 28), (9770, 28), (9789, 28), (9806, 28), (9835, 28), (9895, 28), (9898, 28), (9909, 28), (9911, 28), (9914, 28), (9920, 28), (9940, 28), (9988, 28), (9992, 28), (10006, 28), (10015, 28), (10049, 28), (10070, 28), (10075, 28), (10081, 28), (10093, 28), (10095, 28), (10109, 28), (10110, 28), (10112, 28), (10126, 28), (10136, 28), (10147, 28), (10168, 28), (10190, 28), (10194, 28), (10201, 28), (10209, 28), (10241, 28), (10246, 28), (10247, 28), (10251, 28), (10256, 28), (10264, 28), (10279, 28), (10320, 28), (10329, 28), (10334, 28), (10369, 28), (10380, 28), (10415, 28), (10431, 28), (10433, 28), (10449, 28), (10460, 28), (10485, 28), (10507, 28), (10510, 28), (10516, 28), (10537, 28), (10557, 28), (10587, 28), (10629, 28), (10669, 28), (10678, 28), (10679, 28), (10709, 28), (10725, 28), (10734, 28), (10751, 28), (10774, 28), (10796, 28), (10797, 28), (10799, 28), (10809, 28), (10828, 28), (10835, 28), (10839, 28), (10905, 28), (10929, 28), (10932, 28), (10958, 28), (10970, 28), (10978, 28), (10988, 28), (10989, 28), (11026, 28), (11027, 28), (11034, 28), (11072, 28), (11082, 28), (11090, 28), (11137, 28), (11156, 28), (11186, 28), (11212, 28), (11266, 28), (11316, 28), (11332, 28), (11348, 28), (11358, 28), (11410, 28), (11447, 28), (11448, 28), (11469, 28), (11512, 28), (11517, 28), (11522, 28), (11539, 28), (11579, 28), (11589, 28), (11615, 28), (11633, 28), (11659, 28), (11668, 28), (11669, 28), (11730, 28), (11752, 28), (11761, 28), (11763, 28), (11772, 28), (11781, 28), (11793, 28), (11855, 28), (11871, 28), (11890, 28), (11898, 28), (11899, 28), (11901, 28), (11934, 28), (11937, 28), (11952, 28), (11969, 28), (11976, 28), (11983, 28), (11993, 28), (11994, 28), (12019, 28), (12035, 28), (12050, 28), (12078, 28), (12110, 28), (12146, 28), (12171, 28), (12239, 28), (12254, 28), (12268, 28), (12276, 28), (12286, 28), (12311, 28), (12315, 28), (12318, 28), (12322, 28), (12353, 28), (12368, 28), (12374, 28), (12386, 28), (12387, 28), (12432, 28), (12436, 28), (12438, 28), (12454, 28), (12478, 28), (12500, 28), (12519, 28), (12523, 28), (12544, 28), (12553, 28), (12557, 28), (12567, 28), (12588, 28), (12590, 28), (12648, 28), (12655, 28), (12673, 28), (12682, 28), (12690, 28), (12698, 28), (12739, 28), (12746, 28), (12780, 28), (33, 29), (59, 29), (99, 29), (111, 29), (117, 29), (120, 29), (151, 29), (159, 29), (169, 29), (170, 29), (222, 29), (268, 29), (294, 29), (295, 29), (346, 29), (354, 29), (382, 29), (401, 29), (450, 29), (451, 29), (504, 29), (520, 29), (531, 29), (594, 29), (603, 29), (605, 29), (608, 29), (618, 29), (704, 29), (732, 29), (752, 29), (775, 29), (809, 29), (824, 29), (828, 29), (882, 29), (903, 29), (939, 29), (962, 29), (981, 29), (1050, 29), (1077, 29), (1084, 29), (1098, 29), (1101, 29), (1106, 29), (1129, 29), (1170, 29), (1173, 29), (1181, 29), (1210, 29), (1214, 29), (1231, 29), (1232, 29), (1234, 29), (1257, 29), (1271, 29), (1333, 29), (1358, 29), (1428, 29), (1466, 29), (1493, 29), (1596, 29), (1598, 29), (1605, 29), (1609, 29), (1630, 29), (1658, 29), (1685, 29), (1699, 29), (1719, 29), (1756, 29), (1777, 29), (1783, 29), (1792, 29), (1867, 29), (1885, 29), (1898, 29), (1912, 29), (1925, 29), (1927, 29), (1928, 29), (1953, 29), (1988, 29), (2007, 29), (2012, 29), (2046, 29), (2058, 29), (2081, 29), (2098, 29), (2122, 29), (2167, 29), (2189, 29), (2230, 29), (2234, 29), (2238, 29), (2250, 29), (2297, 29), (2314, 29), (2315, 29), (2319, 29), (2374, 29), (2391, 29), (2410, 29), (2424, 29), (2454, 29), (2461, 29), (2471, 29), (2505, 29), (2533, 29), (2551, 29), (2561, 29), (2580, 29), (2616, 29), (2661, 29), (2685, 29), (2731, 29), (2744, 29), (2754, 29), (2769, 29), (2775, 29), (2794, 29), (2877, 29), (2881, 29), (2885, 29), (2888, 29), (2902, 29), (2904, 29), (2913, 29), (2934, 29), (2953, 29), (3024, 29), (3033, 29), (3048, 29), (3056, 29), (3059, 29), (3096, 29), (3114, 29), (3124, 29), (3142, 29), (3146, 29), (3175, 29), (3208, 29), (3209, 29), (3238, 29), (3267, 29), (3300, 29), (3306, 29), (3307, 29), (3342, 29), (3350, 29), (3374, 29), (3387, 29), (3398, 29), (3414, 29), (3466, 29), (3505, 29), (3563, 29), (3570, 29), (3572, 29), (3598, 29), (3599, 29), (3604, 29), (3616, 29), (3653, 29), (3657, 29), (3659, 29), (3662, 29), (3663, 29), (3673, 29), (3703, 29), (3735, 29), (3758, 29), (3809, 29), (3814, 29), (3844, 29), (3883, 29), (3902, 29), (3907, 29), (3908, 29), (3974, 29), (3976, 29), (3985, 29), (3988, 29), (4072, 29), (4073, 29), (4079, 29), (4088, 29), (4092, 29), (4111, 29), (4113, 29), (4145, 29), (4166, 29), (4194, 29), (4201, 29), (4285, 29), (4312, 29), (4317, 29), (4333, 29), (4347, 29), (4349, 29), (4359, 29), (4394, 29), (4461, 29), (4513, 29), (4537, 29), (4635, 29), (4649, 29), (4682, 29), (4689, 29), (4711, 29), (4716, 29), (4717, 29), (4724, 29), (4735, 29), (4762, 29), (4771, 29), (4774, 29), (4805, 29), (4826, 29), (4833, 29), (4835, 29), (4841, 29), (4857, 29), (4872, 29), (4904, 29), (4919, 29), (4979, 29), (5005, 29), (5006, 29), (5034, 29), (5044, 29), (5055, 29), (5056, 29), (5064, 29), (5123, 29), (5132, 29), (5133, 29), (5146, 29), (5155, 29), (5185, 29), (5192, 29), (5250, 29), (5263, 29), (5280, 29), (5335, 29), (5366, 29), (5374, 29), (5439, 29), (5454, 29), (5477, 29), (5512, 29), (5515, 29), (5521, 29), (5526, 29), (5536, 29), (5564, 29), (5628, 29), (5667, 29), (5677, 29), (5702, 29), (5717, 29), (5720, 29), (5744, 29), (5758, 29), (5770, 29), (5777, 29), (5796, 29), (5845, 29), (5860, 29), (5891, 29), (5916, 29), (5920, 29), (5928, 29), (5957, 29), (5979, 29), (5983, 29), (6000, 29), (6014, 29), (6020, 29), (6037, 29), (6057, 29), (6077, 29), (6078, 29), (6097, 29), (6100, 29), (6176, 29), (6187, 29), (6224, 29), (6244, 29), (6280, 29), (6300, 29), (6311, 29), (6315, 29), (6344, 29), (6360, 29), (6366, 29), (6375, 29), (6427, 29), (6443, 29), (6485, 29), (6515, 29), (6527, 29), (6536, 29), (6558, 29), (6577, 29), (6606, 29), (6616, 29), (6618, 29), (6673, 29), (6729, 29), (6792, 29), (6793, 29), (6794, 29), (6812, 29), (6888, 29), (6912, 29), (6951, 29), (6952, 29), (6980, 29), (6997, 29), (7031, 29), (7052, 29), (7092, 29), (7097, 29), (7116, 29), (7117, 29), (7129, 29), (7145, 29), (7158, 29), (7183, 29), (7192, 29), (7195, 29), (7228, 29), (7229, 29), (7267, 29), (7297, 29), (7316, 29), (7318, 29), (7350, 29), (7369, 29), (7390, 29), (7402, 29), (7452, 29), (7457, 29), (7468, 29), (7492, 29), (7570, 29), (7583, 29), (7587, 29), (7601, 29), (7621, 29), (7659, 29), (7736, 29), (7749, 29), (7815, 29), (7854, 29), (7859, 29), (7864, 29), (7904, 29), (8024, 29), (8084, 29), (8086, 29), (8091, 29), (8096, 29), (8148, 29), (8174, 29), (8179, 29), (8182, 29), (8192, 29), (8204, 29), (8247, 29), (8250, 29), (8264, 29), (8304, 29), (8322, 29), (8367, 29), (8399, 29), (8406, 29), (8410, 29), (8428, 29), (8466, 29), (8472, 29), (8491, 29), (8507, 29), (8508, 29), (8522, 29), (8526, 29), (8550, 29), (8583, 29), (8584, 29), (8586, 29), (8597, 29), (8599, 29), (8609, 29), (8690, 29), (8720, 29), (8813, 29), (8847, 29), (8924, 29), (8928, 29), (8976, 29), (8982, 29), (8990, 29), (8991, 29), (9017, 29), (9048, 29), (9051, 29), (9053, 29), (9063, 29), (9069, 29), (9148, 29), (9154, 29), (9158, 29), (9163, 29), (9195, 29), (9238, 29), (9251, 29), (9252, 29), (9291, 29), (9319, 29), (9342, 29), (9345, 29), (9375, 29), (9380, 29), (9395, 29), (9399, 29), (9401, 29), (9421, 29), (9465, 29), (9518, 29), (9538, 29), (9551, 29), (9552, 29), (9611, 29), (9623, 29), (9643, 29), (9652, 29), (9682, 29), (9690, 29), (9692, 29), (9703, 29), (9711, 29), (9742, 29), (9777, 29), (9790, 29), (9816, 29), (9825, 29), (9830, 29), (9836, 29), (9849, 29), (9851, 29), (9875, 29), (9880, 29), (9941, 29), (9948, 29), (9968, 29), (9977, 29), (9997, 29), (10010, 29), (10018, 29), (10023, 29), (10032, 29), (10040, 29), (10067, 29), (10096, 29), (10115, 29), (10131, 29), (10150, 29), (10156, 29), (10171, 29), (10193, 29), (10211, 29), (10221, 29), (10250, 29), (10275, 29), (10299, 29), (10343, 29), (10344, 29), (10388, 29), (10390, 29), (10471, 29), (10486, 29), (10555, 29), (10569, 29), (10615, 29), (10642, 29), (10647, 29), (10667, 29), (10683, 29), (10697, 29), (10727, 29), (10738, 29), (10745, 29), (10769, 29), (10772, 29), (10802, 29), (10833, 29), (10849, 29), (10928, 29), (10930, 29), (10936, 29), (10942, 29), (10972, 29), (10987, 29), (10990, 29), (11004, 29), (11008, 29), (11036, 29), (11040, 29), (11060, 29), (11102, 29), (11107, 29), (11162, 29), (11165, 29), (11178, 29), (11193, 29), (11198, 29), (11232, 29), (11237, 29), (11272, 29), (11323, 29), (11339, 29), (11340, 29), (11343, 29), (11408, 29), (11409, 29), (11411, 29), (11420, 29), (11428, 29), (11468, 29), (11476, 29), (11488, 29), (11495, 29), (11518, 29), (11520, 29), (11524, 29), (11525, 29), (11545, 29), (11562, 29), (11608, 29), (11612, 29), (11618, 29), (11632, 29), (11650, 29), (11655, 29), (11672, 29), (11674, 29), (11675, 29), (11683, 29), (11691, 29), (11711, 29), (11860, 29), (11874, 29), (11900, 29), (11914, 29), (11916, 29), (11924, 29), (11959, 29), (11968, 29), (12067, 29), (12088, 29), (12093, 29), (12095, 29), (12119, 29), (12137, 29), (12140, 29), (12228, 29), (12252, 29), (12256, 29), (12263, 29), (12270, 29), (12297, 29), (12313, 29), (12329, 29), (12333, 29), (12380, 29), (12382, 29), (12468, 29), (12476, 29), (12485, 29), (12494, 29), (12497, 29), (12507, 29), (12530, 29), (12538, 29), (12543, 29), (12568, 29), (12582, 29), (12616, 29), (12618, 29), (12620, 29), (12625, 29), (12666, 29), (12681, 29), (12694, 29), (12713, 29), (12714, 29), (12719, 29), (12734, 29), (12747, 29), (12758, 29), (12781, 29), (12789, 29), (11, 30), (22, 30), (72, 30), (172, 30), (211, 30), (242, 30), (305, 30), (315, 30), (379, 30), (413, 30), (430, 30), (460, 30), (465, 30), (482, 30), (490, 30), (528, 30), (535, 30), (537, 30), (570, 30), (597, 30), (630, 30), (631, 30), (638, 30), (672, 30), (689, 30), (693, 30), (723, 30), (755, 30), (759, 30), (786, 30), (804, 30), (849, 30), (868, 30), (878, 30), (890, 30), (945, 30), (958, 30), (991, 30), (1011, 30), (1023, 30), (1035, 30), (1038, 30), (1058, 30), (1059, 30), (1086, 30), (1095, 30), (1139, 30), (1167, 30), (1177, 30), (1178, 30), (1199, 30), (1215, 30), (1237, 30), (1273, 30), (1277, 30), (1320, 30), (1336, 30), (1349, 30), (1427, 30), (1429, 30), (1482, 30), (1485, 30), (1508, 30), (1516, 30), (1539, 30), (1549, 30), (1588, 30), (1636, 30), (1721, 30), (1748, 30), (1750, 30), (1754, 30), (1785, 30), (1790, 30), (1830, 30), (1846, 30), (1849, 30), (1933, 30), (1989, 30), (2048, 30), (2069, 30), (2076, 30), (2078, 30), (2101, 30), (2150, 30), (2174, 30), (2186, 30), (2241, 30), (2278, 30), (2302, 30), (2316, 30), (2341, 30), (2352, 30), (2398, 30), (2402, 30), (2422, 30), (2489, 30), (2491, 30), (2492, 30), (2495, 30), (2499, 30), (2541, 30), (2606, 30), (2610, 30), (2612, 30), (2621, 30), (2633, 30), (2676, 30), (2691, 30), (2703, 30), (2711, 30), (2717, 30), (2732, 30), (2757, 30), (2761, 30), (2765, 30), (2773, 30), (2776, 30), (2779, 30), (2786, 30), (2793, 30), (2855, 30), (2879, 30), (2912, 30), (2986, 30), (2988, 30), (3035, 30), (3051, 30), (3092, 30), (3097, 30), (3100, 30), (3102, 30), (3120, 30), (3140, 30), (3189, 30), (3199, 30), (3203, 30), (3215, 30), (3248, 30), (3273, 30), (3284, 30), (3294, 30), (3321, 30), (3343, 30), (3364, 30), (3379, 30), (3472, 30), (3559, 30), (3591, 30), (3665, 30), (3670, 30), (3693, 30), (3715, 30), (3732, 30), (3736, 30), (3749, 30), (3761, 30), (3796, 30), (3811, 30), (3836, 30), (3899, 30), (3927, 30), (3965, 30), (3999, 30), (4022, 30), (4036, 30), (4137, 30), (4178, 30), (4187, 30), (4226, 30), (4304, 30), (4341, 30), (4355, 30), (4369, 30), (4371, 30), (4462, 30), (4472, 30), (4508, 30), (4529, 30), (4544, 30), (4570, 30), (4579, 30), (4614, 30), (4617, 30), (4651, 30), (4659, 30), (4714, 30), (4723, 30), (4741, 30), (4744, 30), (4746, 30), (4758, 30), (4769, 30), (4819, 30), (4825, 30), (4865, 30), (4866, 30), (4880, 30), (4914, 30), (4921, 30), (4932, 30), (4972, 30), (4989, 30), (4992, 30), (5017, 30), (5021, 30), (5049, 30), (5066, 30), (5116, 30), (5117, 30), (5121, 30), (5134, 30), (5137, 30), (5150, 30), (5179, 30), (5249, 30), (5256, 30), (5268, 30), (5276, 30), (5282, 30), (5286, 30), (5320, 30), (5337, 30), (5375, 30), (5393, 30), (5405, 30), (5423, 30), (5444, 30), (5448, 30), (5569, 30), (5581, 30), (5598, 30), (5600, 30), (5635, 30), (5642, 30), (5649, 30), (5668, 30), (5673, 30), (5682, 30), (5748, 30), (5751, 30), (5865, 30), (5897, 30), (5914, 30), (5938, 30), (5952, 30), (5986, 30), (5999, 30), (6050, 30), (6080, 30), (6133, 30), (6143, 30), (6194, 30), (6210, 30), (6211, 30), (6232, 30), (6259, 30), (6284, 30), (6305, 30), (6310, 30), (6384, 30), (6394, 30), (6403, 30), (6455, 30), (6456, 30), (6480, 30), (6497, 30), (6503, 30), (6508, 30), (6518, 30), (6526, 30), (6547, 30), (6590, 30), (6627, 30), (6657, 30), (6664, 30), (6665, 30), (6712, 30), (6735, 30), (6752, 30), (6759, 30), (6767, 30), (6802, 30), (6865, 30), (6885, 30), (6896, 30), (6930, 30), (6953, 30), (6967, 30), (6974, 30), (6999, 30), (7038, 30), (7086, 30), (7148, 30), (7238, 30), (7242, 30), (7253, 30), (7261, 30), (7302, 30), (7307, 30), (7315, 30), (7333, 30), (7354, 30), (7382, 30), (7384, 30), (7387, 30), (7391, 30), (7455, 30), (7482, 30), (7519, 30), (7520, 30), (7533, 30), (7537, 30), (7588, 30), (7614, 30), (7620, 30), (7656, 30), (7674, 30), (7723, 30), (7754, 30), (7759, 30), (7761, 30), (7798, 30), (7808, 30), (7821, 30), (7862, 30), (7882, 30), (7885, 30), (7903, 30), (7911, 30), (7949, 30), (7952, 30), (7973, 30), (8013, 30), (8021, 30), (8065, 30), (8067, 30), (8069, 30), (8077, 30), (8112, 30), (8132, 30), (8159, 30), (8160, 30), (8166, 30), (8248, 30), (8255, 30), (8293, 30), (8297, 30), (8301, 30), (8317, 30), (8349, 30), (8361, 30), (8392, 30), (8402, 30), (8404, 30), (8414, 30), (8485, 30), (8487, 30), (8494, 30), (8521, 30), (8540, 30), (8545, 30), (8601, 30), (8603, 30), (8634, 30), (8684, 30), (8702, 30), (8709, 30), (8726, 30), (8747, 30), (8797, 30), (8822, 30), (8826, 30), (8843, 30), (8889, 30), (8902, 30), (8934, 30), (8958, 30), (8966, 30), (8970, 30), (8977, 30), (9025, 30), (9056, 30), (9098, 30), (9207, 30), (9231, 30), (9249, 30), (9250, 30), (9263, 30), (9275, 30), (9284, 30), (9311, 30), (9322, 30), (9377, 30), (9393, 30), (9426, 30), (9488, 30), (9512, 30), (9577, 30), (9585, 30), (9596, 30), (9616, 30), (9639, 30), (9673, 30), (9715, 30), (9741, 30), (9744, 30), (9761, 30), (9762, 30), (9773, 30), (9809, 30), (9839, 30), (9843, 30), (9855, 30), (9861, 30), (9882, 30), (9905, 30), (9910, 30), (9970, 30), (9973, 30), (10033, 30), (10041, 30), (10051, 30), (10057, 30), (10072, 30), (10090, 30), (10099, 30), (10107, 30), (10175, 30), (10257, 30), (10286, 30), (10290, 30), (10338, 30), (10376, 30), (10395, 30), (10396, 30), (10402, 30), (10436, 30), (10476, 30), (10520, 30), (10521, 30), (10525, 30), (10528, 30), (10546, 30), (10548, 30), (10549, 30), (10573, 30), (10576, 30), (10640, 30), (10651, 30), (10655, 30), (10718, 30), (10732, 30), (10783, 30), (10807, 30), (10830, 30), (10847, 30), (10856, 30), (10861, 30), (10871, 30), (10874, 30), (10903, 30), (10908, 30), (10913, 30), (10957, 30), (10995, 30), (11018, 30), (11025, 30), (11100, 30), (11101, 30), (11117, 30), (11121, 30), (11123, 30), (11131, 30), (11154, 30), (11175, 30), (11194, 30), (11202, 30), (11205, 30), (11234, 30), (11241, 30), (11249, 30), (11268, 30), (11306, 30), (11317, 30), (11335, 30), (11336, 30), (11350, 30), (11361, 30), (11377, 30), (11425, 30), (11467, 30), (11472, 30), (11554, 30), (11595, 30), (11629, 30), (11631, 30), (11652, 30), (11667, 30), (11680, 30), (11681, 30), (11723, 30), (11728, 30), (11732, 30), (11736, 30), (11737, 30), (11788, 30), (11795, 30), (11805, 30), (11816, 30), (11843, 30), (11849, 30), (11850, 30), (11851, 30), (11895, 30), (11927, 30), (11928, 30), (11936, 30), (11957, 30), (11967, 30), (12022, 30), (12030, 30), (12058, 30), (12059, 30), (12061, 30), (12133, 30), (12153, 30), (12188, 30), (12207, 30), (12224, 30), (12259, 30), (12371, 30), (12423, 30), (12451, 30), (12459, 30), (12499, 30), (12556, 30), (12560, 30), (12597, 30), (12609, 30), (12621, 30), (12643, 30), (12651, 30), (12669, 30), (12679, 30), (12735, 30), (12770, 30), (12784, 30), (12787, 30), (12791, 30), (31, 31), (36, 31), (60, 31), (65, 31), (77, 31), (78, 31), (86, 31), (140, 31), (165, 31), (185, 31), (187, 31), (229, 31), (230, 31), (251, 31), (282, 31), (296, 31), (298, 31), (301, 31), (389, 31), (429, 31), (519, 31), (540, 31), (585, 31), (612, 31), (619, 31), (620, 31), (670, 31), (688, 31), (718, 31), (720, 31), (829, 31), (837, 31), (850, 31), (863, 31), (904, 31), (926, 31), (969, 31), (1115, 31), (1117, 31), (1126, 31), (1128, 31), (1137, 31), (1138, 31), (1174, 31), (1192, 31), (1254, 31), (1272, 31), (1276, 31), (1279, 31), (1310, 31), (1322, 31), (1331, 31), (1337, 31), (1381, 31), (1423, 31), (1500, 31), (1528, 31), (1639, 31), (1674, 31), (1693, 31), (1742, 31), (1763, 31), (1780, 31), (1834, 31), (1850, 31), (1859, 31), (1861, 31), (1892, 31), (1929, 31), (1931, 31), (1969, 31), (1985, 31), (2010, 31), (2043, 31), (2045, 31), (2055, 31), (2070, 31), (2159, 31), (2242, 31), (2245, 31), (2262, 31), (2290, 31), (2298, 31), (2306, 31), (2356, 31), (2369, 31), (2411, 31), (2427, 31), (2467, 31), (2468, 31), (2504, 31), (2524, 31), (2546, 31), (2563, 31), (2598, 31), (2622, 31), (2659, 31), (2678, 31), (2695, 31), (2758, 31), (2812, 31), (2813, 31), (2828, 31), (2871, 31), (2886, 31), (2896, 31), (2917, 31), (2932, 31), (2939, 31), (2950, 31), (2961, 31), (2987, 31), (3025, 31), (3026, 31), (3068, 31), (3076, 31), (3077, 31), (3088, 31), (3138, 31), (3196, 31), (3205, 31), (3304, 31), (3308, 31), (3311, 31), (3381, 31), (3415, 31), (3449, 31), (3467, 31), (3477, 31), (3490, 31), (3517, 31), (3527, 31), (3531, 31), (3540, 31), (3573, 31), (3577, 31), (3635, 31), (3639, 31), (3645, 31), (3731, 31), (3740, 31), (3756, 31), (3783, 31), (3784, 31), (3803, 31), (3815, 31), (3820, 31), (3827, 31), (3851, 31), (3935, 31), (4014, 31), (4041, 31), (4042, 31), (4049, 31), (4076, 31), (4086, 31), (4138, 31), (4139, 31), (4141, 31), (4148, 31), (4162, 31), (4177, 31), (4263, 31), (4339, 31), (4368, 31), (4373, 31), (4411, 31), (4429, 31), (4466, 31), (4477, 31), (4480, 31), (4493, 31), (4505, 31), (4547, 31), (4562, 31), (4602, 31), (4620, 31), (4624, 31), (4695, 31), (4728, 31), (4755, 31), (4775, 31), (4836, 31), (4864, 31), (4912, 31), (4926, 31), (4930, 31), (4941, 31), (4969, 31), (4999, 31), (5063, 31), (5103, 31), (5194, 31), (5218, 31), (5251, 31), (5289, 31), (5318, 31), (5328, 31), (5330, 31), (5372, 31), (5402, 31), (5455, 31), (5467, 31), (5519, 31), (5520, 31), (5543, 31), (5552, 31), (5565, 31), (5579, 31), (5585, 31), (5640, 31), (5660, 31), (5662, 31), (5679, 31), (5701, 31), (5710, 31), (5743, 31), (5776, 31), (5802, 31), (5823, 31), (5867, 31), (5970, 31), (5988, 31), (5996, 31), (5997, 31), (6045, 31), (6067, 31), (6073, 31), (6146, 31), (6150, 31), (6158, 31), (6179, 31), (6183, 31), (6303, 31), (6334, 31), (6351, 31), (6410, 31), (6412, 31), (6431, 31), (6433, 31), (6462, 31), (6487, 31), (6516, 31), (6551, 31), (6561, 31), (6591, 31), (6600, 31), (6677, 31), (6710, 31), (6720, 31), (6765, 31), (6808, 31), (6814, 31), (6822, 31), (6840, 31), (6856, 31), (6927, 31), (6938, 31), (6984, 31), (6996, 31), (7033, 31), (7063, 31), (7102, 31), (7106, 31), (7180, 31), (7199, 31), (7213, 31), (7248, 31), (7260, 31), (7270, 31), (7328, 31), (7418, 31), (7465, 31), (7487, 31), (7489, 31), (7497, 31), (7526, 31), (7568, 31), (7571, 31), (7604, 31), (7644, 31), (7647, 31), (7657, 31), (7769, 31), (7794, 31), (7814, 31), (7831, 31), (7893, 31), (7913, 31), (7916, 31), (7922, 31), (7941, 31), (7974, 31), (7979, 31), (7984, 31), (8012, 31), (8066, 31), (8135, 31), (8158, 31), (8162, 31), (8217, 31), (8240, 31), (8245, 31), (8267, 31), (8274, 31), (8275, 31), (8277, 31), (8313, 31), (8351, 31), (8375, 31), (8528, 31), (8534, 31), (8587, 31), (8589, 31), (8612, 31), (8624, 31), (8644, 31), (8649, 31), (8659, 31), (8725, 31), (8738, 31), (8741, 31), (8760, 31), (8788, 31), (8805, 31), (8834, 31), (8885, 31), (8913, 31), (8914, 31), (8920, 31), (8935, 31), (8951, 31), (8957, 31), (9003, 31), (9015, 31), (9062, 31), (9097, 31), (9111, 31), (9127, 31), (9155, 31), (9164, 31), (9180, 31), (9183, 31), (9192, 31), (9203, 31), (9222, 31), (9277, 31), (9278, 31), (9280, 31), (9326, 31), (9381, 31), (9407, 31), (9409, 31), (9458, 31), (9467, 31), (9541, 31), (9582, 31), (9609, 31), (9640, 31), (9727, 31), (9746, 31), (9760, 31), (9778, 31), (9797, 31), (9800, 31), (9803, 31), (9884, 31), (9916, 31), (9969, 31), (9978, 31), (10009, 31), (10025, 31), (10029, 31), (10091, 31), (10145, 31), (10149, 31), (10180, 31), (10207, 31), (10255, 31), (10263, 31), (10272, 31), (10295, 31), (10304, 31), (10319, 31), (10326, 31), (10327, 31), (10339, 31), (10340, 31), (10347, 31), (10360, 31), (10371, 31), (10445, 31), (10571, 31), (10591, 31), (10601, 31), (10604, 31), (10636, 31), (10643, 31), (10649, 31), (10664, 31), (10673, 31), (10686, 31), (10703, 31), (10791, 31), (10822, 31), (10907, 31), (10920, 31), (10961, 31), (10976, 31), (10985, 31), (11033, 31), (11087, 31), (11105, 31), (11125, 31), (11143, 31), (11150, 31), (11153, 31), (11183, 31), (11204, 31), (11252, 31), (11256, 31), (11265, 31), (11299, 31), (11330, 31), (11450, 31), (11452, 31), (11499, 31), (11549, 31), (11565, 31), (11607, 31), (11614, 31), (11656, 31), (11699, 31), (11702, 31), (11751, 31), (11853, 31), (11858, 31), (11870, 31), (11875, 31), (11977, 31), (12004, 31), (12005, 31), (12042, 31), (12126, 31), (12135, 31), (12147, 31), (12177, 31), (12269, 31), (12300, 31), (12306, 31), (12310, 31), (12328, 31), (12338, 31), (12350, 31), (12365, 31), (12385, 31), (12392, 31), (12401, 31), (12412, 31), (12458, 31), (12488, 31), (12502, 31), (12508, 31), (12511, 31), (12513, 31), (12514, 31), (12646, 31), (12691, 31), (12695, 31), (12, 32), (23, 32), (57, 32), (69, 32), (74, 32), (136, 32), (153, 32), (214, 32), (228, 32), (249, 32), (307, 32), (308, 32), (324, 32), (330, 32), (342, 32), (345, 32), (383, 32), (387, 32), (391, 32), (414, 32), (428, 32), (432, 32), (435, 32), (458, 32), (471, 32), (477, 32), (532, 32), (544, 32), (555, 32), (567, 32), (600, 32), (614, 32), (652, 32), (715, 32), (810, 32), (822, 32), (823, 32), (892, 32), (907, 32), (931, 32), (941, 32), (959, 32), (986, 32), (998, 32), (1006, 32), (1022, 32), (1061, 32), (1122, 32), (1136, 32), (1143, 32), (1151, 32), (1155, 32), (1196, 32), (1201, 32), (1205, 32), (1246, 32), (1342, 32), (1469, 32), (1484, 32), (1505, 32), (1531, 32), (1535, 32), (1559, 32), (1586, 32), (1653, 32), (1655, 32), (1697, 32), (1744, 32), (1775, 32), (1812, 32), (1841, 32), (1847, 32), (1863, 32), (1873, 32), (1897, 32), (1911, 32), (1949, 32), (1966, 32), (1967, 32), (2047, 32), (2050, 32), (2063, 32), (2089, 32), (2092, 32), (2138, 32), (2140, 32), (2146, 32), (2149, 32), (2184, 32), (2185, 32), (2191, 32), (2273, 32), (2307, 32), (2325, 32), (2333, 32), (2366, 32), (2370, 32), (2381, 32), (2386, 32), (2426, 32), (2432, 32), (2457, 32), (2542, 32), (2547, 32), (2556, 32), (2564, 32), (2600, 32), (2611, 32), (2620, 32), (2654, 32), (2693, 32), (2783, 32), (2788, 32), (2867, 32), (2966, 32), (2978, 32), (3058, 32), (3107, 32), (3194, 32), (3204, 32), (3227, 32), (3228, 32), (3240, 32), (3319, 32), (3328, 32), (3330, 32), (3358, 32), (3463, 32), (3475, 32), (3478, 32), (3482, 32), (3497, 32), (3613, 32), (3617, 32), (3674, 32), (3799, 32), (3872, 32), (3896, 32), (3924, 32), (3949, 32), (3989, 32), (4015, 32), (4019, 32), (4021, 32), (4037, 32), (4062, 32), (4067, 32), (4095, 32), (4132, 32), (4134, 32), (4152, 32), (4184, 32), (4195, 32), (4203, 32), (4239, 32), (4261, 32), (4290, 32), (4303, 32), (4307, 32), (4332, 32), (4399, 32), (4400, 32), (4419, 32), (4420, 32), (4433, 32), (4447, 32), (4448, 32), (4468, 32), (4501, 32), (4514, 32), (4566, 32), (4589, 32), (4667, 32), (4752, 32), (4795, 32), (4798, 32), (4807, 32), (4816, 32), (4830, 32), (4929, 32), (4938, 32), (4967, 32), (5033, 32), (5043, 32), (5053, 32), (5102, 32), (5151, 32), (5164, 32), (5190, 32), (5203, 32), (5253, 32), (5255, 32), (5309, 32), (5321, 32), (5338, 32), (5470, 32), (5533, 32), (5548, 32), (5571, 32), (5576, 32), (5608, 32), (5646, 32), (5648, 32), (5651, 32), (5653, 32), (5695, 32), (5707, 32), (5737, 32), (5763, 32), (5764, 32), (5815, 32), (5837, 32), (5856, 32), (5882, 32), (5889, 32), (6002, 32), (6012, 32), (6021, 32), (6023, 32), (6029, 32), (6032, 32), (6049, 32), (6060, 32), (6062, 32), (6084, 32), (6111, 32), (6112, 32), (6197, 32), (6218, 32), (6228, 32), (6355, 32), (6383, 32), (6430, 32), (6464, 32), (6493, 32), (6543, 32), (6556, 32), (6604, 32), (6683, 32), (6694, 32), (6707, 32), (6743, 32), (6774, 32), (6820, 32), (6834, 32), (6852, 32), (6900, 32), (6908, 32), (6918, 32), (6921, 32), (6924, 32), (6941, 32), (6954, 32), (7042, 32), (7217, 32), (7295, 32), (7320, 32), (7372, 32), (7392, 32), (7436, 32), (7478, 32), (7509, 32), (7527, 32), (7541, 32), (7596, 32), (7598, 32), (7685, 32), (7722, 32), (7728, 32), (7744, 32), (7745, 32), (7773, 32), (7795, 32), (7802, 32), (7890, 32), (7935, 32), (7965, 32), (7976, 32), (8054, 32), (8117, 32), (8141, 32), (8163, 32), (8288, 32), (8310, 32), (8368, 32), (8400, 32), (8447, 32), (8452, 32), (8482, 32), (8513, 32), (8518, 32), (8524, 32), (8535, 32), (8553, 32), (8627, 32), (8631, 32), (8641, 32), (8680, 32), (8689, 32), (8699, 32), (8713, 32), (8740, 32), (8751, 32), (8768, 32), (8793, 32), (8803, 32), (8809, 32), (8828, 32), (8831, 32), (8838, 32), (8854, 32), (8886, 32), (8899, 32), (8980, 32), (8986, 32), (8992, 32), (9011, 32), (9019, 32), (9080, 32), (9088, 32), (9114, 32), (9118, 32), (9138, 32), (9149, 32), (9205, 32), (9220, 32), (9228, 32), (9290, 32), (9312, 32), (9386, 32), (9404, 32), (9430, 32), (9437, 32), (9452, 32), (9507, 32), (9523, 32), (9633, 32), (9657, 32), (9671, 32), (9691, 32), (9720, 32), (9781, 32), (9845, 32), (9891, 32), (9943, 32), (9972, 32), (9979, 32), (9989, 32), (10086, 32), (10113, 32), (10143, 32), (10197, 32), (10213, 32), (10228, 32), (10289, 32), (10306, 32), (10323, 32), (10377, 32), (10378, 32), (10400, 32), (10416, 32), (10426, 32), (10452, 32), (10472, 32), (10473, 32), (10489, 32), (10498, 32), (10544, 32), (10561, 32), (10608, 32), (10613, 32), (10653, 32), (10665, 32), (10685, 32), (10696, 32), (10744, 32), (10805, 32), (10837, 32), (10889, 32), (10912, 32), (10947, 32), (10968, 32), (10997, 32), (11041, 32), (11089, 32), (11130, 32), (11152, 32), (11159, 32), (11170, 32), (11188, 32), (11211, 32), (11238, 32), (11321, 32), (11329, 32), (11351, 32), (11375, 32), (11386, 32), (11390, 32), (11513, 32), (11531, 32), (11540, 32), (11571, 32), (11592, 32), (11616, 32), (11688, 32), (11762, 32), (11775, 32), (11776, 32), (11792, 32), (11809, 32), (11822, 32), (11929, 32), (11971, 32), (11988, 32), (12031, 32), (12040, 32), (12045, 32), (12052, 32), (12074, 32), (12079, 32), (12098, 32), (12184, 32), (12198, 32), (12235, 32), (12237, 32), (12261, 32), (12274, 32), (12312, 32), (12331, 32), (12355, 32), (12400, 32), (12415, 32), (12584, 32), (12596, 32), (12612, 32), (12652, 32), (12680, 32), (12726, 32), (12771, 32), (12779, 32), (2, 33), (3, 33), (19, 33), (34, 33), (42, 33), (43, 33), (73, 33), (104, 33), (219, 33), (221, 33), (297, 33), (333, 33), (338, 33), (344, 33), (351, 33), (455, 33), (498, 33), (521, 33), (538, 33), (546, 33), (562, 33), (646, 33), (656, 33), (657, 33), (696, 33), (735, 33), (745, 33), (748, 33), (836, 33), (943, 33), (960, 33), (1005, 33), (1108, 33), (1160, 33), (1161, 33), (1328, 33), (1348, 33), (1362, 33), (1440, 33), (1441, 33), (1468, 33), (1593, 33), (1681, 33), (1769, 33), (1795, 33), (1802, 33), (1926, 33), (1947, 33), (1955, 33), (1975, 33), (2052, 33), (2088, 33), (2104, 33), (2106, 33), (2136, 33), (2168, 33), (2171, 33), (2176, 33), (2232, 33), (2257, 33), (2258, 33), (2279, 33), (2330, 33), (2345, 33), (2405, 33), (2415, 33), (2474, 33), (2490, 33), (2514, 33), (2536, 33), (2630, 33), (2755, 33), (2822, 33), (2840, 33), (2849, 33), (2941, 33), (3011, 33), (3065, 33), (3117, 33), (3156, 33), (3179, 33), (3181, 33), (3184, 33), (3233, 33), (3236, 33), (3281, 33), (3301, 33), (3354, 33), (3395, 33), (3400, 33), (3406, 33), (3408, 33), (3411, 33), (3413, 33), (3423, 33), (3437, 33), (3459, 33), (3473, 33), (3485, 33), (3518, 33), (3546, 33), (3551, 33), (3622, 33), (3638, 33), (3652, 33), (3660, 33), (3726, 33), (3738, 33), (3766, 33), (3791, 33), (3810, 33), (3855, 33), (3857, 33), (3913, 33), (3914, 33), (3915, 33), (3917, 33), (3957, 33), (3959, 33), (3981, 33), (3982, 33), (3987, 33), (4020, 33), (4024, 33), (4050, 33), (4058, 33), (4070, 33), (4090, 33), (4155, 33), (4163, 33), (4173, 33), (4197, 33), (4234, 33), (4253, 33), (4267, 33), (4284, 33), (4288, 33), (4408, 33), (4479, 33), (4504, 33), (4509, 33), (4511, 33), (4556, 33), (4574, 33), (4582, 33), (4593, 33), (4597, 33), (4650, 33), (4671, 33), (4697, 33), (4706, 33), (4732, 33), (4794, 33), (4797, 33), (4823, 33), (4849, 33), (4896, 33), (4931, 33), (4933, 33), (5020, 33), (5026, 33), (5051, 33), (5184, 33), (5189, 33), (5193, 33), (5204, 33), (5344, 33), (5362, 33), (5390, 33), (5394, 33), (5437, 33), (5440, 33), (5464, 33), (5514, 33), (5525, 33), (5530, 33), (5559, 33), (5570, 33), (5602, 33), (5671, 33), (5675, 33), (5731, 33), (5772, 33), (5874, 33), (5973, 33), (5995, 33), (5998, 33), (6089, 33), (6110, 33), (6118, 33), (6182, 33), (6189, 33), (6219, 33), (6291, 33), (6304, 33), (6317, 33), (6331, 33), (6376, 33), (6391, 33), (6392, 33), (6396, 33), (6414, 33), (6436, 33), (6499, 33), (6534, 33), (6572, 33), (6607, 33), (6629, 33), (6672, 33), (6688, 33), (6776, 33), (6788, 33), (6795, 33), (6807, 33), (6817, 33), (6823, 33), (6873, 33), (6875, 33), (6925, 33), (6940, 33), (6978, 33), (6993, 33), (7054, 33), (7122, 33), (7142, 33), (7211, 33), (7227, 33), (7251, 33), (7279, 33), (7309, 33), (7310, 33), (7405, 33), (7407, 33), (7425, 33), (7439, 33), (7460, 33), (7462, 33), (7464, 33), (7474, 33), (7484, 33), (7490, 33), (7535, 33), (7562, 33), (7615, 33), (7622, 33), (7624, 33), (7638, 33), (7664, 33), (7669, 33), (7695, 33), (7704, 33), (7706, 33), (7775, 33), (7986, 33), (8031, 33), (8063, 33), (8110, 33), (8156, 33), (8243, 33), (8281, 33), (8286, 33), (8409, 33), (8431, 33), (8446, 33), (8451, 33), (8483, 33), (8497, 33), (8500, 33), (8510, 33), (8544, 33), (8554, 33), (8558, 33), (8562, 33), (8581, 33), (8638, 33), (8650, 33), (8693, 33), (8724, 33), (8730, 33), (8808, 33), (8870, 33), (8942, 33), (9000, 33), (9014, 33), (9075, 33), (9085, 33), (9110, 33), (9157, 33), (9210, 33), (9214, 33), (9224, 33), (9283, 33), (9288, 33), (9352, 33), (9431, 33), (9526, 33), (9591, 33), (9597, 33), (9667, 33), (9708, 33), (9766, 33), (9801, 33), (9808, 33), (9877, 33), (9894, 33), (9902, 33), (9903, 33), (9912, 33), (9917, 33), (9993, 33), (10007, 33), (10008, 33), (10046, 33), (10047, 33), (10130, 33), (10159, 33), (10173, 33), (10216, 33), (10217, 33), (10252, 33), (10305, 33), (10325, 33), (10336, 33), (10351, 33), (10370, 33), (10435, 33), (10439, 33), (10450, 33), (10527, 33), (10623, 33), (10626, 33), (10661, 33), (10715, 33), (10763, 33), (10786, 33), (10886, 33), (10888, 33), (10911, 33), (11019, 33), (11066, 33), (11075, 33), (11114, 33), (11139, 33), (11185, 33), (11220, 33), (11247, 33), (11344, 33), (11405, 33), (11430, 33), (11445, 33), (11456, 33), (11504, 33), (11523, 33), (11569, 33), (11649, 33), (11657, 33), (11673, 33), (11696, 33), (11746, 33), (11749, 33), (11848, 33), (11885, 33), (11932, 33), (11951, 33), (11955, 33), (11987, 33), (12044, 33), (12046, 33), (12047, 33), (12062, 33), (12115, 33), (12122, 33), (12175, 33), (12221, 33), (12240, 33), (12258, 33), (12272, 33), (12290, 33), (12336, 33), (12343, 33), (12344, 33), (12361, 33), (12395, 33), (12396, 33), (12398, 33), (12422, 33), (12429, 33), (12446, 33), (12466, 33), (12493, 33), (12577, 33), (12580, 33), (12635, 33), (12677, 33), (12703, 33), (12717, 33), (12765, 33), (49, 34), (92, 34), (94, 34), (113, 34), (124, 34), (126, 34), (181, 34), (216, 34), (386, 34), (408, 34), (425, 34), (445, 34), (454, 34), (492, 34), (635, 34), (677, 34), (724, 34), (761, 34), (764, 34), (817, 34), (835, 34), (842, 34), (860, 34), (870, 34), (928, 34), (963, 34), (1069, 34), (1112, 34), (1140, 34), (1166, 34), (1248, 34), (1295, 34), (1374, 34), (1385, 34), (1400, 34), (1454, 34), (1529, 34), (1597, 34), (1602, 34), (1617, 34), (1642, 34), (1686, 34), (1720, 34), (1814, 34), (1869, 34), (1871, 34), (1906, 34), (1939, 34), (1944, 34), (1984, 34), (2005, 34), (2030, 34), (2034, 34), (2086, 34), (2099, 34), (2151, 34), (2157, 34), (2160, 34), (2177, 34), (2203, 34), (2235, 34), (2309, 34), (2367, 34), (2425, 34), (2429, 34), (2456, 34), (2460, 34), (2497, 34), (2515, 34), (2537, 34), (2559, 34), (2577, 34), (2604, 34), (2648, 34), (2652, 34), (2698, 34), (2719, 34), (2748, 34), (2759, 34), (2764, 34), (2771, 34), (2778, 34), (2792, 34), (2909, 34), (3010, 34), (3020, 34), (3022, 34), (3034, 34), (3050, 34), (3113, 34), (3324, 34), (3436, 34), (3438, 34), (3534, 34), (3537, 34), (3550, 34), (3579, 34), (3648, 34), (3667, 34), (3679, 34), (3689, 34), (3695, 34), (3739, 34), (3767, 34), (3789, 34), (3832, 34), (3885, 34), (3916, 34), (3940, 34), (3973, 34), (3979, 34), (3990, 34), (3991, 34), (4045, 34), (4188, 34), (4235, 34), (4247, 34), (4273, 34), (4412, 34), (4445, 34), (4545, 34), (4552, 34), (4565, 34), (4672, 34), (4683, 34), (4722, 34), (4781, 34), (4792, 34), (4806, 34), (4883, 34), (4922, 34), (4962, 34), (4991, 34), (5002, 34), (5042, 34), (5048, 34), (5085, 34), (5106, 34), (5114, 34), (5148, 34), (5149, 34), (5208, 34), (5284, 34), (5291, 34), (5339, 34), (5346, 34), (5451, 34), (5486, 34), (5494, 34), (5517, 34), (5560, 34), (5573, 34), (5616, 34), (5754, 34), (5761, 34), (5812, 34), (5825, 34), (5873, 34), (5885, 34), (5937, 34), (5939, 34), (5965, 34), (5976, 34), (5982, 34), (6081, 34), (6098, 34), (6104, 34), (6105, 34), (6128, 34), (6137, 34), (6138, 34), (6265, 34), (6285, 34), (6298, 34), (6302, 34), (6327, 34), (6450, 34), (6452, 34), (6469, 34), (6495, 34), (6540, 34), (6552, 34), (6559, 34), (6566, 34), (6580, 34), (6609, 34), (6613, 34), (6619, 34), (6691, 34), (6760, 34), (6766, 34), (6768, 34), (6780, 34), (6785, 34), (6790, 34), (6796, 34), (6806, 34), (6882, 34), (7007, 34), (7034, 34), (7056, 34), (7081, 34), (7101, 34), (7111, 34), (7175, 34), (7184, 34), (7240, 34), (7299, 34), (7381, 34), (7426, 34), (7456, 34), (7510, 34), (7530, 34), (7553, 34), (7561, 34), (7581, 34), (7600, 34), (7662, 34), (7681, 34), (7705, 34), (7735, 34), (7737, 34), (7844, 34), (7857, 34), (7884, 34), (7943, 34), (8016, 34), (8039, 34), (8068, 34), (8070, 34), (8082, 34), (8123, 34), (8225, 34), (8323, 34), (8363, 34), (8365, 34), (8370, 34), (8384, 34), (8456, 34), (8499, 34), (8519, 34), (8549, 34), (8694, 34), (8731, 34), (8743, 34), (8755, 34), (8856, 34), (8884, 34), (8956, 34), (8989, 34), (9067, 34), (9094, 34), (9116, 34), (9191, 34), (9218, 34), (9253, 34), (9281, 34), (9286, 34), (9299, 34), (9397, 34), (9402, 34), (9434, 34), (9560, 34), (9589, 34), (9649, 34), (9704, 34), (9707, 34), (9724, 34), (9786, 34), (9857, 34), (9858, 34), (9929, 34), (9933, 34), (10062, 34), (10169, 34), (10205, 34), (10233, 34), (10317, 34), (10330, 34), (10337, 34), (10497, 34), (10511, 34), (10543, 34), (10566, 34), (10579, 34), (10580, 34), (10594, 34), (10616, 34), (10619, 34), (10631, 34), (10671, 34), (10691, 34), (10729, 34), (10785, 34), (10827, 34), (10875, 34), (10919, 34), (11053, 34), (11057, 34), (11071, 34), (11076, 34), (11083, 34), (11167, 34), (11278, 34), (11308, 34), (11314, 34), (11383, 34), (11429, 34), (11521, 34), (11552, 34), (11635, 34), (11661, 34), (11705, 34), (11724, 34), (11738, 34), (11780, 34), (11864, 34), (11865, 34), (11922, 34), (11938, 34), (11978, 34), (12008, 34), (12039, 34), (12091, 34), (12100, 34), (12172, 34), (12340, 34), (12359, 34), (12388, 34), (12457, 34), (12473, 34), (12600, 34), (12624, 34), (12630, 34), (12637, 34), (12653, 34), (12686, 34), (12692, 34), (12711, 34), (12712, 34), (44, 35), (91, 35), (96, 35), (236, 35), (271, 35), (288, 35), (317, 35), (360, 35), (365, 35), (371, 35), (405, 35), (452, 35), (518, 35), (563, 35), (673, 35), (678, 35), (703, 35), (757, 35), (778, 35), (779, 35), (801, 35), (889, 35), (902, 35), (1008, 35), (1085, 35), (1148, 35), (1149, 35), (1152, 35), (1186, 35), (1211, 35), (1245, 35), (1258, 35), (1263, 35), (1268, 35), (1285, 35), (1384, 35), (1416, 35), (1461, 35), (1465, 35), (1498, 35), (1556, 35), (1560, 35), (1576, 35), (1599, 35), (1663, 35), (1692, 35), (1707, 35), (1709, 35), (1724, 35), (1740, 35), (1797, 35), (1805, 35), (1808, 35), (1877, 35), (1913, 35), (1917, 35), (1974, 35), (2001, 35), (2003, 35), (2111, 35), (2197, 35), (2217, 35), (2219, 35), (2220, 35), (2221, 35), (2259, 35), (2368, 35), (2390, 35), (2540, 35), (2725, 35), (2737, 35), (2884, 35), (3052, 35), (3084, 35), (3154, 35), (3162, 35), (3192, 35), (3193, 35), (3276, 35), (3318, 35), (3355, 35), (3363, 35), (3453, 35), (3515, 35), (3541, 35), (3606, 35), (3661, 35), (3683, 35), (3716, 35), (3770, 35), (3848, 35), (3871, 35), (3892, 35), (3898, 35), (3936, 35), (4044, 35), (4048, 35), (4097, 35), (4119, 35), (4147, 35), (4189, 35), (4248, 35), (4353, 35), (4374, 35), (4391, 35), (4436, 35), (4528, 35), (4558, 35), (4561, 35), (4634, 35), (4652, 35), (4693, 35), (4730, 35), (4801, 35), (4884, 35), (5009, 35), (5022, 35), (5037, 35), (5187, 35), (5220, 35), (5233, 35), (5277, 35), (5301, 35), (5314, 35), (5351, 35), (5354, 35), (5417, 35), (5421, 35), (5478, 35), (5489, 35), (5497, 35), (5522, 35), (5544, 35), (5674, 35), (5683, 35), (5730, 35), (5733, 35), (5786, 35), (5827, 35), (5858, 35), (5861, 35), (5975, 35), (5981, 35), (6040, 35), (6127, 35), (6190, 35), (6253, 35), (6328, 35), (6346, 35), (6359, 35), (6388, 35), (6509, 35), (6560, 35), (6614, 35), (6633, 35), (6751, 35), (6821, 35), (6826, 35), (6831, 35), (6877, 35), (6889, 35), (6920, 35), (6970, 35), (6988, 35), (7021, 35), (7082, 35), (7095, 35), (7118, 35), (7124, 35), (7137, 35), (7206, 35), (7212, 35), (7361, 35), (7379, 35), (7383, 35), (7397, 35), (7431, 35), (7554, 35), (7578, 35), (7631, 35), (7648, 35), (7650, 35), (7672, 35), (7682, 35), (7726, 35), (7739, 35), (7753, 35), (7771, 35), (7817, 35), (7863, 35), (7909, 35), (7910, 35), (7921, 35), (7963, 35), (8019, 35), (8026, 35), (8085, 35), (8087, 35), (8189, 35), (8350, 35), (8390, 35), (8488, 35), (8525, 35), (8572, 35), (8673, 35), (8677, 35), (8727, 35), (8749, 35), (8818, 35), (8873, 35), (8874, 35), (8894, 35), (8929, 35), (8959, 35), (8978, 35), (9016, 35), (9026, 35), (9029, 35), (9030, 35), (9033, 35), (9050, 35), (9084, 35), (9099, 35), (9102, 35), (9126, 35), (9160, 35), (9188, 35), (9258, 35), (9480, 35), (9517, 35), (9525, 35), (9547, 35), (9607, 35), (9795, 35), (9899, 35), (9915, 35), (9925, 35), (9981, 35), (9990, 35), (10016, 35), (10058, 35), (10138, 35), (10219, 35), (10231, 35), (10399, 35), (10504, 35), (10524, 35), (10526, 35), (10529, 35), (10538, 35), (10586, 35), (10714, 35), (10746, 35), (10759, 35), (10804, 35), (10813, 35), (10850, 35), (10865, 35), (10883, 35), (10941, 35), (10952, 35), (10975, 35), (10981, 35), (11038, 35), (11136, 35), (11246, 35), (11315, 35), (11356, 35), (11370, 35), (11417, 35), (11475, 35), (11484, 35), (11509, 35), (11548, 35), (11557, 35), (11573, 35), (11605, 35), (11610, 35), (11613, 35), (11644, 35), (11651, 35), (11690, 35), (11767, 35), (11787, 35), (11808, 35), (11833, 35), (11842, 35), (11872, 35), (11879, 35), (11897, 35), (11943, 35), (11999, 35), (12033, 35), (12076, 35), (12081, 35), (12108, 35), (12121, 35), (12138, 35), (12139, 35), (12176, 35), (12226, 35), (12265, 35), (12346, 35), (12431, 35), (12463, 35), (12528, 35), (12581, 35), (12583, 35), (12634, 35), (12659, 35), (12707, 35), (12708, 35), (12718, 35), (12721, 35), (12799, 35), (62, 36), (84, 36), (87, 36), (95, 36), (146, 36), (152, 36), (293, 36), (309, 36), (418, 36), (550, 36), (559, 36), (622, 36), (725, 36), (728, 36), (920, 36), (925, 36), (967, 36), (999, 36), (1012, 36), (1029, 36), (1041, 36), (1097, 36), (1213, 36), (1249, 36), (1372, 36), (1434, 36), (1488, 36), (1523, 36), (1537, 36), (1587, 36), (1590, 36), (1736, 36), (1752, 36), (1801, 36), (1845, 36), (1921, 36), (1938, 36), (1999, 36), (2041, 36), (2077, 36), (2091, 36), (2180, 36), (2182, 36), (2229, 36), (2247, 36), (2272, 36), (2295, 36), (2303, 36), (2350, 36), (2383, 36), (2399, 36), (2494, 36), (2583, 36), (2605, 36), (2626, 36), (2631, 36), (2694, 36), (2713, 36), (2791, 36), (2906, 36), (2989, 36), (3067, 36), (3082, 36), (3170, 36), (3191, 36), (3200, 36), (3226, 36), (3237, 36), (3351, 36), (3390, 36), (3441, 36), (3465, 36), (3510, 36), (3556, 36), (3566, 36), (3621, 36), (3642, 36), (3682, 36), (3751, 36), (3819, 36), (3856, 36), (3859, 36), (3928, 36), (4101, 36), (4213, 36), (4255, 36), (4306, 36), (4318, 36), (4325, 36), (4348, 36), (4365, 36), (4376, 36), (4385, 36), (4392, 36), (4395, 36), (4499, 36), (4610, 36), (4862, 36), (4889, 36), (4898, 36), (4903, 36), (4925, 36), (4994, 36), (5201, 36), (5229, 36), (5336, 36), (5353, 36), (5360, 36), (5414, 36), (5428, 36), (5485, 36), (5498, 36), (5599, 36), (5613, 36), (5774, 36), (5775, 36), (5900, 36), (5901, 36), (5943, 36), (5956, 36), (6076, 36), (6091, 36), (6209, 36), (6292, 36), (6341, 36), (6453, 36), (6473, 36), (6490, 36), (6557, 36), (6579, 36), (6582, 36), (6663, 36), (6668, 36), (6679, 36), (6705, 36), (6726, 36), (6828, 36), (6857, 36), (6860, 36), (6903, 36), (6904, 36), (6939, 36), (7039, 36), (7057, 36), (7084, 36), (7088, 36), (7096, 36), (7121, 36), (7162, 36), (7205, 36), (7312, 36), (7376, 36), (7385, 36), (7420, 36), (7543, 36), (7584, 36), (7589, 36), (7591, 36), (7603, 36), (7634, 36), (7651, 36), (7699, 36), (7784, 36), (7816, 36), (7825, 36), (7838, 36), (7846, 36), (7871, 36), (7880, 36), (7946, 36), (7969, 36), (8002, 36), (8022, 36), (8023, 36), (8036, 36), (8052, 36), (8133, 36), (8257, 36), (8356, 36), (8387, 36), (8448, 36), (8477, 36), (8481, 36), (8511, 36), (8582, 36), (8671, 36), (8804, 36), (8824, 36), (8836, 36), (8848, 36), (8851, 36), (8878, 36), (8953, 36), (9008, 36), (9141, 36), (9264, 36), (9285, 36), (9347, 36), (9348, 36), (9442, 36), (9466, 36), (9500, 36), (9557, 36), (9569, 36), (9590, 36), (9626, 36), (9677, 36), (9714, 36), (9796, 36), (9799, 36), (9838, 36), (9860, 36), (9953, 36), (9959, 36), (9961, 36), (10027, 36), (10061, 36), (10076, 36), (10105, 36), (10108, 36), (10186, 36), (10188, 36), (10234, 36), (10324, 36), (10359, 36), (10392, 36), (10403, 36), (10478, 36), (10588, 36), (10589, 36), (10599, 36), (10603, 36), (10607, 36), (10609, 36), (10659, 36), (10663, 36), (10692, 36), (10781, 36), (10801, 36), (10864, 36), (10880, 36), (10909, 36), (10966, 36), (10971, 36), (10974, 36), (11006, 36), (11009, 36), (11021, 36), (11032, 36), (11222, 36), (11242, 36), (11258, 36), (11347, 36), (11368, 36), (11388, 36), (11389, 36), (11500, 36), (11537, 36), (11558, 36), (11643, 36), (11671, 36), (11814, 36), (11981, 36), (11991, 36), (12145, 36), (12168, 36), (12213, 36), (12294, 36), (12295, 36), (12450, 36), (12471, 36), (12603, 36), (12628, 36), (12661, 36), (1, 37), (46, 37), (63, 37), (112, 37), (134, 37), (178, 37), (184, 37), (195, 37), (218, 37), (270, 37), (286, 37), (792, 37), (807, 37), (839, 37), (855, 37), (857, 37), (896, 37), (899, 37), (957, 37), (976, 37), (1051, 37), (1060, 37), (1107, 37), (1267, 37), (1347, 37), (1376, 37), (1382, 37), (1386, 37), (1439, 37), (1478, 37), (1486, 37), (1501, 37), (1507, 37), (1546, 37), (1581, 37), (1640, 37), (1650, 37), (1755, 37), (1820, 37), (1826, 37), (1827, 37), (1889, 37), (1896, 37), (1954, 37), (2036, 37), (2187, 37), (2233, 37), (2431, 37), (2466, 37), (2568, 37), (2599, 37), (2629, 37), (2645, 37), (2650, 37), (2766, 37), (2799, 37), (2841, 37), (2927, 37), (2928, 37), (2930, 37), (2938, 37), (3112, 37), (3185, 37), (3256, 37), (3322, 37), (3341, 37), (3460, 37), (3508, 37), (3554, 37), (3578, 37), (3586, 37), (3589, 37), (3620, 37), (3780, 37), (3794, 37), (3918, 37), (3962, 37), (4031, 37), (4035, 37), (4047, 37), (4124, 37), (4181, 37), (4413, 37), (4435, 37), (4476, 37), (4521, 37), (4642, 37), (4670, 37), (4674, 37), (4704, 37), (4721, 37), (4742, 37), (4753, 37), (5109, 37), (5226, 37), (5304, 37), (5361, 37), (5392, 37), (5452, 37), (5510, 37), (5609, 37), (5625, 37), (5658, 37), (5664, 37), (5693, 37), (5810, 37), (5818, 37), (5909, 37), (5993, 37), (6013, 37), (6082, 37), (6085, 37), (6167, 37), (6264, 37), (6370, 37), (6380, 37), (6507, 37), (6564, 37), (6639, 37), (6649, 37), (6696, 37), (6730, 37), (6747, 37), (6841, 37), (6843, 37), (6879, 37), (6910, 37), (6936, 37), (6964, 37), (7026, 37), (7406, 37), (7427, 37), (7481, 37), (7512, 37), (7555, 37), (7590, 37), (7636, 37), (7667, 37), (7683, 37), (7694, 37), (7740, 37), (7742, 37), (7750, 37), (7847, 37), (7876, 37), (7906, 37), (8094, 37), (8149, 37), (8151, 37), (8211, 37), (8261, 37), (8309, 37), (8413, 37), (8416, 37), (8429, 37), (8453, 37), (8458, 37), (8578, 37), (8619, 37), (8626, 37), (8711, 37), (8766, 37), (8785, 37), (8853, 37), (8864, 37), (9058, 37), (9090, 37), (9104, 37), (9124, 37), (9129, 37), (9169, 37), (9186, 37), (9305, 37), (9321, 37), (9328, 37), (9506, 37), (9524, 37), (9602, 37), (9617, 37), (9656, 37), (9658, 37), (9663, 37), (9739, 37), (9758, 37), (9844, 37), (10001, 37), (10060, 37), (10153, 37), (10154, 37), (10238, 37), (10265, 37), (10283, 37), (10313, 37), (10401, 37), (10506, 37), (10559, 37), (10677, 37), (10706, 37), (10784, 37), (10787, 37), (10794, 37), (10810, 37), (10901, 37), (10906, 37), (10940, 37), (10954, 37), (11146, 37), (11177, 37), (11214, 37), (11218, 37), (11240, 37), (11244, 37), (11380, 37), (11427, 37), (11434, 37), (11457, 37), (11590, 37), (11628, 37), (11704, 37), (11709, 37), (11889, 37), (11913, 37), (12013, 37), (12053, 37), (12068, 37), (12071, 37), (12167, 37), (12173, 37), (12217, 37), (12357, 37), (12393, 37), (12419, 37), (12484, 37), (12558, 37), (76, 38), (143, 38), (168, 38), (186, 38), (201, 38), (233, 38), (235, 38), (278, 38), (361, 38), (395, 38), (553, 38), (583, 38), (628, 38), (669, 38), (683, 38), (738, 38), (773, 38), (832, 38), (979, 38), (980, 38), (989, 38), (1010, 38), (1034, 38), (1036, 38), (1076, 38), (1078, 38), (1130, 38), (1218, 38), (1296, 38), (1303, 38), (1326, 38), (1368, 38), (1415, 38), (1424, 38), (1453, 38), (1483, 38), (1511, 38), (1543, 38), (1614, 38), (1773, 38), (1774, 38), (1831, 38), (1842, 38), (1887, 38), (1922, 38), (1950, 38), (2035, 38), (2090, 38), (2132, 38), (2141, 38), (2142, 38), (2358, 38), (2435, 38), (2445, 38), (2573, 38), (2647, 38), (2718, 38), (2726, 38), (2751, 38), (2752, 38), (2784, 38), (2821, 38), (2852, 38), (2931, 38), (3061, 38), (3106, 38), (3229, 38), (3352, 38), (3401, 38), (3619, 38), (3688, 38), (3729, 38), (3742, 38), (3849, 38), (3867, 38), (3876, 38), (3886, 38), (3888, 38), (3997, 38), (4007, 38), (4061, 38), (4063, 38), (4080, 38), (4112, 38), (4144, 38), (4159, 38), (4179, 38), (4393, 38), (4542, 38), (4546, 38), (4571, 38), (4631, 38), (4632, 38), (4657, 38), (4814, 38), (4842, 38), (4850, 38), (4871, 38), (4961, 38), (5087, 38), (5130, 38), (5293, 38), (5500, 38), (5505, 38), (5587, 38), (5634, 38), (5641, 38), (5745, 38), (5811, 38), (5839, 38), (5907, 38), (5934, 38), (5960, 38), (6043, 38), (6093, 38), (6114, 38), (6166, 38), (6169, 38), (6192, 38), (6220, 38), (6247, 38), (6323, 38), (6361, 38), (6420, 38), (6505, 38), (6555, 38), (6884, 38), (7071, 38), (7100, 38), (7139, 38), (7275, 38), (7346, 38), (7386, 38), (7446, 38), (7471, 38), (7521, 38), (7594, 38), (7637, 38), (7807, 38), (7824, 38), (7833, 38), (7855, 38), (7870, 38), (7968, 38), (7987, 38), (8025, 38), (8103, 38), (8234, 38), (8291, 38), (8382, 38), (8415, 38), (8421, 38), (8423, 38), (8506, 38), (8557, 38), (8576, 38), (8633, 38), (8640, 38), (8663, 38), (8714, 38), (8794, 38), (8845, 38), (8900, 38), (8979, 38), (9022, 38), (9070, 38), (9079, 38), (9101, 38), (9133, 38), (9237, 38), (9246, 38), (9304, 38), (9338, 38), (9364, 38), (9453, 38), (9496, 38), (9612, 38), (9615, 38), (9651, 38), (9774, 38), (9815, 38), (9817, 38), (9818, 38), (9863, 38), (9890, 38), (9927, 38), (9942, 38), (10050, 38), (10052, 38), (10063, 38), (10097, 38), (10120, 38), (10139, 38), (10214, 38), (10223, 38), (10296, 38), (10382, 38), (10407, 38), (10425, 38), (10444, 38), (10461, 38), (10474, 38), (10505, 38), (10552, 38), (10724, 38), (10770, 38), (10773, 38), (10778, 38), (10867, 38), (10892, 38), (11122, 38), (11173, 38), (11231, 38), (11301, 38), (11384, 38), (11437, 38), (11580, 38), (11617, 38), (11778, 38), (11803, 38), (11933, 38), (12027, 38), (12182, 38), (12362, 38), (12420, 38), (12441, 38), (12449, 38), (12464, 38), (12479, 38), (12546, 38), (12578, 38), (12636, 38), (12740, 38), (12764, 38), (67, 39), (70, 39), (177, 39), (522, 39), (529, 39), (780, 39), (856, 39), (929, 39), (953, 39), (1127, 39), (1247, 39), (1259, 39), (1426, 39), (1436, 39), (1479, 39), (1522, 39), (1557, 39), (1585, 39), (1625, 39), (1627, 39), (1743, 39), (1761, 39), (1979, 39), (1996, 39), (2000, 39), (2083, 39), (2094, 39), (2108, 39), (2270, 39), (2285, 39), (2318, 39), (2320, 39), (2396, 39), (2416, 39), (2487, 39), (2498, 39), (2548, 39), (2585, 39), (2763, 39), (2870, 39), (2990, 39), (3029, 39), (3036, 39), (3108, 39), (3132, 39), (3155, 39), (3197, 39), (3678, 39), (3728, 39), (3744, 39), (3757, 39), (3781, 39), (3795, 39), (3817, 39), (3834, 39), (3911, 39), (3941, 39), (4056, 39), (4069, 39), (4196, 39), (4249, 39), (4278, 39), (4283, 39), (4386, 39), (4467, 39), (4490, 39), (4751, 39), (4950, 39), (5347, 39), (5458, 39), (5493, 39), (5575, 39), (5756, 39), (5759, 39), (5789, 39), (5853, 39), (5933, 39), (6022, 39), (6038, 39), (6087, 39), (6174, 39), (6200, 39), (6390, 39), (6407, 39), (6415, 39), (6583, 39), (6648, 39), (6658, 39), (6662, 39), (6770, 39), (7087, 39), (7150, 39), (7234, 39), (7285, 39), (7367, 39), (7368, 39), (7435, 39), (7450, 39), (7472, 39), (7701, 39), (7774, 39), (7788, 39), (8055, 39), (8081, 39), (8181, 39), (8311, 39), (8347, 39), (8354, 39), (8419, 39), (8424, 39), (8551, 39), (8815, 39), (8859, 39), (8905, 39), (8948, 39), (9013, 39), (9043, 39), (9071, 39), (9109, 39), (9244, 39), (9255, 39), (9298, 39), (9455, 39), (9468, 39), (9540, 39), (9556, 39), (9574, 39), (9581, 39), (9608, 39), (9628, 39), (9689, 39), (9807, 39), (9840, 39), (9908, 39), (10044, 39), (10157, 39), (10163, 39), (10172, 39), (10271, 39), (10386, 39), (10447, 39), (10567, 39), (10632, 39), (10654, 39), (10658, 39), (10753, 39), (10836, 39), (10848, 39), (10855, 39), (10953, 39), (11014, 39), (11088, 39), (11148, 39), (11181, 39), (11251, 39), (11285, 39), (11391, 39), (11542, 39), (11771, 39), (11794, 39), (11797, 39), (11859, 39), (11868, 39), (11996, 39), (12003, 39), (12016, 39), (12051, 39), (12085, 39), (12162, 39), (12200, 39), (12247, 39), (12332, 39), (12516, 39), (12550, 39), (12589, 39), (12639, 39), (12683, 39), (12776, 39), (64, 40), (225, 40), (410, 40), (533, 40), (549, 40), (582, 40), (661, 40), (706, 40), (736, 40), (751, 40), (798, 40), (880, 40), (917, 40), (921, 40), (984, 40), (1063, 40), (1064, 40), (1072, 40), (1074, 40), (1110, 40), (1293, 40), (1425, 40), (1552, 40), (1637, 40), (1840, 40), (1908, 40), (1909, 40), (1992, 40), (1994, 40), (2020, 40), (2032, 40), (2074, 40), (2291, 40), (2331, 40), (2340, 40), (2379, 40), (2393, 40), (2562, 40), (2596, 40), (2839, 40), (3012, 40), (3073, 40), (3136, 40), (3412, 40), (3584, 40), (3904, 40), (3921, 40), (3956, 40), (3958, 40), (3970, 40), (4081, 40), (4105, 40), (4245, 40), (4294, 40), (4321, 40), (4422, 40), (4452, 40), (4458, 40), (4491, 40), (4522, 40), (4608, 40), (4653, 40), (4684, 40), (4702, 40), (4710, 40), (4873, 40), (4879, 40), (5007, 40), (5171, 40), (5285, 40), (5324, 40), (5462, 40), (5531, 40), (5684, 40), (5760, 40), (5908, 40), (5917, 40), (5924, 40), (5954, 40), (6006, 40), (6010, 40), (6132, 40), (6144, 40), (6336, 40), (6378, 40), (6413, 40), (6481, 40), (6628, 40), (6661, 40), (6758, 40), (6931, 40), (6994, 40), (7005, 40), (7008, 40), (7051, 40), (7064, 40), (7172, 40), (7177, 40), (7362, 40), (7373, 40), (7547, 40), (7608, 40), (7645, 40), (7654, 40), (7680, 40), (7764, 40), (7796, 40), (7811, 40), (7934, 40), (8000, 40), (8061, 40), (8118, 40), (8131, 40), (8220, 40), (8236, 40), (8341, 40), (8385, 40), (8495, 40), (8703, 40), (8710, 40), (8782, 40), (9293, 40), (9323, 40), (9361, 40), (9449, 40), (9460, 40), (9728, 40), (9729, 40), (9738, 40), (9892, 40), (9896, 40), (10082, 40), (10332, 40), (10598, 40), (10736, 40), (10779, 40), (10815, 40), (10842, 40), (10917, 40), (11157, 40), (11225, 40), (11486, 40), (11666, 40), (11735, 40), (11784, 40), (11902, 40), (11925, 40), (11962, 40), (12209, 40), (12212, 40), (12243, 40), (12287, 40), (12406, 40), (12506, 40), (12611, 40), (12632, 40), (12640, 40), (12760, 40), (202, 41), (205, 41), (212, 41), (290, 41), (569, 41), (586, 41), (796, 41), (961, 41), (1002, 41), (1437, 41), (1494, 41), (1562, 41), (1579, 41), (1582, 41), (1664, 41), (1725, 41), (1728, 41), (1747, 41), (1793, 41), (1823, 41), (1860, 41), (1910, 41), (2042, 41), (2054, 41), (2082, 41), (2107, 41), (2144, 41), (2200, 41), (2237, 41), (2299, 41), (2342, 41), (2365, 41), (2449, 41), (2459, 41), (2475, 41), (2651, 41), (2689, 41), (2697, 41), (2743, 41), (2907, 41), (2920, 41), (2991, 41), (3074, 41), (3075, 41), (3182, 41), (3312, 41), (3391, 41), (3429, 41), (3625, 41), (3734, 41), (3813, 41), (3824, 41), (3863, 41), (3901, 41), (3923, 41), (3984, 41), (4161, 41), (4446, 41), (4455, 41), (4502, 41), (4573, 41), (4609, 41), (4786, 41), (4963, 41), (4990, 41), (5008, 41), (5010, 41), (5110, 41), (5217, 41), (5237, 41), (5260, 41), (5287, 41), (5305, 41), (5329, 41), (5491, 41), (5617, 41), (5620, 41), (5721, 41), (5739, 41), (5888, 41), (5927, 41), (5991, 41), (6435, 41), (6441, 41), (6461, 41), (6532, 41), (6537, 41), (6539, 41), (6608, 41), (6810, 41), (6862, 41), (7378, 41), (7469, 41), (7508, 41), (7689, 41), (7834, 41), (7938, 41), (7962, 41), (8126, 41), (8134, 41), (8155, 41), (8200, 41), (8215, 41), (8228, 41), (8316, 41), (8329, 41), (8411, 41), (8412, 41), (8465, 41), (8752, 41), (8817, 41), (8871, 41), (8896, 41), (8908, 41), (8975, 41), (9054, 41), (9081, 41), (9406, 41), (9443, 41), (9554, 41), (9655, 41), (9674, 41), (9784, 41), (9798, 41), (9854, 41), (10004, 41), (10375, 41), (10533, 41), (10596, 41), (10605, 41), (10740, 41), (11011, 41), (11039, 41), (11155, 41), (11190, 41), (11226, 41), (11394, 41), (11403, 41), (11502, 41), (11535, 41), (11568, 41), (11663, 41), (11720, 41), (11828, 41), (12001, 41), (12087, 41), (12222, 41), (12245, 41), (12654, 41), (12657, 41), (12748, 41), (154, 42), (173, 42), (179, 42), (312, 42), (406, 42), (470, 42), (500, 42), (523, 42), (590, 42), (617, 42), (623, 42), (770, 42), (867, 42), (891, 42), (975, 42), (1018, 42), (1262, 42), (1323, 42), (1404, 42), (1510, 42), (1578, 42), (1608, 42), (1796, 42), (2170, 42), (2516, 42), (2543, 42), (2554, 42), (2572, 42), (2590, 42), (2608, 42), (2624, 42), (2728, 42), (2972, 42), (3006, 42), (3346, 42), (3433, 42), (3476, 42), (3506, 42), (3575, 42), (3720, 42), (3826, 42), (3843, 42), (4005, 42), (4390, 42), (4398, 42), (4434, 42), (4437, 42), (4705, 42), (4811, 42), (4839, 42), (5018, 42), (5092, 42), (5144, 42), (5188, 42), (5561, 42), (5593, 42), (5614, 42), (5629, 42), (5676, 42), (5783, 42), (5864, 42), (5879, 42), (5940, 42), (6212, 42), (6239, 42), (6398, 42), (6451, 42), (6492, 42), (6646, 42), (6689, 42), (6737, 42), (6746, 42), (6756, 42), (6819, 42), (6842, 42), (6916, 42), (7105, 42), (7126, 42), (7127, 42), (7136, 42), (7159, 42), (7219, 42), (7223, 42), (7290, 42), (7339, 42), (7507, 42), (7595, 42), (7613, 42), (7629, 42), (7671, 42), (7678, 42), (7944, 42), (8104, 42), (8106, 42), (8276, 42), (8536, 42), (8563, 42), (8585, 42), (8648, 42), (8672, 42), (8881, 42), (8945, 42), (9034, 42), (9042, 42), (9318, 42), (9354, 42), (9513, 42), (9530, 42), (9563, 42), (9579, 42), (9587, 42), (9625, 42), (9687, 42), (9954, 42), (9956, 42), (10056, 42), (10111, 42), (10128, 42), (10262, 42), (10443, 42), (10539, 42), (10775, 42), (10798, 42), (10852, 42), (10866, 42), (11259, 42), (11310, 42), (11362, 42), (11402, 42), (11516, 42), (11602, 42), (11677, 42), (11785, 42), (11854, 42), (12064, 42), (12118, 42), (12191, 42), (12246, 42), (12321, 42), (12369, 42), (12428, 42), (12486, 42), (12490, 42), (12572, 42), (12574, 42), (12650, 42), (12796, 42), (25, 43), (133, 43), (174, 43), (204, 43), (237, 43), (263, 43), (276, 43), (283, 43), (456, 43), (542, 43), (557, 43), (794, 43), (847, 43), (1009, 43), (1033, 43), (1311, 43), (1312, 43), (1361, 43), (1401, 43), (1517, 43), (1772, 43), (1987, 43), (2125, 43), (2153, 43), (2207, 43), (2421, 43), (2447, 43), (2502, 43), (2528, 43), (2574, 43), (2672, 43), (2690, 43), (2848, 43), (2868, 43), (2947, 43), (3157, 43), (3507, 43), (3655, 43), (3668, 43), (3699, 43), (3845, 43), (3955, 43), (4268, 43), (4330, 43), (4381, 43), (4633, 43), (4861, 43), (4978, 43), (5023, 43), (5216, 43), (5242, 43), (5450, 43), (5594, 43), (5665, 43), (5723, 43), (5842, 43), (5850, 43), (6024, 43), (6054, 43), (6198, 43), (6322, 43), (6419, 43), (6437, 43), (6594, 43), (6859, 43), (6955, 43), (7018, 43), (7120, 43), (7125, 43), (7343, 43), (7448, 43), (7718, 43), (7900, 43), (7997, 43), (8083, 43), (8139, 43), (8146, 43), (8378, 43), (8628, 43), (8692, 43), (8814, 43), (9132, 43), (9446, 43), (9769, 43), (9964, 43), (10240, 43), (10276, 43), (10393, 43), (10412, 43), (10523, 43), (10876, 43), (11044, 43), (11297, 43), (11303, 43), (11373, 43), (11378, 43), (11444, 43), (11538, 43), (11904, 43), (11921, 43), (11980, 43), (12112, 43), (12156, 43), (12339, 43), (12444, 43), (12498, 43), (12619, 43), (38, 44), (190, 44), (483, 44), (866, 44), (1339, 44), (1366, 44), (1496, 44), (1497, 44), (1575, 44), (1628, 44), (1687, 44), (1904, 44), (1973, 44), (1976, 44), (1990, 44), (2120, 44), (2354, 44), (2413, 44), (2552, 44), (2789, 44), (2824, 44), (3002, 44), (3094, 44), (3250, 44), (3539, 44), (3777, 44), (3833, 44), (3860, 44), (3905, 44), (4272, 44), (4336, 44), (4495, 44), (4668, 44), (4756, 44), (4858, 44), (5052, 44), (5173, 44), (5238, 44), (5271, 44), (5334, 44), (5396, 44), (5432, 44), (5527, 44), (5691, 44), (6035, 44), (6099, 44), (6193, 44), (6240, 44), (6329, 44), (6354, 44), (6541, 44), (6836, 44), (6948, 44), (6987, 44), (7141, 44), (7176, 44), (7208, 44), (7298, 44), (7393, 44), (7411, 44), (7440, 44), (7626, 44), (7867, 44), (8037, 44), (8111, 44), (8119, 44), (8374, 44), (8454, 44), (8561, 44), (9171, 44), (9173, 44), (9179, 44), (9230, 44), (9279, 44), (9358, 44), (9683, 44), (10381, 44), (10419, 44), (10494, 44), (10540, 44), (10584, 44), (10873, 44), (10959, 44), (11030, 44), (11099, 44), (11357, 44), (11829, 44), (11896, 44), (12055, 44), (12160, 44), (12724, 44), (147, 45), (215, 45), (238, 45), (239, 45), (287, 45), (461, 45), (496, 45), (515, 45), (912, 45), (1026, 45), (1180, 45), (1278, 45), (1408, 45), (1714, 45), (2009, 45), (2051, 45), (2113, 45), (2236, 45), (2322, 45), (2395, 45), (2527, 45), (2549, 45), (2609, 45), (2636, 45), (3070, 45), (3365, 45), (4082, 45), (4264, 45), (4603, 45), (4821, 45), (4853, 45), (4939, 45), (4964, 45), (5753, 45), (5833, 45), (5849, 45), (6140, 45), (6188, 45), (6245, 45), (6268, 45), (7130, 45), (7156, 45), (7245, 45), (7711, 45), (7743, 45), (7810, 45), (8531, 45), (8635, 45), (8750, 45), (8849, 45), (8915, 45), (9004, 45), (9023, 45), (9175, 45), (9412, 45), (9495, 45), (9694, 45), (9856, 45), (10224, 45), (10373, 45), (10394, 45), (10508, 45), (10621, 45), (10851, 45), (10927, 45), (10939, 45), (11001, 45), (11424, 45), (11440, 45), (11622, 45), (11715, 45), (11800, 45), (12038, 45), (12107, 45), (12141, 45), (12142, 45), (12164, 45), (12223, 45), (12266, 45), (58, 46), (156, 46), (359, 46), (369, 46), (373, 46), (509, 46), (556, 46), (710, 46), (831, 46), (916, 46), (934, 46), (988, 46), (1025, 46), (1354, 46), (1809, 46), (1837, 46), (1866, 46), (2275, 46), (2394, 46), (2437, 46), (2746, 46), (2981, 46), (3288, 46), (3291, 46), (3800, 46), (3944, 46), (3968, 46), (4180, 46), (4217, 46), (4258, 46), (4299, 46), (4658, 46), (5083, 46), (5488, 46), (5655, 46), (6204, 46), (6459, 46), (6844, 46), (6966, 46), (7550, 46), (7566, 46), (8124, 46), (8157, 46), (8629, 46), (8712, 46), (9176, 46), (9344, 46), (9372, 46), (9396, 46), (9567, 46), (9793, 46), (9986, 46), (9987, 46), (9994, 46), (10038, 46), (10043, 46), (10152, 46), (10311, 46), (10951, 46), (11059, 46), (11168, 46), (11228, 46), (11342, 46), (11574, 46), (11648, 46), (12054, 46), (12370, 46), (12424, 46), (12433, 46), (12440, 46), (12684, 46), (12774, 46), (5, 47), (107, 47), (192, 47), (402, 47), (489, 47), (682, 47), (820, 47), (827, 47), (1573, 47), (1621, 47), (1764, 47), (1816, 47), (1907, 47), (1970, 47), (1978, 47), (2121, 47), (2328, 47), (2586, 47), (2627, 47), (2722, 47), (2745, 47), (2749, 47), (2809, 47), (2861, 47), (3125, 47), (3218, 47), (3371, 47), (3530, 47), (3547, 47), (3754, 47), (3938, 47), (4128, 47), (4172, 47), (4410, 47), (4510, 47), (4868, 47), (4940, 47), (5061, 47), (5065, 47), (5377, 47), (5424, 47), (5792, 47), (5820, 47), (5915, 47), (6866, 47), (7140, 47), (7269, 47), (7289, 47), (7349, 47), (7961, 47), (8357, 47), (8639, 47), (8707, 47), (9317, 47), (9456, 47), (9598, 47), (9669, 47), (9866, 47), (10155, 47), (10652, 47), (10747, 47), (10844, 47), (11074, 47), (11197, 47), (11436, 47), (11514, 47), (12178, 47), (12367, 47), (12475, 47), (12766, 47), (253, 48), (273, 48), (466, 48), (602, 48), (632, 48), (637, 48), (771, 48), (819, 48), (1028, 48), (1421, 48), (1450, 48), (2335, 48), (2485, 48), (2688, 48), (2816, 48), (2829, 48), (3000, 48), (3458, 48), (3614, 48), (3868, 48), (4242, 48), (4482, 48), (4788, 48), (5073, 48), (5382, 48), (5459, 48), (5495, 48), (6126, 48), (6156, 48), (6256, 48), (6506, 48), (6601, 48), (7763, 48), (7924, 48), (8164, 48), (8168, 48), (8169, 48), (8282, 48), (8473, 48), (8479, 48), (8505, 48), (9044, 48), (9661, 48), (9676, 48), (9958, 48), (10042, 48), (10071, 48), (10237, 48), (10761, 48), (10793, 48), (10868, 48), (11081, 48), (11216, 48), (11353, 48), (11369, 48), (11588, 48), (11813, 48), (11893, 48), (12279, 48), (12405, 48), (12531, 48), (135, 49), (364, 49), (385, 49), (423, 49), (577, 49), (654, 49), (721, 49), (731, 49), (737, 49), (739, 49), (950, 49), (1162, 49), (1284, 49), (1550, 49), (1583, 49), (1594, 49), (1832, 49), (2346, 49), (2544, 49), (2736, 49), (2834, 49), (2903, 49), (3259, 49), (3315, 49), (3452, 49), (3713, 49), (4106, 49), (4191, 49), (4200, 49), (4557, 49), (4815, 49), (4892, 49), (5457, 49), (5851, 49), (5969, 49), (6123, 49), (6171, 49), (6178, 49), (6202, 49), (6307, 49), (6320, 49), (7078, 49), (7272, 49), (7363, 49), (7451, 49), (7532, 49), (7625, 49), (7649, 49), (7772, 49), (8326, 49), (8432, 49), (8439, 49), (8679, 49), (8872, 49), (9028, 49), (9261, 49), (9641, 49), (9693, 49), (10259, 49), (10682, 49), (10926, 49), (10965, 49), (11506, 49), (11536, 49), (12043, 49), (12455, 49), (12534, 49), (606, 50), (691, 50), (746, 50), (1088, 50), (1334, 50), (1451, 50), (2059, 50), (2183, 50), (2704, 50), (2887, 50), (2926, 50), (2994, 50), (3430, 50), (4205, 50), (4779, 50), (4983, 50), (5197, 50), (5303, 50), (6033, 50), (6975, 50), (7048, 50), (7115, 50), (7170, 50), (7276, 50), (7755, 50), (7813, 50), (7932, 50), (8235, 50), (8512, 50), (8739, 50), (8762, 50), (8769, 50), (8770, 50), (8961, 50), (8984, 50), (9041, 50), (9706, 50), (9713, 50), (9931, 50), (10036, 50), (10140, 50), (10195, 50), (11005, 50), (11015, 50), (11091, 50), (11261, 50), (11698, 50), (11786, 50), (12225, 50), (12527, 50), (871, 51), (1037, 51), (1243, 51), (1770, 51), (2397, 51), (2532, 51), (2545, 51), (3016, 51), (3152, 51), (3504, 51), (3877, 51), (4464, 51), (4540, 51), (4612, 51), (4800, 51), (5228, 51), (5460, 51), (5699, 51), (5862, 51), (6090, 51), (6122, 51), (6172, 51), (6738, 51), (7107, 51), (7179, 51), (7231, 51), (7498, 51), (7633, 51), (7783, 51), (8090, 51), (8130, 51), (8210, 51), (9384, 51), (9501, 51), (9675, 51), (9826, 51), (10028, 51), (10318, 51), (10357, 51), (10517, 51), (11355, 51), (11986, 51), (11997, 51), (12066, 51), (12570, 51), (12757, 51), (12794, 51), (55, 52), (517, 52), (1057, 52), (1208, 52), (1656, 52), (1844, 52), (2582, 52), (2587, 52), (3268, 52), (3605, 52), (3649, 52), (3730, 52), (4192, 52), (4906, 52), (4910, 52), (6070, 52), (6186, 52), (6589, 52), (6632, 52), (6968, 52), (7119, 52), (7265, 52), (7353, 52), (7836, 52), (8303, 52), (8610, 52), (9423, 52), (10098, 52), (10335, 52), (10408, 52), (10558, 52), (10832, 52), (10937, 52), (11020, 52), (11065, 52), (11077, 52), (11338, 52), (11718, 52), (11819, 52), (11935, 52), (12196, 52), (12435, 52), (12489, 52), (348, 53), (665, 53), (797, 53), (968, 53), (1435, 53), (1491, 53), (1541, 53), (2085, 53), (3895, 53), (5162, 53), (6252, 53), (6460, 53), (6958, 53), (6991, 53), (7321, 53), (7408, 53), (7653, 53), (8205, 53), (8883, 53), (9057, 53), (9454, 53), (9621, 53), (9946, 53), (10410, 53), (11770, 53), (12181, 53), (12759, 53), (844, 54), (858, 54), (885, 54), (2312, 54), (2819, 54), (2860, 54), (3628, 54), (3709, 54), (4257, 54), (4352, 54), (4748, 54), (5016, 54), (5160, 54), (5568, 54), (5588, 54), (6522, 54), (6894, 54), (7332, 54), (7441, 54), (7899, 54), (8862, 54), (9320, 54), (9415, 54), (10005, 54), (10307, 54), (11227, 54), (12025, 54), (12120, 54), (12238, 54), (12537, 54), (121, 55), (209, 55), (473, 55), (873, 55), (1198, 55), (1829, 55), (2634, 55), (2830, 55), (3900, 55), (5425, 55), (5476, 55), (5631, 55), (8285, 55), (8389, 55), (8471, 55), (8567, 55), (8658, 55), (9200, 55), (9256, 55), (9700, 55), (10277, 55), (10765, 55), (11359, 55), (11626, 55), (11907, 55), (12194, 55), (12472, 55), (12521, 55), (122, 56), (802, 56), (1260, 56), (1392, 56), (1624, 56), (1646, 56), (2385, 56), (3447, 56), (4040, 56), (4059, 56), (4084, 56), (4361, 56), (4726, 56), (4747, 56), (4783, 56), (5153, 56), (5408, 56), (5490, 56), (5893, 56), (6650, 56), (6676, 56), (7032, 56), (7274, 56), (7395, 56), (7896, 56), (7950, 56), (8272, 56), (8718, 56), (9159, 56), (9400, 56), (9752, 56), (10891, 56), (11239, 56), (12324, 56), (12645, 56), (284, 57), (464, 57), (580, 57), (1252, 57), (1766, 57), (2412, 57), (3116, 57), (3931, 57), (4644, 57), (5119, 57), (5537, 57), (7931, 57), (8178, 57), (8687, 57), (8850, 57), (8893, 57), (9121, 57), (9481, 57), (9944, 57), (10513, 57), (10693, 57), (11104, 57), (11341, 57), (12782, 57), (240, 58), (599, 58), (741, 58), (1209, 58), (1373, 58), (1882, 58), (1919, 58), (1956, 58), (2964, 58), (3425, 58), (7188, 58), (7413, 58), (8732, 58), (8835, 58), (9353, 58), (9918, 58), (9939, 58), (10230, 58), (10800, 58), (12608, 58), (607, 59), (1487, 59), (1711, 59), (2409, 59), (2712, 59), (3098, 59), (3243, 59), (3353, 59), (4388, 59), (4580, 59), (4707, 59), (5407, 59), (5605, 59), (5735, 59), (8918, 59), (9040, 59), (10668, 59), (10862, 59), (12023, 59), (12467, 59), (12617, 59), (12728, 59), (12753, 59), (2486, 60), (3866, 60), (4223, 60), (4475, 60), (4942, 60), (5156, 60), (6066, 60), (6867, 60), (7167, 60), (7569, 60), (7878, 60), (8006, 60), (8075, 60), (8122, 60), (8254, 60), (8296, 60), (8700, 60), (9308, 60), (9490, 60), (9967, 60), (10448, 60), (10570, 60), (10900, 60), (11575, 60), (12709, 60), (1536, 61), (1613, 61), (2918, 61), (2967, 61), (3003, 61), (4218, 61), (4720, 61), (4875, 61), (5553, 61), (6387, 61), (8120, 61), (8594, 61), (9202, 61), (11319, 61), (12505, 61), (12736, 61), (139, 62), (4078, 62), (5456, 62), (6670, 62), (7093, 62), (7313, 62), (7377, 62), (7690, 62), (7914, 62), (8335, 62), (8360, 62), (9306, 62), (10341, 62), (12798, 62), (910, 63), (1266, 63), (2442, 63), (5111, 63), (5213, 63), (5239, 63), (5481, 63), (7147, 63), (8040, 63), (8704, 63), (8943, 63), (12166, 63), (51, 64), (845, 64), (2518, 64), (2686, 64), (3805, 64), (4109, 64), (4822, 64), (5098, 64), (8569, 64), (8606, 64), (8840, 64), (9072, 64), (11363, 64), (12520, 64), (3909, 65), (5545, 65), (8011, 65), (8450, 65), (8911, 65), (10821, 65), (90, 66), (1443, 66), (1980, 66), (1997, 66), (7326, 66), (8939, 66), (12283, 66), (978, 67), (1329, 67), (2164, 67), (2225, 67), (2256, 67), (2558, 67), (4360, 67), (4970, 67), (5709, 67), (5788, 67), (6789, 67), (8035, 67), (8949, 67), (12587, 67), (992, 68), (1638, 68), (2601, 68), (3062, 68), (3632, 68), (4439, 68), (5290, 68), (6136, 68), (7244, 68), (7311, 68), (9001, 68), (9936, 68), (12174, 68), (3201, 69), (5572, 69), (6229, 69), (6733, 69), (9834, 69), (4221, 70), (5580, 70), (5795, 70), (8844, 70), (8964, 70), (9952, 70), (11639, 70), (12358, 70), (2995, 71), (3597, 71), (6695, 71), (9351, 71), (11281, 71), (1525, 72), (4656, 72), (7945, 72), (9046, 72), (11941, 72), (12161, 72), (547, 73), (4219, 73), (4259, 73), (9422, 73), (10106, 73), (1054, 74), (2087, 74), (2656, 74), (2970, 74), (3071, 74), (11302, 74), (11328, 74), (12154, 74), (2017, 75), (2377, 75), (6895, 75), (6944, 75), (12032, 75), (2181, 76), (2820, 76), (8467, 76), (9852, 76), (1397, 77), (3338, 77), (9532, 77), (1256, 78), (7658, 78), (7756, 78), (11372, 78), (1506, 79), (7713, 79), (8981, 79), (2401, 80), (8799, 80), (9638, 80), (12326, 80), (160, 81), (5447, 81), (6901, 81), (11577, 81), (5313, 82), (8337, 82), (8605, 82), (834, 83), (5105, 83), (5647, 83), (7632, 84), (3583, 85), (10660, 85), (4377, 86), (6839, 86), (10949, 86), (12130, 86), (381, 88), (1353, 88), (7861, 88), (8227, 88), (8574, 88), (11543, 88), (5804, 89), (7792, 89), (1294, 90), (5869, 91), (3702, 92), (4292, 92), (4824, 92), (4270, 94), (8212, 95), (1444, 97), (5196, 98), (6005, 98), (10924, 98), (10127, 99), (1817, 100), (398, 107), (11354, 107), (1184, 108), (4772, 110), (8742, 113), (6479, 114), (2805, 117), (3786, 118), (11946, 118), (8298, 119), (1923, 127), (12301, 127), (12249, 129), (7466, 130), (7403, 146), (6935, 155)]
```

## yield函数
首先，如果你还没有对yield有个初步分认识，那么你先把yield看做“return”，这个是直观的，它首先是个return，普通的return是什么意思，就是在程序中返回某个值，返回之后程序就不再往下运行了。看做return之后再把它看做一个是生成器（generator）的一部分（带yield的函数才是真正的迭代器），好了，如果你对这些不明白的话，那先把yield看做return,然后直接看下面的程序，你就会明白yield的全部意思了：
```
def foo():
    print("starting...")
    while True:
        res = yield 4
        print("res:",res)
g = foo()
print(next(g))
print("*"*20)
print(next(g))
```
输出为：
```
starting...
4
********************
res: None
4
```
1.程序开始执行以后，因为foo函数中有yield关键字，所以foo函数并不会真的执行，而是先得到一个生成器g(相当于一个对象)

2.直到调用next方法，foo函数正式开始执行，先执行foo函数中的print方法，然后进入while循环

3.程序遇到yield关键字，然后把yield想想成return,return了一个4之后，程序停止，*并没有执行赋值给res操作*，此时next(g)语句执行完成，所以输出的前两行（第一个是while上面的print的结果,第二个是return出的结果）是执行print(next(g))的结果，

4.程序执行print("*"*20)，输出20个*

5.又开始执行下面的print(next(g)),这个时候和上面那个差不多，不过不同的是，*这个时候是从刚才那个next程序停止的地方开始执行的*，也就是要执行res的赋值操作，**这时候要注意，这个时候赋值操作的右边是没有值的（因为刚才那个是return出去了，并没有给赋值操作的左边传参数）**，所以这个时候res赋值是None,所以接着下面的输出就是res:None,

6.程序会继续在while里执行，又一次碰到yield,这个时候同样return 出4，然后程序停止，print函数输出的4就是这次return出的4.

*到这里你可能就明白yield和return的关系和区别了，带yield的函数是一个生成器，而不是一个函数了，这个生成器有一个函数就是next函数，next就相当于“下一步”生成哪个数，这一次的next开始的地方是接着上一次的next停止的地方执行的，所以调用next的时候，生成器并不会从foo函数的开始执行，只是接着上一步停止的地方开始，然后遇到yield后，return出要生成的数，此步就结束。*

原文链接：https://blog.csdn.net/mieleizhi0522/article/details/82142856/


In [ ]:
from torch.utils.data import BatchSampler, SubsetRandomSampler
import math
class BucketBatchSampler(BatchSampler):
    """ `BucketBatchSampler` toggles between `sampler` batches and sorted batches.

    Typically, the `sampler` will be a `RandomSampler` allowing the user to toggle between
    random batches and sorted batches. A larger `bucket_size_multiplier` is more sorted and vice
    versa.

    Background:
        ``BucketBatchSampler`` is similar to a ``BucketIterator`` found in popular libraries like
        ``AllenNLP`` and ``torchtext``. A ``BucketIterator`` pools together examples with a similar
        size length to reduce the padding required for each batch while maintaining some noise
        through bucketing.

        **AllenNLP Implementation:**
        https://github.com/allenai/allennlp/blob/master/allennlp/data/iterators/bucket_iterator.py

        **torchtext Implementation:**
        https://github.com/pytorch/text/blob/master/torchtext/data/iterator.py#L225

    Args:
        sampler (torch.data.utils.sampler.Sampler):
        batch_size (int): Size of mini-batch.
        drop_last (bool): If `True` the sampler will drop the last batch if its size would be less
            than `batch_size`.
        sort_key (callable, optional): Callable to specify a comparison key for sorting.
        bucket_size_multiplier (int, optional): Buckets are of size
            `batch_size * bucket_size_multiplier`.
    """
    def __init__(self, sampler, batch_size, drop_last, sort_key, bucket_size_multiplier = 100):
      super().__init__(sampler, batch_size, drop_last)
      self.sort_key = sort_key
      self.bucket_sampler = BatchSampler(sampler, min(batch_size * bucket_size_multiplier, len(sampler)), False)

    def __iter__(self):
      for bucket in self.bucket_sampler:
        sorted_sampler = SortedSampler(bucket, self.sort_key)
        # print('list:',list(BatchSampler(sorted_sampler, self.batch_size, self.drop_last)))
        for batch in SubsetRandomSampler(list(BatchSampler(sorted_sampler, self.batch_size, self.drop_last))):
          yield [bucket[i] for i in batch]

    def __len__(self):
      if self.drop_last:
        return len(self.sampler) // self.batch_size

      else:
        return math.ceil(len(self.sampler) / self.batch_size)


```
list: [[1458, 3168, 5913, 9646, 11686, 12642, 795, 1735, 1848, 2093, 2195, 2438, 4590, 4951, 5029, 5855, 5857, 6340, 6982, 7203, 7218, 7599, 9213, 9225, 9267, 10583, 11142, 11845, 12585, 101, 318, 777, 851, 883, 1156, 1236, 1518, 1584, 1629, 1652, 1957, 2162, 2210, 2419, 2584, 2762, 2796, 3042, 3078, 3198, 3426, 3555, 3631, 3712, 3986, 4110, 4125, 4216, 4313, 4351, 4533, 4539, 5047, 5054], [5257, 5331, 5348, 5652, 5780, 5870, 5918, 6055, 6094, 6115, 6385, 6546, 6554, 6721, 6880, 6977, 7012, 7202, 7282, 7548, 8003, 8007, 8099, 8564, 8607, 8701, 8880, 8931, 9339, 9432, 9542, 9545, 9841, 9938, 9982, 10254, 10285, 10322, 10345, 10547, 10637, 10704, 11176, 11267, 11863, 12060, 12116, 12144, 12210, 12391, 12606, 12778, 116, 144, 188, 223, 262, 332, 353, 434, 717, 774, 933, 995], [1212, 1432, 1580, 1667, 1677, 1690, 1762, 1884, 1941, 2028, 2414, 2430, 2668, 2699, 2908, 3123, 3147, 3296, 3464, 3646, 3685, 3701, 3752, 3840, 3842, 4001, 4075, 4098, 4103, 4114, 4135, 4685, 4893, 4954, 4988, 5124, 5254, 5380, 5427, 5475, 5499, 5539, 5586, 5636, 5992, 6003, 6160, 6251, 6287, 6393, 6471, 6484, 6524, 6553, 6675, 6755, 6972, 7712, 7839, 7891, 7897, 7901, 8088, 8138], [8930, 9082, 9137, 9349, 9350, 9521, 9595, 9622, 9665, 9853, 9870, 9924, 10017, 10089, 10470, 10635, 10716, 10771, 10860, 10969, 11079, 11092, 11208, 11300, 11426, 11464, 11624, 11734, 11744, 11918, 12077, 12104, 12204, 12605, 12729, 0, 24, 100, 210, 217, 234, 322, 350, 393, 397, 431, 543, 545, 578, 767, 815, 872, 914, 956, 1001, 1027, 1203, 1233, 1290, 1445, 1606, 1779, 1899, 2060], [2103, 2196, 2296, 2362, 2373, 2511, 2512, 2640, 2665, 2696, 2730, 2767, 2797, 3144, 3195, 3314, 3331, 3373, 3380, 3384, 3536, 3611, 3696, 3778, 3862, 3887, 3906, 4023, 4099, 4121, 4149, 4498, 4519, 4600, 4606, 4645, 4733, 4987, 5000, 5019, 5118, 5236, 5302, 5332, 5340, 5487, 5638, 5644, 5881, 6031, 6058, 6096, 6121, 6162, 6165, 6201, 6386, 6592, 6593, 6682, 6702, 6848, 6934, 6962], [7047, 7388, 7423, 7454, 7493, 7544, 7545, 7580, 7677, 7698, 7776, 7799, 7826, 7999, 8008, 8125, 8393, 8394, 8430, 8440, 8580, 8636, 8642, 8651, 8681, 8800, 8812, 8973, 9018, 9039, 9089, 9093, 9103, 9130, 9162, 9184, 9325, 9370, 9385, 9420, 9477, 9519, 9529, 9546, 9634, 9782, 9848, 9871, 9919, 9949, 10094, 10206, 10321, 10348, 10362, 10430, 10466, 10556, 10568, 10935, 11085, 11119, 11124, 11140], [11151, 11215, 11262, 11283, 11477, 11498, 11576, 11599, 11653, 11742, 11824, 11903, 11975, 12103, 12296, 12345, 12389, 12456, 12607, 12668, 12670, 12769, 14, 79, 98, 103, 125, 131, 155, 176, 227, 277, 300, 303, 329, 340, 392, 399, 424, 441, 472, 474, 633, 694, 744, 750, 762, 799, 875, 877, 918, 919, 1048, 1113, 1164, 1242, 1280, 1340, 1371, 1463, 1571, 1610, 1620, 1657], [1670, 1683, 1739, 1768, 1781, 1786, 1804, 1914, 1965, 2062, 2072, 2269, 2271, 2283, 2294, 2304, 2439, 2565, 2597, 2729, 2747, 2815, 3013, 3086, 3148, 3234, 3252, 3261, 3287, 3313, 3370, 3439, 3455, 3543, 3775, 3806, 3853, 3870, 3912, 3933, 3950, 3998, 4017, 4028, 4175, 4238, 4240, 4250, 4316, 4329, 4425, 4484, 4531, 4763, 4813, 4882, 4886, 4915, 4924, 5024, 5028, 5040, 5060, 5067], [5078, 5088, 5176, 5180, 5209, 5214, 5225, 5235, 5333, 5511, 5603, 5633, 5654, 5672, 5718, 5836, 5868, 5896, 5948, 5967, 6009, 6015, 6052, 6184, 6257, 6277, 6278, 6316, 6319, 6421, 6434, 6467, 6562, 6631, 6716, 6719, 6772, 6850, 6863, 6878, 6883, 6891, 6932, 7138, 7166, 7181, 7189, 7257, 7258, 7277, 7314, 7415, 7523, 7567, 7670, 7691, 7716, 7873, 8009, 8053, 8056, 8064, 8076, 8186], [8194, 8208, 8233, 8318, 8344, 8345, 8397, 8420, 8435, 8490, 8686, 8695, 8736, 8776, 8789, 8827, 8830, 8832, 9006, 9032, 9066, 9119, 9232, 9273, 9332, 9411, 9427, 9516, 9522, 9534, 9553, 9629, 9695, 9820, 9879, 9904, 9906, 9966, 10101, 10142, 10182, 10192, 10367, 10384, 10438, 10456, 10499, 10536, 10572, 10602, 10644, 10672, 10749, 10846, 11042, 11046, 11050, 11080, 11109, 11113, 11200, 11260, 11322, 11364], [11413, 11422, 11473, 11481, 11507, 11555, 11603, 11625, 11637, 11645, 11664, 11693, 11773, 11801, 11820, 11856, 12012, 12041, 12073, 12134, 12214, 12308, 12341, 12379, 12410, 12512, 12535, 12547, 12573, 12591, 12595, 12731, 68, 80, 97, 137, 158, 220, 285, 314, 320, 331, 343, 356, 404, 407, 447, 491, 495, 534, 576, 624, 649, 663, 700, 766, 781, 788, 881, 884, 913, 938, 946, 1019], [1024, 1047, 1053, 1062, 1124, 1150, 1175, 1216, 1217, 1250, 1269, 1289, 1317, 1318, 1335, 1345, 1356, 1389, 1393, 1409, 1410, 1430, 1447, 1467, 1477, 1527, 1545, 1547, 1615, 1679, 1691, 1702, 1726, 1730, 1753, 1767, 1784, 1787, 1799, 1819, 1870, 1879, 2040, 2095, 2130, 2169, 2175, 2190, 2205, 2214, 2261, 2280, 2287, 2293, 2310, 2338, 2359, 2372, 2407, 2446, 2462, 2470, 2482, 2613], [2638, 2669, 2679, 2715, 2721, 2777, 2795, 2843, 2845, 2875, 2901, 2910, 2983, 3017, 3023, 3069, 3158, 3221, 3262, 3278, 3290, 3298, 3348, 3357, 3360, 3397, 3431, 3456, 3474, 3533, 3561, 3582, 3698, 3719, 3723, 3801, 3828, 3831, 3980, 4108, 4116, 4220, 4266, 4296, 4297, 4320, 4424, 4507, 4515, 4518, 4550, 4587, 4588, 4592, 4622, 4638, 4640, 4660, 4676, 4691, 4696, 4738, 4740, 4754], [4770, 4791, 4793, 4877, 4894, 4901, 4902, 4911, 4927, 4959, 5062, 5094, 5095, 5142, 5145, 5157, 5170, 5174, 5182, 5224, 5385, 5433, 5441, 5504, 5532, 5624, 5715, 5736, 5741, 5779, 5784, 5826, 5829, 5841, 5854, 5946, 5962, 5971, 5987, 6018, 6061, 6086, 6107, 6116, 6180, 6269, 6294, 6301, 6306, 6333, 6352, 6395, 6417, 6418, 6447, 6454, 6463, 6478, 6502, 6517, 6545, 6620, 6644, 6655], [6690, 6731, 6736, 6833, 6845, 6890, 6960, 7058, 7112, 7163, 7178, 7200, 7239, 7256, 7325, 7364, 7375, 7396, 7438, 7447, 7458, 7479, 7513, 7618, 7639, 7708, 7731, 7785, 7819, 7852, 7888, 7951, 7977, 7994, 7995, 8051, 8078, 8079, 8114, 8184, 8191, 8226, 8259, 8268, 8270, 8289, 8299, 8332, 8342, 8348, 8355, 8422, 8442, 8444, 8461, 8539, 8560, 8565, 8570, 8621, 8643, 8655, 8661, 8685], [8734, 8757, 8784, 8855, 8861, 8867, 8891, 8916, 8994, 8998, 8999, 9005, 9007, 9045, 9076, 9115, 9117, 9134, 9150, 9153, 9161, 9187, 9233, 9235, 9245, 9295, 9327, 9331, 9343, 9359, 9367, 9376, 9413, 9416, 9471, 9473, 9497, 9505, 9510, 9618, 9630, 9647, 9688, 9748, 9833, 9867, 9907, 9991, 10000, 10039, 10065, 10084, 10146, 10170, 10176, 10185, 10196, 10253, 10270, 10281, 10291, 10316, 10328, 10349], [10364, 10467, 10493, 10614, 10641, 10646, 10687, 10698, 10721, 10722, 10756, 10811, 10825, 10838, 10859, 10881, 10884, 10938, 10983, 11017, 11043, 11093, 11116, 11120, 11127, 11129, 11163, 11184, 11213, 11219, 11264, 11276, 11326, 11334, 11365, 11385, 11407, 11446, 11460, 11508, 11511, 11515, 11598, 11621, 11636, 11638, 11684, 11689, 11706, 11712, 11719, 11756, 11791, 11802, 11807, 11812, 11815, 11878, 11880, 11908, 11917, 11931, 11985, 12094], [12157, 12169, 12189, 12215, 12244, 12260, 12281, 12285, 12293, 12316, 12319, 12375, 12416, 12425, 12437, 12442, 12481, 12482, 12504, 12539, 12555, 12594, 12601, 12647, 12671, 12693, 12720, 29, 81, 108, 213, 243, 246, 310, 355, 367, 396, 416, 484, 503, 554, 566, 596, 613, 625, 697, 701, 716, 733, 765, 785, 805, 843, 894, 908, 923, 924, 927, 944, 952, 982, 985, 1014, 1032], [1075, 1083, 1094, 1104, 1132, 1165, 1197, 1228, 1240, 1241, 1265, 1302, 1309, 1321, 1341, 1346, 1390, 1418, 1442, 1459, 1476, 1532, 1631, 1651, 1704, 1708, 1715, 1788, 1807, 1821, 1822, 1838, 1856, 1868, 1901, 1902, 1945, 2024, 2031, 2133, 2152, 2173, 2201, 2216, 2243, 2284, 2286, 2337, 2343, 2355, 2389, 2417, 2481, 2483, 2510, 2520, 2550, 2576, 2578, 2579, 2607, 2614, 2644, 2657], [2660, 2667, 2671, 2681, 2683, 2692, 2716, 2738, 2740, 2741, 2742, 2825, 2832, 2851, 2898, 2921, 2929, 2933, 2940, 2944, 2962, 2976, 3015, 3030, 3054, 3105, 3115, 3121, 3122, 3127, 3164, 3169, 3246, 3247, 3269, 3289, 3305, 3329, 3345, 3359, 3366, 3377, 3440, 3450, 3457, 3469, 3479, 3480, 3484, 3491, 3496, 3498, 3500, 3513, 3520, 3588, 3618, 3634, 3647, 3669, 3706, 3725, 3802, 3808], [3838, 3847, 3893, 3920, 3954, 3983, 3996, 4002, 4010, 4018, 4039, 4091, 4100, 4123, 4129, 4151, 4158, 4167, 4190, 4207, 4222, 4228, 4232, 4236, 4243, 4246, 4260, 4265, 4274, 4280, 4311, 4343, 4354, 4363, 4387, 4406, 4430, 4457, 4459, 4488, 4551, 4553, 4560, 4584, 4586, 4604, 4621, 4648, 4680, 4713, 4727, 4731, 4750, 4785, 4808, 4809, 4828, 4840, 4847, 4888, 4909, 4928, 4966, 4968], [5030, 5038, 5050, 5081, 5101, 5175, 5199, 5202, 5211, 5247, 5281, 5288, 5295, 5298, 5306, 5323, 5326, 5345, 5350, 5381, 5401, 5468, 5483, 5524, 5538, 5547, 5557, 5558, 5562, 5567, 5578, 5596, 5610, 5611, 5627, 5645, 5669, 5692, 5698, 5708, 5801, 5803, 5817, 5819, 5847, 5903, 5906, 5910, 6001, 6034, 6039, 6051, 6056, 6064, 6065, 6079, 6102, 6125, 6129, 6152, 6221, 6222, 6234, 6235], [6241, 6250, 6321, 6326, 6365, 6373, 6381, 6402, 6428, 6446, 6482, 6494, 6521, 6530, 6549, 6568, 6570, 6578, 6598, 6643, 6667, 6750, 6778, 6783, 6805, 6818, 6829, 6838, 6871, 6872, 6898, 6909, 6914, 6933, 6945, 6971, 6990, 6995, 7000, 7004, 7010, 7014, 7029, 7030, 7046, 7104, 7196, 7204, 7209, 7236, 7286, 7317, 7337, 7394, 7400, 7412, 7433, 7437, 7483, 7494, 7564, 7574, 7606, 7619], [7665, 7702, 7703, 7714, 7730, 7777, 7780, 7851, 7858, 7860, 7874, 7892, 7993, 8015, 8017, 8030, 8045, 8080, 8161, 8172, 8196, 8202, 8209, 8213, 8237, 8241, 8251, 8260, 8280, 8283, 8328, 8336, 8376, 8418, 8434, 8438, 8468, 8478, 8515, 8556, 8571, 8602, 8615, 8646, 8682, 8708, 8723, 8728, 8754, 8763, 8780, 8781, 8837, 8839, 8858, 8898, 8906, 8912, 8925, 8932, 8940, 8944, 8960, 8969], [8985, 8993, 9038, 9074, 9092, 9096, 9106, 9139, 9166, 9172, 9178, 9199, 9268, 9292, 9302, 9310, 9366, 9374, 9403, 9410, 9435, 9462, 9469, 9499, 9539, 9555, 9564, 9570, 9592, 9594, 9627, 9642, 9670, 9717, 9723, 9733, 9734, 9764, 9768, 9821, 9822, 9829, 9888, 9893, 9900, 9921, 9937, 9951, 9983, 10014, 10079, 10085, 10104, 10122, 10167, 10198, 10226, 10245, 10249, 10260, 10269, 10292, 10297, 10302], [10312, 10333, 10342, 10352, 10397, 10413, 10420, 10432, 10453, 10482, 10550, 10593, 10622, 10638, 10639, 10645, 10719, 10737, 10739, 10754, 10757, 10764, 10841, 10854, 10858, 10863, 10910, 10916, 10921, 10922, 10923, 10977, 11002, 11037, 11051, 11094, 11096, 11103, 11106, 11115, 11160, 11171, 11217, 11235, 11236, 11274, 11277, 11280, 11288, 11296, 11331, 11346, 11382, 11396, 11397, 11398, 11443, 11505, 11541, 11572, 11587, 11593, 11627, 11741], [11753, 11806, 11839, 11867, 11910, 11919, 11956, 11958, 11982, 11995, 12002, 12021, 12024, 12048, 12086, 12099, 12105, 12106, 12109, 12127, 12180, 12185, 12199, 12203, 12229, 12248, 12262, 12356, 12373, 12377, 12397, 12434, 12439, 12554, 12565, 12586, 12641, 12710, 12752, 12763, 12785, 54, 105, 106, 182, 191, 193, 226, 244, 247, 248, 260, 313, 325, 328, 335, 363, 368, 372, 446, 478, 480, 487, 494], [497, 506, 507, 510, 516, 558, 572, 575, 588, 610, 616, 645, 659, 684, 687, 707, 719, 768, 776, 811, 840, 848, 862, 874, 879, 906, 909, 974, 983, 987, 1021, 1044, 1068, 1071, 1082, 1146, 1157, 1221, 1235, 1244, 1299, 1307, 1350, 1394, 1396, 1405, 1438, 1490, 1495, 1503, 1509, 1520, 1538, 1544, 1563, 1564, 1591, 1595, 1611, 1633, 1695, 1698, 1717, 1723], [1757, 1765, 1794, 1872, 1875, 1893, 1940, 1962, 2025, 2039, 2049, 2096, 2124, 2126, 2127, 2158, 2165, 2204, 2213, 2231, 2240, 2253, 2282, 2326, 2336, 2357, 2371, 2441, 2448, 2507, 2517, 2522, 2557, 2566, 2593, 2642, 2646, 2680, 2684, 2705, 2753, 2787, 2803, 2807, 2866, 2916, 2946, 2949, 2960, 2969, 2979, 3001, 3007, 3043, 3053, 3081, 3093, 3133, 3161, 3173, 3180, 3188, 3190, 3230], [3244, 3251, 3254, 3283, 3292, 3333, 3347, 3361, 3386, 3404, 3409, 3420, 3432, 3434, 3501, 3526, 3538, 3565, 3587, 3600, 3607, 3637, 3675, 3686, 3714, 3717, 3721, 3829, 3852, 3891, 3945, 3952, 3992, 4012, 4029, 4038, 4051, 4068, 4074, 4077, 4127, 4131, 4153, 4156, 4170, 4202, 4276, 4289, 4291, 4295, 4356, 4367, 4383, 4407, 4409, 4443, 4454, 4460, 4463, 4465, 4503, 4506, 4524, 4548], [4567, 4625, 4675, 4709, 4848, 4863, 4913, 4920, 4955, 4956, 5058, 5071, 5080, 5084, 5113, 5122, 5127, 5141, 5154, 5200, 5205, 5210, 5223, 5231, 5245, 5261, 5275, 5310, 5327, 5404, 5416, 5431, 5435, 5466, 5492, 5496, 5501, 5503, 5540, 5546, 5574, 5583, 5696, 5700, 5719, 5806, 5884, 5887, 5892, 5904, 5926, 5932, 5950, 5980, 6007, 6075, 6101, 6117, 6164, 6231, 6261, 6262, 6271, 6275], [6299, 6356, 6399, 6416, 6422, 6426, 6458, 6468, 6475, 6491, 6504, 6511, 6529, 6550, 6584, 6596, 6597, 6671, 6685, 6722, 6740, 6754, 6762, 6769, 6784, 6811, 6902, 6969, 6973, 6983, 7041, 7076, 7146, 7247, 7249, 7273, 7303, 7331, 7419, 7459, 7470, 7477, 7504, 7531, 7534, 7556, 7559, 7602, 7607, 7709, 7727, 7767, 7789, 7806, 7843, 7848, 7872, 7915, 7929, 7930, 7940, 7966, 8005, 8041], [8048, 8049, 8058, 8062, 8093, 8108, 8180, 8239, 8279, 8302, 8307, 8315, 8320, 8334, 8359, 8379, 8395, 8445, 8460, 8462, 8470, 8480, 8496, 8509, 8555, 8577, 8675, 8722, 8748, 8759, 8772, 8795, 8810, 8875, 8887, 8897, 8962, 8996, 9002, 9009, 9031, 9037, 9055, 9068, 9078, 9095, 9107, 9140, 9156, 9174, 9185, 9204, 9297, 9333, 9394, 9414, 9419, 9440, 9482, 9493, 9543, 9571, 9583, 9584], [9599, 9613, 9619, 9620, 9648, 9654, 9686, 9710, 9731, 9745, 9749, 9759, 9783, 9794, 9819, 9823, 9828, 9842, 9847, 9878, 9963, 9965, 10002, 10048, 10069, 10083, 10087, 10088, 10132, 10134, 10144, 10179, 10184, 10210, 10220, 10236, 10273, 10288, 10346, 10354, 10361, 10379, 10418, 10422, 10457, 10458, 10492, 10518, 10522, 10553, 10575, 10620, 10650, 10699, 10705, 10712, 10731, 10733, 10780, 10818, 10834, 10896, 10918, 10964], [10979, 11062, 11108, 11112, 11128, 11161, 11180, 11230, 11253, 11279, 11287, 11289, 11376, 11421, 11449, 11458, 11478, 11485, 11491, 11493, 11501, 11532, 11534, 11676, 11745, 11782, 11818, 11821, 11827, 11838, 11966, 11992, 12020, 12080, 12082, 12083, 12159, 12186, 12187, 12219, 12227, 12267, 12335, 12363, 12366, 12376, 12411, 12460, 12461, 12515, 12518, 12525, 12536, 12548, 12549, 12551, 12562, 12564, 12566, 12569, 12592, 12593, 12667, 12672], [12705, 12715, 12733, 12790, 12792, 9, 18, 28, 40, 61, 162, 166, 189, 207, 231, 232, 241, 245, 275, 319, 352, 412, 437, 449, 468, 479, 485, 488, 511, 524, 536, 560, 589, 626, 629, 636, 651, 690, 695, 708, 712, 713, 747, 758, 760, 789, 793, 816, 833, 859, 886, 900, 922, 940, 949, 990, 1004, 1031, 1040, 1096, 1099, 1120, 1125, 1144], [1145, 1147, 1171, 1185, 1188, 1190, 1207, 1225, 1270, 1291, 1308, 1324, 1338, 1351, 1377, 1383, 1387, 1460, 1464, 1471, 1475, 1519, 1555, 1589, 1601, 1619, 1626, 1676, 1703, 1718, 1745, 1758, 1789, 1852, 1853, 1862, 1937, 1943, 1961, 1982, 2013, 2016, 2019, 2021, 2056, 2079, 2115, 2139, 2155, 2192, 2208, 2244, 2260, 2349, 2384, 2420, 2444, 2477, 2508, 2529, 2530, 2603, 2635, 2701], [2723, 2724, 2798, 2831, 2835, 2838, 2853, 2854, 2880, 2895, 2958, 2965, 2974, 2996, 3021, 3046, 3060, 3085, 3087, 3099, 3151, 3159, 3176, 3186, 3217, 3224, 3255, 3258, 3264, 3272, 3286, 3337, 3388, 3399, 3419, 3427, 3435, 3443, 3481, 3495, 3503, 3509, 3511, 3512, 3524, 3562, 3571, 3574, 3590, 3594, 3603, 3624, 3633, 3641, 3643, 3658, 3680, 3687, 3690, 3691, 3711, 3737, 3741, 3763], [3771, 3825, 3837, 3869, 3878, 4030, 4118, 4143, 4157, 4169, 4174, 4183, 4206, 4251, 4254, 4344, 4366, 4382, 4405, 4474, 4486, 4525, 4563, 4599, 4605, 4626, 4628, 4641, 4669, 4687, 4694, 4699, 4734, 4767, 4834, 4845, 4869, 4952, 4974, 4981, 4995, 4997, 5077, 5082, 5089, 5093, 5129, 5131, 5158, 5230, 5244, 5259, 5267, 5270, 5315, 5349, 5356, 5359, 5371, 5406, 5419, 5422, 5443, 5453], [5463, 5471, 5554, 5650, 5659, 5697, 5705, 5762, 5766, 5767, 5800, 5808, 5816, 5843, 5846, 5852, 5899, 5911, 5923, 5945, 5958, 5990, 6016, 6092, 6130, 6134, 6139, 6141, 6163, 6246, 6254, 6270, 6309, 6350, 6367, 6371, 6405, 6440, 6457, 6466, 6472, 6477, 6501, 6525, 6542, 6586, 6595, 6652, 6693, 6698, 6703, 6704, 6711, 6757, 6763, 6764, 6775, 6779, 6797, 6832, 6837, 6864, 6876, 6892], [6942, 6957, 6965, 6985, 7043, 7065, 7085, 7090, 7113, 7123, 7131, 7153, 7154, 7155, 7169, 7171, 7191, 7215, 7220, 7230, 7233, 7241, 7266, 7292, 7296, 7327, 7370, 7389, 7409, 7430, 7443, 7501, 7525, 7666, 7700, 7715, 7721, 7725, 7748, 7757, 7849, 7886, 7894, 7927, 7948, 7956, 7967, 7990, 8014, 8038, 8050, 8057, 8113, 8121, 8154, 8167, 8176, 8216, 8244, 8269, 8273, 8287, 8292, 8312], [8314, 8331, 8338, 8377, 8381, 8474, 8476, 8486, 8501, 8503, 8523, 8548, 8588, 8592, 8596, 8625, 8697, 8706, 8764, 8778, 8801, 8846, 8865, 8892, 8901, 8926, 8947, 8950, 8983, 9020, 9024, 9035, 9105, 9122, 9125, 9131, 9142, 9165, 9167, 9168, 9181, 9201, 9206, 9215, 9223, 9241, 9287, 9313, 9424, 9459, 9461, 9472, 9605, 9637, 9644, 9659, 9666, 9697, 9716, 9719, 9754, 9792, 9865, 9869], [9932, 9935, 9971, 9995, 9996, 10003, 10053, 10073, 10114, 10133, 10158, 10177, 10178, 10181, 10222, 10229, 10274, 10278, 10303, 10309, 10310, 10315, 10356, 10365, 10368, 10404, 10406, 10424, 10440, 10451, 10469, 10475, 10479, 10531, 10562, 10577, 10582, 10648, 10657, 10675, 10702, 10711, 10720, 10726, 10728, 10762, 10790, 10819, 10829, 10879, 10893, 10915, 10934, 10950, 10955, 10967, 10994, 11003, 11016, 11022, 11031, 11061, 11069, 11097], [11111, 11118, 11132, 11196, 11199, 11206, 11209, 11223, 11255, 11257, 11309, 11311, 11312, 11345, 11392, 11404, 11432, 11439, 11492, 11503, 11533, 11546, 11551, 11556, 11559, 11583, 11586, 11596, 11619, 11641, 11665, 11755, 11760, 11765, 11817, 11831, 11835, 11852, 11857, 11862, 11876, 11882, 11883, 11906, 11915, 11930, 11939, 12007, 12018, 12037, 12070, 12089, 12101, 12132, 12197, 12205, 12231, 12282, 12309, 12317, 12323, 12372, 12403, 12413], [12418, 12427, 12470, 12477, 12633, 12658, 12664, 12697, 12702, 12749, 12768, 12795, 39, 53, 56, 75, 82, 129, 138, 164, 198, 254, 259, 267, 316, 337, 339, 341, 380, 388, 390, 394, 403, 417, 444, 525, 526, 530, 581, 584, 666, 674, 679, 692, 699, 705, 726, 763, 782, 787, 806, 808, 812, 821, 826, 841, 846, 876, 898, 915, 935, 954, 970, 1000], [1013, 1073, 1089, 1092, 1111, 1135, 1141, 1163, 1168, 1169, 1202, 1230, 1286, 1319, 1352, 1359, 1367, 1378, 1419, 1433, 1470, 1473, 1492, 1504, 1534, 1542, 1558, 1572, 1634, 1641, 1665, 1716, 1722, 1729, 1732, 1771, 1776, 1782, 1803, 1828, 1833, 1881, 1883, 1891, 1900, 1916, 1918, 1930, 1934, 1948, 1959, 1977, 1983, 2023, 2027, 2066, 2067, 2114, 2116, 2131, 2147, 2163, 2198, 2199], [2224, 2248, 2267, 2348, 2353, 2364, 2376, 2380, 2436, 2451, 2452, 2463, 2476, 2484, 2509, 2513, 2538, 2560, 2571, 2575, 2595, 2615, 2619, 2639, 2655, 2670, 2706, 2750, 2772, 2833, 2857, 2862, 2869, 2873, 2874, 2876, 2878, 2890, 2891, 2924, 2963, 2980, 2992, 2997, 3019, 3041, 3045, 3055, 3064, 3089, 3103, 3104, 3128, 3206, 3211, 3214, 3231, 3239, 3265, 3279, 3295, 3297, 3299, 3320], [3335, 3344, 3349, 3356, 3369, 3393, 3396, 3444, 3454, 3493, 3494, 3542, 3553, 3567, 3568, 3595, 3601, 3612, 3623, 3626, 3627, 3630, 3644, 3654, 3700, 3704, 3724, 3733, 3745, 3753, 3759, 3760, 3788, 3807, 3816, 3846, 3874, 3884, 3903, 3919, 3966, 3972, 4011, 4107, 4115, 4126, 4140, 4146, 4150, 4185, 4204, 4225, 4227, 4241, 4256, 4269, 4286, 4293, 4298, 4302, 4315, 4323, 4327, 4340], [4362, 4375, 4384, 4416, 4426, 4442, 4444, 4453, 4512, 4530, 4536, 4568, 4569, 4572, 4576, 4623, 4627, 4637, 4654, 4663, 4679, 4686, 4703, 4715, 4718, 4729, 4736, 4743, 4759, 4765, 4777, 4820, 4843, 4851, 4852, 4856, 4870, 4876, 4895, 4905, 4937, 4947, 4949, 4984, 5031, 5045, 5057, 5086, 5099, 5100, 5138, 5152, 5168, 5183, 5222, 5342, 5343, 5388, 5389, 5391, 5403, 5420, 5429, 5480], [5507, 5508, 5550, 5555, 5563, 5607, 5621, 5656, 5661, 5666, 5711, 5747, 5749, 5787, 5809, 5814, 5822, 5835, 5844, 5863, 5880, 5894, 5898, 5912, 5919, 5930, 5941, 5942, 5949, 5984, 5985, 6027, 6044, 6046, 6083, 6177, 6181, 6203, 6215, 6216, 6217, 6223, 6226, 6230, 6237, 6243, 6258, 6263, 6295, 6324, 6357, 6358, 6362, 6363, 6369, 6372, 6374, 6377, 6400, 6406, 6411, 6445, 6496, 6513], [6573, 6574, 6576, 6599, 6625, 6634, 6642, 6647, 6654, 6692, 6727, 6739, 6745, 6748, 6799, 6804, 6846, 6849, 6858, 6915, 6947, 6949, 6963, 6976, 6989, 7006, 7015, 7016, 7044, 7049, 7059, 7069, 7089, 7103, 7108, 7133, 7160, 7161, 7173, 7207, 7214, 7222, 7224, 7225, 7268, 7293, 7294, 7308, 7330, 7398, 7399, 7401, 7444, 7445, 7463, 7467, 7480, 7485, 7488, 7502, 7522, 7524, 7529, 7540], [7597, 7612, 7623, 7652, 7655, 7668, 7675, 7684, 7692, 7696, 7707, 7717, 7747, 7752, 7760, 7765, 7778, 7782, 7818, 7823, 7827, 7865, 7877, 7883, 7887, 7912, 7919, 7920, 7958, 7980, 7982, 7998, 8018, 8028, 8032, 8042, 8060, 8100, 8127, 8128, 8144, 8147, 8165, 8185, 8198, 8207, 8222, 8229, 8249, 8278, 8324, 8364, 8386, 8403, 8408, 8426, 8433, 8457, 8464, 8489, 8516, 8530, 8538, 8552], [8559, 8566, 8575, 8590, 8604, 8611, 8623, 8652, 8653, 8660, 8666, 8729, 8753, 8786, 8791, 8796, 8806, 8821, 8857, 8868, 8909, 8936, 8937, 9027, 9113, 9145, 9151, 9177, 9198, 9208, 9216, 9221, 9239, 9270, 9274, 9294, 9300, 9329, 9337, 9356, 9373, 9408, 9445, 9451, 9484, 9492, 9498, 9504, 9558, 9561, 9573, 9576, 9610, 9645, 9660, 9664, 9698, 9699, 9709, 9722, 9765, 9775, 9805, 9846], [9886, 9887, 9889, 9913, 9922, 9928, 9950, 9957, 9960, 9962, 9985, 10068, 10077, 10100, 10102, 10118, 10129, 10137, 10203, 10268, 10294, 10301, 10391, 10411, 10434, 10480, 10503, 10530, 10532, 10535, 10542, 10551, 10554, 10560, 10585, 10597, 10606, 10625, 10662, 10684, 10710, 10723, 10735, 10748, 10767, 10777, 10795, 10803, 10816, 10824, 10853, 10857, 10887, 10890, 10956, 10984, 10991, 10998, 11028, 11047, 11073, 11110, 11179, 11182], [11191, 11201, 11221, 11270, 11271, 11282, 11291, 11298, 11337, 11352, 11416, 11461, 11463, 11479, 11490, 11496, 11510, 11526, 11528, 11530, 11553, 11584, 11585, 11601, 11611, 11620, 11630, 11642, 11654, 11662, 11710, 11716, 11729, 11740, 11766, 11823, 11825, 11830, 11836, 11846, 11877, 11884, 11886, 11888, 11920, 11949, 11950, 11964, 11973, 11979, 11989, 12010, 12026, 12029, 12034, 12056, 12090, 12114, 12123, 12129, 12151, 12163, 12208, 12233], [12242, 12255, 12277, 12289, 12307, 12334, 12349, 12351, 12352, 12378, 12404, 12443, 12447, 12462, 12474, 12483, 12487, 12510, 12533, 12576, 12599, 12604, 12610, 12614, 12638, 12656, 12662, 12700, 12701, 12704, 12706, 12738, 12744, 12755, 12756, 12761, 12767, 12772, 12783, 27, 37, 50, 85, 89, 114, 118, 123, 142, 148, 149, 163, 258, 269, 299, 306, 311, 362, 366, 421, 422, 453, 475, 486, 493], [499, 512, 541, 564, 573, 593, 604, 615, 639, 640, 643, 653, 658, 660, 662, 680, 727, 730, 734, 740, 790, 830, 852, 853, 854, 888, 901, 930, 951, 973, 994, 1007, 1015, 1017, 1030, 1046, 1065, 1067, 1079, 1121, 1123, 1133, 1158, 1183, 1189, 1191, 1195, 1206, 1224, 1255, 1305, 1315, 1327, 1332, 1343, 1364, 1398, 1411, 1412, 1413, 1414, 1420, 1431, 1455], [1457, 1472, 1526, 1540, 1554, 1569, 1604, 1635, 1660, 1675, 1701, 1705, 1706, 1727, 1731, 1741, 1746, 1751, 1825, 1835, 1839, 1857, 1880, 1886, 1903, 1993, 1995, 2004, 2011, 2065, 2068, 2080, 2100, 2110, 2112, 2117, 2123, 2135, 2137, 2143, 2161, 2178, 2194, 2277, 2281, 2292, 2300, 2301, 2375, 2403, 2408, 2418, 2440, 2472, 2479, 2493, 2503, 2519, 2521, 2567, 2569, 2617, 2618, 2625], [2628, 2643, 2666, 2675, 2682, 2700, 2707, 2770, 2774, 2781, 2817, 2818, 2844, 2846, 2850, 2856, 2865, 2872, 2889, 2911, 2925, 2936, 2943, 2954, 2982, 2984, 2998, 3008, 3057, 3072, 3080, 3090, 3091, 3101, 3109, 3110, 3118, 3137, 3141, 3165, 3172, 3212, 3219, 3225, 3235, 3249, 3263, 3293, 3302, 3317, 3334, 3339, 3378, 3382, 3389, 3402, 3410, 3417, 3418, 3421, 3462, 3470, 3492, 3499], [3516, 3532, 3545, 3549, 3593, 3609, 3650, 3651, 3664, 3676, 3694, 3727, 3748, 3755, 3762, 3772, 3773, 3779, 3792, 3804, 3858, 3861, 3865, 3880, 3881, 3889, 3890, 3922, 3932, 3934, 3937, 3947, 3963, 3964, 3977, 3993, 3995, 4000, 4003, 4034, 4057, 4064, 4083, 4087, 4096, 4117, 4142, 4160, 4193, 4208, 4209, 4231, 4244, 4282, 4308, 4310, 4314, 4324, 4328, 4342, 4345, 4358, 4370, 4372], [4378, 4389, 4396, 4421, 4428, 4441, 4450, 4451, 4456, 4483, 4494, 4496, 4497, 4500, 4559, 4578, 4583, 4585, 4591, 4594, 4598, 4607, 4616, 4618, 4673, 4698, 4700, 4708, 4737, 4757, 4773, 4789, 4796, 4799, 4802, 4804, 4832, 4859, 4878, 4887, 4897, 4907, 4943, 4948, 4953, 4960, 4965, 4971, 4985, 4993, 4996, 5003, 5011, 5012, 5036, 5046, 5070, 5079, 5147, 5159, 5163, 5169, 5212, 5219], [5241, 5252, 5258, 5262, 5272, 5279, 5311, 5325, 5341, 5358, 5363, 5365, 5368, 5399, 5400, 5418, 5430, 5446, 5465, 5474, 5513, 5577, 5592, 5601, 5612, 5615, 5623, 5639, 5643, 5670, 5689, 5704, 5706, 5716, 5728, 5729, 5738, 5746, 5752, 5757, 5768, 5769, 5771, 5778, 5799, 5824, 5831, 5840, 5848, 5878, 5890, 5931, 5947, 5955, 5968, 5974, 6011, 6017, 6069, 6103, 6113, 6131, 6135, 6161], [6185, 6195, 6199, 6214, 6227, 6248, 6255, 6267, 6290, 6325, 6345, 6348, 6349, 6353, 6368, 6423, 6424, 6439, 6442, 6470, 6483, 6486, 6500, 6510, 6520, 6523, 6548, 6563, 6567, 6575, 6603, 6610, 6611, 6623, 6630, 6635, 6641, 6645, 6651, 6674, 6699, 6717, 6718, 6723, 6725, 6728, 6734, 6742, 6749, 6761, 6786, 6816, 6851, 6899, 6906, 6907, 6913, 6917, 6923, 6928, 6943, 6998, 7002, 7013], [7017, 7025, 7036, 7045, 7053, 7077, 7083, 7134, 7143, 7152, 7165, 7182, 7187, 7193, 7194, 7226, 7235, 7237, 7243, 7262, 7264, 7280, 7284, 7341, 7345, 7352, 7357, 7360, 7366, 7404, 7422, 7429, 7432, 7461, 7476, 7486, 7495, 7499, 7503, 7518, 7542, 7552, 7579, 7585, 7586, 7635, 7640, 7641, 7661, 7687, 7710, 7734, 7770, 7779, 7805, 7809, 7820, 7840, 7841, 7881, 7917, 7937, 7939, 7957], [7964, 7985, 7991, 8004, 8020, 8043, 8074, 8089, 8095, 8107, 8115, 8143, 8150, 8171, 8183, 8197, 8258, 8290, 8294, 8305, 8306, 8346, 8366, 8369, 8372, 8391, 8398, 8443, 8537, 8547, 8617, 8657, 8667, 8668, 8674, 8683, 8705, 8733, 8774, 8783, 8798, 8820, 8823, 8860, 8869, 8876, 8882, 8903, 8907, 8910, 8922, 8933, 8946, 8954, 8963, 8974, 8987, 8988, 8997, 9036, 9083, 9108, 9120, 9123], [9136, 9144, 9189, 9197, 9227, 9248, 9314, 9335, 9340, 9341, 9360, 9363, 9378, 9388, 9398, 9433, 9441, 9463, 9464, 9470, 9475, 9483, 9486, 9515, 9520, 9527, 9550, 9578, 9603, 9604, 9650, 9680, 9712, 9725, 9735, 9743, 9756, 9771, 9772, 9779, 9788, 9804, 9813, 9814, 9832, 9850, 9874, 9881, 9883, 9901, 9923, 9930, 9980, 9999, 10011, 10020, 10034, 10059, 10080, 10116, 10117, 10141, 10161, 10162], [10165, 10183, 10187, 10189, 10200, 10215, 10227, 10232, 10280, 10284, 10355, 10358, 10366, 10374, 10383, 10389, 10405, 10427, 10437, 10455, 10463, 10464, 10465, 10477, 10500, 10501, 10502, 10515, 10534, 10541, 10564, 10574, 10610, 10617, 10624, 10628, 10630, 10676, 10680, 10688, 10695, 10766, 10789, 10806, 10823, 10831, 10843, 10845, 10870, 10872, 10902, 10960, 10973, 10982, 10986, 11010, 11023, 11029, 11064, 11166, 11174, 11187, 11229, 11250], [11254, 11263, 11269, 11273, 11293, 11305, 11381, 11395, 11406, 11419, 11431, 11433, 11453, 11455, 11470, 11480, 11494, 11527, 11560, 11563, 11570, 11640, 11682, 11685, 11692, 11694, 11697, 11726, 11727, 11750, 11757, 11759, 11774, 11783, 11789, 11798, 11799, 11832, 11834, 11881, 11891, 11953, 11961, 11965, 11972, 12015, 12049, 12063, 12069, 12075, 12149, 12158, 12183, 12193, 12206, 12257, 12275, 12284, 12298, 12303, 12304, 12320, 12390, 12399], [12407, 12408, 12421, 12452, 12480, 12495, 12501, 12509, 12552, 12559, 12561, 12571, 12660, 12685, 12689, 12699, 12722, 12725, 12732, 12737, 12750, 12751, 12777, 12786, 6, 10, 17, 30, 47, 52, 93, 119, 180, 183, 194, 197, 200, 208, 261, 264, 347, 357, 384, 409, 411, 419, 436, 438, 439, 440, 448, 459, 462, 463, 467, 469, 501, 502, 552, 598, 621, 627, 634, 667], [668, 675, 681, 711, 714, 783, 791, 818, 838, 887, 936, 942, 947, 948, 971, 1003, 1045, 1066, 1100, 1102, 1116, 1119, 1153, 1154, 1194, 1251, 1253, 1287, 1288, 1298, 1300, 1301, 1355, 1365, 1375, 1388, 1391, 1422, 1452, 1481, 1561, 1565, 1577, 1603, 1612, 1616, 1623, 1632, 1654, 1661, 1678, 1696, 1733, 1778, 1806, 1824, 1864, 1874, 1905, 1924, 1942, 1991, 1998, 2029], [2038, 2064, 2071, 2073, 2118, 2145, 2172, 2179, 2188, 2209, 2222, 2249, 2251, 2254, 2263, 2276, 2288, 2313, 2323, 2334, 2344, 2443, 2450, 2453, 2455, 2458, 2496, 2539, 2581, 2588, 2658, 2709, 2710, 2727, 2734, 2735, 2756, 2847, 2864, 2919, 2945, 2951, 2952, 2959, 2971, 2973, 3004, 3005, 3018, 3028, 3040, 3066, 3095, 3119, 3126, 3145, 3150, 3153, 3171, 3187, 3223, 3253, 3277, 3282], [3303, 3325, 3372, 3383, 3403, 3405, 3468, 3471, 3502, 3514, 3521, 3557, 3564, 3580, 3666, 3684, 3697, 3746, 3747, 3750, 3764, 3768, 3769, 3782, 3785, 3821, 3822, 3835, 3839, 3873, 3910, 3930, 3975, 3978, 3994, 4043, 4071, 4133, 4164, 4171, 4199, 4211, 4229, 4237, 4252, 4350, 4379, 4403, 4418, 4492, 4517, 4523, 4526, 4575, 4611, 4613, 4636, 4646, 4655, 4692, 4725, 4764, 4776, 4778], [4803, 4812, 4831, 4837, 4854, 4874, 4885, 4891, 4899, 4900, 4934, 4980, 4986, 5013, 5041, 5059, 5068, 5090, 5091, 5126, 5136, 5139, 5161, 5167, 5181, 5215, 5234, 5243, 5274, 5278, 5297, 5307, 5319, 5364, 5367, 5369, 5370, 5373, 5379, 5386, 5397, 5412, 5413, 5415, 5426, 5436, 5473, 5506, 5516, 5542, 5549, 5551, 5556, 5582, 5618, 5619, 5622, 5657, 5686, 5687, 5703, 5732, 5734, 5750], [5755, 5793, 5794, 5871, 5872, 5886, 5895, 5905, 5922, 5925, 5936, 5951, 5966, 6047, 6048, 6095, 6108, 6109, 6119, 6145, 6147, 6153, 6155, 6159, 6173, 6175, 6207, 6208, 6225, 6279, 6281, 6286, 6288, 6293, 6308, 6312, 6314, 6337, 6338, 6382, 6425, 6448, 6474, 6488, 6512, 6519, 6531, 6535, 6544, 6612, 6615, 6621, 6622, 6656, 6678, 6684, 6687, 6706, 6714, 6715, 6724, 6773, 6787, 6791], [6800, 6801, 6847, 6874, 6887, 6959, 7020, 7035, 7050, 7062, 7067, 7070, 7073, 7079, 7110, 7114, 7128, 7149, 7157, 7185, 7186, 7197, 7255, 7278, 7288, 7301, 7304, 7323, 7340, 7342, 7344, 7351, 7365, 7417, 7421, 7424, 7442, 7449, 7536, 7546, 7551, 7557, 7573, 7577, 7592, 7605, 7627, 7628, 7630, 7663, 7673, 7679, 7697, 7729, 7758, 7766, 7768, 7793, 7797, 7822, 7832, 7918, 7923, 7926], [7942, 7953, 7955, 7975, 7983, 7989, 7992, 8027, 8034, 8044, 8046, 8059, 8071, 8072, 8129, 8140, 8142, 8177, 8187, 8199, 8246, 8253, 8295, 8325, 8339, 8352, 8358, 8373, 8388, 8405, 8407, 8437, 8459, 8502, 8529, 8532, 8542, 8573, 8595, 8598, 8620, 8622, 8656, 8664, 8669, 8670, 8678, 8698, 8758, 8771, 8777, 8779, 8802, 8807, 8863, 8890, 8917, 8921, 8965, 9012, 9064, 9100, 9112, 9128], [9135, 9152, 9170, 9209, 9240, 9243, 9254, 9266, 9269, 9272, 9276, 9296, 9301, 9303, 9309, 9315, 9316, 9346, 9365, 9382, 9383, 9391, 9405, 9447, 9448, 9485, 9487, 9489, 9491, 9509, 9562, 9566, 9632, 9668, 9681, 9702, 9726, 9730, 9736, 9740, 9751, 9780, 9812, 9831, 9837, 9934, 9947, 9974, 10013, 10019, 10030, 10031, 10045, 10054, 10066, 10074, 10119, 10125, 10148, 10174, 10212, 10218, 10225, 10243], [10287, 10300, 10331, 10353, 10363, 10387, 10409, 10414, 10417, 10423, 10446, 10459, 10484, 10487, 10491, 10509, 10519, 10563, 10565, 10581, 10592, 10595, 10612, 10618, 10627, 10666, 10670, 10690, 10700, 10741, 10752, 10758, 10768, 10812, 10826, 10882, 10914, 10931, 10933, 10946, 10948, 11012, 11048, 11054, 11067, 11068, 11084, 11095, 11135, 11141, 11145, 11149, 11169, 11192, 11195, 11203, 11275, 11286, 11290, 11295, 11304, 11307, 11349, 11366], [11379, 11387, 11393, 11400, 11414, 11435, 11459, 11483, 11487, 11489, 11519, 11547, 11566, 11578, 11581, 11591, 11606, 11623, 11646, 11647, 11679, 11695, 11700, 11701, 11721, 11733, 11747, 11754, 11779, 11796, 11837, 11844, 11847, 11866, 11887, 11894, 11926, 11940, 11945, 11970, 12036, 12072, 12084, 12113, 12117, 12128, 12143, 12148, 12150, 12190, 12201, 12220, 12234, 12250, 12251, 12264, 12278, 12299, 12327, 12330, 12347, 12384, 12394, 12409], [12448, 12465, 12491, 12492, 12496, 12503, 12615, 12627, 12644, 12649, 12674, 12688, 12696, 12730, 12742, 12745, 12754, 12775, 12788, 12797, 4, 15, 20, 26, 32, 35, 41, 66, 83, 110, 145, 171, 199, 250, 252, 256, 257, 272, 279, 291, 302, 304, 326, 334, 358, 374, 375, 377, 400, 505, 508, 513, 527, 539, 551, 561, 565, 574, 579, 601, 650, 664, 686, 698], [722, 729, 742, 743, 769, 772, 813, 814, 869, 905, 911, 966, 1016, 1020, 1039, 1052, 1070, 1080, 1087, 1093, 1109, 1114, 1131, 1159, 1172, 1182, 1187, 1204, 1222, 1223, 1227, 1229, 1274, 1275, 1283, 1292, 1297, 1344, 1363, 1369, 1370, 1379, 1395, 1399, 1402, 1448, 1456, 1515, 1521, 1524, 1533, 1551, 1574, 1592, 1644, 1648, 1649, 1666, 1668, 1671, 1700, 1738, 1759, 1760], [1858, 1890, 1915, 1920, 1932, 1946, 1952, 1958, 1963, 1968, 1971, 2006, 2018, 2053, 2075, 2102, 2119, 2128, 2129, 2154, 2156, 2218, 2239, 2317, 2360, 2378, 2382, 2387, 2434, 2464, 2469, 2473, 2506, 2525, 2526, 2555, 2602, 2637, 2641, 2653, 2674, 2677, 2687, 2733, 2768, 2782, 2785, 2800, 2801, 2802, 2804, 2810, 2827, 2837, 2842, 2894, 2899, 2935, 2942, 2956, 2968, 2977, 2993, 3014], [3031, 3039, 3049, 3079, 3130, 3135, 3160, 3163, 3202, 3207, 3232, 3242, 3270, 3275, 3280, 3285, 3309, 3316, 3367, 3445, 3446, 3483, 3486, 3487, 3488, 3489, 3523, 3525, 3560, 3569, 3576, 3581, 3596, 3671, 3672, 3677, 3707, 3722, 3743, 3787, 3790, 3818, 3875, 3897, 3942, 3951, 3953, 3961, 3967, 3969, 3971, 4004, 4008, 4052, 4053, 4054, 4065, 4102, 4122, 4136, 4182, 4215, 4224, 4230], [4233, 4262, 4275, 4300, 4309, 4322, 4331, 4337, 4346, 4402, 4404, 4431, 4432, 4469, 4473, 4527, 4534, 4549, 4555, 4564, 4577, 4581, 4596, 4601, 4619, 4630, 4664, 4665, 4666, 4688, 4745, 4749, 4761, 4766, 4782, 4818, 4846, 4855, 4867, 4908, 4936, 4958, 4976, 5001, 5014, 5032, 5035, 5076, 5107, 5108, 5120, 5143, 5165, 5207, 5227, 5232, 5246, 5248, 5269, 5296, 5300, 5317, 5322, 5383], [5395, 5398, 5410, 5434, 5438, 5469, 5484, 5523, 5528, 5529, 5534, 5541, 5566, 5590, 5606, 5626, 5637, 5678, 5685, 5690, 5712, 5725, 5765, 5782, 5791, 5805, 5828, 5838, 5875, 5959, 5994, 6004, 6008, 6028, 6041, 6053, 6071, 6148, 6154, 6157, 6170, 6205, 6213, 6233, 6236, 6238, 6260, 6283, 6289, 6297, 6330, 6342, 6379, 6389, 6408, 6432, 6498, 6538, 6565, 6605, 6653, 6659, 6669, 6681], [6701, 6732, 6744, 6777, 6781, 6815, 6827, 6855, 6861, 6868, 6897, 6946, 6950, 6961, 6979, 6986, 7001, 7003, 7011, 7024, 7028, 7061, 7066, 7068, 7091, 7098, 7190, 7198, 7221, 7232, 7246, 7263, 7271, 7283, 7287, 7305, 7334, 7335, 7336, 7347, 7358, 7371, 7380, 7410, 7414, 7416, 7428, 7491, 7496, 7500, 7511, 7514, 7558, 7560, 7609, 7610, 7643, 7676, 7686, 7688, 7693, 7720, 7732, 7733], [7781, 7786, 7790, 7801, 7829, 7837, 7868, 7907, 7925, 7928, 7970, 7978, 7988, 7996, 8010, 8033, 8073, 8101, 8109, 8145, 8170, 8175, 8190, 8221, 8223, 8224, 8230, 8232, 8238, 8242, 8271, 8327, 8362, 8396, 8436, 8441, 8455, 8463, 8492, 8493, 8504, 8593, 8600, 8613, 8632, 8645, 8654, 8716, 8721, 8744, 8745, 8756, 8765, 8787, 8792, 8816, 8819, 8841, 8842, 8888, 8938, 8952, 8967, 8971], [8995, 9047, 9049, 9059, 9086, 9091, 9147, 9182, 9193, 9211, 9212, 9226, 9229, 9234, 9247, 9257, 9260, 9262, 9282, 9334, 9355, 9387, 9389, 9390, 9428, 9429, 9438, 9450, 9457, 9502, 9514, 9528, 9535, 9536, 9559, 9565, 9572, 9588, 9601, 9606, 9662, 9684, 9685, 9732, 9747, 9753, 9785, 9791, 9811, 9827, 9859, 9868, 9873, 9976, 9984, 10022, 10024, 10026, 10037, 10055, 10064, 10078, 10124, 10135], [10191, 10199, 10202, 10204, 10208, 10239, 10248, 10261, 10266, 10385, 10428, 10462, 10468, 10481, 10488, 10490, 10495, 10496, 10590, 10600, 10611, 10633, 10681, 10694, 10701, 10707, 10717, 10743, 10788, 10820, 10877, 10878, 10899, 10925, 10993, 11013, 11024, 11049, 11052, 11086, 11098, 11126, 11138, 11144, 11147, 11164, 11189, 11207, 11210, 11233, 11248, 11284, 11294, 11313, 11318, 11325, 11327, 11333, 11367, 11371, 11374, 11399, 11401, 11415], [11418, 11423, 11438, 11441, 11451, 11462, 11465, 11466, 11474, 11529, 11544, 11550, 11564, 11594, 11600, 11604, 11634, 11658, 11660, 11687, 11703, 11708, 11713, 11714, 11717, 11722, 11731, 11743, 11768, 11790, 11811, 11826, 11840, 11841, 11861, 11869, 11873, 11892, 11905, 11911, 11912, 11923, 11947, 11954, 11960, 11963, 12006, 12011, 12065, 12092, 12096, 12097, 12102, 12111, 12125, 12131, 12136, 12165, 12170, 12202, 12216, 12230, 12232, 12236], [12241, 12253, 12280, 12292, 12305, 12342, 12348, 12354, 12360, 12383, 12402, 12430, 12517, 12524, 12526, 12529, 12532, 12540, 12563, 12579, 12598, 12623, 12629, 12675, 12676, 12678, 12687, 12727, 12741, 12743, 7, 48, 102, 109, 115, 128, 130, 141, 150, 157, 161, 196, 206, 224, 274, 280, 281, 292, 321, 327, 370, 378, 415, 426, 427, 433, 443, 457, 514, 548, 568, 571, 587, 595], [642, 647, 671, 676, 749, 753, 754, 825, 864, 865, 937, 977, 997, 1042, 1056, 1090, 1103, 1142, 1176, 1179, 1200, 1219, 1220, 1238, 1239, 1261, 1304, 1316, 1330, 1360, 1380, 1417, 1489, 1499, 1502, 1512, 1553, 1566, 1570, 1600, 1607, 1622, 1647, 1680, 1684, 1689, 1694, 1710, 1712, 1713, 1734, 1737, 1798, 1810, 1818, 1836, 1843, 1854, 1865, 1876, 1888, 1894, 1935, 1960], [1964, 2008, 2015, 2037, 2057, 2084, 2105, 2148, 2193, 2202, 2211, 2212, 2215, 2223, 2227, 2228, 2246, 2252, 2255, 2265, 2268, 2289, 2311, 2321, 2327, 2351, 2392, 2400, 2406, 2433, 2465, 2478, 2488, 2500, 2501, 2523, 2591, 2592, 2594, 2623, 2632, 2662, 2673, 2708, 2714, 2739, 2814, 2823, 2826, 2836, 2858, 2859, 2892, 2915, 2923, 2948, 2955, 2957, 2975, 3038, 3044, 3063, 3083, 3111], [3149, 3220, 3245, 3257, 3260, 3266, 3274, 3310, 3323, 3326, 3327, 3362, 3385, 3407, 3416, 3428, 3448, 3461, 3535, 3544, 3552, 3558, 3585, 3592, 3602, 3608, 3610, 3636, 3640, 3681, 3692, 3718, 3793, 3798, 3830, 3841, 3879, 3926, 3929, 3939, 3943, 3948, 4009, 4027, 4032, 4046, 4060, 4085, 4089, 4094, 4104, 4271, 4279, 4281, 4287, 4305, 4326, 4334, 4338, 4357, 4380, 4397, 4401, 4414], [4415, 4417, 4427, 4440, 4449, 4481, 4487, 4516, 4532, 4554, 4629, 4661, 4678, 4690, 4784, 4787, 4810, 4827, 4829, 4838, 4844, 4881, 4890, 4916, 4917, 4918, 4935, 4946, 4973, 4975, 4977, 4998, 5015, 5069, 5072, 5075, 5096, 5104, 5135, 5166, 5178, 5198, 5206, 5221, 5265, 5266, 5273, 5294, 5299, 5308, 5312, 5357, 5376, 5384, 5409, 5449, 5472, 5502, 5535, 5584, 5589, 5595, 5597, 5630], [5632, 5680, 5688, 5713, 5722, 5727, 5740, 5742, 5773, 5785, 5813, 5821, 5832, 5859, 5876, 5883, 5953, 5961, 5964, 5972, 5977, 6026, 6030, 6042, 6059, 6072, 6074, 6106, 6120, 6149, 6168, 6242, 6249, 6266, 6274, 6276, 6318, 6332, 6335, 6339, 6343, 6364, 6397, 6409, 6438, 6444, 6449, 6465, 6528, 6571, 6581, 6585, 6587, 6602, 6617, 6624, 6626, 6636, 6637, 6640, 6660, 6709, 6713, 6741], [6753, 6809, 6825, 6835, 6853, 6854, 6869, 6870, 6893, 6905, 6911, 6922, 6926, 6929, 6956, 6992, 7009, 7022, 7027, 7037, 7055, 7072, 7094, 7109, 7135, 7151, 7168, 7174, 7201, 7210, 7216, 7252, 7254, 7281, 7300, 7338, 7348, 7356, 7374, 7434, 7453, 7473, 7505, 7515, 7528, 7539, 7563, 7575, 7582, 7593, 7611, 7646, 7660, 7719, 7741, 7751, 7762, 7800, 7828, 7835, 7845, 7850, 7879, 7889], [7905, 7908, 7933, 7947, 7954, 7959, 8001, 8029, 8092, 8098, 8152, 8153, 8173, 8188, 8193, 8195, 8214, 8218, 8252, 8256, 8262, 8266, 8284, 8319, 8330, 8333, 8340, 8353, 8371, 8401, 8417, 8425, 8427, 8469, 8484, 8517, 8520, 8527, 8533, 8541, 8591, 8616, 8630, 8662, 8676, 8691, 8735, 8737, 8790, 8811, 8825, 8833, 8852, 8879, 8895, 8919, 8923, 8955, 8972, 9010, 9052, 9065, 9073, 9087], [9194, 9259, 9265, 9271, 9324, 9336, 9362, 9368, 9369, 9371, 9418, 9425, 9436, 9439, 9474, 9476, 9478, 9479, 9511, 9533, 9537, 9544, 9548, 9549, 9568, 9580, 9586, 9614, 9631, 9635, 9636, 9653, 9737, 9750, 9767, 9776, 9787, 9802, 9810, 9824, 9862, 9864, 9872, 9876, 9885, 9897, 9926, 9945, 9955, 9975, 9998, 10012, 10021, 10035, 10092, 10103, 10121, 10123, 10151, 10160, 10164, 10166, 10235, 10242], [10244, 10258, 10267, 10282, 10293, 10298, 10308, 10314, 10350, 10372, 10398, 10421, 10429, 10441, 10442, 10454, 10483, 10512, 10514, 10545, 10578, 10634, 10656, 10674, 10689, 10708, 10713, 10730, 10742, 10750, 10755, 10760, 10776, 10782, 10792, 10808, 10814, 10817, 10840, 10869, 10885, 10894, 10895, 10897, 10898, 10904, 10943, 10944, 10945, 10962, 10963, 10980, 10992, 10996, 10999, 11000, 11007, 11035, 11045, 11055, 11056, 11058, 11063, 11070], [11078, 11133, 11134, 11158, 11172, 11224, 11243, 11245, 11292, 11320, 11324, 11360, 11412, 11442, 11454, 11471, 11482, 11497, 11561, 11567, 11582, 11597, 11609, 11670, 11678, 11707, 11725, 11739, 11748, 11758, 11764, 11769, 11777, 11804, 11810, 11909, 11942, 11944, 11948, 11974, 11984, 11990, 11998, 12000, 12009, 12014, 12017, 12028, 12057, 12124, 12152, 12155, 12179, 12192, 12195, 12211, 12218, 12271, 12273, 12288, 12291, 12302, 12314, 12325], [12337, 12364, 12381, 12414, 12417, 12426, 12445, 12453, 12469, 12522, 12541, 12542, 12545, 12575, 12602, 12613, 12622, 12626, 12631, 12663, 12665, 12716, 12723, 12762, 12773, 12793, 8, 13, 16, 21, 45, 71, 88, 127, 132, 167, 175, 203, 255, 265, 266, 289, 323, 336, 349, 376, 420, 442, 476, 481, 591, 592, 609, 611, 641, 644, 648, 655, 685, 702, 709, 756, 784, 800], [803, 861, 893, 895, 897, 932, 955, 964, 965, 972, 993, 996, 1043, 1049, 1055, 1081, 1091, 1105, 1118, 1134, 1193, 1226, 1264, 1281, 1282, 1306, 1313, 1314, 1325, 1357, 1403, 1406, 1407, 1446, 1449, 1462, 1474, 1480, 1513, 1514, 1530, 1548, 1567, 1568, 1618, 1643, 1645, 1659, 1662, 1669, 1672, 1673, 1682, 1688, 1749, 1791, 1800, 1811, 1813, 1815, 1851, 1855, 1878, 1895], [1936, 1951, 1972, 1981, 1986, 2002, 2014, 2022, 2026, 2033, 2044, 2061, 2097, 2109, 2134, 2166, 2206, 2226, 2264, 2266, 2274, 2305, 2308, 2324, 2329, 2332, 2339, 2347, 2361, 2363, 2388, 2404, 2423, 2428, 2480, 2531, 2534, 2535, 2553, 2570, 2589, 2649, 2663, 2664, 2702, 2720, 2760, 2780, 2790, 2806, 2808, 2811, 2863, 2882, 2883, 2893, 2897, 2900, 2905, 2914, 2922, 2937, 2985, 2999], [3009, 3027, 3032, 3037, 3047, 3129, 3131, 3134, 3139, 3143, 3166, 3167, 3174, 3177, 3178, 3183, 3210, 3213, 3216, 3222, 3241, 3271, 3332, 3336, 3340, 3368, 3375, 3376, 3392, 3394, 3422, 3424, 3442, 3451, 3519, 3522, 3528, 3529, 3548, 3615, 3629, 3656, 3705, 3708, 3710, 3765, 3774, 3776, 3797, 3812, 3823, 3850, 3854, 3864, 3882, 3894, 3925, 3946, 3960, 4006, 4013, 4016, 4025, 4026], [4033, 4055, 4066, 4093, 4120, 4130, 4154, 4165, 4168, 4176, 4186, 4198, 4210, 4212, 4214, 4277, 4301, 4319, 4335, 4364, 4423, 4438, 4470, 4471, 4478, 4485, 4489, 4520, 4535, 4538, 4541, 4543, 4595, 4615, 4639, 4643, 4647, 4662, 4677, 4681, 4701, 4712, 4719, 4739, 4760, 4768, 4780, 4790, 4817, 4860, 4923, 4944, 4945, 4957, 4982, 5004, 5025, 5027, 5039, 5074, 5097, 5112, 5115, 5125], [5128, 5140, 5172, 5177, 5186, 5191, 5195, 5240, 5264, 5283, 5292, 5316, 5352, 5355, 5378, 5387, 5411, 5442, 5445, 5461, 5479, 5482, 5509, 5518, 5591, 5604, 5663, 5681, 5694, 5714, 5724, 5726, 5781, 5790, 5797, 5798, 5807, 5830, 5834, 5866, 5877, 5902, 5921, 5929, 5935, 5944, 5963, 5978, 5989, 6019, 6025, 6036, 6063, 6068, 6088, 6124, 6142, 6151, 6191, 6196, 6206, 6272, 6273, 6282], [6296, 6313, 6347, 6401, 6404, 6429, 6476, 6489, 6514, 6533, 6569, 6588, 6638, 6666, 6680, 6686, 6697, 6700, 6708, 6771, 6782, 6798, 6803, 6813, 6824, 6830, 6881, 6886, 6919, 6937, 6981, 7019, 7023, 7040, 7060, 7074, 7075, 7080, 7099, 7132, 7144, 7164, 7250, 7259, 7291, 7306, 7319, 7322, 7324, 7329, 7355, 7359, 7475, 7506, 7516, 7517, 7538, 7549, 7565, 7572, 7576, 7616, 7617, 7642], [7724, 7738, 7746, 7787, 7791, 7803, 7804, 7812, 7830, 7842, 7853, 7856, 7866, 7869, 7875, 7895, 7898, 7902, 7936, 7960, 7971, 7972, 7981, 8047, 8097, 8102, 8105, 8116, 8136, 8137, 8201, 8203, 8206, 8219, 8231, 8263, 8265, 8300, 8308, 8321, 8343, 8380, 8383, 8449, 8475, 8498, 8514, 8543, 8546, 8568, 8579, 8608, 8614, 8618, 8637, 8647, 8665, 8688, 8696, 8715, 8717, 8719, 8746, 8761], [8767, 8773, 8775, 8829, 8866, 8877, 8904, 8927, 8941, 8968, 9021, 9060, 9061, 9077, 9143, 9146, 9190, 9196, 9217, 9219, 9236, 9242, 9289, 9307, 9330, 9357, 9379, 9392, 9417, 9444, 9494, 9503, 9508, 9531, 9575, 9593, 9600, 9624, 9672, 9678, 9679, 9696, 9701, 9705, 9718, 9721, 9755, 9757, 9763, 9770, 9789, 9806, 9835, 9895, 9898, 9909, 9911, 9914, 9920, 9940, 9988, 9992, 10006, 10015], [10049, 10070, 10075, 10081, 10093, 10095, 10109, 10110, 10112, 10126, 10136, 10147, 10168, 10190, 10194, 10201, 10209, 10241, 10246, 10247, 10251, 10256, 10264, 10279, 10320, 10329, 10334, 10369, 10380, 10415, 10431, 10433, 10449, 10460, 10485, 10507, 10510, 10516, 10537, 10557, 10587, 10629, 10669, 10678, 10679, 10709, 10725, 10734, 10751, 10774, 10796, 10797, 10799, 10809, 10828, 10835, 10839, 10905, 10929, 10932, 10958, 10970, 10978, 10988], [10989, 11026, 11027, 11034, 11072, 11082, 11090, 11137, 11156, 11186, 11212, 11266, 11316, 11332, 11348, 11358, 11410, 11447, 11448, 11469, 11512, 11517, 11522, 11539, 11579, 11589, 11615, 11633, 11659, 11668, 11669, 11730, 11752, 11761, 11763, 11772, 11781, 11793, 11855, 11871, 11890, 11898, 11899, 11901, 11934, 11937, 11952, 11969, 11976, 11983, 11993, 11994, 12019, 12035, 12050, 12078, 12110, 12146, 12171, 12239, 12254, 12268, 12276, 12286], [12311, 12315, 12318, 12322, 12353, 12368, 12374, 12386, 12387, 12432, 12436, 12438, 12454, 12478, 12500, 12519, 12523, 12544, 12553, 12557, 12567, 12588, 12590, 12648, 12655, 12673, 12682, 12690, 12698, 12739, 12746, 12780, 33, 59, 99, 111, 117, 120, 151, 159, 169, 170, 222, 268, 294, 295, 346, 354, 382, 401, 450, 451, 504, 520, 531, 594, 603, 605, 608, 618, 704, 732, 752, 775], [809, 824, 828, 882, 903, 939, 962, 981, 1050, 1077, 1084, 1098, 1101, 1106, 1129, 1170, 1173, 1181, 1210, 1214, 1231, 1232, 1234, 1257, 1271, 1333, 1358, 1428, 1466, 1493, 1596, 1598, 1605, 1609, 1630, 1658, 1685, 1699, 1719, 1756, 1777, 1783, 1792, 1867, 1885, 1898, 1912, 1925, 1927, 1928, 1953, 1988, 2007, 2012, 2046, 2058, 2081, 2098, 2122, 2167, 2189, 2230, 2234, 2238], [2250, 2297, 2314, 2315, 2319, 2374, 2391, 2410, 2424, 2454, 2461, 2471, 2505, 2533, 2551, 2561, 2580, 2616, 2661, 2685, 2731, 2744, 2754, 2769, 2775, 2794, 2877, 2881, 2885, 2888, 2902, 2904, 2913, 2934, 2953, 3024, 3033, 3048, 3056, 3059, 3096, 3114, 3124, 3142, 3146, 3175, 3208, 3209, 3238, 3267, 3300, 3306, 3307, 3342, 3350, 3374, 3387, 3398, 3414, 3466, 3505, 3563, 3570, 3572], [3598, 3599, 3604, 3616, 3653, 3657, 3659, 3662, 3663, 3673, 3703, 3735, 3758, 3809, 3814, 3844, 3883, 3902, 3907, 3908, 3974, 3976, 3985, 3988, 4072, 4073, 4079, 4088, 4092, 4111, 4113, 4145, 4166, 4194, 4201, 4285, 4312, 4317, 4333, 4347, 4349, 4359, 4394, 4461, 4513, 4537, 4635, 4649, 4682, 4689, 4711, 4716, 4717, 4724, 4735, 4762, 4771, 4774, 4805, 4826, 4833, 4835, 4841, 4857], [4872, 4904, 4919, 4979, 5005, 5006, 5034, 5044, 5055, 5056, 5064, 5123, 5132, 5133, 5146, 5155, 5185, 5192, 5250, 5263, 5280, 5335, 5366, 5374, 5439, 5454, 5477, 5512, 5515, 5521, 5526, 5536, 5564, 5628, 5667, 5677, 5702, 5717, 5720, 5744, 5758, 5770, 5777, 5796, 5845, 5860, 5891, 5916, 5920, 5928, 5957, 5979, 5983, 6000, 6014, 6020, 6037, 6057, 6077, 6078, 6097, 6100, 6176, 6187], [6224, 6244, 6280, 6300, 6311, 6315, 6344, 6360, 6366, 6375, 6427, 6443, 6485, 6515, 6527, 6536, 6558, 6577, 6606, 6616, 6618, 6673, 6729, 6792, 6793, 6794, 6812, 6888, 6912, 6951, 6952, 6980, 6997, 7031, 7052, 7092, 7097, 7116, 7117, 7129, 7145, 7158, 7183, 7192, 7195, 7228, 7229, 7267, 7297, 7316, 7318, 7350, 7369, 7390, 7402, 7452, 7457, 7468, 7492, 7570, 7583, 7587, 7601, 7621], [7659, 7736, 7749, 7815, 7854, 7859, 7864, 7904, 8024, 8084, 8086, 8091, 8096, 8148, 8174, 8179, 8182, 8192, 8204, 8247, 8250, 8264, 8304, 8322, 8367, 8399, 8406, 8410, 8428, 8466, 8472, 8491, 8507, 8508, 8522, 8526, 8550, 8583, 8584, 8586, 8597, 8599, 8609, 8690, 8720, 8813, 8847, 8924, 8928, 8976, 8982, 8990, 8991, 9017, 9048, 9051, 9053, 9063, 9069, 9148, 9154, 9158, 9163, 9195], [9238, 9251, 9252, 9291, 9319, 9342, 9345, 9375, 9380, 9395, 9399, 9401, 9421, 9465, 9518, 9538, 9551, 9552, 9611, 9623, 9643, 9652, 9682, 9690, 9692, 9703, 9711, 9742, 9777, 9790, 9816, 9825, 9830, 9836, 9849, 9851, 9875, 9880, 9941, 9948, 9968, 9977, 9997, 10010, 10018, 10023, 10032, 10040, 10067, 10096, 10115, 10131, 10150, 10156, 10171, 10193, 10211, 10221, 10250, 10275, 10299, 10343, 10344, 10388], [10390, 10471, 10486, 10555, 10569, 10615, 10642, 10647, 10667, 10683, 10697, 10727, 10738, 10745, 10769, 10772, 10802, 10833, 10849, 10928, 10930, 10936, 10942, 10972, 10987, 10990, 11004, 11008, 11036, 11040, 11060, 11102, 11107, 11162, 11165, 11178, 11193, 11198, 11232, 11237, 11272, 11323, 11339, 11340, 11343, 11408, 11409, 11411, 11420, 11428, 11468, 11476, 11488, 11495, 11518, 11520, 11524, 11525, 11545, 11562, 11608, 11612, 11618, 11632], [11650, 11655, 11672, 11674, 11675, 11683, 11691, 11711, 11860, 11874, 11900, 11914, 11916, 11924, 11959, 11968, 12067, 12088, 12093, 12095, 12119, 12137, 12140, 12228, 12252, 12256, 12263, 12270, 12297, 12313, 12329, 12333, 12380, 12382, 12468, 12476, 12485, 12494, 12497, 12507, 12530, 12538, 12543, 12568, 12582, 12616, 12618, 12620, 12625, 12666, 12681, 12694, 12713, 12714, 12719, 12734, 12747, 12758, 12781, 12789, 11, 22, 72, 172], [211, 242, 305, 315, 379, 413, 430, 460, 465, 482, 490, 528, 535, 537, 570, 597, 630, 631, 638, 672, 689, 693, 723, 755, 759, 786, 804, 849, 868, 878, 890, 945, 958, 991, 1011, 1023, 1035, 1038, 1058, 1059, 1086, 1095, 1139, 1167, 1177, 1178, 1199, 1215, 1237, 1273, 1277, 1320, 1336, 1349, 1427, 1429, 1482, 1485, 1508, 1516, 1539, 1549, 1588, 1636], [1721, 1748, 1750, 1754, 1785, 1790, 1830, 1846, 1849, 1933, 1989, 2048, 2069, 2076, 2078, 2101, 2150, 2174, 2186, 2241, 2278, 2302, 2316, 2341, 2352, 2398, 2402, 2422, 2489, 2491, 2492, 2495, 2499, 2541, 2606, 2610, 2612, 2621, 2633, 2676, 2691, 2703, 2711, 2717, 2732, 2757, 2761, 2765, 2773, 2776, 2779, 2786, 2793, 2855, 2879, 2912, 2986, 2988, 3035, 3051, 3092, 3097, 3100, 3102], [3120, 3140, 3189, 3199, 3203, 3215, 3248, 3273, 3284, 3294, 3321, 3343, 3364, 3379, 3472, 3559, 3591, 3665, 3670, 3693, 3715, 3732, 3736, 3749, 3761, 3796, 3811, 3836, 3899, 3927, 3965, 3999, 4022, 4036, 4137, 4178, 4187, 4226, 4304, 4341, 4355, 4369, 4371, 4462, 4472, 4508, 4529, 4544, 4570, 4579, 4614, 4617, 4651, 4659, 4714, 4723, 4741, 4744, 4746, 4758, 4769, 4819, 4825, 4865], [4866, 4880, 4914, 4921, 4932, 4972, 4989, 4992, 5017, 5021, 5049, 5066, 5116, 5117, 5121, 5134, 5137, 5150, 5179, 5249, 5256, 5268, 5276, 5282, 5286, 5320, 5337, 5375, 5393, 5405, 5423, 5444, 5448, 5569, 5581, 5598, 5600, 5635, 5642, 5649, 5668, 5673, 5682, 5748, 5751, 5865, 5897, 5914, 5938, 5952, 5986, 5999, 6050, 6080, 6133, 6143, 6194, 6210, 6211, 6232, 6259, 6284, 6305, 6310], [6384, 6394, 6403, 6455, 6456, 6480, 6497, 6503, 6508, 6518, 6526, 6547, 6590, 6627, 6657, 6664, 6665, 6712, 6735, 6752, 6759, 6767, 6802, 6865, 6885, 6896, 6930, 6953, 6967, 6974, 6999, 7038, 7086, 7148, 7238, 7242, 7253, 7261, 7302, 7307, 7315, 7333, 7354, 7382, 7384, 7387, 7391, 7455, 7482, 7519, 7520, 7533, 7537, 7588, 7614, 7620, 7656, 7674, 7723, 7754, 7759, 7761, 7798, 7808], [7821, 7862, 7882, 7885, 7903, 7911, 7949, 7952, 7973, 8013, 8021, 8065, 8067, 8069, 8077, 8112, 8132, 8159, 8160, 8166, 8248, 8255, 8293, 8297, 8301, 8317, 8349, 8361, 8392, 8402, 8404, 8414, 8485, 8487, 8494, 8521, 8540, 8545, 8601, 8603, 8634, 8684, 8702, 8709, 8726, 8747, 8797, 8822, 8826, 8843, 8889, 8902, 8934, 8958, 8966, 8970, 8977, 9025, 9056, 9098, 9207, 9231, 9249, 9250], [9263, 9275, 9284, 9311, 9322, 9377, 9393, 9426, 9488, 9512, 9577, 9585, 9596, 9616, 9639, 9673, 9715, 9741, 9744, 9761, 9762, 9773, 9809, 9839, 9843, 9855, 9861, 9882, 9905, 9910, 9970, 9973, 10033, 10041, 10051, 10057, 10072, 10090, 10099, 10107, 10175, 10257, 10286, 10290, 10338, 10376, 10395, 10396, 10402, 10436, 10476, 10520, 10521, 10525, 10528, 10546, 10548, 10549, 10573, 10576, 10640, 10651, 10655, 10718], [10732, 10783, 10807, 10830, 10847, 10856, 10861, 10871, 10874, 10903, 10908, 10913, 10957, 10995, 11018, 11025, 11100, 11101, 11117, 11121, 11123, 11131, 11154, 11175, 11194, 11202, 11205, 11234, 11241, 11249, 11268, 11306, 11317, 11335, 11336, 11350, 11361, 11377, 11425, 11467, 11472, 11554, 11595, 11629, 11631, 11652, 11667, 11680, 11681, 11723, 11728, 11732, 11736, 11737, 11788, 11795, 11805, 11816, 11843, 11849, 11850, 11851, 11895, 11927], [11928, 11936, 11957, 11967, 12022, 12030, 12058, 12059, 12061, 12133, 12153, 12188, 12207, 12224, 12259, 12371, 12423, 12451, 12459, 12499, 12556, 12560, 12597, 12609, 12621, 12643, 12651, 12669, 12679, 12735, 12770, 12784, 12787, 12791, 31, 36, 60, 65, 77, 78, 86, 140, 165, 185, 187, 229, 230, 251, 282, 296, 298, 301, 389, 429, 519, 540, 585, 612, 619, 620, 670, 688, 718, 720], [829, 837, 850, 863, 904, 926, 969, 1115, 1117, 1126, 1128, 1137, 1138, 1174, 1192, 1254, 1272, 1276, 1279, 1310, 1322, 1331, 1337, 1381, 1423, 1500, 1528, 1639, 1674, 1693, 1742, 1763, 1780, 1834, 1850, 1859, 1861, 1892, 1929, 1931, 1969, 1985, 2010, 2043, 2045, 2055, 2070, 2159, 2242, 2245, 2262, 2290, 2298, 2306, 2356, 2369, 2411, 2427, 2467, 2468, 2504, 2524, 2546, 2563], [2598, 2622, 2659, 2678, 2695, 2758, 2812, 2813, 2828, 2871, 2886, 2896, 2917, 2932, 2939, 2950, 2961, 2987, 3025, 3026, 3068, 3076, 3077, 3088, 3138, 3196, 3205, 3304, 3308, 3311, 3381, 3415, 3449, 3467, 3477, 3490, 3517, 3527, 3531, 3540, 3573, 3577, 3635, 3639, 3645, 3731, 3740, 3756, 3783, 3784, 3803, 3815, 3820, 3827, 3851, 3935, 4014, 4041, 4042, 4049, 4076, 4086, 4138, 4139], [4141, 4148, 4162, 4177, 4263, 4339, 4368, 4373, 4411, 4429, 4466, 4477, 4480, 4493, 4505, 4547, 4562, 4602, 4620, 4624, 4695, 4728, 4755, 4775, 4836, 4864, 4912, 4926, 4930, 4941, 4969, 4999, 5063, 5103, 5194, 5218, 5251, 5289, 5318, 5328, 5330, 5372, 5402, 5455, 5467, 5519, 5520, 5543, 5552, 5565, 5579, 5585, 5640, 5660, 5662, 5679, 5701, 5710, 5743, 5776, 5802, 5823, 5867, 5970], [5988, 5996, 5997, 6045, 6067, 6073, 6146, 6150, 6158, 6179, 6183, 6303, 6334, 6351, 6410, 6412, 6431, 6433, 6462, 6487, 6516, 6551, 6561, 6591, 6600, 6677, 6710, 6720, 6765, 6808, 6814, 6822, 6840, 6856, 6927, 6938, 6984, 6996, 7033, 7063, 7102, 7106, 7180, 7199, 7213, 7248, 7260, 7270, 7328, 7418, 7465, 7487, 7489, 7497, 7526, 7568, 7571, 7604, 7644, 7647, 7657, 7769, 7794, 7814], [7831, 7893, 7913, 7916, 7922, 7941, 7974, 7979, 7984, 8012, 8066, 8135, 8158, 8162, 8217, 8240, 8245, 8267, 8274, 8275, 8277, 8313, 8351, 8375, 8528, 8534, 8587, 8589, 8612, 8624, 8644, 8649, 8659, 8725, 8738, 8741, 8760, 8788, 8805, 8834, 8885, 8913, 8914, 8920, 8935, 8951, 8957, 9003, 9015, 9062, 9097, 9111, 9127, 9155, 9164, 9180, 9183, 9192, 9203, 9222, 9277, 9278, 9280, 9326], [9381, 9407, 9409, 9458, 9467, 9541, 9582, 9609, 9640, 9727, 9746, 9760, 9778, 9797, 9800, 9803, 9884, 9916, 9969, 9978, 10009, 10025, 10029, 10091, 10145, 10149, 10180, 10207, 10255, 10263, 10272, 10295, 10304, 10319, 10326, 10327, 10339, 10340, 10347, 10360, 10371, 10445, 10571, 10591, 10601, 10604, 10636, 10643, 10649, 10664, 10673, 10686, 10703, 10791, 10822, 10907, 10920, 10961, 10976, 10985, 11033, 11087, 11105, 11125], [11143, 11150, 11153, 11183, 11204, 11252, 11256, 11265, 11299, 11330, 11450, 11452, 11499, 11549, 11565, 11607, 11614, 11656, 11699, 11702, 11751, 11853, 11858, 11870, 11875, 11977, 12004, 12005, 12042, 12126, 12135, 12147, 12177, 12269, 12300, 12306, 12310, 12328, 12338, 12350, 12365, 12385, 12392, 12401, 12412, 12458, 12488, 12502, 12508, 12511, 12513, 12514, 12646, 12691, 12695, 12, 23, 57, 69, 74, 136, 153, 214, 228], [249, 307, 308, 324, 330, 342, 345, 383, 387, 391, 414, 428, 432, 435, 458, 471, 477, 532, 544, 555, 567, 600, 614, 652, 715, 810, 822, 823, 892, 907, 931, 941, 959, 986, 998, 1006, 1022, 1061, 1122, 1136, 1143, 1151, 1155, 1196, 1201, 1205, 1246, 1342, 1469, 1484, 1505, 1531, 1535, 1559, 1586, 1653, 1655, 1697, 1744, 1775, 1812, 1841, 1847, 1863], [1873, 1897, 1911, 1949, 1966, 1967, 2047, 2050, 2063, 2089, 2092, 2138, 2140, 2146, 2149, 2184, 2185, 2191, 2273, 2307, 2325, 2333, 2366, 2370, 2381, 2386, 2426, 2432, 2457, 2542, 2547, 2556, 2564, 2600, 2611, 2620, 2654, 2693, 2783, 2788, 2867, 2966, 2978, 3058, 3107, 3194, 3204, 3227, 3228, 3240, 3319, 3328, 3330, 3358, 3463, 3475, 3478, 3482, 3497, 3613, 3617, 3674, 3799, 3872], [3896, 3924, 3949, 3989, 4015, 4019, 4021, 4037, 4062, 4067, 4095, 4132, 4134, 4152, 4184, 4195, 4203, 4239, 4261, 4290, 4303, 4307, 4332, 4399, 4400, 4419, 4420, 4433, 4447, 4448, 4468, 4501, 4514, 4566, 4589, 4667, 4752, 4795, 4798, 4807, 4816, 4830, 4929, 4938, 4967, 5033, 5043, 5053, 5102, 5151, 5164, 5190, 5203, 5253, 5255, 5309, 5321, 5338, 5470, 5533, 5548, 5571, 5576, 5608], [5646, 5648, 5651, 5653, 5695, 5707, 5737, 5763, 5764, 5815, 5837, 5856, 5882, 5889, 6002, 6012, 6021, 6023, 6029, 6032, 6049, 6060, 6062, 6084, 6111, 6112, 6197, 6218, 6228, 6355, 6383, 6430, 6464, 6493, 6543, 6556, 6604, 6683, 6694, 6707, 6743, 6774, 6820, 6834, 6852, 6900, 6908, 6918, 6921, 6924, 6941, 6954, 7042, 7217, 7295, 7320, 7372, 7392, 7436, 7478, 7509, 7527, 7541, 7596], [7598, 7685, 7722, 7728, 7744, 7745, 7773, 7795, 7802, 7890, 7935, 7965, 7976, 8054, 8117, 8141, 8163, 8288, 8310, 8368, 8400, 8447, 8452, 8482, 8513, 8518, 8524, 8535, 8553, 8627, 8631, 8641, 8680, 8689, 8699, 8713, 8740, 8751, 8768, 8793, 8803, 8809, 8828, 8831, 8838, 8854, 8886, 8899, 8980, 8986, 8992, 9011, 9019, 9080, 9088, 9114, 9118, 9138, 9149, 9205, 9220, 9228, 9290, 9312], [9386, 9404, 9430, 9437, 9452, 9507, 9523, 9633, 9657, 9671, 9691, 9720, 9781, 9845, 9891, 9943, 9972, 9979, 9989, 10086, 10113, 10143, 10197, 10213, 10228, 10289, 10306, 10323, 10377, 10378, 10400, 10416, 10426, 10452, 10472, 10473, 10489, 10498, 10544, 10561, 10608, 10613, 10653, 10665, 10685, 10696, 10744, 10805, 10837, 10889, 10912, 10947, 10968, 10997, 11041, 11089, 11130, 11152, 11159, 11170, 11188, 11211, 11238, 11321], [11329, 11351, 11375, 11386, 11390, 11513, 11531, 11540, 11571, 11592, 11616, 11688, 11762, 11775, 11776, 11792, 11809, 11822, 11929, 11971, 11988, 12031, 12040, 12045, 12052, 12074, 12079, 12098, 12184, 12198, 12235, 12237, 12261, 12274, 12312, 12331, 12355, 12400, 12415, 12584, 12596, 12612, 12652, 12680, 12726, 12771, 12779, 2, 3, 19, 34, 42, 43, 73, 104, 219, 221, 297, 333, 338, 344, 351, 455, 498], [521, 538, 546, 562, 646, 656, 657, 696, 735, 745, 748, 836, 943, 960, 1005, 1108, 1160, 1161, 1328, 1348, 1362, 1440, 1441, 1468, 1593, 1681, 1769, 1795, 1802, 1926, 1947, 1955, 1975, 2052, 2088, 2104, 2106, 2136, 2168, 2171, 2176, 2232, 2257, 2258, 2279, 2330, 2345, 2405, 2415, 2474, 2490, 2514, 2536, 2630, 2755, 2822, 2840, 2849, 2941, 3011, 3065, 3117, 3156, 3179], [3181, 3184, 3233, 3236, 3281, 3301, 3354, 3395, 3400, 3406, 3408, 3411, 3413, 3423, 3437, 3459, 3473, 3485, 3518, 3546, 3551, 3622, 3638, 3652, 3660, 3726, 3738, 3766, 3791, 3810, 3855, 3857, 3913, 3914, 3915, 3917, 3957, 3959, 3981, 3982, 3987, 4020, 4024, 4050, 4058, 4070, 4090, 4155, 4163, 4173, 4197, 4234, 4253, 4267, 4284, 4288, 4408, 4479, 4504, 4509, 4511, 4556, 4574, 4582], [4593, 4597, 4650, 4671, 4697, 4706, 4732, 4794, 4797, 4823, 4849, 4896, 4931, 4933, 5020, 5026, 5051, 5184, 5189, 5193, 5204, 5344, 5362, 5390, 5394, 5437, 5440, 5464, 5514, 5525, 5530, 5559, 5570, 5602, 5671, 5675, 5731, 5772, 5874, 5973, 5995, 5998, 6089, 6110, 6118, 6182, 6189, 6219, 6291, 6304, 6317, 6331, 6376, 6391, 6392, 6396, 6414, 6436, 6499, 6534, 6572, 6607, 6629, 6672], [6688, 6776, 6788, 6795, 6807, 6817, 6823, 6873, 6875, 6925, 6940, 6978, 6993, 7054, 7122, 7142, 7211, 7227, 7251, 7279, 7309, 7310, 7405, 7407, 7425, 7439, 7460, 7462, 7464, 7474, 7484, 7490, 7535, 7562, 7615, 7622, 7624, 7638, 7664, 7669, 7695, 7704, 7706, 7775, 7986, 8031, 8063, 8110, 8156, 8243, 8281, 8286, 8409, 8431, 8446, 8451, 8483, 8497, 8500, 8510, 8544, 8554, 8558, 8562], [8581, 8638, 8650, 8693, 8724, 8730, 8808, 8870, 8942, 9000, 9014, 9075, 9085, 9110, 9157, 9210, 9214, 9224, 9283, 9288, 9352, 9431, 9526, 9591, 9597, 9667, 9708, 9766, 9801, 9808, 9877, 9894, 9902, 9903, 9912, 9917, 9993, 10007, 10008, 10046, 10047, 10130, 10159, 10173, 10216, 10217, 10252, 10305, 10325, 10336, 10351, 10370, 10435, 10439, 10450, 10527, 10623, 10626, 10661, 10715, 10763, 10786, 10886, 10888], [10911, 11019, 11066, 11075, 11114, 11139, 11185, 11220, 11247, 11344, 11405, 11430, 11445, 11456, 11504, 11523, 11569, 11649, 11657, 11673, 11696, 11746, 11749, 11848, 11885, 11932, 11951, 11955, 11987, 12044, 12046, 12047, 12062, 12115, 12122, 12175, 12221, 12240, 12258, 12272, 12290, 12336, 12343, 12344, 12361, 12395, 12396, 12398, 12422, 12429, 12446, 12466, 12493, 12577, 12580, 12635, 12677, 12703, 12717, 12765, 49, 92, 94, 113], [124, 126, 181, 216, 386, 408, 425, 445, 454, 492, 635, 677, 724, 761, 764, 817, 835, 842, 860, 870, 928, 963, 1069, 1112, 1140, 1166, 1248, 1295, 1374, 1385, 1400, 1454, 1529, 1597, 1602, 1617, 1642, 1686, 1720, 1814, 1869, 1871, 1906, 1939, 1944, 1984, 2005, 2030, 2034, 2086, 2099, 2151, 2157, 2160, 2177, 2203, 2235, 2309, 2367, 2425, 2429, 2456, 2460, 2497], [2515, 2537, 2559, 2577, 2604, 2648, 2652, 2698, 2719, 2748, 2759, 2764, 2771, 2778, 2792, 2909, 3010, 3020, 3022, 3034, 3050, 3113, 3324, 3436, 3438, 3534, 3537, 3550, 3579, 3648, 3667, 3679, 3689, 3695, 3739, 3767, 3789, 3832, 3885, 3916, 3940, 3973, 3979, 3990, 3991, 4045, 4188, 4235, 4247, 4273, 4412, 4445, 4545, 4552, 4565, 4672, 4683, 4722, 4781, 4792, 4806, 4883, 4922, 4962], [4991, 5002, 5042, 5048, 5085, 5106, 5114, 5148, 5149, 5208, 5284, 5291, 5339, 5346, 5451, 5486, 5494, 5517, 5560, 5573, 5616, 5754, 5761, 5812, 5825, 5873, 5885, 5937, 5939, 5965, 5976, 5982, 6081, 6098, 6104, 6105, 6128, 6137, 6138, 6265, 6285, 6298, 6302, 6327, 6450, 6452, 6469, 6495, 6540, 6552, 6559, 6566, 6580, 6609, 6613, 6619, 6691, 6760, 6766, 6768, 6780, 6785, 6790, 6796], [6806, 6882, 7007, 7034, 7056, 7081, 7101, 7111, 7175, 7184, 7240, 7299, 7381, 7426, 7456, 7510, 7530, 7553, 7561, 7581, 7600, 7662, 7681, 7705, 7735, 7737, 7844, 7857, 7884, 7943, 8016, 8039, 8068, 8070, 8082, 8123, 8225, 8323, 8363, 8365, 8370, 8384, 8456, 8499, 8519, 8549, 8694, 8731, 8743, 8755, 8856, 8884, 8956, 8989, 9067, 9094, 9116, 9191, 9218, 9253, 9281, 9286, 9299, 9397], [9402, 9434, 9560, 9589, 9649, 9704, 9707, 9724, 9786, 9857, 9858, 9929, 9933, 10062, 10169, 10205, 10233, 10317, 10330, 10337, 10497, 10511, 10543, 10566, 10579, 10580, 10594, 10616, 10619, 10631, 10671, 10691, 10729, 10785, 10827, 10875, 10919, 11053, 11057, 11071, 11076, 11083, 11167, 11278, 11308, 11314, 11383, 11429, 11521, 11552, 11635, 11661, 11705, 11724, 11738, 11780, 11864, 11865, 11922, 11938, 11978, 12008, 12039, 12091], [12100, 12172, 12340, 12359, 12388, 12457, 12473, 12600, 12624, 12630, 12637, 12653, 12686, 12692, 12711, 12712, 44, 91, 96, 236, 271, 288, 317, 360, 365, 371, 405, 452, 518, 563, 673, 678, 703, 757, 778, 779, 801, 889, 902, 1008, 1085, 1148, 1149, 1152, 1186, 1211, 1245, 1258, 1263, 1268, 1285, 1384, 1416, 1461, 1465, 1498, 1556, 1560, 1576, 1599, 1663, 1692, 1707, 1709], [1724, 1740, 1797, 1805, 1808, 1877, 1913, 1917, 1974, 2001, 2003, 2111, 2197, 2217, 2219, 2220, 2221, 2259, 2368, 2390, 2540, 2725, 2737, 2884, 3052, 3084, 3154, 3162, 3192, 3193, 3276, 3318, 3355, 3363, 3453, 3515, 3541, 3606, 3661, 3683, 3716, 3770, 3848, 3871, 3892, 3898, 3936, 4044, 4048, 4097, 4119, 4147, 4189, 4248, 4353, 4374, 4391, 4436, 4528, 4558, 4561, 4634, 4652, 4693], [4730, 4801, 4884, 5009, 5022, 5037, 5187, 5220, 5233, 5277, 5301, 5314, 5351, 5354, 5417, 5421, 5478, 5489, 5497, 5522, 5544, 5674, 5683, 5730, 5733, 5786, 5827, 5858, 5861, 5975, 5981, 6040, 6127, 6190, 6253, 6328, 6346, 6359, 6388, 6509, 6560, 6614, 6633, 6751, 6821, 6826, 6831, 6877, 6889, 6920, 6970, 6988, 7021, 7082, 7095, 7118, 7124, 7137, 7206, 7212, 7361, 7379, 7383, 7397], [7431, 7554, 7578, 7631, 7648, 7650, 7672, 7682, 7726, 7739, 7753, 7771, 7817, 7863, 7909, 7910, 7921, 7963, 8019, 8026, 8085, 8087, 8189, 8350, 8390, 8488, 8525, 8572, 8673, 8677, 8727, 8749, 8818, 8873, 8874, 8894, 8929, 8959, 8978, 9016, 9026, 9029, 9030, 9033, 9050, 9084, 9099, 9102, 9126, 9160, 9188, 9258, 9480, 9517, 9525, 9547, 9607, 9795, 9899, 9915, 9925, 9981, 9990, 10016], [10058, 10138, 10219, 10231, 10399, 10504, 10524, 10526, 10529, 10538, 10586, 10714, 10746, 10759, 10804, 10813, 10850, 10865, 10883, 10941, 10952, 10975, 10981, 11038, 11136, 11246, 11315, 11356, 11370, 11417, 11475, 11484, 11509, 11548, 11557, 11573, 11605, 11610, 11613, 11644, 11651, 11690, 11767, 11787, 11808, 11833, 11842, 11872, 11879, 11897, 11943, 11999, 12033, 12076, 12081, 12108, 12121, 12138, 12139, 12176, 12226, 12265, 12346, 12431], [12463, 12528, 12581, 12583, 12634, 12659, 12707, 12708, 12718, 12721, 12799, 62, 84, 87, 95, 146, 152, 293, 309, 418, 550, 559, 622, 725, 728, 920, 925, 967, 999, 1012, 1029, 1041, 1097, 1213, 1249, 1372, 1434, 1488, 1523, 1537, 1587, 1590, 1736, 1752, 1801, 1845, 1921, 1938, 1999, 2041, 2077, 2091, 2180, 2182, 2229, 2247, 2272, 2295, 2303, 2350, 2383, 2399, 2494, 2583], [2605, 2626, 2631, 2694, 2713, 2791, 2906, 2989, 3067, 3082, 3170, 3191, 3200, 3226, 3237, 3351, 3390, 3441, 3465, 3510, 3556, 3566, 3621, 3642, 3682, 3751, 3819, 3856, 3859, 3928, 4101, 4213, 4255, 4306, 4318, 4325, 4348, 4365, 4376, 4385, 4392, 4395, 4499, 4610, 4862, 4889, 4898, 4903, 4925, 4994, 5201, 5229, 5336, 5353, 5360, 5414, 5428, 5485, 5498, 5599, 5613, 5774, 5775, 5900], [5901, 5943, 5956, 6076, 6091, 6209, 6292, 6341, 6453, 6473, 6490, 6557, 6579, 6582, 6663, 6668, 6679, 6705, 6726, 6828, 6857, 6860, 6903, 6904, 6939, 7039, 7057, 7084, 7088, 7096, 7121, 7162, 7205, 7312, 7376, 7385, 7420, 7543, 7584, 7589, 7591, 7603, 7634, 7651, 7699, 7784, 7816, 7825, 7838, 7846, 7871, 7880, 7946, 7969, 8002, 8022, 8023, 8036, 8052, 8133, 8257, 8356, 8387, 8448], [8477, 8481, 8511, 8582, 8671, 8804, 8824, 8836, 8848, 8851, 8878, 8953, 9008, 9141, 9264, 9285, 9347, 9348, 9442, 9466, 9500, 9557, 9569, 9590, 9626, 9677, 9714, 9796, 9799, 9838, 9860, 9953, 9959, 9961, 10027, 10061, 10076, 10105, 10108, 10186, 10188, 10234, 10324, 10359, 10392, 10403, 10478, 10588, 10589, 10599, 10603, 10607, 10609, 10659, 10663, 10692, 10781, 10801, 10864, 10880, 10909, 10966, 10971, 10974], [11006, 11009, 11021, 11032, 11222, 11242, 11258, 11347, 11368, 11388, 11389, 11500, 11537, 11558, 11643, 11671, 11814, 11981, 11991, 12145, 12168, 12213, 12294, 12295, 12450, 12471, 12603, 12628, 12661, 1, 46, 63, 112, 134, 178, 184, 195, 218, 270, 286, 792, 807, 839, 855, 857, 896, 899, 957, 976, 1051, 1060, 1107, 1267, 1347, 1376, 1382, 1386, 1439, 1478, 1486, 1501, 1507, 1546, 1581], [1640, 1650, 1755, 1820, 1826, 1827, 1889, 1896, 1954, 2036, 2187, 2233, 2431, 2466, 2568, 2599, 2629, 2645, 2650, 2766, 2799, 2841, 2927, 2928, 2930, 2938, 3112, 3185, 3256, 3322, 3341, 3460, 3508, 3554, 3578, 3586, 3589, 3620, 3780, 3794, 3918, 3962, 4031, 4035, 4047, 4124, 4181, 4413, 4435, 4476, 4521, 4642, 4670, 4674, 4704, 4721, 4742, 4753, 5109, 5226, 5304, 5361, 5392, 5452], [5510, 5609, 5625, 5658, 5664, 5693, 5810, 5818, 5909, 5993, 6013, 6082, 6085, 6167, 6264, 6370, 6380, 6507, 6564, 6639, 6649, 6696, 6730, 6747, 6841, 6843, 6879, 6910, 6936, 6964, 7026, 7406, 7427, 7481, 7512, 7555, 7590, 7636, 7667, 7683, 7694, 7740, 7742, 7750, 7847, 7876, 7906, 8094, 8149, 8151, 8211, 8261, 8309, 8413, 8416, 8429, 8453, 8458, 8578, 8619, 8626, 8711, 8766, 8785], [8853, 8864, 9058, 9090, 9104, 9124, 9129, 9169, 9186, 9305, 9321, 9328, 9506, 9524, 9602, 9617, 9656, 9658, 9663, 9739, 9758, 9844, 10001, 10060, 10153, 10154, 10238, 10265, 10283, 10313, 10401, 10506, 10559, 10677, 10706, 10784, 10787, 10794, 10810, 10901, 10906, 10940, 10954, 11146, 11177, 11214, 11218, 11240, 11244, 11380, 11427, 11434, 11457, 11590, 11628, 11704, 11709, 11889, 11913, 12013, 12053, 12068, 12071, 12167], [12173, 12217, 12357, 12393, 12419, 12484, 12558, 76, 143, 168, 186, 201, 233, 235, 278, 361, 395, 553, 583, 628, 669, 683, 738, 773, 832, 979, 980, 989, 1010, 1034, 1036, 1076, 1078, 1130, 1218, 1296, 1303, 1326, 1368, 1415, 1424, 1453, 1483, 1511, 1543, 1614, 1773, 1774, 1831, 1842, 1887, 1922, 1950, 2035, 2090, 2132, 2141, 2142, 2358, 2435, 2445, 2573, 2647, 2718], [2726, 2751, 2752, 2784, 2821, 2852, 2931, 3061, 3106, 3229, 3352, 3401, 3619, 3688, 3729, 3742, 3849, 3867, 3876, 3886, 3888, 3997, 4007, 4061, 4063, 4080, 4112, 4144, 4159, 4179, 4393, 4542, 4546, 4571, 4631, 4632, 4657, 4814, 4842, 4850, 4871, 4961, 5087, 5130, 5293, 5500, 5505, 5587, 5634, 5641, 5745, 5811, 5839, 5907, 5934, 5960, 6043, 6093, 6114, 6166, 6169, 6192, 6220, 6247], [6323, 6361, 6420, 6505, 6555, 6884, 7071, 7100, 7139, 7275, 7346, 7386, 7446, 7471, 7521, 7594, 7637, 7807, 7824, 7833, 7855, 7870, 7968, 7987, 8025, 8103, 8234, 8291, 8382, 8415, 8421, 8423, 8506, 8557, 8576, 8633, 8640, 8663, 8714, 8794, 8845, 8900, 8979, 9022, 9070, 9079, 9101, 9133, 9237, 9246, 9304, 9338, 9364, 9453, 9496, 9612, 9615, 9651, 9774, 9815, 9817, 9818, 9863, 9890], [9927, 9942, 10050, 10052, 10063, 10097, 10120, 10139, 10214, 10223, 10296, 10382, 10407, 10425, 10444, 10461, 10474, 10505, 10552, 10724, 10770, 10773, 10778, 10867, 10892, 11122, 11173, 11231, 11301, 11384, 11437, 11580, 11617, 11778, 11803, 11933, 12027, 12182, 12362, 12420, 12441, 12449, 12464, 12479, 12546, 12578, 12636, 12740, 12764, 67, 70, 177, 522, 529, 780, 856, 929, 953, 1127, 1247, 1259, 1426, 1436, 1479], [1522, 1557, 1585, 1625, 1627, 1743, 1761, 1979, 1996, 2000, 2083, 2094, 2108, 2270, 2285, 2318, 2320, 2396, 2416, 2487, 2498, 2548, 2585, 2763, 2870, 2990, 3029, 3036, 3108, 3132, 3155, 3197, 3678, 3728, 3744, 3757, 3781, 3795, 3817, 3834, 3911, 3941, 4056, 4069, 4196, 4249, 4278, 4283, 4386, 4467, 4490, 4751, 4950, 5347, 5458, 5493, 5575, 5756, 5759, 5789, 5853, 5933, 6022, 6038], [6087, 6174, 6200, 6390, 6407, 6415, 6583, 6648, 6658, 6662, 6770, 7087, 7150, 7234, 7285, 7367, 7368, 7435, 7450, 7472, 7701, 7774, 7788, 8055, 8081, 8181, 8311, 8347, 8354, 8419, 8424, 8551, 8815, 8859, 8905, 8948, 9013, 9043, 9071, 9109, 9244, 9255, 9298, 9455, 9468, 9540, 9556, 9574, 9581, 9608, 9628, 9689, 9807, 9840, 9908, 10044, 10157, 10163, 10172, 10271, 10386, 10447, 10567, 10632], [10654, 10658, 10753, 10836, 10848, 10855, 10953, 11014, 11088, 11148, 11181, 11251, 11285, 11391, 11542, 11771, 11794, 11797, 11859, 11868, 11996, 12003, 12016, 12051, 12085, 12162, 12200, 12247, 12332, 12516, 12550, 12589, 12639, 12683, 12776, 64, 225, 410, 533, 549, 582, 661, 706, 736, 751, 798, 880, 917, 921, 984, 1063, 1064, 1072, 1074, 1110, 1293, 1425, 1552, 1637, 1840, 1908, 1909, 1992, 1994], [2020, 2032, 2074, 2291, 2331, 2340, 2379, 2393, 2562, 2596, 2839, 3012, 3073, 3136, 3412, 3584, 3904, 3921, 3956, 3958, 3970, 4081, 4105, 4245, 4294, 4321, 4422, 4452, 4458, 4491, 4522, 4608, 4653, 4684, 4702, 4710, 4873, 4879, 5007, 5171, 5285, 5324, 5462, 5531, 5684, 5760, 5908, 5917, 5924, 5954, 6006, 6010, 6132, 6144, 6336, 6378, 6413, 6481, 6628, 6661, 6758, 6931, 6994, 7005], [7008, 7051, 7064, 7172, 7177, 7362, 7373, 7547, 7608, 7645, 7654, 7680, 7764, 7796, 7811, 7934, 8000, 8061, 8118, 8131, 8220, 8236, 8341, 8385, 8495, 8703, 8710, 8782, 9293, 9323, 9361, 9449, 9460, 9728, 9729, 9738, 9892, 9896, 10082, 10332, 10598, 10736, 10779, 10815, 10842, 10917, 11157, 11225, 11486, 11666, 11735, 11784, 11902, 11925, 11962, 12209, 12212, 12243, 12287, 12406, 12506, 12611, 12632, 12640], [12760, 202, 205, 212, 290, 569, 586, 796, 961, 1002, 1437, 1494, 1562, 1579, 1582, 1664, 1725, 1728, 1747, 1793, 1823, 1860, 1910, 2042, 2054, 2082, 2107, 2144, 2200, 2237, 2299, 2342, 2365, 2449, 2459, 2475, 2651, 2689, 2697, 2743, 2907, 2920, 2991, 3074, 3075, 3182, 3312, 3391, 3429, 3625, 3734, 3813, 3824, 3863, 3901, 3923, 3984, 4161, 4446, 4455, 4502, 4573, 4609, 4786], [4963, 4990, 5008, 5010, 5110, 5217, 5237, 5260, 5287, 5305, 5329, 5491, 5617, 5620, 5721, 5739, 5888, 5927, 5991, 6435, 6441, 6461, 6532, 6537, 6539, 6608, 6810, 6862, 7378, 7469, 7508, 7689, 7834, 7938, 7962, 8126, 8134, 8155, 8200, 8215, 8228, 8316, 8329, 8411, 8412, 8465, 8752, 8817, 8871, 8896, 8908, 8975, 9054, 9081, 9406, 9443, 9554, 9655, 9674, 9784, 9798, 9854, 10004, 10375], [10533, 10596, 10605, 10740, 11011, 11039, 11155, 11190, 11226, 11394, 11403, 11502, 11535, 11568, 11663, 11720, 11828, 12001, 12087, 12222, 12245, 12654, 12657, 12748, 154, 173, 179, 312, 406, 470, 500, 523, 590, 617, 623, 770, 867, 891, 975, 1018, 1262, 1323, 1404, 1510, 1578, 1608, 1796, 2170, 2516, 2543, 2554, 2572, 2590, 2608, 2624, 2728, 2972, 3006, 3346, 3433, 3476, 3506, 3575, 3720], [3826, 3843, 4005, 4390, 4398, 4434, 4437, 4705, 4811, 4839, 5018, 5092, 5144, 5188, 5561, 5593, 5614, 5629, 5676, 5783, 5864, 5879, 5940, 6212, 6239, 6398, 6451, 6492, 6646, 6689, 6737, 6746, 6756, 6819, 6842, 6916, 7105, 7126, 7127, 7136, 7159, 7219, 7223, 7290, 7339, 7507, 7595, 7613, 7629, 7671, 7678, 7944, 8104, 8106, 8276, 8536, 8563, 8585, 8648, 8672, 8881, 8945, 9034, 9042], [9318, 9354, 9513, 9530, 9563, 9579, 9587, 9625, 9687, 9954, 9956, 10056, 10111, 10128, 10262, 10443, 10539, 10775, 10798, 10852, 10866, 11259, 11310, 11362, 11402, 11516, 11602, 11677, 11785, 11854, 12064, 12118, 12191, 12246, 12321, 12369, 12428, 12486, 12490, 12572, 12574, 12650, 12796, 25, 133, 174, 204, 237, 263, 276, 283, 456, 542, 557, 794, 847, 1009, 1033, 1311, 1312, 1361, 1401, 1517, 1772], [1987, 2125, 2153, 2207, 2421, 2447, 2502, 2528, 2574, 2672, 2690, 2848, 2868, 2947, 3157, 3507, 3655, 3668, 3699, 3845, 3955, 4268, 4330, 4381, 4633, 4861, 4978, 5023, 5216, 5242, 5450, 5594, 5665, 5723, 5842, 5850, 6024, 6054, 6198, 6322, 6419, 6437, 6594, 6859, 6955, 7018, 7120, 7125, 7343, 7448, 7718, 7900, 7997, 8083, 8139, 8146, 8378, 8628, 8692, 8814, 9132, 9446, 9769, 9964], [10240, 10276, 10393, 10412, 10523, 10876, 11044, 11297, 11303, 11373, 11378, 11444, 11538, 11904, 11921, 11980, 12112, 12156, 12339, 12444, 12498, 12619, 38, 190, 483, 866, 1339, 1366, 1496, 1497, 1575, 1628, 1687, 1904, 1973, 1976, 1990, 2120, 2354, 2413, 2552, 2789, 2824, 3002, 3094, 3250, 3539, 3777, 3833, 3860, 3905, 4272, 4336, 4495, 4668, 4756, 4858, 5052, 5173, 5238, 5271, 5334, 5396, 5432], [5527, 5691, 6035, 6099, 6193, 6240, 6329, 6354, 6541, 6836, 6948, 6987, 7141, 7176, 7208, 7298, 7393, 7411, 7440, 7626, 7867, 8037, 8111, 8119, 8374, 8454, 8561, 9171, 9173, 9179, 9230, 9279, 9358, 9683, 10381, 10419, 10494, 10540, 10584, 10873, 10959, 11030, 11099, 11357, 11829, 11896, 12055, 12160, 12724, 147, 215, 238, 239, 287, 461, 496, 515, 912, 1026, 1180, 1278, 1408, 1714, 2009], [2051, 2113, 2236, 2322, 2395, 2527, 2549, 2609, 2636, 3070, 3365, 4082, 4264, 4603, 4821, 4853, 4939, 4964, 5753, 5833, 5849, 6140, 6188, 6245, 6268, 7130, 7156, 7245, 7711, 7743, 7810, 8531, 8635, 8750, 8849, 8915, 9004, 9023, 9175, 9412, 9495, 9694, 9856, 10224, 10373, 10394, 10508, 10621, 10851, 10927, 10939, 11001, 11424, 11440, 11622, 11715, 11800, 12038, 12107, 12141, 12142, 12164, 12223, 12266], [58, 156, 359, 369, 373, 509, 556, 710, 831, 916, 934, 988, 1025, 1354, 1809, 1837, 1866, 2275, 2394, 2437, 2746, 2981, 3288, 3291, 3800, 3944, 3968, 4180, 4217, 4258, 4299, 4658, 5083, 5488, 5655, 6204, 6459, 6844, 6966, 7550, 7566, 8124, 8157, 8629, 8712, 9176, 9344, 9372, 9396, 9567, 9793, 9986, 9987, 9994, 10038, 10043, 10152, 10311, 10951, 11059, 11168, 11228, 11342, 11574], [11648, 12054, 12370, 12424, 12433, 12440, 12684, 12774, 5, 107, 192, 402, 489, 682, 820, 827, 1573, 1621, 1764, 1816, 1907, 1970, 1978, 2121, 2328, 2586, 2627, 2722, 2745, 2749, 2809, 2861, 3125, 3218, 3371, 3530, 3547, 3754, 3938, 4128, 4172, 4410, 4510, 4868, 4940, 5061, 5065, 5377, 5424, 5792, 5820, 5915, 6866, 7140, 7269, 7289, 7349, 7961, 8357, 8639, 8707, 9317, 9456, 9598], [9669, 9866, 10155, 10652, 10747, 10844, 11074, 11197, 11436, 11514, 12178, 12367, 12475, 12766, 253, 273, 466, 602, 632, 637, 771, 819, 1028, 1421, 1450, 2335, 2485, 2688, 2816, 2829, 3000, 3458, 3614, 3868, 4242, 4482, 4788, 5073, 5382, 5459, 5495, 6126, 6156, 6256, 6506, 6601, 7763, 7924, 8164, 8168, 8169, 8282, 8473, 8479, 8505, 9044, 9661, 9676, 9958, 10042, 10071, 10237, 10761, 10793], [10868, 11081, 11216, 11353, 11369, 11588, 11813, 11893, 12279, 12405, 12531, 135, 364, 385, 423, 577, 654, 721, 731, 737, 739, 950, 1162, 1284, 1550, 1583, 1594, 1832, 2346, 2544, 2736, 2834, 2903, 3259, 3315, 3452, 3713, 4106, 4191, 4200, 4557, 4815, 4892, 5457, 5851, 5969, 6123, 6171, 6178, 6202, 6307, 6320, 7078, 7272, 7363, 7451, 7532, 7625, 7649, 7772, 8326, 8432, 8439, 8679], [8872, 9028, 9261, 9641, 9693, 10259, 10682, 10926, 10965, 11506, 11536, 12043, 12455, 12534, 606, 691, 746, 1088, 1334, 1451, 2059, 2183, 2704, 2887, 2926, 2994, 3430, 4205, 4779, 4983, 5197, 5303, 6033, 6975, 7048, 7115, 7170, 7276, 7755, 7813, 7932, 8235, 8512, 8739, 8762, 8769, 8770, 8961, 8984, 9041, 9706, 9713, 9931, 10036, 10140, 10195, 11005, 11015, 11091, 11261, 11698, 11786, 12225, 12527], [871, 1037, 1243, 1770, 2397, 2532, 2545, 3016, 3152, 3504, 3877, 4464, 4540, 4612, 4800, 5228, 5460, 5699, 5862, 6090, 6122, 6172, 6738, 7107, 7179, 7231, 7498, 7633, 7783, 8090, 8130, 8210, 9384, 9501, 9675, 9826, 10028, 10318, 10357, 10517, 11355, 11986, 11997, 12066, 12570, 12757, 12794, 55, 517, 1057, 1208, 1656, 1844, 2582, 2587, 3268, 3605, 3649, 3730, 4192, 4906, 4910, 6070, 6186], [6589, 6632, 6968, 7119, 7265, 7353, 7836, 8303, 8610, 9423, 10098, 10335, 10408, 10558, 10832, 10937, 11020, 11065, 11077, 11338, 11718, 11819, 11935, 12196, 12435, 12489, 348, 665, 797, 968, 1435, 1491, 1541, 2085, 3895, 5162, 6252, 6460, 6958, 6991, 7321, 7408, 7653, 8205, 8883, 9057, 9454, 9621, 9946, 10410, 11770, 12181, 12759, 844, 858, 885, 2312, 2819, 2860, 3628, 3709, 4257, 4352, 4748], [5016, 5160, 5568, 5588, 6522, 6894, 7332, 7441, 7899, 8862, 9320, 9415, 10005, 10307, 11227, 12025, 12120, 12238, 12537, 121, 209, 473, 873, 1198, 1829, 2634, 2830, 3900, 5425, 5476, 5631, 8285, 8389, 8471, 8567, 8658, 9200, 9256, 9700, 10277, 10765, 11359, 11626, 11907, 12194, 12472, 12521, 122, 802, 1260, 1392, 1624, 1646, 2385, 3447, 4040, 4059, 4084, 4361, 4726, 4747, 4783, 5153, 5408], [5490, 5893, 6650, 6676, 7032, 7274, 7395, 7896, 7950, 8272, 8718, 9159, 9400, 9752, 10891, 11239, 12324, 12645, 284, 464, 580, 1252, 1766, 2412, 3116, 3931, 4644, 5119, 5537, 7931, 8178, 8687, 8850, 8893, 9121, 9481, 9944, 10513, 10693, 11104, 11341, 12782, 240, 599, 741, 1209, 1373, 1882, 1919, 1956, 2964, 3425, 7188, 7413, 8732, 8835, 9353, 9918, 9939, 10230, 10800, 12608, 607, 1487], [1711, 2409, 2712, 3098, 3243, 3353, 4388, 4580, 4707, 5407, 5605, 5735, 8918, 9040, 10668, 10862, 12023, 12467, 12617, 12728, 12753, 2486, 3866, 4223, 4475, 4942, 5156, 6066, 6867, 7167, 7569, 7878, 8006, 8075, 8122, 8254, 8296, 8700, 9308, 9490, 9967, 10448, 10570, 10900, 11575, 12709, 1536, 1613, 2918, 2967, 3003, 4218, 4720, 4875, 5553, 6387, 8120, 8594, 9202, 11319, 12505, 12736, 139, 4078], [5456, 6670, 7093, 7313, 7377, 7690, 7914, 8335, 8360, 9306, 10341, 12798, 910, 1266, 2442, 5111, 5213, 5239, 5481, 7147, 8040, 8704, 8943, 12166, 51, 845, 2518, 2686, 3805, 4109, 4822, 5098, 8569, 8606, 8840, 9072, 11363, 12520, 3909, 5545, 8011, 8450, 8911, 10821, 90, 1443, 1980, 1997, 7326, 8939, 12283, 978, 1329, 2164, 2225, 2256, 2558, 4360, 4970, 5709, 5788, 6789, 8035, 8949], [12587, 992, 1638, 2601, 3062, 3632, 4439, 5290, 6136, 7244, 7311, 9001, 9936, 12174, 3201, 5572, 6229, 6733, 9834, 4221, 5580, 5795, 8844, 8964, 9952, 11639, 12358, 2995, 3597, 6695, 9351, 11281, 1525, 4656, 7945, 9046, 11941, 12161, 547, 4219, 4259, 9422, 10106, 1054, 2087, 2656, 2970, 3071, 11302, 11328, 12154, 2017, 2377, 6895, 6944, 12032, 2181, 2820, 8467, 9852, 1397, 3338, 9532, 1256], [7658, 7756, 11372, 1506, 7713, 8981, 2401, 8799, 9638, 12326, 160, 5447, 6901, 11577, 5313, 8337, 8605, 834, 5105, 5647, 7632, 3583, 10660, 4377, 6839, 10949, 12130, 381, 1353, 7861, 8227, 8574, 11543, 5804, 7792, 1294, 5869, 3702, 4292, 4824, 4270, 8212, 1444, 5196, 6005, 10924, 10127, 1817, 398, 11354, 1184, 4772, 8742, 6479, 2805, 3786, 11946, 8298, 1923, 12301, 12249, 7466, 7403, 6935]]
```

In [ ]:
from torch.utils.data import DataLoader, RandomSampler
def build_dataloader(config, data, tokenizer):
  train_dataset = AFQMCDataset(data['train'])
  dev_dataset = AFQMCDataset(data['dev'])
  test_dataset = AFQMCDataset(data['test'])

  # 使用桶采样
  if config['use_bucket']:
    # 基采样器RandomSampler
    train_sampler = RandomSampler(train_dataset)
    # drop_last 最后一个batch小于size 丢弃
    bucket_sampler = BucketBatchSampler(train_sampler, batch_size = config['batch_size'],
                      drop_last = False, sort_key = lambda x: len(train_dataset[x][0]),# 以 input_id 长度作为排序的指标
                      bucket_size_multiplier = config['bucket_multiplier'])
    train_dataloader = DataLoader(dataset = train_dataset, batch_sampler = bucket_sampler, num_workers = 4, collate_fn = collate_fn)
  else:
    train_dataloader = DataLoader(dataset = train_dataset, batch_size = config['batch_size'], shuffle = True, num_workers = 4, collate_fn = collate_fn)


  dev_dataloader = DataLoader(dataset = dev_dataset, batch_size = config['batch_size'], shuffle = False, num_workers = 4, collate_fn = collate_fn)
  test_dataloader = DataLoader(dataset = test_dataset, batch_size = config['batch_size'], shuffle = False, num_workers = 4, collate_fn = collate_fn)

  return train_dataloader, dev_dataloader, test_dataloader

In [ ]:
train_dataloader, dev_dataloader, test_dataloader = build_dataloader(config, data, tokenizer)

In [ ]:
for i in train_dataloader:
    print('一个batch：',i)
    break

一个batch： {'input_ids': tensor([[ 101, 5709, 1446,  ..., 1408,  102,    0],
        [ 101, 5709, 1446,  ..., 4500,  102,    0],
        [ 101,  711,  784,  ...,  749,  102,    0],
        ...,
        [ 101, 2769, 6820,  ..., 1384,  677,  102],
        [ 101,  955, 1446,  ...,  955, 3621,  102],
        [ 101,  955, 1446,  ..., 1510, 1408,  102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 1, 1, 0],
        [0, 0, 0,  ..., 1, 1, 0],
        [0, 0, 0,  ..., 1, 1, 0],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
      

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
def evaluation(config, model, val_dataloader):
  model.eval()
  preds = []
  labels = []
  val_loss = 0.
  val_iterator = tqdm(val_dataloader, desc = 'Evaluation', total = len(val_dataloader))

  with torch.no_grad():
    for batch in val_iterator:
      labels.append(batch['labels'])
      batch_cuda = {item: value.to(config['device']) for item, value in list(batch.items())}
      loss, logits = model(**batch_cuda)[:2]

      if config['n_gpus'] > 1:
        loss = loss.mean()

      val_loss += loss.item()
      preds.append(logits.argmax(dim = -1).detach().cpu())

  avg_val_loss = val_loss / len(val_dataloader)
  labels = torch.cat(labels, dim = 0).numpy()
  preds = torch.cat(preds, dim = 0).numpy()
  f1 = f1_score(labels, preds)
  acc = accuracy_score(labels, preds)
  return avg_val_loss, f1, acc

# 自集成 和 自蒸馏
![自监督](https://img-blog.csdnimg.cn/e0bff606718a480b84b26b37c02c1651.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

![自集成](https://img-blog.csdnimg.cn/e7013e8b74424092be6658ff9ff866bc.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

![self1](https://img-blog.csdnimg.cn/9857a9457bba49a2953f06206a09b2cf.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

self-ensemble 为了进一步降低集成模型的复杂度，从而使用了一种更高效的集成方法，该方法将多个基本模型 与 参数平均相结合，而不是保留多个基本模型。\
self-distillation 使用知识蒸馏来提高微调效率。当前的BERT模型（学生模型），自集成模型（教师模型）。教师模型是 之前几个时间步长 的学生模型的平均值。


![在这里插入图片描述](https://img-blog.csdnimg.cn/9da9862031d64d628b10f1d6b5644729.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)
![在这里插入图片描述](https://img-blog.csdnimg.cn/1ae9a6f0c02e43789351952ef79d8f2c.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

![在这里插入图片描述](https://img-blog.csdnimg.cn/2484928631e64646a4d17bc3452bc315.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

自集成模型，在单个训练阶段组合不同时间步长的中间模型。将每个时间步的BERT视为基础模型，并将它们组合成一个自集成模型。

*自集成BERT：只有一个BERT模型，单位时间步的BERT视为基础模型，再取平均*

![在这里插入图片描述](https://img-blog.csdnimg.cn/16055adb9e0f4668ac25025956364ef7.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

![在这里插入图片描述](https://img-blog.csdnimg.cn/3027d580da3d443ca50a38f674077bc2.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)
![在这里插入图片描述](https://img-blog.csdnimg.cn/5e7a8e4517684d23a9175a61f9a83394.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

Self-Distillation-Average(SDA)自蒸馏平均：\
计算两个loss CE 和 MSE\
MSE（当前时间步的参数，前几个时间步的参数求平均（不包括当前时间步））

# EMA:指数移动平均(moving average)
### \theta_{t} ：取平均之后的参数  \tilde\theta_{t}：当前时间步的参数


EMA公式
$$\theta_{t}   = 0.99\theta_{t-1} + (1-0.99)\tilde\theta_{t}$$
$$\theta_{t-1} = 0.99\theta_{t-2} + (1-0.99)\tilde\theta_{t-1}$$
$$\theta_{t-2} = 0.99\theta_{t-3} + (1-0.99)\tilde\theta_{t-2}$$
$$\theta_{t-3} = 0.99*0 + (1-0.99)\tilde\theta_{t-3} 【最初状态】$$
**公式推导**
$$\theta_{t}=0.99\theta_{t-1} + (1-0.99)\tilde\theta_{t}$$
$$\theta_{t}=0.99\theta_{t-1} + 0.01 * \tilde\theta_{t}$$
$$\theta_{t}=0.01*\tilde\theta_{t}+0.99\theta_{t-1}$$
$$\theta_{t}=0.01*\tilde\theta_{t}+0.99(0.99\theta_{t-2} + 0.01*\tilde\theta_{t-1})$$
$$\theta_{t}=0.01*\tilde\theta_{t}+0.01*0.99*\tilde\theta_{t-1}+0.99*0.99*\theta_{t-2}$$
$$\theta_{t}=0.01*\tilde\theta_{t}+0.01*0.99*\tilde\theta_{t-1}+0.99*0.99*(0.99*\theta_{t-3} + 0.01*\tilde\theta_{t-2})$$
$$\theta_{t}=0.01*\tilde\theta_{t}+0.01*0.99*\tilde\theta_{t-1}+0.01*0.99*0.99*\tilde\theta_{t-2}+0.99*0.99*0.99*\theta_{t-3}$$
$$\theta_{t}=0.01*\tilde\theta_{t}+0.01*0.99*\tilde\theta_{t-1}+0.01*0.99*0.99*\tilde\theta_{t-2}+0.01*0.99*0.99*0.99*\tilde\theta_{t-3})$$
$$\theta_{t}=0.01*0.99^{0}*\tilde\theta_{t}+0.01*0.99^{1}*\tilde\theta_{t-1}+0.01*0.99^{2}*\tilde\theta_{t-2}+0.01*0.99^{3}*\tilde\theta_{t-3})$$
**总结**
$$\theta_{t} = decay\theta_{t-1} + (1-decay)\tilde\theta_{t}$$



![在这里插入图片描述](https://img-blog.csdnimg.cn/a3c4790799e34d81940a380c102c2a24.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

**这里使用SDA**

In [ ]:
class EMA:
  def __init__(self, model, decay):
    self.model = model
    self.decay = decay
    self.shadow = {}
    self.backup = {}
    self.register()

  # EMA初始化
  def register(self):
    for name, param in self.model.named_parameters():
      if param.requires_grad():
        # 把当前模型参数保存到副本中
        self.shadow[name] = param.data.clone()

  # 每一个时间步(batch)都要更新
  def update(self):
    for name, param in self.model.named_parameters():
      if param.requires_grad:
        # 判断name是否在self.shadow
        assert name in self.shadow
        # 更新的参数
        new_average = (1.0 - self.decay) * param.data + self.decay * self.shadow[name]
        # 保存平均参数
        self.shadow[name] = new_average.clone()

  # 评估的时候用
  def apply_shadow(self):
    for name, param in self.model.named_parameters():
      if param.requires_grad:
        assert name in self.shadow
        # 把当前参数备份
        self.backup[name] = param.data
        # 用维护的平均参数 替换当前模型的参数 进行模型评估
        param.data = self.shadow[name]
        
  # 恢复到训练时的参数(因为Average不进行反向传播，当模型再次训练的时候要把之前的BERT参数拿回来)
  def restore(self):
    for name, param in self.model.name_parameters():
      if param.requires_grad:
        assert name in self.backup
        param.data = self.backup[name]

    self.backup = {}

**总结的公式：**
$$ \theta_{t} = decay\theta_{t-1} + (1-decay)\tilde\theta_{t}$$
$$ \theta_{t} = decay\theta_{t-1} + \tilde\theta_{t} - decay\tilde\theta_{t}$$
$$ \theta_{t} = decay(\theta_{t-1} - \tilde\theta_{t}) + \tilde\theta_{t} $$

**train函数中**

```
one_minus_decay = 1.0 - decay
for s_param, param in zip(kd_model.parameters(), parameters):
     # 指数移动平均
     s_param.sub_(one_minus_decay * (s_param - param))
```
s_param：平均参数
param：当前参数
$$\theta_{t} = \theta_{t-1} - (1 - decay) * (\theta_{t-1} - \tilde\theta_{t})$$
$$\theta_{t} = \theta_{t-1} - (\theta_{t-1} - \tilde\theta_{t}) + decay(\theta_{t-1} - \tilde\theta_{t})$$
$$\theta_{t} = \tilde\theta_{t} + decay(\theta_{t-1} - \tilde\theta_{t})$$

与总结的公式相同

```             #\theta_{t-1} param \theta_{t-1}
                for s_param, param in zip(kd_model.parameters(), parameters):
                    s_param.sub_(one_minus_decay * (s_param - param))
```
$$ \theta_{t} = \theta_{t-1} - (1-decay)( \theta_{t-1} - \tilde\theta_{t})$$
$$ \theta_{t} = \theta_{t-1} - ( \theta_{t-1} - \tilde\theta_{t}) + decay( \theta_{t-1} - \tilde\theta_{t})$$
$$ \theta_{t} = \tilde\theta_{t} + decay( \theta_{t-1} - \tilde\theta_{t})$$

In [ ]:
from transformers import AdamW, BertForSequenceClassification
from torch.cuda import amp
from extra_optim import *
from extra_pgd import *
from extra_fgm import *
from extra_loss import *
from tqdm import trange
import copy, os

def train(config, train_dataloader, dev_dataloader):
  model = BertForSequenceClassification.from_pretrained(config['model_path'])

  param_optimizer = list(model.named_parameters())

  scaler = amp.GradScaler(enabled = config['use_amp'])

  no_decay = ['bias', 'LayerNorm.weight']

  optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                    "weight_decay": config['weight_decay']},
                    {"params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                    "weight_decay": 0.0}
                    ]
  optimizer = AdamW(optimizer_grouped_parameters, lr = config['learning_rate'], correct_bias = False, eps = 1e-8)
  optimizer = Lookahead(optimizer, 5, 1)                  

  model.to(config['device'])
  total_steps = len(train_dataloader) * config['num_epochs']
  
  lr_scheduler = WarmupLinearSchedule(optimizer, warmup_steps = int(config['warmup_ratio'] * total_steps), t_total = total_steps)
  epoch_iterator = trange(config['num_epochs'])
  global_steps = 0
  train_loss = 0.
  logging_loss = 0.
  best_acc = 0.
  SDA_model_path = ''

  if config['n_gpus'] > 1:
    model =nn.DataParallel(model)

  # -----------    new----------------#
  # 定义 MSE loss. (x-y)**2
  kd_loss_fct = nn.MSELoss()
  # 复制BERT模型，得到Average(参考SDA右边的图)
  kd_model = copy.deepcopy(model)
  # Average不需要反向传播
  kd_model.eval()

  train_iterator = iter(train_dataloader)
  for i in epoch_iterator:
    train_iterator = tqdm(train_dataloader, desc = 'Training', total = len(train_dataloader))
    model.train()
    
    for batch in train_iterator:
      batch_cuda = {item: value.to(config['device']) for item, value in list(batch.items())}
      with amp.autocast(enabled = config['use_amp']):
        loss, logits = model(**batch_cuda)[:2]
        if config['n_gpus'] > 1:
          loss = loss.mean()

        # --------------------     new -----------------#
        with torch.no_grad():
          # SDA 拿到标签
          kd_logits = kd_model(**batch_cuda)[1]
        # SDA 使用 MSE计算损失
        kd_loss = kd_loss_fct(logits, kd_logits)
        # SDA 加权损失
        loss += config['kd_coeff'] * kd_loss
        # --------------------     new -----------------#
      scaler.scale(loss).backward()

      scaler.step(optimizer)
      scaler.update()
      lr_scheduler.step()
      optimizer.zero_grad()
      if config['ema_start']:
        ema.update()
      train_loss += loss.item()
      global_steps += 1  
      
      # --------------------     new -----------------#
      # 修正偏差
      decay = min(config['decay'], (1 + global_steps) / (10 + global_steps))

      one_minus_decay = 1.0 - decay
      # SDA 更新Average参数
      with torch.no_grad():
        # parameters：可训练参数都要有这样的操作
        parameters = [p for p in model.parameters() if p.requires_grad]
        # 指数移动平均
        # s_param平均参数  ；param当前参数 
        for s_param, param in zip(kd_model.parameters(), parameters):
          s_param.sub_(one_minus_decay * (s_param - param))
          # sub_：在原位置修改
      # --------------------     new -----------------#
      train_iterator.set_postfix_str(f'running training loss: {loss.item():.5f}')
      
     

      if global_steps % config['logging_step'] == 0:
        if global_steps >= config['ema_start_step'] and not config['ema_start']:
          print('\n>>> EMA starting .....')
          config['ema_start'] = True
          # --------------------     new -----------------#
          # if 多张GPU卡， model 会放在 model.module 属性里， else 返回 model
          ema = EMA(model.module if hasattr(model, 'module') else model, decay = 0.99)
          # --------------------     new -----------------#

        print_train_loss = (train_loss - logging_loss) / config['logging_step']
        logging_loss = train_loss

        if config['ema_start']:
          ema.apply_shadow()
        val_loss, f1, acc = evaluation(config, model, dev_dataloader)

        print_log = f'\n>>> training loss: {print_train_loss:.6f}, valid loss: {val_loss:.6f},' 

        if acc > best_acc:
          model_save_path = os.path.join(config['output_path'],
                          f'checkpoint- {global_steps} - {acc:.6f}')
          model_to_save = model.module if hasattr(model, 'module') else model
          model_to_save.save_pretrained(model_save_path)
          best_acc = acc
          SDA_model_path = model_save_path
        print_log += f'valid f1: {f1:.6f}, valid acc:{acc:.6f}'

        print(print_log)
        model.train()

        if config['ema_start']:
          ema.restore()


  return model, SDA_model_path

In [ ]:
model = train(config, train_dataloader, dev_dataloader)

Some weights of the model checkpoint at /content/drive/MyDrive/Colab Notebooks/dataset/BERT_model were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 


>>> training loss: 0.635384, valid loss: 0.555733,valid f1: 0.000000, valid acc:0.689991



100%|██████████| 1/1 [02:19<00:00, 139.49s/it]
